# Process Illinois County-level & Zipcode-level Data from IDPH

In [1]:
import pandas as pd
import json
import numpy as np
import geopandas as gpd
import shapely.wkt
import requests
from urllib.request import urlopen
from datetime import datetime, timedelta

URL preparation

In [2]:
# Illinois Geometry URL
zipcode_geometry_url = "http://www.dph.illinois.gov/sites/default/files/COVID19/il_illinois_zip_codes.json?nocache=1"
county_geometry_url = "http://www.dph.illinois.gov/sites/default/files/Illinois_County_Geo_ch.json"

Download data

In [3]:
with open("./idph_COVIDZip.json") as f:
    zipcode_dict = json.load(f)
    zipcode_data = pd.DataFrame(zipcode_dict['zip_values'])

In [4]:
with open("./idph_COVIDHistoricalTestResults.json") as f:
    history_dict = json.load(f)

In [5]:
zipcode_geometry = gpd.read_file(zipcode_geometry_url)
county_geometry = gpd.read_file(county_geometry_url)

In [6]:
with urlopen(zipcode_geometry_url) as response:
    tmp = json.load(response)
    zipcode_gpd = gpd.GeoDataFrame(tmp['features'])

In [7]:
zipcode_gpd['geometry'] = zipcode_geometry
zipcode_gpd = zipcode_gpd[['id','geometry']]
zipcode_gpd['id'] = zipcode_gpd['id'].astype(str)

In [8]:
county_geometry

,id,OBJECTID,County,Percent_1,Shape__Area,Shape__Length,geometry
0,McHenry,1,McHenry County,27.7,17027625788.537001,522324.30501520802,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
1,Boone,2,Boone County,33.7,7862723082.6885996,375171.14457663603,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
2,Ogle,3,Ogle County,31.2,21284584781.1903,651729.42136120505,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
3,Will,4,Will County,28.0,23669428994.175301,779085.21483073698,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
4,LaSalle,5,LaSalle County,27.1,32008439057.989899,825360.16391267197,"POLYGON ((-88.93885 41.62837, -88.93891 41.628..."
...,...,...,...,...,...,...,...
98,Lawrence,99,Lawrence County,28.9,10426836159.952499,442943.67976590397,"POLYGON ((-87.90806 38.85013, -87.90819 38.835..."
99,Marion,100,Marion County,33.5,16056558800.8808,509529.79577371402,"POLYGON ((-89.13844 38.73633, -89.13847 38.721..."
100,Union,101,Union County,28.2,11769540678.443001,489617.64749037399,"POLYGON ((-89.04143 37.59650, -89.06017 37.597..."
101,Pope,102,Pope County,29.5,10432553057.307899,537878.380730576,"POLYGON ((-88.70860 37.59926, -88.70876 37.584..."


In [9]:
# Move the target row "Chicago" to the first element of list
target_row = 102
idx = [target_row] + [i for i in range(len(county_geometry)) if i != target_row]
county_geometry = county_geometry.iloc[idx]
county_geometry = county_geometry.reset_index(drop=True)
county_geometry

,id,OBJECTID,County,Percent_1,Shape__Area,Shape__Length,geometry
0,Chicago,103,Chicago,28.2,6450276623.31,845282.931362,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ..."
1,McHenry,1,McHenry County,27.7,17027625788.537001,522324.30501520802,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
2,Boone,2,Boone County,33.7,7862723082.6885996,375171.14457663603,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
3,Ogle,3,Ogle County,31.2,21284584781.1903,651729.42136120505,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
4,Will,4,Will County,28.0,23669428994.175301,779085.21483073698,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
...,...,...,...,...,...,...,...
98,Jefferson,98,Jefferson County,30.2,16276278165.5205,510233.04148936202,"POLYGON ((-89.14445 38.47387, -89.14462 38.460..."
99,Lawrence,99,Lawrence County,28.9,10426836159.952499,442943.67976590397,"POLYGON ((-87.90806 38.85013, -87.90819 38.835..."
100,Marion,100,Marion County,33.5,16056558800.8808,509529.79577371402,"POLYGON ((-89.13844 38.73633, -89.13847 38.721..."
101,Union,101,Union County,28.2,11769540678.443001,489617.64749037399,"POLYGON ((-89.04143 37.59650, -89.06017 37.597..."


In [10]:
county_gpd = county_geometry[['id','geometry']]

In [11]:
# Generate necessary polygons
polygonOOS = shapely.wkt.loads('POLYGON ((-89.6536529004544 39.7828177522233, -89.6536529004544 39.7799814663649, -89.64987635016143 39.7799814663649, -89.64987635016143 39.7828177522233, -89.6536529004544 39.7828177522233))')
polygonIllinois = shapely.wkt.loads('POLYGON ((-89.64923261999786 39.78268583454302, -89.64923261999786 39.7799814663649, -89.64575647711456 39.7799814663649, -89.64575647711456 39.78268583454302, -89.64923261999786 39.78268583454302))')
polyCookWithoutChicago = shapely.wkt.loads('MULTIPOLYGON (((-87.93992864392526 41.99344156938823, -87.940059952909 41.9934377560448, -87.9592864584299 41.9930952543991, -87.9787642642879 41.9928203538893, -87.97886586432659 41.9928202540587, -87.9789527640426 41.9928185534356, -87.9982849696229 41.9925979522091, -88.0179535757124 41.99244465174, -88.0312664795014 41.9925176507714, -88.0509826848682 41.9918497491432, -88.0703918903931 41.9913568484372, -88.0900793965758 41.9906078468331, -88.0907670966521 41.9905772473565, -88.1096834024695 41.9897596454982, -88.1103157024931 41.9897324454626, -88.12904890807501 41.9888092443022, -88.1298918077994 41.9887773443969, -88.1449342126437 41.9882207431457, -88.164536517955 41.9877591424364, -88.18375842345409 41.9876541409864, -88.18413892402531 41.9876527416196, -88.20363452925351 41.9873353400353, -88.2229406346837 41.9870596391324, -88.2231518351291 41.987028739309, -88.2423458409274 41.9867116381146, -88.24255094037311 41.9867096380104, -88.2426562411228 41.9867065380408, -88.26273304687879 41.9863761367198, -88.26312604863 42.0010993388119, -88.2632611499117 42.0156900410801, -88.26328285113109 42.0301140440916, -88.2632826514928 42.0302242443359, -88.2632844510472 42.0303262436609, -88.26334355238269 42.0447283463752, -88.263556654589 42.059304048841, -88.2635589541746 42.0594845491014, -88.2635617546798 42.0596361487435, -88.26364075551911 42.0671681502163, -88.2576660537848 42.0670757505957, -88.2425203491039 42.0668392513013, -88.238085847463 42.0669560519229, -88.23828124918511 42.0812984540228, -88.23830895070751 42.0959089566692, -88.2384144525836 42.1103160587092, -88.2383625532828 42.1248314616329, -88.23858955393921 42.1249688609648, -88.238649155315 42.1394576637292, -88.23857735674061 42.1542480667897, -88.2193774513445 42.1542163676744, -88.2192016509199 42.1542158676613, -88.19985024569441 42.1542607683223, -88.1806949394871 42.1542179697689, -88.1608575341823 42.1542114705957, -88.1413099282908 42.1541499715578, -88.12157142234391 42.1540086729736, -88.10183901650559 42.1539304744313, -88.08241721051409 42.1538746752888, -88.0628704050111 42.1537729765465, -88.0434471996785 42.1537798772474, -88.0241445934565 42.153685878487, -88.0044718882224 42.1535816796345, -87.9850542820747 42.1536591807235, -87.9656263768652 42.1535268816348, -87.9462160704968 42.1534302829378, -87.9265300651734 42.1532526841042, -87.9069992594988 42.1530876847653, -87.88739635336189 42.1529929863455, -87.86794574746349 42.1529578868981, -87.86771064752421 42.1529574876123, -87.86744594729031 42.1529552869858, -87.8482148416805 42.1528744884679, -87.8288342363241 42.1527747889033, -87.8286381363958 42.1527706889045, -87.8284270357716 42.1527691897463, -87.8091018304677 42.1526753906309, -87.78953262462881 42.1525626913735, -87.77002361852141 42.1524146928165, -87.7589142153644 42.1523945935908, -87.75898641565 42.1523211936068, -87.75881741498139 42.1521258932104, -87.7583309148183 42.1514603930119, -87.75782821516761 42.1508469932474, -87.75740521438129 42.1504468929188, -87.75752761483059 42.1502970928133, -87.75726291454539 42.1497917924629, -87.7568049148245 42.1493112927317, -87.7562058141183 42.1487075930618, -87.75599351375899 42.14832149247, -87.7556891138003 42.1479202922637, -87.755534314318 42.1476039925612, -87.75530631359371 42.1472562921288, -87.7550560141584 42.1467676921909, -87.7547401136527 42.1462368924618, -87.7542668137493 42.1456376920938, -87.7539129132826 42.145144892327, -87.7540632133999 42.1449154923127, -87.7531916126169 42.1441094919281, -87.75258101226351 42.1433786919517, -87.75286041281331 42.1433138921743, -87.7521910120672 42.1425629923501, -87.75161581264371 42.1418961917368, -87.7511366122952 42.1413822920796, -87.7512312120012 42.141303691314, -87.75115641174111 42.1410463916399, -87.7504635117086 42.1403502920152, -87.7494381114722 42.1390541916177, -87.74897171117151 42.1384743914277, -87.7481289111896 42.1377072912231, -87.7487231104916 42.1373506910521, -87.7476897108005 42.136223890757, -87.74708031015319 42.1355923912397, -87.7466582104704 42.1350213909149, -87.7463851096346 42.1347033906008, -87.7460396097343 42.1341775911047, -87.745687109522 42.1337784908184, -87.7459010097253 42.1336685909357, -87.745257609316 42.1332073906692, -87.74453580957331 42.1326320907018, -87.7437754090956 42.1318330906349, -87.7438844087518 42.1317684905165, -87.7434884092152 42.1313356908312, -87.74336210834799 42.1309232904562, -87.74294120837131 42.1304514907658, -87.74296060851979 42.1301320901905, -87.7426184082348 42.1296228903834, -87.7422304081534 42.1290248906056, -87.74182620768551 42.1284816901276, -87.7415983076711 42.1279933896626, -87.7415076073822 42.1276338901153, -87.7409407078308 42.127226089798, -87.7404242071633 42.1264470897437, -87.73994060733369 42.1259688895807, -87.73994620739199 42.1257540900704, -87.7397080074503 42.1253757893015, -87.7393668072224 42.1249713893088, -87.73926800676961 42.1249285896715, -87.7391848068431 42.1247096892571, -87.73863280636439 42.1243021893126, -87.7381525061068 42.1234133892103, -87.7376872065733 42.1228003896419, -87.736851505844 42.1220553893805, -87.7360145055561 42.1209354895734, -87.7360704056162 42.1207764894055, -87.73566520513511 42.1205611889959, -87.7352119052705 42.1200585893206, -87.73462050459111 42.119741389421, -87.73404620450241 42.1191957886615, -87.7337453046937 42.1186705890108, -87.7332023046262 42.1181997884742, -87.7317741042509 42.117186988981, -87.7317795037898 42.1172642884217, -87.7307695035923 42.1163953885507, -87.7304625037186 42.1162475885618, -87.72987320300631 42.1164181891168, -87.729580802717 42.1161328890581, -87.7305220037089 42.1155264886369, -87.73086530378851 42.1152806885721, -87.7301333030162 42.1145397883148, -87.72966300325621 42.1139817885943, -87.72897570237021 42.1129466885669, -87.72840540242331 42.111968487749, -87.727826402452 42.1111828875725, -87.7270160014756 42.1106117874446, -87.7270648013853 42.1104443876078, -87.7264152019015 42.1095309874555, -87.7259716012052 42.1088025877604, -87.7254353010927 42.1083676872663, -87.7248817007263 42.1080261876787, -87.72475650065449 42.1078589875011, -87.72475980114029 42.1075916873596, -87.7236987005483 42.1067082877196, -87.7219767994704 42.1054761876252, -87.7220315996499 42.1052233872526, -87.7210808998671 42.1043635867202, -87.7210927996824 42.104190086783, -87.7206543989053 42.1038337870434, -87.7202784992318 42.103492086897, -87.7202300994762 42.1032212865965, -87.7194503986766 42.1027579873195, -87.7187077985047 42.1021463871119, -87.71787879863081 42.1014480868531, -87.7172606979468 42.1008878864995, -87.71659409769759 42.1002057866217, -87.7158694974498 42.099475886873, -87.7154708971272 42.0995775867462, -87.71528669700059 42.0992661865526, -87.71526109686179 42.0989709867401, -87.71489349705929 42.0988801866949, -87.7148024972842 42.0984020863345, -87.714402197192 42.0980048863186, -87.71353189684289 42.0973307868225, -87.7130466962131 42.096786285929, -87.7124445962817 42.0963255865825, -87.71145689572489 42.0958976864376, -87.7114588957958 42.0956800864382, -87.7104820949976 42.0949767859563, -87.7089930950618 42.0941970862837, -87.7081270943707 42.0935036864336, -87.7078778948469 42.0934118863354, -87.7071144942589 42.0928962860757, -87.70666479435511 42.092131885614, -87.70586279394639 42.0915385857171, -87.7049199929538 42.0910948853779, -87.7050526933443 42.0909727861941, -87.7040162926601 42.0902905856129, -87.70232849209999 42.0891992851985, -87.7012542920169 42.0886873856671, -87.7002964912286 42.0882543854642, -87.7002972915939 42.0879429852602, -87.6984681913144 42.0868852853296, -87.6984846911599 42.086678885488, -87.6969827904537 42.0859787857232, -87.6960675905418 42.085334184897, -87.6952573901162 42.0847792851591, -87.69400828980631 42.0840332848743, -87.69327998887709 42.0835980849165, -87.6916618885254 42.0828162853336, -87.6893981880483 42.0818318846832, -87.6873558872507 42.0810078853292, -87.6848249864275 42.0800608848061, -87.6835454854298 42.0793500848932, -87.6823092850352 42.0789596846991, -87.681946885095 42.078678685203, -87.68170238539849 42.0782781846658, -87.67998458448839 42.0786025846685, -87.679875084797 42.0785485848732, -87.6813307854119 42.0782037846604, -87.6813027849082 42.0771781850353, -87.6817999851766 42.0775355846427, -87.68312038545589 42.077271384391, -87.6837115855147 42.0768805847955, -87.6843914856918 42.0760804847763, -87.68566098602579 42.0748150844114, -87.685281586218 42.0747515841915, -87.68408988534399 42.0758802839804, -87.68346238586599 42.0755512840761, -87.68247128509741 42.0763687841986, -87.68220248491819 42.077012385045, -87.6822755853529 42.0762197845076, -87.6822462853612 42.0755193842865, -87.68187048453061 42.0743590842019, -87.6809876844525 42.0732076838382, -87.68027168442239 42.0724611843997, -87.6797274837395 42.0717889842537, -87.6792603837313 42.070861483618, -87.6785877831735 42.0698786836718, -87.6777940834388 42.0685825832605, -87.67721628316259 42.0677856831002, -87.6770011823136 42.0673966830936, -87.6766926823353 42.0669014830329, -87.6762736826508 42.0666664829712, -87.6758778818805 42.066390482809, -87.675880282149 42.0661645829649, -87.67572338187421 42.0660877835559, -87.6756452820004 42.0658275835676, -87.675731282377 42.0656552833219, -87.6754986815976 42.0650922833564, -87.6747866817107 42.0639241825699, -87.6747814817376 42.0630173825067, -87.6744360816102 42.0619732829424, -87.6735043809965 42.0619620828359, -87.6734974813663 42.0616698822618, -87.6735226812516 42.061419382357, -87.6739460812623 42.0610812821524, -87.6740087810596 42.0605365824778, -87.6728020810811 42.0596144823133, -87.67181068074311 42.0594893821543, -87.6717061804347 42.0599342822482, -87.6710826802984 42.0601536827255, -87.6701453795705 42.0598034822169, -87.66955497980869 42.0592047827774, -87.669754379241 42.058121981751, -87.6698738792349 42.0565721815967, -87.669911079591 42.0554675819086, -87.66987387901401 42.054381181808, -87.669723379137 42.0535190816635, -87.6695009793326 42.0525786811364, -87.6691911789123 42.0518657812517, -87.6694422786683 42.0516021809078, -87.6698710791306 42.0514735814065, -87.6704898794535 42.0517005810869, -87.6703798788965 42.0522115813627, -87.66987277889839 42.0519558807074, -87.6699969789815 42.0526963808008, -87.67019227947 42.0532725810197, -87.6703725792903 42.0541268813411, -87.6704630790701 42.0548833812777, -87.6704726793967 42.0554870818804, -87.6703746794763 42.0567890818954, -87.6702998793145 42.0575045817421, -87.670222279559 42.0581868822107, -87.6702236798539 42.05868298251, -87.670806580098 42.0585979821404, -87.6732599808622 42.0589571820813, -87.6738454810927 42.059324281773, -87.6743905810247 42.0592717820105, -87.67440108137561 42.0579187814385, -87.6742349810124 42.0576323815015, -87.67427328131851 42.0573105815334, -87.67397828108641 42.0570029818814, -87.6740015806277 42.0560001818698, -87.6734840808914 42.0559923811781, -87.6735592803923 42.0556738817203, -87.67427728043759 42.0556514813627, -87.6742964809594 42.0555567818348, -87.674418880461 42.0549260817735, -87.67439728046629 42.0542174815321, -87.674141980766 42.053397881562, -87.6738133800274 42.0528307806926, -87.6736520801735 42.0518168812567, -87.67333557986839 42.0517625810993, -87.673278679654 42.050852280877, -87.6732930795214 42.0500449809215, -87.67357177950061 42.0499939803439, -87.67310047963289 42.0488239801753, -87.67237877969281 42.0480248802808, -87.67318787926671 42.0477751798295, -87.6726678792994 42.0466264801099, -87.672120279137 42.0455434794757, -87.67210667865341 42.0452235797783, -87.6719899788766 42.0440670793153, -87.6718458785808 42.0432408792133, -87.67106107859141 42.0424546792, -87.6705575785672 42.0420667790804, -87.67051287863841 42.0419420790668, -87.66950337755701 42.0409898794764, -87.6690841772561 42.0400837791089, -87.66866757775119 42.0387394786037, -87.66825437672451 42.0367902786984, -87.6679938767427 42.0319977781212, -87.6676035760435 42.0298904773719, -87.6674578757425 42.027961777501, -87.66705567528901 42.0266355767535, -87.6664487756138 42.0255681766647, -87.665575574843 42.0238242767547, -87.6652881747289 42.0229710763102, -87.6652826951226 42.02294890466851, -87.66562049307413 42.02295177620253, -87.66728810995436 42.02296597102081, -87.67069738802076 42.02299491797321, -87.67557756531933 42.02302847606026, -87.67705606068898 42.02303858614759, -87.67676592504883 42.02262881241506, -87.67665666306539 42.02247451364826, -87.67664628739456 42.02245990959261, -87.67624075402881 42.02192774301023, -87.67559528114492 42.02140237710148, -87.67452056319178 42.02052757195246, -87.67370228173777 42.01986146941876, -87.6735518536952 42.01973898335073, -87.67323202028757 42.01947864274607, -87.67311983931955 42.0193873298531, -87.67311770517385 42.01938559231194, -87.67312467960541 42.01938563599528, -87.67318420276976 42.01938600716044, -87.67350741974928 42.01938802176841, -87.67352618665861 42.01938828085255, -87.67373382370046 42.01939114570978, -87.6739778160281 42.01939451184009, -87.6744080492424 42.01939519874416, -87.67488838648171 42.0193976807649, -87.67536619137216 42.01939946021425, -87.67568144230945 42.0194016785191, -87.67607045620684 42.019395372608, -87.67630675847633 42.01940269845062, -87.67646995030911 42.01940775778638, -87.6765784253373 42.01940917431479, -87.67675715286175 42.0194106084335, -87.67682977234753 42.01941102380663, -87.67688203032283 42.01941132268639, -87.67690633556225 42.01941146168758, -87.67697605414642 42.01940986179198, -87.67707864116863 42.01940750742844, -87.67715477894089 42.01941145526159, -87.67728044249431 42.01942688237946, -87.6773529364871 42.01942726432301, -87.67747474098876 42.01942788270879, -87.67763305153053 42.01942866011819, -87.6780050166826 42.01943048499232, -87.67815659227287 42.01943122844148, -87.67833771504546 42.01943236230655, -87.67889871032075 42.01943587339085, -87.67903700019637 42.01943663816948, -87.67922640411035 42.01943768556743, -87.67938327780885 42.0194385528116, -87.67974151049441 42.01944098205966, -87.68024292592946 42.01944438028124, -87.68044759497714 42.01944700699906, -87.68065772901298 42.01944970339436, -87.68147052977034 42.01945552969489, -87.68163041236896 42.01945726494441, -87.68188412712462 42.01946001827427, -87.6826758837548 42.01946527753208, -87.68281067257166 42.01946657718164, -87.68297589283992 42.01946816957079, -87.68315929139037 42.01946993768774, -87.68390097634376 42.0194773414155, -87.68439762953687 42.01948229659448, -87.68465309465869 42.0194847724209, -87.68485281322879 42.01948670772154, -87.68542364310046 42.01949662708502, -87.68588219367783 42.01950294661916, -87.68629446523019 42.01950763136195, -87.68650871368179 42.01950829856822, -87.68655236552121 42.01950776053493, -87.68668322762556 42.01950795614504, -87.68758356331006 42.01951541226423, -87.6877350766631 42.01951506617155, -87.68788541198859 42.01951472258477, -87.68838321200079 42.01951911236752, -87.68860769858108 42.01952272939013, -87.68880160850424 42.01952585321355, -87.68895391063931 42.01952753706724, -87.68912439084073 42.01952942153381, -87.68999118410451 42.01953129218816, -87.69017356947235 42.01952843072509, -87.69035851694792 42.01952552926393, -87.69044048390533 42.01952439676167, -87.6907820724842 42.01951695456376, -87.69097255925169 42.0195128040591, -87.69144569428222 42.01950351371235, -87.69209102058079 42.01949043607064, -87.69262768635853 42.01947832320497, -87.69271416588813 42.01947637125506, -87.69316316706967 42.0194662346992, -87.69330944672456 42.01946296243532, -87.69345480650243 42.01945971065224, -87.69475357363341 42.01943168080385, -87.69487912182714 42.01942898081307, -87.69504149327391 42.01942548849905, -87.6956248042278 42.01941286963874, -87.69575359872123 42.01941008025273, -87.69583270145962 42.01940837146943, -87.69632041629497 42.01939812665292, -87.69644888274568 42.01939501414569, -87.69660241707233 42.01939129413828, -87.69709003315397 42.01937989294708, -87.69723265835182 42.01937766483302, -87.69736494195971 42.0193755987778, -87.69819556439585 42.0193583542539, -87.6987822873361 42.01934667097137, -87.69909886932776 42.01933722442636, -87.69941366144963 42.01933012711883, -87.69958999054704 42.01932612505889, -87.69979780291936 42.01932140848479, -87.70016467284763 42.01931009608742, -87.7002045723778 42.01930939440479, -87.70050550885749 42.01930409990933, -87.70107676497479 42.0192934172949, -87.70170963787815 42.0192815341745, -87.70194315118744 42.01927783766754, -87.70213158918932 42.0192748543824, -87.70251212367708 42.01926607938554, -87.70301582188144 42.01925446697979, -87.70345024572359 42.01924436571954, -87.70414264293234 42.01922686837962, -87.7043096219145 42.01923051292734, -87.70450358193855 42.0192347463132, -87.70479948827237 42.01922824452788, -87.70495566966552 42.01922470072313, -87.7050129434277 42.01922348546938, -87.70546491990622 42.01921347266413, -87.70590636028108 42.0192046079268, -87.70632649676627 42.01919526837646, -87.70655301263913 42.01919085248989, -87.70666071468575 42.01918853748336, -87.70705898399051 42.01917997567989, -87.7072997850097 42.01917489533135, -87.70759300767953 42.01916801496249, -87.70785331831766 42.01916243581311, -87.70812665491115 42.01915700962434, -87.7084944478117 42.01914891375925, -87.7086615358199 42.01914390647527, -87.70901447098903 42.01913110715609, -87.7090166084599 42.0190405053462, -87.70901770412374 42.01899380560321, -87.70903239019403 42.0183677177322, -87.70903908694568 42.01808175657062, -87.70904658175597 42.01776242827985, -87.70906800649061 42.01684933621078, -87.70907216206238 42.01667235873169, -87.70907492706188 42.01655448338911, -87.70908083968285 42.01630210639907, -87.70909273573071 42.01579449662375, -87.70909981616859 42.01549207204125, -87.7091162451832 42.01479030838971, -87.70912301854372 42.01450401761119, -87.70912849096226 42.01426804698445, -87.70914743235029 42.01346031725674, -87.70916460488986 42.01272870148493, -87.70918288195756 42.01194939740851, -87.70918742812705 42.01176814096928, -87.7091900795898 42.01166848602311, -87.70919009065311 42.01166806951592, -87.70931580129734 42.01054037620479, -87.70946850884485 42.00479334147273, -87.70947531037487 42.00451890522452, -87.70947747121735 42.0011718285843, -87.70946523217847 41.99922333082218, -87.70945596510167 41.99775420596532, -87.70945423006752 41.99747983138403, -87.70945364626277 41.99738755118066, -87.70964156652001 41.99738463772634, -87.70980275969994 41.99738213860577, -87.7106709029225 41.99736566521522, -87.71078055278299 41.99736357172203, -87.71159697623226 41.99735032338869, -87.71173473348638 41.99734810597317, -87.71206308500084 41.99734277606775, -87.71230143545807 41.99733907055204, -87.71254194310215 41.99733669506624, -87.71276845791311 41.99733193721285, -87.71305993705698 41.99732744763664, -87.71340763786904 41.99732208150076, -87.71368135028347 41.99731760443929, -87.71395462088148 41.99731293279655, -87.71428685982301 41.99730874207443, -87.7143683896674 41.9973079742718, -87.71551146809051 41.99729728332444, -87.71603287903935 41.99729240468299, -87.71673891061315 41.99728939335967, -87.71709106228738 41.99728788101894, -87.7175873549438 41.9972859619564, -87.71796428058025 41.99728287814769, -87.71858399085167 41.99727777499614, -87.72745077208791 41.99724482938733, -87.72754170311637 41.99724443142476, -87.7279962050708 41.99724249460746, -87.72808716244464 41.99724214986345, -87.72817817735461 41.99724175074125, -87.72836004124886 41.99724095332682, -87.72879015523763 41.99723916199823, -87.72903230264895 41.99723873888282, -87.72948803269705 41.99723927404385, -87.73000298215105 41.99723989610442, -87.73649758567144 41.99724717148989, -87.73661056771267 41.99724726600009, -87.73673073121394 41.99724739689974, -87.73684374219062 41.99724754456818, -87.73696403885752 41.9972476764644, -87.73707710797329 41.99724782283302, -87.73719733158332 41.99724789922139, -87.73729571648657 41.99724802646985, -87.73753857443677 41.99724829819846, -87.73763906330355 41.99724838146313, -87.73775003778331 41.99724851904333, -87.73786105128404 41.99724865397445, -87.73797200347117 41.99724873634051, -87.73808289518648 41.99724887316966, -87.73819384723328 41.99724901020443, -87.73830480076603 41.99724914576758, -87.73841587606368 41.99724928185451, -87.73851637340543 41.99724936412063, -87.73862886866377 41.99724950735923, -87.73883100154521 41.99724967540421, -87.7392091542239 41.99725009728275, -87.73972764336321 41.99725063915199, -87.73997043700635 41.99725090650043, -87.7404542725419 41.99725143088362, -87.74093785464996 41.99725195191947, -87.74118011733385 41.99725270847286, -87.74166622947816 41.99725532507865, -87.74215254474984 41.99725794039745, -87.74239533582951 41.99725924567164, -87.74342000333837 41.99726473604695, -87.74354143067211 41.99726541460147, -87.74469994329327 41.99727158088064, -87.74531054627043 41.99727482007528, -87.74539885439877 41.99727532618431, -87.74561240311597 41.99727641972085, -87.74579636393625 41.99727741630957, -87.74603931150311 41.99727860463489, -87.74643377589801 41.99728023872555, -87.74663086287643 41.99728102639232, -87.74682797658166 41.99728186874112, -87.7470123887759 41.99728259099331, -87.74725516772895 41.99728355744616, -87.74766498083319 41.99728520913774, -87.74772375727008 41.99728550899834, -87.74816179305283 41.99728730343837, -87.75215967045726 41.99729883214376, -87.7524445596995 41.99729967095723, -87.75284612930938 41.9973008244622, -87.7528983393028 41.99730097869976, -87.7529000642633 41.99730098330549, -87.75300932737618 41.99735203329875, -87.7532052702954 41.9974435821246, -87.7533305545867 41.99750211798557, -87.75342008486713 41.9975439524918, -87.75352163136969 41.99759139116935, -87.75393478209813 41.9977843636727, -87.75434785929524 41.99797744380801, -87.75444325136567 41.99804022113356, -87.75546545813066 41.99852400261438, -87.75561144628155 41.99859088019373, -87.75566460235349 41.9986157356828, -87.75573416985796 41.99864830258533, -87.7558467100467 41.99870092588596, -87.7559389550109 41.99874406239963, -87.75607571061623 41.9988080315767, -87.75621635084521 41.99887377585581, -87.75636676940404 41.99894434428234, -87.7565014466029 41.99899962947943, -87.75661975094467 41.99904819378276, -87.75666399887459 41.99906962842022, -87.75675520447923 41.99911382901636, -87.7568587190938 41.99916399137214, -87.75693981612821 41.9992032838332, -87.75706911720613 41.99926595178945, -87.75715805230632 41.99930909787284, -87.75725713480168 41.99935709715094, -87.75737856163352 41.99941593852788, -87.75753142115506 41.99948973139722, -87.75766508241088 41.99955425586801, -87.75771443084187 41.9995771978169, -87.75776381646429 41.99960013993101, -87.75786255102803 41.99964602391145, -87.75793849827693 41.99968136565118, -87.75801063449954 41.99971487707234, -87.75804492595117 41.99973082859469, -87.75811076118636 41.99976145386435, -87.75819447590304 41.99980037436682, -87.75829075543312 41.99984514800016, -87.75841612555023 41.99990340403335, -87.75855839467326 41.99997227648652, -87.75865257401256 42.00001786832064, -87.75871050159994 42.0000442281479, -87.75875999904945 42.00006708806638, -87.75882352965492 42.0000963298428, -87.75888566730143 42.00012501577454, -87.75900111343185 42.00017822707228, -87.75907259983862 42.00021117887411, -87.75912678212305 42.00023615451807, -87.75918397470699 42.00026251042925, -87.75926014502582 42.00029763287274, -87.75933635295296 42.0003327554536, -87.75945029878467 42.00038519042233, -87.75958666773037 42.00045133318638, -87.7597146976465 42.00051343088836, -87.75981969950394 42.00056228078616, -87.75989326642882 42.00059651193409, -87.75998808383571 42.00064061795552, -87.76007418128152 42.00068067410405, -87.76018695086293 42.00073312990764, -87.76031756283885 42.00079390724729, -87.76032243579562 42.00079611238247, -87.76045587004447 42.00085809819979, -87.76061879447313 42.00093421211321, -87.7607576114287 42.00099906357649, -87.76089877662966 42.00106469523017, -87.76104990233689 42.0011355903347, -87.76118585923821 42.00119954915282, -87.76129734206262 42.00125202489134, -87.76143326267263 42.00131598323485, -87.76176081411275 42.00146344108148, -87.76202451634273 42.00158009225044, -87.76227014378563 42.0016872449783, -87.7622256901189 42.00181849344116, -87.76220232934907 42.0018841555186, -87.76217696679183 42.00195557043538, -87.76214096291633 42.00205687153926, -87.76210276831729 42.00216439104591, -87.76206849969232 42.00226490494232, -87.76204238149201 42.00234265514888, -87.76200522460688 42.00245344509666, -87.76199316699538 42.00248938916523, -87.76195816714102 42.00263684510826, -87.76195348098868 42.00265658737819, -87.76193486288821 42.00280484574941, -87.76192796063077 42.00286897061068, -87.76191962122311 42.0029457664857, -87.76190995441182 42.00303517903608, -87.76190036950048 42.00312365896653, -87.76189278156757 42.00318254037359, -87.7618853849682 42.00323993812403, -87.76187862145292 42.00332555069307, -87.76187245364427 42.00340227504341, -87.7618673607259 42.00346561308598, -87.76185849995711 42.00354309240094, -87.76185401961568 42.00369905628419, -87.76184866685028 42.00388540884312, -87.76184526510966 42.00396129598145, -87.7618424889306 42.00402483791851, -87.76183964260714 42.0040879947699, -87.76183520084646 42.0041896452372, -87.76183124068046 42.00427859249245, -87.76182786917626 42.0043551658235, -87.76182220280297 42.0044826879159, -87.76181532316492 42.00465290303666, -87.76180783699391 42.00483809841759, -87.76180435516898 42.00491469918197, -87.76180087481073 42.00499113502661, -87.76179761347161 42.00506768172826, -87.76179331970721 42.00516930519708, -87.76179238151894 42.00519076010192, -87.76178744664826 42.005307528312, -87.7617805063085 42.00547044382981, -87.76177356273324 42.00563371634572, -87.76176720571272 42.0057766293309, -87.76175822140343 42.00597468774254, -87.76174783151836 42.00620649269496, -87.76173974718046 42.00638164157899, -87.76173368129007 42.00651305783436, -87.76172695503978 42.00665613415701, -87.76172109467262 42.00678464286131, -87.76171553691155 42.00696167035695, -87.76171077579778 42.0071112324682, -87.76170364970973 42.00733594617071, -87.76168507611592 42.00773787753884, -87.76166484838132 42.00819298303789, -87.76166410866762 42.00827231388287, -87.76183977273998 42.00827486199849, -87.76240658878095 42.00827960434774, -87.76260585254093 42.00828125359133, -87.76264470567803 42.00828155653016, -87.76274993015249 42.00828246435787, -87.76293933378879 42.00828406373555, -87.76310857805991 42.00828545426288, -87.7632557452189 42.00828667809566, -87.76327730468101 42.008286840049, -87.76340298558407 42.00828790429856, -87.76344654676475 42.00828823006714, -87.76355015386231 42.00828912775942, -87.76361579058876 42.0082896733561, -87.76369732102982 42.00829035267257, -87.76378503307305 42.00829106150506, -87.76384448784688 42.00829157629765, -87.76395427555768 42.00829245050215, -87.76399165686949 42.00829280084264, -87.76412351757294 42.00829389303351, -87.76413882370447 42.008294023816, -87.76428599053492 42.00829524797275, -87.7642927619176 42.00829528154075, -87.76443315959045 42.00829647085416, -87.76446200407456 42.00829666868992, -87.76458032643637 42.0082976946336, -87.76463124561967 42.00829816672622, -87.76472749365792 42.00829891822624, -87.7648004881675 42.00829955310382, -87.76487466200319 42.00830014026361, -87.76496973218494 42.00830094061128, -87.76502182887225 42.00830136347712, -87.76513897474565 42.00830232731313, -87.76516899649465 42.00830258540798, -87.76530821682576 42.00830376892129, -87.76531616485086 42.00830380825147, -87.76546333175281 42.00830502980138, -87.76547745941167 42.00830515457535, -87.76561049902065 42.00830625226212, -87.76564670346964 42.00830654108483, -87.7657576674122 42.00830747316763, -87.76581594522003 42.00830798111949, -87.76590483469546 42.00830869525103, -87.76598518856247 42.00830936712632, -87.76605200161848 42.00830991714399, -87.76615443082095 42.00831075178061, -87.76619917076654 42.00831113776157, -87.76629497611779 42.00831194064384, -87.76632367480001 42.00831219217533, -87.76635369618904 42.0083124505074, -87.76649291659159 42.00831363121202, -87.76650822175162 42.00831370678996, -87.76666274866055 42.00831501940147, -87.76684729747647 42.00831653320574, -87.7670935075021 42.00831862614089, -87.76713184522961 42.00831892501841, -87.76718843107388 42.00831936904389, -87.76746186859137 42.00832165008312, -87.76746519231534 42.00817496288605, -87.76747118475615 42.0079097387231, -87.76747238074026 42.00785798864055, -87.76747431903192 42.00777205015366, -87.76747547236067 42.00772079463709, -87.76747741527363 42.00763441655158, -87.76747856470287 42.00758360035944, -87.76748054913344 42.00749672961986, -87.76748169418654 42.00744640653589, -87.76748364497637 42.00735909573462, -87.76748478563889 42.00730926740537, -87.76748677881486 42.00722140742391, -87.76748787757082 42.00717207229233, -87.76748981180684 42.00708679320396, -87.76748987462207 42.00708377380617, -87.76749097131854 42.00703487800816, -87.76749297125481 42.0069460866774, -87.76749406320768 42.00689768398588, -87.76749606666093 42.0068084530509, -87.76749715508032 42.00676048996018, -87.76749920080051 42.00667076472818, -87.76750011949305 42.006629331974, -87.76750024657096 42.0066232956548, -87.76750229654125 42.00653313109657, -87.76750333842028 42.0064861005246, -87.76750539372733 42.0063954985664, -87.76750646813902 42.00634890667534, -87.76750852597962 42.00625780994979, -87.7675095594592 42.00621176751731, -87.76751162351006 42.00612017631686, -87.76751265309119 42.00607457320907, -87.76751475572648 42.00598248796773, -87.76751500598517 42.00597085437933, -87.76751574486462 42.00593737916271, -87.76751785100399 42.00584485568906, -87.76751887452662 42.00580018420209, -87.76752098502652 42.00570716734205, -87.76752196590147 42.00566298987265, -87.76752408065082 42.00556953368624, -87.76752505762529 42.00552579581599, -87.76752717858439 42.00543184515458, -87.76752814933258 42.00538860175588, -87.7675304358407 42.00528834026761, -87.76753426358292 42.00511794456206, -87.76753849240473 42.00493130521325, -87.76754347406916 42.00470553766792, -87.76731963428601 42.00460103328688, -87.76721923849894 42.0045541070268, -87.76718340460316 42.00453740998808, -87.76712946865722 42.0045122272468, -87.76699873411805 42.00445115494081, -87.76685539688555 42.00438420260617, -87.76673690033118 42.00432889831321, -87.76671191180239 42.00431719502731, -87.76661840630703 42.00427353930068, -87.76656842849077 42.00425018755181, -87.76649991064232 42.00421818098072, -87.76642494365896 42.00418317796808, -87.76638141507428 42.00416287632476, -87.76628153441619 42.00411622510833, -87.76626292203918 42.00410751667484, -87.76614435305143 42.00405215680146, -87.76613812412083 42.00404921745434, -87.76602585920557 42.00399685178608, -87.76599456843961 42.0039822086278, -87.76590736493995 42.00394149285986, -87.76585123332825 42.00391525422344, -87.76578887052244 42.00388613271257, -87.76570782345152 42.00384830091647, -87.76567037815074 42.0038307724528, -87.76555188402573 42.00377546694558, -87.76546688216722 42.00373539289627, -87.76555279077232 42.00363206049455, -87.76560515446148 42.00356903841267, -87.76564520704451 42.00352088328026, -87.76571289275766 42.00343949622276, -87.76579496047081 42.00332524964824, -87.76593465745209 42.00339575222315, -87.76646756443516 42.00364283802742, -87.76655191199646 42.00368194818787, -87.76665596900304 42.00373056706795, -87.76675833637438 42.00377901286237, -87.76684948458731 42.003822931813, -87.76692850947188 42.00386097153804, -87.76701192774192 42.00390117445111, -87.76708476428563 42.00393569868806, -87.76718127390552 42.00398074067535, -87.76729009849811 42.00403149579548, -87.76742163673715 42.0040929004898, -87.76749356246269 42.004126460058, -87.76760850483738 42.00418009899182, -87.76771531109745 42.00423013109413, -87.76785847119019 42.00428810797674, -87.76865079646583 42.00465813764313, -87.76919792655656 42.00491362277675, -87.76994862177872 42.00637562814637, -87.77026541642596 42.00622526356142, -87.77035947703132 42.00619408521005, -87.77037210412608 42.00621837807227, -87.77042137530592 42.00631370510558, -87.77046439012537 42.00639690092746, -87.77052538046793 42.00651517220631, -87.77061145876598 42.00667615765749, -87.77069543458494 42.00683323433714, -87.77072079396292 42.00688242586548, -87.77077712100245 42.00699145311518, -87.77083366548879 42.0071010873183, -87.77088891721334 42.00721101537187, -87.7709403336642 42.00731368021742, -87.7709993058545 42.00742746913241, -87.77105487981358 42.0075345455441, -87.77106964098992 42.00756307428284, -87.77114291141096 42.00770357897135, -87.77119534127317 42.00780410846396, -87.77125426262815 42.00791537288396, -87.77132024349024 42.00803989801057, -87.77135205622086 42.00809998715698, -87.77138859393526 42.00817941742778, -87.77143280482267 42.00825647173175, -87.77148189593534 42.00834765394546, -87.77154978957178 42.0084765238404, -87.77160902119532 42.00859025963084, -87.77167429876199 42.00872361601952, -87.7717421447723 42.00886225624889, -87.77177535294976 42.00892701749495, -87.77182223285132 42.00901862808097, -87.77185768305519 42.00908778932638, -87.77188864912947 42.00914820347581, -87.77193140669418 42.00922760937351, -87.77197119260155 42.00930159549087, -87.77205878918204 42.00947051764672, -87.77212858505257 42.00960510494174, -87.77220707957206 42.00975875167411, -87.77224626311866 42.00983465540385, -87.77233269602314 42.01000208951186, -87.77241385918123 42.0101579162222, -87.7724454499904 42.01021858024131, -87.77253049078266 42.01038128683358, -87.77258972686508 42.01049491096661, -87.77263529685007 42.01058108267662, -87.77269730004147 42.01069809534536, -87.77273655830238 42.01077383433584, -87.77279191883549 42.0108806884505, -87.7728797730486 42.01105005109021, -87.77291162526618 42.01111008453666, -87.77297645432765 42.01123246255907, -87.77304973813892 42.01137238973651, -87.77310296565231 42.0114750343964, -87.77322413504238 42.01169324071105, -87.77334945723824 42.01195828136238, -87.77415196032658 42.01196230330604, -87.77628423712105 42.01197294579325, -87.77657741420128 42.01197939031618, -87.77705324493108 42.0119823281598, -87.77705210899174 42.01210424751299, -87.77704499986083 42.01239646915936, -87.77703202309421 42.01293135783187, -87.77702652925501 42.01315630004132, -87.7770225195128 42.01332093190332, -87.77701965071563 42.01343891836603, -87.77701678313164 42.01355684857628, -87.77701391567012 42.01367488992424, -87.77701144482613 42.01377262674965, -87.77701097408784 42.01379282114294, -87.77700810524112 42.01391080732103, -87.77700523858444 42.01402879405607, -87.77700237008935 42.01414677913318, -87.77699950120136 42.01426476557799, -87.77699663266841 42.01438275202202, -87.77699376596233 42.01450073847243, -87.77699089751381 42.01461867002813, -87.77698802895395 42.01473665509246, -87.77698516036919 42.01485464152626, -87.77698149035328 42.01500555487098, -87.776973577908 42.01533070244452, -87.77697221681727 42.01538557876897, -87.77734702971233 42.01539623169144, -87.77762273085324 42.0154040979378, -87.77778085263742 42.01540859543724, -87.77793904756309 42.01541314795649, -87.77809716939667 42.01541764502041, -87.77825536484924 42.01542214222255, -87.77841348710004 42.01542663885277, -87.7785716822339 42.01543113561742, -87.77872980379797 42.01543563180862, -87.77888351141149 42.01543999669384, -87.77903501047508 42.01544435125704, -87.77910377185975 42.01544193827377, -87.77911097380461 42.01512952069536, -87.77911670032879 42.01487691761875, -87.77911942689815 42.01475766921328, -87.77912202273644 42.01464516977815, -87.7791221549387 42.01463841944032, -87.77912488332989 42.01451917021532, -87.77912768529694 42.01439992189187, -87.77913136964163 42.01428067672841, -87.7791340979979 42.01416142694677, -87.77913589275725 42.01403959394165, -87.77913859600548 42.01392297927745, -87.77913907703288 42.01390152232386, -87.7791412503653 42.01380372913097, -87.77914397829328 42.0136844807092, -87.77914781083791 42.01356523652033, -87.77915691626623 42.013448982058, -87.77916609294206 42.01333278199704, -87.77917527041943 42.01321652787635, -87.77918437562802 42.01310032883627, -87.77919355304606 42.01298407333698, -87.7792027316691 42.01286792925445, -87.77921183552095 42.01275161988212, -87.77922094071032 42.01263536429924, -87.77923011749523 42.01251916531475, -87.77925980889196 42.0121423111587, -87.77926053705868 42.01210933062075, -87.77926623029512 42.01199171417358, -87.77971963446419 42.01199239251725, -87.77996962156175 42.01199351504938, -87.77998327295796 42.01199358085903, -87.78036683091457 42.01199490755319, -87.78062951711691 42.01199979557593, -87.78108829811383 42.01200834333213, -87.78126396346936 42.01200866693492, -87.78142023799826 42.01200895192027, -87.78153783781568 42.0120091325621, -87.78180908018277 42.01200341183647, -87.7820706790593 42.01199791747877, -87.78245510330292 42.01200116228728, -87.78268981899377 42.01200242513953, -87.78286555711659 42.01200280152394, -87.78311249869843 42.01200310788884, -87.78346993776631 42.01200353146234, -87.78358709918241 42.01200351608565, -87.7837430078595 42.01200338480239, -87.78387989633491 42.01200277752922, -87.78434361095175 42.01200266287998, -87.78477821947813 42.01200257328954, -87.78503318572766 42.01200236420983, -87.7852674768063 42.01200161760227, -87.78550221379869 42.01200040613247, -87.78571711895874 42.01199896126518, -87.78606833204978 42.01199616241271, -87.7862632591069 42.01199437511867, -87.7865164069695 42.01199165796319, -87.78672600152839 42.01199172237997, -87.78723504660854 42.01199183989099, -87.78742758102494 42.01199462258162, -87.78778604892082 42.01200818172521, -87.78781938802486 42.01200803830987, -87.78797909197904 42.01200734193172, -87.7887227350944 42.01199671255794, -87.78882191154152 42.01199529423481, -87.78879306235839 42.0119612740503, -87.78760312872244 42.01055799559525, -87.7875232691862 42.01059141937637, -87.78752250552347 42.01059053266484, -87.78749822239841 42.01056229912828, -87.7873985375027 42.01044670629337, -87.78733248037756 42.01037021366142, -87.78724624065745 42.01027013415992, -87.78720827875851 42.01022596418676, -87.78708154487526 42.01007648856275, -87.78692887583745 42.00989640248734, -87.7868940821177 42.00985584237809, -87.78677556727726 42.0097177939243, -87.78670445317825 42.00963496410863, -87.78663814610424 42.00955764689523, -87.78658146067342 42.00949151578808, -87.78653264147802 42.00943453338527, -87.78645555456008 42.00934454198544, -87.7863989423341 42.00927832798592, -87.78627497123883 42.00913317381756, -87.78615931344346 42.00899694949515, -87.78605979457498 42.00887976545571, -87.7860020197506 42.00881187285442, -87.78595379138635 42.00875489301706, -87.78590530733621 42.00869774701371, -87.78580112618499 42.00857505185419, -87.78568328580187 42.00843626480725, -87.78556262517785 42.00829589944075, -87.78543243151947 42.00814319583861, -87.78533334754049 42.0080271375259, -87.78525473513855 42.00793505106147, -87.78515255403921 42.00781535519464, -87.78506085831553 42.00770871760704, -87.78485091199978 42.00746446841995, -87.78480686810119 42.0074130513504, -87.7846795459071 42.0072649977312, -87.7845357196402 42.00709776494222, -87.78444986499171 42.00699675348086, -87.78435585161665 42.00688615177614, -87.78426183709199 42.00677555026161, -87.78413008571637 42.00662064097254, -87.78407369914433 42.00655437336827, -87.7839952759946 42.00646217770861, -87.78390421256897 42.00635523951799, -87.78382735422208 42.0062650270489, -87.78375829177618 42.00618401772723, -87.78347701281609 42.00585419037775, -87.78338114089166 42.00574168518695, -87.78333255142292 42.00568462126505, -87.78329175526692 42.00563673145372, -87.78320813074485 42.00554201310553, -87.78310268145594 42.00542254613867, -87.78306873601065 42.00538670919516, -87.78302490868403 42.00534050615571, -87.78298589377981 42.0052994863512, -87.78298078856123 42.00529430169541, -87.782954527103 42.00526763971811, -87.78291086791096 42.00522324943071, -87.78282947544628 42.0051429748582, -87.78276919464318 42.00508500310072, -87.78267535411813 42.00500165174721, -87.78262025047781 42.00495347407082, -87.78255126340552 42.00489384080574, -87.78245101732887 42.00481536898966, -87.7822906178765 42.00468982095275, -87.78223570126424 42.00464979396517, -87.78215858409682 42.00459860153352, -87.78205595716599 42.00453191824037, -87.78200312521733 42.00449771911514, -87.78215566207028 42.00442570309166, -87.78220080751171 42.0044042956152, -87.78246642662045 42.00427884378216, -87.78277726332801 42.00413198397199, -87.78539002503116 42.0028972496955, -87.78800169373334 42.00165811454737, -87.79015067625393 42.00063357743079, -87.79027035794118 42.00057700955173, -87.79053948923347 42.00045012855178, -87.79064217642407 42.00040171223328, -87.79064803315305 42.00039914418738, -87.79075282598012 42.00051611992903, -87.79076273878465 42.00052719872112, -87.79084276818459 42.00061692767014, -87.79093540713821 42.00072071181214, -87.7910368661524 42.0008344163727, -87.7911662715791 42.00098014065526, -87.79155845407412 42.00097611915839, -87.79174264048413 42.00097451962588, -87.79174440696119 42.00097449969495, -87.7919528706266 42.00097350749483, -87.79204543856601 42.00097303789331, -87.79229617812929 42.0009714768581, -87.79246255458099 42.00097044863831, -87.79263602927182 42.00096934521634, -87.79282127965885 42.00096813205415, -87.79304122493991 42.0009667805847, -87.79313353511135 42.00096633688322, -87.79333051857935 42.00096539682575, -87.79358541299021 42.00096420844142, -87.79370119673153 42.00096365539205, -87.79394016797323 42.00096154011522, -87.79415312823942 42.0009596591892, -87.79431523727094 42.00095841834134, -87.7944657553941 42.0009572309751, -87.79468544477689 42.00095576511147, -87.79484769796025 42.00095487901243, -87.79507934047777 42.00095355051764, -87.79528773365492 42.00095222207802, -87.79555271780686 42.00094997893857, -87.7958090899708 42.00094783179646, -87.79593661448921 42.00094680979749, -87.79612212334338 42.00094542786618, -87.79639986801143 42.00094381904329, -87.79656208294234 42.00094301349099, -87.79674736823766 42.00094209546812, -87.79711153762663 42.00093973658944, -87.7971242317126 42.00093965785306, -87.797124242659 42.00088905659334, -87.79716951155099 42.00088859736074, -87.79717051273035 42.00004150709156, -87.79828308487804 42.00003588554343, -87.79936026589762 42.00003041899059, -87.79945464772511 42.00002993922651, -87.79945920290216 42.00005147874725, -87.79963228132419 42.0008698833892, -87.79965934803987 42.00086917361974, -87.80108609939612 42.00087243506873, -87.80107851084156 42.00154019757401, -87.80107812477335 42.00156762758009, -87.80184774401793 42.00156389421807, -87.80250173730944 42.00155500944403, -87.80252396208763 42.00155467369375, -87.80278822294358 42.00155106503005, -87.80350204423057 42.00154135474861, -87.80392474430622 42.00153562151033, -87.80534591033535 42.0015162618017, -87.80535523845508 42.00087771041766, -87.80538846792898 42.00085870135585, -87.80583595476935 42.00084915086931, -87.806635493089 42.00083893680784, -87.80675853375327 42.00083736081486, -87.80675104882876 42.00151300172654, -87.80674914212487 42.00165514284132, -87.80674613213702 42.0019129727845, -87.80674446695994 42.00205258978901, -87.80674246025795 42.00222447777905, -87.8067409554171 42.00235339232832, -87.80673965161232 42.00250258849408, -87.80673780515133 42.00266418537785, -87.8067351145169 42.00290116195932, -87.80673318178413 42.00307299400717, -87.80673085853687 42.00328785405017, -87.8067290894293 42.00344898585123, -87.80672601463012 42.00372816497271, -87.80672390426284 42.00392148459282, -87.80672200933782 42.00409331650906, -87.80671952369448 42.00431893273449, -87.80671798589422 42.00447400089928, -87.80671405040223 42.00486667587304, -87.80671285505582 42.00498480902271, -87.80671182192039 42.00509223946158, -87.80671044915292 42.00523188462311, -87.80670849166783 42.00537992542823, -87.80670809663019 42.00540560844294, -87.80670407039138 42.00571535506345, -87.806702389577 42.00584764435873, -87.80670119794475 42.00594319224678, -87.8066996204923 42.00606749723239, -87.80669752216491 42.00631759226055, -87.80669591099077 42.00651294506056, -87.80669326803803 42.00669942827987, -87.80669249512411 42.00675299137678, -87.80669087145809 42.00686513798698, -87.80668822005994 42.00707958502755, -87.80668698250153 42.00718934787619, -87.80668278296845 42.00756931693925, -87.80668201298879 42.00763585924798, -87.80668147493842 42.00768324870651, -87.80667927464731 42.00786962442525, -87.80667643460791 42.00809345597412, -87.80667207813897 42.00843448389888, -87.80667167824588 42.00854523710177, -87.80667154235009 42.00858414994264, -87.80667100486038 42.00879638272455, -87.80667069055272 42.00891031957556, -87.80666882733685 42.00900237865884, -87.80666499738385 42.0091902567117, -87.80666288133803 42.00933522209451, -87.80666084755045 42.00956388651332, -87.80665782549259 42.00989910361073, -87.80665769103497 42.00991540321546, -87.80665603150308 42.01014316319639, -87.80665406503131 42.01031927608439, -87.80665235304008 42.01046427100881, -87.80664976258001 42.01067128225996, -87.80664736382553 42.01081481987761, -87.80664186522483 42.0111467308632, -87.80664128066154 42.01133369016266, -87.80664097147317 42.01147817010176, -87.80663826091782 42.01171306091638, -87.80663237722497 42.01222556551408, -87.8066317885848 42.01226573775682, -87.80662691669889 42.01258395939774, -87.80661754598391 42.0132649419875, -87.80660503424264 42.01343452982834, -87.80660887431594 42.01352650633915, -87.80661511305468 42.01367568084287, -87.80660704010673 42.01430623188425, -87.80659399397197 42.01522736907922, -87.80659338556161 42.01533677681632, -87.80659244095339 42.01550485376098, -87.80657784258631 42.01652469181178, -87.80657264798542 42.01701146003008, -87.80657226428931 42.01704908053605, -87.80657153523272 42.01715066729835, -87.80657067293149 42.01726844388073, -87.80656696232037 42.01750420773359, -87.80656360573033 42.01782161195646, -87.8065598154916 42.0181783109342, -87.80655479404508 42.01851041687127, -87.80655048935988 42.01879571052234, -87.80654721882817 42.01896457250547, -87.8067805552879 42.01896213166876, -87.80729843770935 42.01895495532165, -87.8075704364901 42.01894560950304, -87.8077680393076 42.01893883044126, -87.80795838924108 42.01893229253577, -87.80885203286542 42.01891321837162, -87.80899098318085 42.01890979234041, -87.80921510902134 42.01890428450118, -87.81006579573105 42.01888621217449, -87.81021394967118 42.0188824418837, -87.81038952610592 42.01887794509138, -87.81130690388265 42.01885893277187, -87.81143669116615 42.01885543384176, -87.81161197448186 42.01885071540919, -87.81249387095679 42.01883241019147, -87.81265803217838 42.0188285990617, -87.81281343494746 42.01882499660342, -87.81372463667856 42.01880492105284, -87.81388091964932 42.01880153929648, -87.81403617272983 42.01879818093312, -87.81495981544758 42.01877790488239, -87.81510307356585 42.018774187947, -87.81528332435484 42.01876948633417, -87.81536212851606 42.01876786642801, -87.81619962825501 42.0187508416866, -87.81632744829187 42.01874949544745, -87.81649560563957 42.0187477288983, -87.81740501559929 42.01873046973989, -87.8175445633327 42.01872596607916, -87.81771135127485 42.0187205958863, -87.81859942622759 42.01870199681296, -87.81876543010981 42.01869804882418, -87.81895428961165 42.01869357020959, -87.82046633691257 42.0186592836085, -87.8206267751538 42.01865607615769, -87.8208098512 42.01865242100133, -87.82120248307946 42.01864245550642, -87.82120976747744 42.01864249102529, -87.82121086909493 42.01854674848388, -87.82121194631605 42.01837584527016, -87.82121241353562 42.01830839521569, -87.82121293830558 42.01822464494071, -87.82121329529103 42.0181710251002, -87.82121382049505 42.0180872202158, -87.82121417623136 42.01803370986156, -87.82121470275452 42.01794974005533, -87.82121491059404 42.01792383584687, -87.82121558456473 42.01781231504773, -87.82121634263471 42.01769031182265, -87.82121646636818 42.01767489003683, -87.82121711346907 42.0175667170227, -87.82121761237245 42.01749531546896, -87.82121795833562 42.01744306766664, -87.82121875205601 42.01731661900756, -87.82121930813187 42.01722891722643, -87.82121991072999 42.01713550828828, -87.82122065657775 42.01702415244586, -87.82122091286068 42.01698298966327, -87.82122091858041 42.01698227674861, -87.82122124353694 42.01694177370064, -87.8212217576198 42.01685939627562, -87.82122187254426 42.01684507184931, -87.82122234420595 42.01677701752637, -87.82122266916492 42.01672756951952, -87.82122275364067 42.01670786662163, -87.82122285975807 42.01669463983113, -87.82122344670717 42.01661226108112, -87.82122357924045 42.01659574165618, -87.82122370686567 42.01657066034036, -87.8212239607803 42.01652988365135, -87.8212243275687 42.01647494607931, -87.82122454816246 42.01644745029157, -87.82122466044062 42.01643345542949, -87.82122506149916 42.01636507230729, -87.82122514956012 42.01635409594473, -87.82122554004685 42.01629624991092, -87.82122564879516 42.01628269492622, -87.82122605997455 42.01622226944643, -87.82122616250159 42.01620031611798, -87.82122649360791 42.01615904499344, -87.82122674979253 42.01611793873452, -87.82122716060249 42.01605751297694, -87.82122733672173 42.0160355602506, -87.82122737327069 42.01602178485921, -87.82122785041423 42.01595318226202, -87.82122810711945 42.01591196486428, -87.82122832681785 42.0158845799351, -87.82122843733839 42.0158708037757, -87.82122895102383 42.01578842605912, -87.82122920677374 42.01574737330732, -87.82122953831092 42.01570604757208, -87.82123005346341 42.0156236698597, -87.8212301617789 42.01561016838318, -87.82123052098281 42.01555621977429, -87.82123064074763 42.01554129109584, -87.82103674268673 42.01554640876341, -87.82077092448085 42.0155535089971, -87.82054640287936 42.01555947699482, -87.82052800089036 42.01555988819087, -87.82042288555377 42.0155621083956, -87.82031246931479 42.01556446864059, -87.82020212849696 42.01556682993979, -87.82013028480229 42.015568319643, -87.82009163872419 42.0155691344854, -87.82006889372856 42.01556963732858, -87.82006951356655 42.01550174831055, -87.82006986615291 42.0154578977433, -87.8200700988844 42.015419754279, -87.82007120870357 42.01528172706922, -87.82007162879955 42.01522947985718, -87.82007164821803 42.01522706478295, -87.82007257490778 42.01511181214963, -87.82007381301999 42.01502180882154, -87.82007403198472 42.01493059376604, -87.82007562708678 42.01479618967248, -87.82007665849881 42.01465898507955, -87.82007722287908 42.01458879095787, -87.82007757589454 42.01454488495292, -87.82007776209873 42.01452172592231, -87.82007886526075 42.01438452027155, -87.82007989451586 42.01424726077533, -87.82008025104648 42.01420291681078, -87.82008141369758 42.01402169516106, -87.82008180685037 42.01397279572883, -87.82008298356449 42.0138355917659, -87.82008430789988 42.01369833330153, -87.82008446498061 42.01367879564496, -87.82008481797764 42.01363488990738, -87.82008541101378 42.0135611279053, -87.82008622018432 42.0134238671309, -87.82008750117885 42.01329204223404, -87.82008895813374 42.01311082214591, -87.82008975101792 42.01301220031218, -87.82009078056832 42.01287494051007, -87.82009164801435 42.01276704309286, -87.82009188363648 42.01273773509391, -87.82009200099738 42.01272313707386, -87.8200929871316 42.01260047561586, -87.82009401658412 42.01246327096166, -87.82009469075787 42.01237941205761, -87.82009512006404 42.01232601147696, -87.82009614809139 42.01219813650968, -87.82009745980208 42.01203497333681, -87.82009759304889 42.0120183987453, -87.82009826553742 42.01192559321598, -87.81973052816636 42.01193344413307, -87.81973079895393 42.01186319350841, -87.81973100602058 42.01181934198922, -87.81973235483208 42.01147764217999, -87.81972901246584 42.0113855865054, -87.81998821722838 42.01137963853872, -87.82005299177459 42.01137808931773, -87.82022888468242 42.01137739356087, -87.82040391191298 42.01137438964326, -87.82057821293328 42.01137031115694, -87.82072395358742 42.01136692784999, -87.82081838856014 42.01136487920975, -87.82094951345093 42.01136252940626, -87.82107302058662 42.01136033590451, -87.8212534613282 42.01135686007224, -87.82125587208502 42.01105629927783, -87.82125635168062 42.01099650521233, -87.82125713631778 42.01089867938558, -87.82125860983136 42.01071946143982, -87.82126008463038 42.01059990496415, -87.8212616387037 42.01047491369363, -87.82126314270103 42.01036084435444, -87.82126489392819 42.01022498695873, -87.82126580585883 42.01015720921503, -87.82126599008136 42.01014346136139, -87.82126718855388 42.01003995634019, -87.8212680137014 42.00990950141259, -87.82126874534242 42.00982745306265, -87.82127069312195 42.00974058118195, -87.82127397396093 42.0096158174053, -87.82127671593003 42.009512592307, -87.82128065215188 42.00936593306957, -87.82128200654269 42.0093162963999, -87.82128300514462 42.00927895267272, -87.8212831372674 42.00927165425958, -87.82128581215559 42.00912164092582, -87.8212872461708 42.00903468359333, -87.82128856213188 42.00895313234074, -87.82129064533974 42.00881724840148, -87.82129226963656 42.00872477663506, -87.82129404817618 42.00862692693777, -87.82129551195266 42.00854543122253, -87.82129735911269 42.00844351959988, -87.82129854366234 42.00837821337321, -87.8213180252509 42.00748141571455, -87.82130835065593 42.00722876958142, -87.82130906989163 42.00716669913889, -87.82131037441668 42.00705907797236, -87.82131160205282 42.00695173005691, -87.82131290553895 42.00684432869159, -87.82131497195726 42.00666931297718, -87.82131537422481 42.00663731692663, -87.82131660109771 42.00653024479129, -87.82132227858159 42.00605137072692, -87.82133079971609 42.00532846126807, -87.82133112581212 42.00529701425794, -87.82133132135932 42.00528175741076, -87.82133152469171 42.00526562135845, -87.8213340015573 42.00505756787665, -87.82133668362934 42.00483332483031, -87.82133835708757 42.004688712427, -87.82134105107306 42.00446293212155, -87.82134504495943 42.00412085918402, -87.82133757089983 42.00392409452736, -87.82133770666415 42.00386122535566, -87.82133818200528 42.00380661799876, -87.82133883506013 42.00373883902416, -87.82133963512332 42.00365745032101, -87.82134003650764 42.00361656415579, -87.82134032613999 42.00358961663009, -87.82134163472919 42.00345397690906, -87.82134217931205 42.00339972632802, -87.82134297899454 42.00331833761832, -87.82134382260189 42.00323151559709, -87.82134433457684 42.00317688013235, -87.82134539771981 42.00306730916118, -87.82134588699502 42.00300628044205, -87.82134662974977 42.00291808559419, -87.82134849124573 42.00278231166237, -87.82134911117957 42.00273706264625, -87.82134989329242 42.00268094596731, -87.82135038151247 42.00260614220165, -87.82135092437242 42.0025247520608, -87.82135144909869 42.00245016689389, -87.82135350446516 42.00233132550216, -87.82135407722683 42.00229664032727, -87.82135586913074 42.00219239605465, -87.82135621084072 42.00213136831456, -87.821356864203 42.00200420437807, -87.82135713329956 42.00195673154131, -87.82135719814255 42.00187975714005, -87.82135725014049 42.00184128319847, -87.82135731302166 42.00175991807361, -87.82135789471411 42.00169208387261, -87.82135860188285 42.00161750060627, -87.82135932829954 42.0015361101724, -87.82136126296942 42.00138642318601, -87.82136309814354 42.0012446944694, -87.82136426653823 42.00117691748847, -87.82136530966491 42.00111586322858, -87.82136750686598 42.00107106085399, -87.82136702254033 42.00101669561924, -87.82136618236281 42.00092429451193, -87.82136563210263 42.00086452373839, -87.82136611107998 42.00083230955948, -87.82137387646775 42.00062526741366, -87.82137443972441 42.00061012196181, -87.82138102336927 42.00054604607321, -87.82138083330331 42.00048273716325, -87.82138032053187 42.00031717722938, -87.82138252677787 42.00016117976995, -87.82138447654262 42.00001881899649, -87.82138594320725 41.99987733503119, -87.821385950468 41.99987642920424, -87.82138747687297 41.99972726147641, -87.8213889131741 41.99958478973203, -87.8213902952923 41.99944912252755, -87.8213928252929 41.9990920314549, -87.82139364794824 41.99897576321755, -87.82139440587787 41.99888120183783, -87.82139478660199 41.99883370148225, -87.82139563580361 41.99866340066253, -87.82139586733813 41.99861156353363, -87.82139688295091 41.99842054475909, -87.82139723073125 41.99836352133884, -87.82139786673399 41.99825658362349, -87.82140064024644 41.99778651466975, -87.82140312908025 41.99756792242655, -87.82140574021372 41.99733851995271, -87.8214044459797 41.99692164196206, -87.82141432255251 41.99685052949793, -87.82157430986813 41.99625405041031, -87.82160914642539 41.99612583155512, -87.8216661459068 41.99588959059857, -87.82172257419023 41.99565576250162, -87.8217695755006 41.9954276540774, -87.82180964131481 41.9952204540559, -87.82185018882727 41.99498062708332, -87.82194896390877 41.99439663449334, -87.82201839870747 41.99399453438203, -87.8220825229903 41.99361227882944, -87.82212783437755 41.99333301213539, -87.82216370401116 41.99311193336957, -87.82219354055245 41.99289549423312, -87.82221901643764 41.99270441979129, -87.82227157424477 41.99231011984151, -87.82230878533937 41.99203616731661, -87.82234932480686 41.9917553139218, -87.82234994897432 41.9917509542444, -87.82236229331926 41.99166536268923, -87.82238398267941 41.99150146524636, -87.82240032620703 41.99138068366158, -87.82241946444923 41.99124147348033, -87.82244196156685 41.9910776347676, -87.8224761127059 41.99082735123994, -87.82249810311832 41.9906578849113, -87.82251976332539 41.99049308142288, -87.82253940142542 41.99035560254509, -87.82255665031541 41.99021802988074, -87.82257358248516 41.99008325543556, -87.82259443387174 41.98993661575474, -87.82262645178301 41.98970901663913, -87.82265684505008 41.98948703644047, -87.82267590782264 41.98934768807486, -87.82269540121369 41.98920079561189, -87.8227440249473 41.98884234442106, -87.82275363092464 41.98877265669741, -87.8227751068424 41.98861679975691, -87.82282896952317 41.98821929380301, -87.82286801913244 41.9879172220957, -87.82289078788692 41.98774679777099, -87.82291711434412 41.98755010144025, -87.8229344423103 41.98742070617735, -87.82294608424955 41.9873355776105, -87.82297744812023 41.98710668588284, -87.82297764260809 41.98710531574374, -87.82299880335405 41.98695211781874, -87.82302622849764 41.98674197917565, -87.82303617029424 41.98666705229049, -87.82305542770084 41.9865215845353, -87.823075754077 41.98637609490572, -87.82309311570849 41.98625161323744, -87.82309675433692 41.98622432376401, -87.82310782987305 41.98614097665175, -87.82312661232861 41.98599059628249, -87.82314344149944 41.98585897620493, -87.82315229324401 41.98579606406483, -87.82317085749177 41.98566382052279, -87.82319006200021 41.98552038456678, -87.82320032624287 41.98544639156442, -87.82321970851598 41.98530836164588, -87.82324351946505 41.98512232810673, -87.82325819325976 41.98500746550063, -87.82327355147586 41.98489891869794, -87.82328134616205 41.9848439869252, -87.82332742116868 41.98450898737942, -87.82334604710459 41.98435509200738, -87.82810783893223 41.98503983357621, -87.82847090166226 41.9850916500401, -87.82879590282862 41.98513802893614, -87.83046015601754 41.98537535014243, -87.83084791218769 41.98543066979448, -87.83161497769446 41.98554006080398, -87.83206570936163 41.98560432324017, -87.83280169685581 41.98570923477203, -87.83355429275474 41.98581657390763, -87.83383881345495 41.98585716411145, -87.83404801520987 41.98588699807986, -87.83416712900178 41.98590398129598, -87.83425795271341 41.98591688974976, -87.83502760138478 41.98602665337681, -87.83502701172148 41.98607401550228, -87.83638195061049 41.98626715365557, -87.83638982214029 41.98626287301871, -87.83658097478063 41.98628165373444, -87.83658099041752 41.98638387566969, -87.83747234261297 41.98648180083246, -87.83765727061015 41.98650696717121, -87.83786737574552 41.98653948740006, -87.83801608952001 41.98656279599787, -87.83821514511799 41.98659718995481, -87.83844167929401 41.98663625603555, -87.83865609697418 41.98667227827645, -87.83892490500659 41.98671473710011, -87.83908928422711 41.98673849623698, -87.83947727036418 41.98679461099427, -87.83966024065143 41.98682130298757, -87.84004988448972 41.98687736777988, -87.8417422612882 41.98712320819486, -87.84320384813718 41.98734869272351, -87.84330495748816 41.98736391562311, -87.84344126225898 41.98738447435477, -87.84352723596299 41.98739716378166, -87.84372715020947 41.987425978916, -87.84383622015351 41.98744400721899, -87.84394616135654 41.98745874581383, -87.84398694591195 41.98746451812242, -87.84412782970982 41.9874781258294, -87.84425015301174 41.98749442621018, -87.84434718500253 41.98750428044684, -87.84440371930599 41.98750913068206, -87.84461130117144 41.98754055813127, -87.84473357834339 41.98755825707001, -87.8450526270481 41.98762144146215, -87.84536554260244 41.98766895693716, -87.84573987214786 41.98772329134731, -87.8460778589608 41.98777286002501, -87.84670880396526 41.98786820752455, -87.84672600451479 41.98797642949717, -87.84723412121757 41.98805421216589, -87.8474792528481 41.98809169318685, -87.8480071555375 41.98817010528708, -87.84841961765586 41.98823134263093, -87.84924821797782 41.98835437748964, -87.84964381263045 41.98840895294354, -87.85019329512396 41.9884751518825, -87.85026937885846 41.98848277164668, -87.85074499421846 41.98853049046429, -87.85128765935805 41.98857415358723, -87.85134229445228 41.98857745595132, -87.85184246546467 41.98860787507009, -87.85239831076721 41.9886306219606, -87.85311026997996 41.98865281160117, -87.85326493552219 41.98865883741041, -87.8535949463551 41.98867970084451, -87.85374741273405 41.98869449826454, -87.85392340376427 41.98871153365659, -87.85424964707003 41.98875416845055, -87.85457294041913 41.98880760217632, -87.85489277057249 41.98887172295154, -87.85510457328301 41.98892046439342, -87.85537817288186 41.98897901230391, -87.85555941165475 41.98903229079603, -87.85568633043549 41.98906959072455, -87.85588197991322 41.98913401607081, -87.85576567097161 41.98892700270258, -87.85568054450063 41.98866265780646, -87.8556358546206 41.98849507433319, -87.85556800021851 41.98818859425462, -87.8555374696562 41.98794272400848, -87.85555885905555 41.98784196326181, -87.85558218840356 41.98773745157215, -87.85570095209988 41.98750219037213, -87.85587386516727 41.98715316056923, -87.85600773517065 41.98680586192574, -87.85608785797531 41.98661744975593, -87.85618449974396 41.98641324276046, -87.85630604828572 41.98615447571326, -87.85636035771751 41.98599405534976, -87.85643789097882 41.98558044213557, -87.85646413239097 41.98541084867716, -87.85647147578187 41.98536324032833, -87.85651884725868 41.98520098014595, -87.85656523837494 41.98504200891415, -87.85657097578805 41.98502230245437, -87.85661302590549 41.98481888420945, -87.85664471830323 41.98466278846924, -87.85664838730328 41.98443226189678, -87.85663951788165 41.98431960396545, -87.85663945108132 41.98431869755136, -87.85657206704934 41.98414624625869, -87.85623353373045 41.98348167503364, -87.85620751903048 41.98341422437264, -87.85597136459756 41.98280202558108, -87.85578014124496 41.9821625708182, -87.85578362073326 41.98169692029536, -87.85582144587946 41.98140435335326, -87.85587717038601 41.98109857831686, -87.85603970881344 41.98080655787753, -87.85627391614879 41.98046159631436, -87.85701213347701 41.97986510768929, -87.8582484003814 41.97883571775691, -87.85845581483244 41.97866415785072, -87.85883009730685 41.97822849842803, -87.85886115939364 41.97819100370986, -87.85937924408788 41.97756570730579, -87.85996441033008 41.97699342935725, -87.86067487806621 41.97653867795663, -87.86092722675338 41.97631139784622, -87.86102050827279 41.97622937276487, -87.86124454254526 41.97600118100098, -87.8615507468804 41.9755567520709, -87.8616837142263 41.97534931473871, -87.86175798154248 41.97516762380888, -87.86186406814441 41.97481992873238, -87.86192712963587 41.97459664430781, -87.8620079404778 41.97431042590856, -87.86201957329112 41.97398454457883, -87.86201070084242 41.97388610047803, -87.8620086460764 41.97386352612173, -87.86238833738511 41.97385172823715, -87.86251052520804 41.97384877142834, -87.86399693147692 41.97381320088032, -87.86400796600944 41.97381291695989, -87.86415489222875 41.97380643662296, -87.86473846439563 41.97379240969493, -87.86535410939695 41.97377768810436, -87.8655012339965 41.97377417114951, -87.86611736714437 41.97376290488612, -87.8667245572403 41.97374770136469, -87.86688632607957 41.97374303552408, -87.8674928519809 41.9737279896182, -87.86749221411129 41.97390641508933, -87.86749189630869 41.97399055158579, -87.86783925649853 41.9739825210151, -87.86783745916516 41.97381517118482, -87.86819320665388 41.97380684342111, -87.86856579888884 41.97379814367601, -87.86860732145034 41.97379757192773, -87.86860568461705 41.97397209603348, -87.8686048703204 41.97405370667274, -87.86965031761024 41.97403235704433, -87.8709683029122 41.97400084568088, -87.87264599493069 41.97396244822873, -87.8725760660205 41.97387319944108, -87.87272709123137 41.97386946961775, -87.87297617595299 41.97386322453617, -87.87300847879376 41.97395303489052, -87.87332715300639 41.97394520117059, -87.87589258616308 41.97388141337372, -87.87588925618753 41.97478874610974, -87.87588599950287 41.97569618876305, -87.87588274121545 41.97660346661412, -87.87415070314402 41.97664655145488, -87.87397973555916 41.97665080762545, -87.87373064191704 41.97665699995123, -87.87502218126899 41.98024396059336, -87.87489387459217 41.98024723969164, -87.87492618554482 41.98033698989552, -87.87547460395326 41.98185985414604, -87.87586299764479 41.9829385033735, -87.87586251636202 41.98318043075186, -87.87586100218574 41.98388530223955, -87.87585909401447 41.98474779845819, -87.87585704767814 41.98570134619797, -87.87691093087743 41.98566860437332, -87.87755076496951 41.98744516636894, -87.87827171977962 41.98944706870054, -87.87967033421599 41.9933297247606, -87.87974370935724 41.99353341553219, -87.88035483860226 41.99522985162517, -87.88060350134701 41.99522207802161, -87.88083877057321 41.99521471869799, -87.88282152059369 41.99515275675134, -87.88420045784503 41.99510964534873, -87.88442152333695 41.99510272721835, -87.88443116367959 41.99510242491036, -87.8845428739279 41.99509891827703, -87.8845424538107 41.99523411796198, -87.88453362206035 41.99808119411392, -87.88453326951172 41.99819503588267, -87.88495539365722 41.9982969464514, -87.88496048384164 41.99846312079151, -87.8850070143409 41.99846939547299, -87.88577005387289 41.99864311656397, -87.88673239598772 41.99886217504024, -87.88675135148789 41.9988661449996, -87.88730481508223 41.9989812475288, -87.88774883312288 41.9990698540883, -87.8883495045706 41.99918974534304, -87.88857068562565 41.99923149026803, -87.88939787951062 41.99938759958227, -87.88982811523628 41.99944717812161, -87.89019387664109 41.99951229890635, -87.89044968837425 41.99955785166751, -87.89075343181558 41.9996097902839, -87.89081079367907 41.99957170212168, -87.8918665658959 41.99974459148385, -87.89292536082282 41.99990677925569, -87.89297034687753 41.99991342814867, -87.8934638927209 41.99998145506328, -87.89411173636361 42.00007081648344, -87.8943546912941 42.00010385350549, -87.89447605358839 42.00012105770774, -87.89506994382174 42.00020296330406, -87.89745824030786 42.00053219440994, -87.89744096152393 42.00154624301159, -87.89921185542113 42.00148759604789, -87.89954277309633 42.001476697089, -87.90311505897485 42.00342009613308, -87.90406237512804 42.00356044197506, -87.90406027817279 42.00397255783736, -87.9024043573 42.00402699967128, -87.90240647376923 42.00361499396824, -87.8992362186724 42.00189917699893, -87.89760829206818 42.0019527776987, -87.89743393679463 42.00195848202206, -87.8969116062203 42.00197570596612, -87.89435736227077 42.00205970884431, -87.89435634694985 42.00211867724386, -87.89435546903849 42.00216964408485, -87.89041298970554 42.00229919164037, -87.89038549265595 42.00230009513167, -87.89038494941808 42.00237841849711, -87.89038364718346 42.00256617180172, -87.89038168143951 42.00284958501649, -87.89026472974176 42.00285342696982, -87.8901703166069 42.00298498188951, -87.89003404648477 42.00317485916895, -87.88999682255277 42.00322672729607, -87.89010295910808 42.00326934621461, -87.89068882074322 42.00350459143254, -87.89126918435917 42.0037376157514, -87.89188956270945 42.003986701877, -87.89216206379196 42.00397776664409, -87.89252974668126 42.00396567467595, -87.89289724764728 42.00395358741796, -87.89326471021482 42.00394155317115, -87.89326474627967 42.0039415519373, -87.89323480220951 42.00343437467134, -87.89322946431786 42.00334390173286, -87.89357787890691 42.00333248677046, -87.89357322422806 42.00324338842513, -87.89354676382268 42.00273698219586, -87.89404061782103 42.00271886164737, -87.89434620660636 42.00270764831209, -87.89429464736084 42.00570217243801, -87.89530933824969 42.0056687826266, -87.89576275154063 42.00587660080792, -87.8957626588051 42.00565388251963, -87.89592024256027 42.00564866485698, -87.89598339378603 42.00564660035298, -87.89598347935487 42.00578452457314, -87.8967053819179 42.00611542353255, -87.89699903088702 42.00625001729347, -87.89717927999126 42.00633116297701, -87.89756397909436 42.00649491554444, -87.8979302709314 42.00663905859586, -87.89835426009392 42.00679346315914, -87.89875888774078 42.00692781455655, -87.89916205354041 42.00704991960579, -87.90238436837384 42.00791623993087, -87.9023834682407 42.00809125727122, -87.90245515359881 42.0081104588572, -87.90403713943296 42.0085342048125, -87.90542557798973 42.00890409315788, -87.90575035197803 42.00899059600335, -87.90768048201831 42.0089218064531, -87.907680347591 42.00882065543801, -87.90768021627397 42.00876302710821, -87.90767354642871 42.00524867321634, -87.90827429680377 42.00522724458902, -87.90827427749394 42.00520803568183, -87.90871442502346 42.00519232022672, -87.90908222286369 42.00517908183176, -87.91133391439806 42.00509506711256, -87.91133545305982 42.00511839762842, -87.9118215433777 42.00510968235062, -87.91302716812352 42.00508805238078, -87.91302985085683 42.00626806723554, -87.91330550110645 42.00625782443156, -87.91331089084669 42.00857081797802, -87.91331059299732 42.0086160955794, -87.91330990301556 42.00872098282461, -87.91338335225043 42.00871904220406, -87.91348151450747 42.00871485332465, -87.91375116495668 42.0087055383831, -87.91377038952881 42.00870453814051, -87.9137704039623 42.00860059873764, -87.91377042728477 42.00843018395062, -87.91377046398465 42.00815576498073, -87.9137708222148 42.00521184686303, -87.91377083923659 42.00507469080995, -87.91381948506788 42.00507382607933, -87.9147639216291 42.0050568571505, -87.9149414348737 42.00505366817389, -87.91526798402138 42.00504779627147, -87.91859619790816 42.00498792019822, -87.91949669615991 42.00497170196592, -87.92012487640459 42.00496038510754, -87.92051082649269 42.00495343028099, -87.92061247527873 42.00495159782799, -87.92059535539354 42.00388539726879, -87.92085739910783 42.00388305627106, -87.92109607158747 42.00416289931431, -87.92219938805863 42.00545649764943, -87.92224793299289 42.00551341342964, -87.92229675848765 42.00556869725835, -87.92255936626337 42.00586605458578, -87.92259325072691 42.00588812941859, -87.92275812637384 42.00599554264519, -87.92368889646058 42.00658966083921, -87.9237323741928 42.00661741438259, -87.92426933934894 42.00696015574768, -87.9243576083803 42.00701639701197, -87.92444696957558 42.00707333363605, -87.92448078513823 42.00709487803261, -87.92516811645795 42.00753281721185, -87.92545391180538 42.00771491167191, -87.92558229210523 42.0076179515093, -87.92565858101597 42.00756327266563, -87.92576128754855 42.00748965381857, -87.92584604507738 42.00742880577509, -87.92593006689543 42.00736848194302, -87.92609834527516 42.00724614449922, -87.92626711474389 42.00712574320632, -87.92643587251763 42.00700649967217, -87.9266041207371 42.00688686166119, -87.9267723646295 42.00676722612944, -87.92693959491559 42.00664603628783, -87.92710681107663 42.00652638838874, -87.92727452204936 42.00640867665288, -87.92744122188078 42.00628902369002, -87.92760740846619 42.00616897899184, -87.92777359442685 42.0060489340478, -87.92793978344206 42.00592888887095, -87.92800092202222 42.00588556257747, -87.92810646362965 42.0058107770999, -87.92827314318342 42.00569266782585, -87.92844033719591 42.00557456286563, -87.92860599764802 42.00545566741119, -87.92877114972931 42.00533561187129, -87.92893628531107 42.00521748246, -87.92910137254647 42.00510398758941, -87.92914460374158 42.00507266660357, -87.92925081867537 42.00499570902647, -87.92963460607348 42.00505599103734, -87.92969094256196 42.00506425434747, -87.92996938302643 42.00510510629875, -87.92998359765528 42.00510719244181, -87.93033310731855 42.00515801041625, -87.93032918771114 42.00507074485945, -87.93032578888544 42.00499510027269, -87.93062395004847 42.00503398346879, -87.9308754468249 42.00506678052012, -87.93091756413928 42.00507253457318, -87.93128495228895 42.00512272627434, -87.93128588603022 42.00512285303363, -87.93201877357934 42.00522313025243, -87.93238233980702 42.00527287424061, -87.93275364123312 42.00532367314501, -87.93311945475374 42.00537372175845, -87.93348284993621 42.00542343971116, -87.93348486797886 42.00542371320222, -87.93350555952615 42.00556497072406, -87.93350595412264 42.00565872791556, -87.93354312249896 42.00719779600236, -87.93354628811328 42.00732875163656, -87.93356294620166 42.00801853379242, -87.93358036208268 42.00817352578867, -87.93359419700913 42.00817332726411, -87.93425976115471 42.00816400683705, -87.9348729939064 42.00815592981334, -87.93503070634885 42.00815422664046, -87.93510113646401 42.00815345509735, -87.93528753192918 42.00815143368313, -87.93528082401019 42.00800805217776, -87.93523344014982 42.00594010699479, -87.935230943712 42.00583122226323, -87.93523091730508 42.00583014644585, -87.9352283461747 42.00572970260782, -87.93520725286693 42.00490641648572, -87.93519183062328 42.0043042829769, -87.93516694540872 42.00333279396756, -87.9350947345648 42.00093785829629, -87.93514385942143 42.00088911607326, -87.93509856300578 42.00089029811226, -87.93465521701097 42.00090186305688, -87.93510867029796 42.00045622418665, -87.93518626678629 42.00037996416515, -87.935373055228 42.00020099621795, -87.93540868723213 42.00016584550313, -87.93552473217561 42.00004039068039, -87.93565499600393 41.99989076138244, -87.9357858884876 41.99973554993394, -87.93590627998418 41.99959521094883, -87.93603225069677 41.99943998958003, -87.93612179262904 41.99932752428951, -87.93624160250138 41.99917415730116, -87.93636080931554 41.99901520529369, -87.93645346008061 41.9988915749983, -87.93652819454812 41.99879304510632, -87.93666781888641 41.99859317950972, -87.93687728799708 41.99828267685973, -87.93708061256233 41.99796657994755, -87.93727409289576 41.99764860980242, -87.93739218042167 41.99744684929293, -87.93745154331265 41.99734085036869, -87.93753009179846 41.99719579675908, -87.93761546043115 41.99703121168776, -87.93776953879232 41.99671783555064, -87.93791500304195 41.99640258678456, -87.93804690436288 41.99609662305135, -87.93817264845212 41.99579064865675, -87.93828361545663 41.99548744841272, -87.93838719757849 41.99518423169353, -87.93848465306293 41.99486891270823, -87.93857101762786 41.99455916161731, -87.93865123032008 41.99424939944689, -87.93872034930983 41.99394427574185, -87.93877544398627 41.99366976285242, -87.93878084519451 41.99364286077103, -87.93880096642351 41.99353262662745, -87.93916328709022 41.99352642872359, -87.93993042467741 41.99349140399112, -87.93992864392526 41.99344156938823), (-87.93514385942143 42.00088911607326, -87.93521503107634 42.00088726087022, -87.93526832666888 42.0008859284254, -87.93539947522872 42.00088265293795, -87.93639423702659 42.00085779605135, -87.93706081655651 42.00084448976504, -87.93802044050877 42.00082532973181, -87.93897153745496 42.00080633188573, -87.9399299489268 42.00079216455487, -87.94011408252348 42.00078931826637, -87.94009181181038 42.00015343250929, -87.9400819857009 41.9998725169338, -87.94005007906628 41.99896069051319, -87.94002020835825 41.99810694262792, -87.93989441399762 41.9981270162374, -87.93982896518334 41.99813745944146, -87.93905445886887 41.99826104013864, -87.93707950926456 41.99858574196458, -87.93706164890665 41.99861236246218, -87.93693408810181 41.99880335510703, -87.9368759749499 41.99888184715304, -87.93677128867272 41.99902324827868, -87.9366154497264 41.99923337944194, -87.93632784066214 41.99960326946243, -87.93598608216818 42.00001374987231, -87.93590356821541 42.00010497538239, -87.93584505940073 42.00015903938191, -87.93574247939097 42.00025812301311, -87.93551196383594 42.00048078193559, -87.93529133950187 42.00072700895027, -87.9352110270025 42.00081664143955, -87.93514385942143 42.00088911607326)), ((-87.52480830724122 41.67578679420264, -87.5248230988926 41.6735441241037, -87.5249769975925 41.6589664214782, -87.5250647961531 41.6448919193047, -87.525266394886 41.6300581166913, -87.52490269315319 41.6157112143425, -87.52500849160531 41.6011401118414, -87.52511359040049 41.5867177095673, -87.5251332886415 41.5721972065333, -87.5251822877046 41.5576825041249, -87.52519718623211 41.5428224017373, -87.52540538513919 41.5283815993215, -87.52540878316159 41.5137882961521, -87.5252563816826 41.4991288937078, -87.525681680594 41.4846606916277, -87.5255518791097 41.4702328889289, -87.5363791819194 41.4701003884985, -87.53838838307431 41.4701026879554, -87.55559788764771 41.4700733870961, -87.557751988512 41.4700707870377, -87.57488469364969 41.4699643863905, -87.5771563943949 41.4699536861306, -87.589064697916 41.4698985858055, -87.5941836989597 41.4698759855581, -87.596537099542 41.4698909852242, -87.6134642052085 41.469784184389, -87.6160417052178 41.4697857841419, -87.6325717106245 41.4697169832884, -87.6351751111153 41.4697089827753, -87.65192061573271 41.469664281946, -87.6544939168074 41.4696814822412, -87.6711264215735 41.469614480925, -87.6740605226727 41.4697112811077, -87.690764127854 41.469586080151, -87.6935547285773 41.4695657795976, -87.70976153283119 41.4695008791206, -87.7127726339065 41.4696005787138, -87.7289588391906 41.4696238774638, -87.73212233941651 41.4696425777958, -87.7482197447183 41.4696949768181, -87.7512604451877 41.4697859763665, -87.7673014503807 41.4697493757685, -87.770806150792 41.4697609750334, -87.7872617561721 41.4698545746832, -87.790061156772 41.469921774582, -87.78999825778671 41.4844643766625, -87.7900281598679 41.4989639794446, -87.7904632614883 41.5132455820051, -87.7904427628628 41.5278569840274, -87.7903752635231 41.5390620863018, -87.7921633643558 41.5377086863297, -87.79226996432929 41.5438673868518, -87.79279656573669 41.5584860895394, -87.81233707233569 41.5582085882058, -87.8318738773217 41.5579168874736, -87.85135988318331 41.5575908865951, -87.870818689031 41.5571677850002, -87.89030529476391 41.5571030839849, -87.9096402000845 41.5567712824851, -87.91000990212351 41.5713289850516, -87.9105210037042 41.5858138874771, -87.910807704587 41.6003204905588, -87.91125770622421 41.6149227932038, -87.9114915080639 41.629345494921, -87.9121462098412 41.6438507975593, -87.9314654147943 41.6435067967512, -87.9507027211488 41.6430345957688, -87.9699843263345 41.6427032942906, -87.9894923323024 41.6422613932558, -88.0090876381699 41.6416796916802, -88.02778374316119 41.641359691116, -88.0281621443672 41.6560231935339, -88.0284736467818 41.6706373960109, -88.02922834782311 41.685256798343, -88.0106714426249 41.6854741997277, -88.0001874399299 41.6856247999084, -87.99389313795911 41.6857880002589, -87.99115093696859 41.6858456002, -87.9898272361548 41.6858732004721, -87.9859663357709 41.6859650007994, -87.9826922341876 41.6859227013621, -87.9824769346573 41.6862185010364, -87.9823001342 41.6863548014196, -87.98216633393641 41.6864586011954, -87.9819605339258 41.6865339014547, -87.9816448340332 41.6866312009398, -87.9813429342413 41.6867281011251, -87.9811447344026 41.6867925015989, -87.98093903421631 41.6868629014515, -87.9806072337629 41.6870256014466, -87.9802917335187 41.6871119013219, -87.97999753372289 41.6871923016423, -87.9797403338444 41.6872621016499, -87.9792929335927 41.6873310014465, -87.9789769333193 41.6873566015699, -87.9786922330558 41.6873434013345, -87.97844313336201 41.6872983012393, -87.97821743313381 41.6872362012554, -87.9778808329732 41.6871575015883, -87.9776189332206 41.6870681017816, -87.9773628332003 41.6870056016009, -87.9770347325338 41.686916001249, -87.97664103259341 41.6867594013969, -87.9762401326781 41.6865972014658, -87.97596883231429 41.6865573012528, -87.9757210321396 41.6864956014684, -87.975436032399 41.686454801934, -87.97524603257349 41.6864428013892, -87.9749445320748 41.6864796016704, -87.9746945321714 41.6865551015817, -87.97430533246489 41.6866515017487, -87.97395263175559 41.6867594015466, -87.973673632307 41.6868565013801, -87.9733932317171 41.6869812014121, -87.97315053161719 41.687100101713, -87.9727022320469 41.6872075014932, -87.9722683315272 41.6873254017585, -87.97194523111131 41.6874227021433, -87.9717065311836 41.6874066015913, -87.9715725312593 41.6873980022798, -87.97131563084589 41.6873747017553, -87.9711262308651 41.6873296016568, -87.97095103118239 41.6872737015972, -87.9707542306744 41.6871899018662, -87.9705068306927 41.6870730016851, -87.9702370312913 41.6869669014656, -87.9699373312238 41.6868825018924, -87.96960153056131 41.6868086019079, -87.9692286301572 41.6867516019882, -87.96861423023481 41.6866763017747, -87.9679779300193 41.6866393018961, -87.9675602298808 41.6866478023128, -87.9670691297534 41.6866554016128, -87.96659413018661 41.6865977020261, -87.9663137296005 41.6867609020415, -87.9658115291699 41.687093001973, -87.9649860295746 41.6875381026563, -87.96438802883991 41.687919301988, -87.9635550288928 41.6883264027115, -87.9629281293364 41.6887237026102, -87.96233132918761 41.6890111026364, -87.96194692828411 41.6892999028953, -87.9614380287988 41.6896043028041, -87.9607304285462 41.6899130032827, -87.9601703279466 41.6902450033927, -87.9597651284061 41.6904349033917, -87.9594851280124 41.6905375034799, -87.9590655276222 41.6907107030666, -87.9586900282 41.6908893030805, -87.95843252795341 41.69101420366, -87.9581150276251 41.6911880030889, -87.9578422281094 41.6913347032394, -87.9576870275732 41.6914493029767, -87.95741312729631 41.6916891036594, -87.9571021272773 41.6919401036036, -87.956828327724 41.6921744031324, -87.95650282703529 41.6924584037055, -87.956251327077 41.6926819032727, -87.9558367269335 41.6930040036359, -87.9556821271418 41.6930910040742, -87.9554676270927 41.6932213041817, -87.9550935270056 41.6932958040536, -87.95479152738901 41.6933926035148, -87.9543730268613 41.6934782037232, -87.9539856266867 41.6934533042109, -87.9535381266096 41.6935165041035, -87.95314852670199 41.6935860037449, -87.95269432667141 41.6936263041966, -87.95242212609379 41.6936649040199, -87.9522468265943 41.6936896039139, -87.95202572641161 41.6937488045081, -87.9516652258815 41.6938731043461, -87.9513713264576 41.6939368042532, -87.95106252616731 41.6940501044785, -87.9507237262861 41.6941912045533, -87.95040742564829 41.6943098044848, -87.9498349260697 41.6944162042255, -87.9491725257509 41.6946704039776, -87.94872242575261 41.6949313045676, -87.94853792529869 41.6950785047363, -87.948198025555 41.6953071043647, -87.94776292535239 41.6955573048183, -87.947526124809 41.6956983050627, -87.9471575249929 41.6959320047489, -87.9468613251168 41.6961392050031, -87.9465889249025 41.6963080050973, -87.9461900251236 41.6965854046496, -87.9455843243916 41.6969719045868, -87.9451116244301 41.6972988046356, -87.9447196242492 41.6975488054373, -87.9442912240044 41.697865105173, -87.9439433238921 41.6981212051754, -87.94360342414789 41.6983443054485, -87.94324822424031 41.69863880561, -87.94284312362851 41.69886110549, -87.94248912430641 41.6990571055887, -87.94227012426779 41.7013997059341, -87.9421252238124 41.7029514063594, -87.9418753237332 41.7030212063665, -87.9416469237546 41.7031134066899, -87.9414781236614 41.7031781066363, -87.9412569236577 41.7032752064666, -87.94109492424199 41.7033620063565, -87.940828524304 41.7035088063314, -87.94047642343931 41.7035836065757, -87.94021942410861 41.7036423062493, -87.9399326239362 41.7037172063148, -87.939741423524 41.7037933060707, -87.939572023585 41.7038800061362, -87.939379823127 41.7039609064741, -87.93919582359339 41.7040861065238, -87.9390473230106 41.7042227063797, -87.9389288238178 41.7043377063134, -87.93875182370709 41.7044353070637, -87.938552223625 41.7045161066779, -87.9382949230268 41.7046300067482, -87.9381034231414 41.7047171070134, -87.9378976231303 41.704786806294, -87.9375739228647 41.7048675069897, -87.9372942228468 41.7049920064727, -87.93701432318041 41.7051277070138, -87.9367918227627 41.7052854067713, -87.9365262232258 41.7054756071362, -87.93634802252529 41.7056284073839, -87.93615502258061 41.7058196068918, -87.9359769226074 41.7060055066931, -87.9358730230135 41.7061254070326, -87.9357754222776 41.7062903067567, -87.93567172233649 41.7064433075491, -87.93557392285081 41.7066130069558, -87.935417922824 41.7067984076321, -87.93526872257161 41.7069681070416, -87.935141422621 41.7071870074682, -87.9349906224928 41.7075435077901, -87.9348787225304 41.7076909076043, -87.9348037226612 41.707833307185, -87.9347436227673 41.7080090077667, -87.93462432228149 41.7081563077565, -87.9344833229455 41.7082875071822, -87.9342980220942 41.7084291072717, -87.93414282245671 41.708538107829, -87.9339661227223 41.7086633078783, -87.93379632258269 41.7087280078248, -87.9335603224324 41.7088311079337, -87.93353612256939 41.7088418080705, -87.9332221223997 41.708977007691, -87.93292002205899 41.7090744078994, -87.932655321833 41.7091495076131, -87.9323827219615 41.7092411080911, -87.9321693217786 41.7093218078813, -87.9319270218749 41.7094193079111, -87.93155932136921 41.7095324075909, -87.93126432207841 41.7096402075608, -87.9310585220673 41.7097106083128, -87.9307716211752 41.709829608441, -87.9305869219672 41.7099381080774, -87.9303728212155 41.7100519083206, -87.9300034210016 41.7102752082449, -87.9296485214654 41.7105477082873, -87.9295300213733 41.7106619078242, -87.9293740213422 41.7108040079352, -87.9292038210102 41.7109293087081, -87.92900432076 41.7110818085834, -87.9287895212414 41.7112616082843, -87.9285528214254 41.7114364086542, -87.9284188206172 41.7115835080961, -87.9281755206753 41.7117637088962, -87.92802692116361 41.7118672082993, -87.92782722126501 41.7120307086837, -87.9277014204738 41.7121394088657, -87.9272577209594 41.7124720088171, -87.9271099209459 41.7125811082995, -87.92698412105401 41.7126897086567, -87.92666732039081 41.7127869083171, -87.926389320976 41.7127571083922, -87.9260522207982 41.7127390086523, -87.925839820653 41.7127321090642, -87.9255180206472 41.7127293085929, -87.92526092059251 41.7127887088385, -87.9250616199831 41.7128529087635, -87.92481121988661 41.7129834085226, -87.9246564204459 41.7130758084914, -87.9245226201817 41.7131740090872, -87.92434521987791 41.7133261087629, -87.92416682043149 41.7135229092486, -87.9239667203361 41.7137030093222, -87.9237585200479 41.7139160093004, -87.92360201999961 41.7141187093385, -87.92345281975101 41.7143263088836, -87.92323061971349 41.7145501088228, -87.9230144199599 41.7148292089831, -87.92277701957831 41.7150694089629, -87.9224575200722 41.715402309281, -87.92210221944001 41.7156913091878, -87.9216577195358 41.7160949097276, -87.9213775194968 41.7162415100538, -87.9211270195718 41.7163333092806, -87.9209863197008 41.7163708092256, -87.9208474193737 41.7164083100698, -87.92045831949029 41.7164880098235, -87.9200406193512 41.7164908094368, -87.9198064187333 41.716478110121, -87.91929351903021 41.7164583097727, -87.918941318333 41.7164558096766, -87.918684618475 41.716498610233, -87.9183768182224 41.7165236096087, -87.9181281187286 41.7164948097284, -87.917997118039 41.7164345101822, -87.91794611836779 41.7164111098213, -87.9177276181106 41.7163159102833, -87.9175597182265 41.7162545099688, -87.9172015181443 41.716191909969, -87.91696661785249 41.7161682101442, -87.9167255176355 41.7161719099691, -87.91648301807891 41.716236309546, -87.91619531768811 41.7163442102396, -87.9156889174031 41.7164837104296, -87.91565031759031 41.716484609754, -87.91534261806881 41.7165805098932, -87.91497441753771 41.7166715096292, -87.914592417805 41.7167623097165, -87.91429101713 41.7167887102358, -87.9145748178628 41.7237651113582, -87.9148661189712 41.7310167122679, -87.9153641210565 41.7456838146629, -87.9160281218378 41.7601902171637, -87.9166021238478 41.7747779201939, -87.9170061251676 41.7893990222605, -87.917639326871 41.8039953246044, -87.91828852841491 41.8185503273729, -87.91890843003959 41.8331074300593, -87.919586032192 41.8477339325364, -87.9201203336137 41.8621874351068, -87.9202381343795 41.8655325350688, -87.9207499353424 41.8768377372471, -87.92064013672839 41.8912521400203, -87.92059303666861 41.8927220398902, -87.9201234380532 41.906815942237, -87.92050963998661 41.9213163444643, -87.92027974138659 41.9358084468807, -87.9203241423829 41.950250149466, -87.9203237422024 41.9504279499614, -87.92032374221 41.9505036495082, -87.92032248838974 41.95337047695071, -87.92019695512401 41.95333872831988, -87.91690271051949 41.95251309104307, -87.91594028102732 41.95227185588003, -87.91594145235712 41.95240769980954, -87.91594748054919 41.95311798072569, -87.91596111501816 41.95472280574447, -87.91596424244553 41.95509053402274, -87.91632672722831 41.95509185117047, -87.91615587512518 41.95583178626499, -87.91591025922307 41.95689532254021, -87.91584929591959 41.95715931859842, -87.91578099414373 41.95713178806096, -87.91542826188814 41.95695262399683, -87.9152277887074 41.95684476054367, -87.91514133894161 41.95656014382389, -87.91504365881929 41.95619414876017, -87.9149604764803 41.95582611971568, -87.91489179283437 41.95545644256224, -87.91483767680667 41.95508544605995, -87.91479819877269 41.95471340518208, -87.91477342812247 41.95434136300079, -87.91474729110888 41.95290837473699, -87.91474668592809 41.9528751921989, -87.91452531777783 41.95282965332774, -87.91230028474516 41.9523719019079, -87.91229803534667 41.95237143976457, -87.91008622351627 41.95191639751901, -87.90849578170905 41.95158917522549, -87.90647421732297 41.95117322775096, -87.9049917427832 41.95086814602494, -87.90455598353728 41.95077843018257, -87.90377631735495 41.95061799437615, -87.90184855495775 41.95064913303904, -87.90162347621532 41.95065278746928, -87.90097853430181 41.95066320440554, -87.90008829233571 41.95067758420223, -87.89999307099316 41.95067912138119, -87.89730344798967 41.95072252105504, -87.89699917557871 41.95072740322441, -87.89531876542144 41.95075448399196, -87.89521435063156 41.95075616572232, -87.89287730506713 41.95079377202208, -87.89288061183304 41.95189394311508, -87.89288823086122 41.95441832729993, -87.89289884515911 41.95796000366475, -87.89289910506837 41.95805056391234, -87.89047705326281 41.9580923453891, -87.89026621410935 41.95809597646716, -87.88934415934189 41.95797474230993, -87.88869488218384 41.95788935419068, -87.88802577034905 41.95780136188581, -87.88680730002594 41.95764108623271, -87.88562916640862 41.95748617006401, -87.8856293796873 41.95753984783217, -87.88550794625627 41.95752356698424, -87.88494317325302 41.9574476534744, -87.88484326487834 41.95743420075555, -87.88457681992816 41.95739841834801, -87.88442732486017 41.95737834969022, -87.88403944883653 41.95732622876678, -87.88384646561668 41.95730028118329, -87.88382845798334 41.95736201133175, -87.88163261362125 41.95711146169589, -87.88078844362624 41.95720045373935, -87.88079337576424 41.95823883117646, -87.88079792983301 41.95902940344055, -87.88079816803292 41.95906859182286, -87.88080337278681 41.95998545546434, -87.88080910456635 41.96097328651003, -87.88080923559872 41.96099633900491, -87.88080959123054 41.96106055455395, -87.88083453376264 41.96541303548606, -87.88083506873281 41.96550387168221, -87.8808340835739 41.96559424771787, -87.88080433188887 41.96831953436504, -87.88079492255092 41.9691812905474, -87.88078383278321 41.9710183325429, -87.8807727473727 41.97285455097844, -87.87807002051488 41.97292180140298, -87.87732285387298 41.97294040499114, -87.87589582869249 41.9729758907771, -87.87305384064425 41.97304652900646, -87.87285345803855 41.97305148856432, -87.87268617739818 41.97305564674235, -87.87243724364998 41.97306183801813, -87.8720843814519 41.97306862633098, -87.87170707972521 41.97307830648126, -87.87138888263632 41.97308674080662, -87.87100786757682 41.97309632084178, -87.87089391936496 41.97309918185335, -87.87060680713478 41.97310639510389, -87.86988726227685 41.97312393731761, -87.86882639065078 41.97316986891063, -87.86875887159769 41.97316992528253, -87.86867474936081 41.9731673345828, -87.86858851533371 41.97316199023783, -87.86848169884311 41.97315535640872, -87.86808381817582 41.97316776813383, -87.86796365400504 41.97317079259181, -87.86770000573154 41.9731774622398, -87.86734399961665 41.97318652836674, -87.86712673390879 41.97319206745961, -87.86701138548642 41.97319511212339, -87.86681082215081 41.97319956589167, -87.86645890724813 41.97320738301259, -87.86605979371365 41.9732175604457, -87.86542945006521 41.97323156731088, -87.86485129694154 41.97324441241878, -87.86427414790026 41.97326071614889, -87.86406751347035 41.97326571687758, -87.86316354371932 41.97328762179323, -87.86227798926538 41.97331083087008, -87.86190219194968 41.97332067678497, -87.86189024144282 41.97328061766098, -87.8618034175672 41.97312595381528, -87.86170113682662 41.9729878288804, -87.86144846917254 41.97271763835635, -87.86133777471659 41.97260856687279, -87.86111135730837 41.97241935542636, -87.86085177452598 41.97222446455562, -87.86042602277828 41.97192910726332, -87.86010604581661 41.97175619374093, -87.85990957640027 41.97166526407828, -87.85974632854337 41.97159700028374, -87.85960456018617 41.97155338668564, -87.85940534761455 41.97149520963131, -87.85920596563601 41.97145997451518, -87.85902956007291 41.97143056910833, -87.85892210226373 41.97142153544895, -87.85879533610999 41.97143248289303, -87.85867604861512 41.97146640163143, -87.85853733165962 41.97153177143604, -87.85838697589854 41.97161432629856, -87.85783321555054 41.97197355732732, -87.85746442719498 41.97214949743699, -87.85704324667496 41.97231355501, -87.85667056182962 41.9724341805311, -87.85659222789336 41.9724595419059, -87.85621357484553 41.97255206759907, -87.85606341270082 41.97257830926294, -87.85584134836523 41.97258635916607, -87.85559564510754 41.97255397036758, -87.85536192711434 41.97251715815305, -87.85515841907932 41.97245805108936, -87.85497932559561 41.97234081432927, -87.8545976281852 41.97203900627951, -87.8544668328306 41.97189532244158, -87.85427078924621 41.97162093464478, -87.85418821164113 41.97143241833891, -87.85415514314217 41.97131093210054, -87.85413499526533 41.97116817588404, -87.85413624887141 41.97100092167733, -87.85417261801673 41.97078477414106, -87.85423604512388 41.97056009605998, -87.85432295443876 41.9702980299745, -87.85464398183399 41.96967410017543, -87.85477705162717 41.96942959702958, -87.85489427210493 41.96911150910954, -87.85502853285979 41.9687077098676, -87.855036915718 41.96832939957832, -87.85497959603354 41.96793028880794, -87.85485695541745 41.96751842123782, -87.85477734388419 41.96719951337391, -87.85477187559368 41.96691293949837, -87.85486729052506 41.96635851254583, -87.85492186921797 41.96613579948268, -87.85496772437027 41.96594864249681, -87.8553014432147 41.96511864609648, -87.85544282921758 41.96470935905154, -87.85550645787336 41.96445603070568, -87.85559467828081 41.9636572266456, -87.85562412381776 41.96328896594795, -87.85567657317358 41.9628437705149, -87.85568620049204 41.96281494196663, -87.85582008444108 41.96241465127055, -87.85629870220842 41.96159079622601, -87.8564745164457 41.96122728728883, -87.85664603899336 41.96090127258495, -87.85684363968228 41.96055420902379, -87.85703032774599 41.96019582027144, -87.8572480307553 41.95984436546551, -87.85741634109282 41.95955900550815, -87.85757725537934 41.95918997170908, -87.8577402503709 41.95895640059629, -87.85792614132193 41.95865942312312, -87.85813159823738 41.95834164281787, -87.85842254480335 41.95792824964695, -87.85868113610282 41.95760865791748, -87.85871814553305 41.95754124825741, -87.85875826582301 41.95741603020941, -87.85878197270466 41.95724359945014, -87.85878657336707 41.95708434401254, -87.85877505125583 41.9569325410891, -87.85874476246364 41.95675854163384, -87.85870843806677 41.95659505583871, -87.85867744197121 41.95651111924934, -87.85863554081378 41.95638580989818, -87.858610264958 41.95622000971429, -87.858587122258 41.95614318510418, -87.85852336750671 41.95600505133668, -87.85844618617976 41.95585816336946, -87.85834588324501 41.95570593933101, -87.85825043251143 41.95557862315464, -87.8580787825354 41.95544459938865, -87.84660472600027 41.95568681243826, -87.84661768182393 41.95253447305257, -87.84662079438546 41.95178717983021, -87.8466124346149 41.95163917628162, -87.85407255572582 41.95144855238202, -87.85411219379246 41.95131488189043, -87.85420269328843 41.95107555130988, -87.8542460133016 41.95092145277584, -87.85432707116189 41.95073002368407, -87.8544348198574 41.95050407543071, -87.85453280422036 41.95030064330159, -87.85466436577403 41.95008549630863, -87.85475115968126 41.94989859159985, -87.85493570997178 41.9494841601921, -87.85501321104442 41.9489421160887, -87.85502174283613 41.94862694995752, -87.85501697097379 41.94846359284827, -87.85500439275239 41.94819888842788, -87.85497588743516 41.94803397936015, -87.85487051586783 41.94779542297335, -87.85464329016901 41.94743860581134, -87.85408173642175 41.94655668389039, -87.85396878825604 41.94624595127517, -87.85382223715034 41.94584273341999, -87.8535827878264 41.94504406953111, -87.8534160668876 41.94435572557096, -87.85329116820974 41.94386513952123, -87.85326069009727 41.94376314701236, -87.85321718772161 41.94368727911417, -87.8531475392957 41.94357837170281, -87.85304727779574 41.9434397251833, -87.85301273187522 41.94335072348672, -87.85301018560331 41.94321973042337, -87.85302521409709 41.9431469618326, -87.85302429980452 41.94310735938789, -87.85305499802129 41.94299876022951, -87.85308191846725 41.94293303914638, -87.85309550863425 41.94288043458212, -87.85309612598587 41.94279816501591, -87.85309239483315 41.94271258528745, -87.85307559146105 41.94260060549244, -87.85303704520528 41.94248164618615, -87.85295495480496 41.94226769224635, -87.85274177181468 41.94182149489594, -87.85253233905389 41.9412582156021, -87.85226713440449 41.94041347527028, -87.85211178647293 41.93979007612949, -87.85180609362935 41.9391696705912, -87.85164285110305 41.93895175285192, -87.85141194452754 41.93870457185066, -87.85099761226242 41.93834800318465, -87.85066017859724 41.93807866598189, -87.8501874214292 41.93775082833277, -87.8497776811724 41.93743047172816, -87.84962055261087 41.93728535290575, -87.84854945507834 41.9373120376873, -87.84694895619494 41.93734300207925, -87.84661919669712 41.93734830089693, -87.84646223570854 41.93735133986716, -87.84617400202634 41.93735747127401, -87.84598218857909 41.93736159798293, -87.84575501286911 41.93736628688639, -87.84558036970132 41.93736987963658, -87.84533595363567 41.93737487866377, -87.84516009725938 41.93737841238025, -87.84490527348416 41.93738438132939, -87.84444642887549 41.93739516119378, -87.84411024899008 41.93739888924866, -87.84384797517532 41.93740339683455, -87.84367348050579 41.93740657705496, -87.84358643805902 41.93740788007018, -87.84339073480872 41.93741121610181, -87.84298176340543 41.93741742878315, -87.84260554731605 41.93742729179447, -87.8421779589992 41.9374361108372, -87.84181576712012 41.93744345249995, -87.84143717930959 41.93745102409515, -87.84107620065399 41.93745820396855, -87.84071507913507 41.93746502507003, -87.84037027560662 41.9374716955584, -87.84005840667189 41.93747795821064, -87.83967948619329 41.93748576956073, -87.83933471863392 41.93749265690612, -87.83891143466334 41.93750068592679, -87.83853014171484 41.93751117315863, -87.83813541116493 41.93751979025449, -87.83779042757921 41.93752595770813, -87.83746234672915 41.93753269163462, -87.83720121854823 41.93753631137949, -87.83688618929295 41.93754252321835, -87.83665581883058 41.93754682355302, -87.83640912066033 41.93755217762028, -87.83606005125448 41.93755909139779, -87.83582269426657 41.93756379805517, -87.8354231557024 41.9375715073882, -87.83481017093838 41.93757642160709, -87.83461770564399 41.93757459786984, -87.8343966320283 41.93757901582974, -87.83431576088088 41.93758055707342, -87.83414130502278 41.9375834485514, -87.83387361816847 41.93758903522362, -87.8336322144352 41.9375942427984, -87.83344925822745 41.93759819364637, -87.8332107273089 41.9376168304651, -87.83292958573855 41.93762274107011, -87.83285139631884 41.9376244861633, -87.83282669418163 41.93762492650848, -87.83272453914222 41.9376267863075, -87.83256743963847 41.93762857159763, -87.83243837746066 41.93763102673679, -87.83231475480461 41.93763339597211, -87.83213808688268 41.93763698880122, -87.83190624251903 41.9376416848053, -87.83173269416002 41.93764575605492, -87.83159032332682 41.93764867487426, -87.8314168623222 41.93765120981001, -87.83124155171585 41.93765502590765, -87.83103925954292 41.93765941003591, -87.83080304305872 41.93766397464184, -87.83069305794302 41.93766609995316, -87.83053576317241 41.93766914285491, -87.83036156047366 41.93767241388942, -87.83022006748379 41.93767571849781, -87.83000539202988 41.93767977266842, -87.82963816638068 41.93768666937893, -87.82940212813349 41.93769172529618, -87.8292128860722 41.93769572495428, -87.82903941682342 41.93769932568676, -87.82888270770377 41.93770256181497, -87.82876423179836 41.93770484089315, -87.82856679510788 41.93770863744886, -87.82818942034463 41.93771603430108, -87.82811075589166 41.9377174450428, -87.82790556299486 41.93772137126027, -87.82773344939993 41.93772530584297, -87.82752590386342 41.93772941366673, -87.82727641423702 41.93773435244291, -87.82705714162424 41.93773881909148, -87.82688345002562 41.9377423889668, -87.82672637414915 41.93774559254265, -87.82656882229409 41.93774865659148, -87.82631660979567 41.93774946382659, -87.82593770281252 41.93775067172695, -87.82571648381327 41.93775468923501, -87.82557573555016 41.93775686518118, -87.82537127906943 41.93776068145425, -87.82523173571559 41.93776390538974, -87.82520666812788 41.93776406942755, -87.82507271473752 41.9377668242216, -87.82487681876735 41.9377708959486, -87.82469140183684 41.93777463066974, -87.82455678699741 41.93777727188812, -87.82437192008649 41.93778111829954, -87.82413467133337 41.93778596647371, -87.82403372717906 41.93778815403985, -87.82384720976563 41.93779146923563, -87.82363040402339 41.93779533710445, -87.82334499943997 41.9378009857748, -87.82310762884909 41.93780717540736, -87.82299039870253 41.93780989259019, -87.82284001904057 41.93781213269024, -87.82262614874057 41.93781664316023, -87.82236599559741 41.93782209874894, -87.82206210080444 41.93782826484914, -87.82187683147913 41.93783188563823, -87.82170810469408 41.93783492150263, -87.82159029240346 41.93783683856167, -87.82140674990971 41.93784043774863, -87.82118791912559 41.93784475904322, -87.82098544397249 41.93784879588108, -87.82078289545004 41.93785299586462, -87.82069878917667 41.93785473398441, -87.82054696882433 41.93785795202106, -87.82037875141373 41.93786175631548, -87.82018505863644 41.93786610457635, -87.82002422916599 41.93786972155696, -87.81975514529191 41.93787537910639, -87.81960365841105 41.93787815940863, -87.81946900622476 41.93788068527381, -87.8192999113444 41.93788398955077, -87.81916529515162 41.9378867901923, -87.81896445131846 41.93788926491556, -87.81877816187527 41.93789158662209, -87.81850428415252 41.93789900240522, -87.81839106261485 41.93790123945353, -87.81822226180314 41.93790457172755, -87.81808764605303 41.93790726078954, -87.81790244941534 41.93791092872881, -87.8177399694618 41.93791417889953, -87.81753069458412 41.93791837065663, -87.8172945509982 41.93792255159435, -87.81710990888263 41.93792581076, -87.81692523059174 41.93792904147485, -87.81663916905121 41.93793406596625, -87.81651876007274 41.93793876579466, -87.81650950762001 41.93775099082328, -87.81650467023762 41.93761263343546, -87.81650087179888 41.93750501495623, -87.81647760931462 41.93684475888392, -87.8164766817472 41.93681843812534, -87.81647054023288 41.93664601789396, -87.81646344072114 41.93644645294291, -87.81645310537751 41.93613784528759, -87.81644431330037 41.93587485388871, -87.81643864818965 41.93572993320135, -87.81643205859496 41.93555822551514, -87.81642860288008 41.93545821059637, -87.8164233440138 41.93530390592086, -87.81641561906733 41.93507714336128, -87.81641094658845 41.93494125535517, -87.81640018656657 41.93462616960065, -87.81639724335977 41.9345400696572, -87.81639040715896 41.93431254295804, -87.8160495071474 41.93432000898483, -87.81604597988338 41.93432007487175, -87.81590357860541 41.93432286296946, -87.81576816312197 41.93432551882427, -87.8157577231752 41.93432571847029, -87.81561252791384 41.93432874169061, -87.81541770956086 41.93433285703486, -87.81517238141252 41.93433836458374, -87.81456351061546 41.93435161298593, -87.81439875849723 41.93435520574678, -87.81395207480664 41.93436416159694, -87.81364242434664 41.93437038943965, -87.8133396396838 41.93437749880056, -87.81323439761022 41.93437995845578, -87.81272897308732 41.93439012588444, -87.81227291476918 41.93439928214539, -87.81213396373371 41.93440271267144, -87.812014018043 41.93440568102501, -87.81150730280953 41.93441624884901, -87.81101488675777 41.93442652410753, -87.81089648877085 41.93442897693087, -87.81066443291355 41.93443376602856, -87.81028725759153 41.934441323135, -87.80976021945565 41.93445187491503, -87.80967626290843 41.93445352304724, -87.80940895786136 41.93445880635747, -87.80906548045792 41.93446670640318, -87.80853765679107 41.93447884008678, -87.8084532227151 41.93448053947766, -87.80820606177066 41.93448550098594, -87.80784530967394 41.93449337353264, -87.8072323206638 41.9345067302304, -87.80713707891584 41.93450881765961, -87.80685639206318 41.93451011137534, -87.80662355692481 41.93451118347628, -87.80661838758954 41.9343756775428, -87.80661430793084 41.93426849593215, -87.80660400175502 41.93394441139012, -87.80659407357382 41.93365913001953, -87.80658693592244 41.93343670507224, -87.80657413108318 41.9330366199783, -87.80656774242563 41.9328573105371, -87.80656054562733 41.93270453351818, -87.80655575319538 41.93256340415027, -87.80652647753159 41.93169960632005, -87.80650911238452 41.93120611380018, -87.80650175753578 41.93099683343628, -87.80649498263665 41.93081524362269, -87.80647996018685 41.93041229443875, -87.80647708671982 41.93035517439564, -87.8064750734523 41.93030950145991, -87.80647031884853 41.93019520856981, -87.8064658015286 41.93002501901194, -87.80646439630773 41.92995020514861, -87.80646401342942 41.92993867680548, -87.80646091755375 41.92984160003224, -87.80645668073532 41.92972673459222, -87.80645285967007 41.92962378019004, -87.80644907620432 41.92952099170989, -87.80644275625947 41.92933764762785, -87.80643763049997 41.92918795382393, -87.80643584811502 41.9291367383195, -87.80642905173242 41.92893989179997, -87.80642220055843 41.92874082109807, -87.80642122997116 41.92868911507897, -87.80641786601096 41.92859774505169, -87.80641393771931 41.92849457114661, -87.80640938165043 41.92837399621473, -87.8064043376839 41.92823670695431, -87.80639184202128 41.92789296194823, -87.80638657039958 41.92775213047611, -87.80638177867473 41.92762425534227, -87.80637828562175 41.92752643521433, -87.80637459096955 41.92741703295793, -87.80637132626892 41.92731361564397, -87.8063648936989 41.92713087442741, -87.80635881600631 41.92695850939764, -87.80635550219911 41.92686101938811, -87.80635180684411 41.92675175515004, -87.80634749770698 41.92663683305627, -87.80634342386672 41.92653368698599, -87.80633885356902 41.92641906665206, -87.80633295750455 41.92623366623828, -87.806324900766 41.92598003486385, -87.80632328395261 41.92592662470438, -87.80631881992035 41.92575896022739, -87.80631509887324 41.92564412396221, -87.80631186053562 41.92554649706418, -87.80630343150999 41.92532052580284, -87.80629635731555 41.92513072917407, -87.80629136561677 41.9250274139339, -87.80628551921662 41.92490735413212, -87.80627967317533 41.92478707451623, -87.80627259346936 41.92463827776422, -87.80626987163072 41.92449581164716, -87.80626740188929 41.92438504393458, -87.80626078955974 41.92418388882146, -87.80625508011852 41.92400688736247, -87.80624729890661 41.92376475411101, -87.80624203253754 41.92359214571861, -87.80623951594913 41.923558599922, -87.80623239117143 41.92346441287889, -87.80622980136717 41.92336824281336, -87.80622944180766 41.92332254914431, -87.80622929273734 41.92330512340734, -87.80622567544272 41.9230926203321, -87.80622657595205 41.92307701005723, -87.80621903878702 41.92282310622498, -87.80621620023163 41.92273939437192, -87.80620938796213 41.9225447964485, -87.80620514929311 41.92242136884002, -87.80620080417015 41.92229297450174, -87.80619227215497 41.92203942254221, -87.80618821945721 41.92191563957468, -87.80618210194899 41.92173026598397, -87.80617705768391 41.92156646671324, -87.80617352071347 41.92144257633345, -87.80616932357633 41.92128728806694, -87.80616233621335 41.92107372663486, -87.8061564916417 41.92089529717993, -87.80615187641229 41.92075090186061, -87.80614757823126 41.92061682713515, -87.80614065477488 41.92039989138623, -87.80613467896683 41.92022420471756, -87.80612930358411 41.9200691857051, -87.80612238992165 41.91985562566191, -87.80611825507647 41.91972854808954, -87.80611399409958 41.91959461017291, -87.80610689359945 41.91936795898147, -87.806100435334 41.91916167240002, -87.80609262654065 41.91891385879284, -87.80608744584571 41.91874882445558, -87.8060822734964 41.91858255579496, -87.80607542799413 41.91836534535174, -87.80606956750147 41.91817997282189, -87.80606505301432 41.91802320057135, -87.80606064899152 41.91787082069734, -87.8060577020883 41.91777799713668, -87.80605177771616 41.9175916915283, -87.80604552998194 41.9173953948637, -87.80603960025003 41.91720974755987, -87.8060339517677 41.91703439324737, -87.8060286236378 41.91686935707935, -87.80602148453104 41.91664759088714, -87.8060203137187 41.91661119639765, -87.80601485402926 41.91643545846571, -87.80601137845946 41.91632202241947, -87.8060072584271 41.91621541793058, -87.80600505153562 41.91618906285761, -87.80599662820721 41.91590894746133, -87.80599215718127 41.91576040860306, -87.80599152159954 41.91573924789893, -87.80598525933908 41.91553145302682, -87.80598022979058 41.91536578810842, -87.80597169400194 41.91508605552389, -87.80596655330265 41.91492055501033, -87.80596197219859 41.91477187868266, -87.80595541226046 41.91456018504235, -87.80595165061661 41.91444581624992, -87.80594918805811 41.91436109008, -87.80594482941159 41.91421222183919, -87.80593987927355 41.91406370900063, -87.80593339710467 41.91386936036541, -87.80592785552274 41.91370352839134, -87.80591904079242 41.91343532078635, -87.80591376435478 41.91326411103772, -87.80590897240393 41.91310969741794, -87.80590279811567 41.91290912046998, -87.80589787478684 41.91274866946335, -87.80589087227129 41.91253258403039, -87.80588568560867 41.9123727083676, -87.80587935721687 41.91219523716654, -87.8058727798056 41.91201249505603, -87.80586770745309 41.91186993631848, -87.80586096618121 41.91167577796339, -87.80585684635713 41.91155586273285, -87.80585098413651 41.91138434864332, -87.80584686357521 41.9112645714382, -87.80584309161864 41.91115604638517, -87.80582618802204 41.91070591392773, -87.80582049014401 41.91055457044494, -87.805811485346 41.91039454079555, -87.80580856512954 41.9103429900311, -87.80580244464869 41.91023423572902, -87.80579767385288 41.91014878456495, -87.80578964186002 41.90999979093716, -87.80578527891913 41.9099140683167, -87.80577807663974 41.90977130709289, -87.80577261257464 41.90966302176607, -87.80576876671471 41.90959472798173, -87.80576711978433 41.90941991211835, -87.80576781171594 41.90935372452641, -87.8057677459331 41.90933489846989, -87.80574541301185 41.90883935037458, -87.8039424544545 41.90889183794991, -87.80256615771111 41.90893188948525, -87.80254413749796 41.90892915295728, -87.80239977463206 41.90893161482023, -87.80227896518358 41.90893369009653, -87.80211323522185 41.90893448774117, -87.80185000095834 41.90893576803414, -87.80174235507577 41.90893622913717, -87.80151713617848 41.90893842449506, -87.80131543088893 41.90894042676253, -87.80118048067948 41.90894202373077, -87.80085682785634 41.90894582030986, -87.80070677671338 41.90894767627906, -87.8005797984419 41.90894928262446, -87.80046443060657 41.90895072289479, -87.80027701925192 41.90895213159973, -87.80011286672294 41.90895337193464, -87.79998062351093 41.9089563535543, -87.7998477549567 41.90895938617788, -87.79973810946876 41.90896186822086, -87.79960128835188 41.90896326281685, -87.79931132930311 41.90896625061146, -87.79919607387161 41.90896755346697, -87.79907501138688 41.90896893786086, -87.79888439767157 41.90897134393842, -87.79876344487731 41.9089729214343, -87.79865953964668 41.90897424942154, -87.79845044168312 41.90897706212448, -87.79824781274141 41.90897979591936, -87.79814251061124 41.90898122587569, -87.7979999856161 41.90898357990508, -87.79788178377216 41.90898549888009, -87.79780668293044 41.90898674141653, -87.79770292544869 41.9089880129702, -87.79755900881509 41.90898983855545, -87.79745499467121 41.90899113695318, -87.79720386503352 41.90899449217275, -87.79655192115298 41.9090031649067, -87.7959884137383 41.90901068359435, -87.79505256779598 41.90902312240757, -87.79479846224191 41.90902651330061, -87.79384148758926 41.90903922789222, -87.7935655934331 41.90904289870436, -87.79262860636972 41.90905536755537, -87.79234198500966 41.90905917618844, -87.79142263430083 41.90907136180902, -87.79111745732888 41.90907543656605, -87.79020776778904 41.90908749314895, -87.78989645615955 41.90909161879491, -87.78874431461105 41.90910188466216, -87.78869640422447 41.90910228888251, -87.78852592079983 41.90910474634581, -87.78826314663134 41.90910789009884, -87.78797546184134 41.90911132658147, -87.78764038558413 41.90911483845685, -87.78734586976061 41.90911788442281, -87.78707409809707 41.90912068080702, -87.78696582036082 41.90912216950774, -87.7866328212974 41.90913186341093, -87.78661393390711 41.90913240438744, -87.78631798883677 41.90913072027333, -87.78604687313074 41.9091342574719, -87.78565796033421 41.90913934752084, -87.78542967944246 41.90914289600514, -87.78530971566997 41.90914473874544, -87.78500237868285 41.90914785417346, -87.78477102187516 41.90915017892094, -87.78452882378525 41.90915297416641, -87.78428609630684 41.90915743898287, -87.78414265487831 41.90915941495207, -87.78386922265702 41.90916299218623, -87.78362893112022 41.90916617830079, -87.78335509843843 41.90916936627922, -87.78305286454416 41.90917285877867, -87.78287305582579 41.90917452194102, -87.78264548483065 41.90917661412094, -87.78241743760317 41.90917867502493, -87.78212113501363 41.90917596383357, -87.78193602798181 41.90917425212474, -87.78171330181482 41.90917699689599, -87.78128779632384 41.90918225145593, -87.78114409847488 41.90918416756146, -87.7808203001625 41.90918774175707, -87.78056792437262 41.9091905334629, -87.7804242254398 41.90919261331653, -87.78010783918792 41.90919704446337, -87.77993121880806 41.90919951448275, -87.77978689403891 41.90920178234482, -87.77951533971755 41.90920494552945, -87.77934368295671 41.90920694407259, -87.77908241234233 41.90921043081872, -87.77886831821712 41.90921312995698, -87.77844024137283 41.90921855419326, -87.77821035365733 41.90922087309437, -87.77801375277862 41.90922285910245, -87.77797899624213 41.90922321241847, -87.77765162551565 41.90922755540605, -87.77729441950693 41.90923230266347, -87.77690782363126 41.90923718109796, -87.77668972612251 41.90923991165941, -87.77665684227264 41.90924032853791, -87.7764374205435 41.90924307909398, -87.77559930525821 41.90925355500868, -87.77559798554219 41.90921115095109, -87.77559052470232 41.90888218928609, -87.77554257705486 41.90743955876924, -87.77548229131726 41.90562524059178, -87.77542204166419 41.9038111980628, -87.77537345707731 41.90234946210946, -87.77536980738975 41.90223704026928, -87.77536171590634 41.90199778574227, -87.77535619162019 41.9018348319356, -87.77534451801881 41.90147110919355, -87.77533332592047 41.90112808034777, -87.7753216966146 41.90075925238737, -87.77531065141392 41.90041625083039, -87.77530240750977 41.90016906352691, -87.77529772381075 41.90002784951947, -87.77529109774559 41.89981014459253, -87.77527770256992 41.89939092367506, -87.77526219010215 41.89889452461222, -87.77525972665788 41.89881051192584, -87.7752526632427 41.89857134375048, -87.775249307615 41.89835760582122, -87.77524622420101 41.89816327155265, -87.77523939546271 41.89790146421154, -87.77523595598461 41.89776903777112, -87.77522385898136 41.8974455981948, -87.77521548570012 41.89722190140145, -87.77520752785443 41.89698846427955, -87.77520313635432 41.89686009439618, -87.77519172940745 41.89653328324908, -87.77518623874273 41.89637515986231, -87.77517613813018 41.89607554939447, -87.77517464979279 41.89603138727701, -87.77516462021158 41.89575754448901, -87.77515969748974 41.8956182781692, -87.77515506817537 41.89548779512757, -87.77512939073127 41.89471192651577, -87.77509926341112 41.89380209094328, -87.77506651333995 41.89281367390264, -87.77503560018576 41.89188020596554, -87.77500649700474 41.89100050617977, -87.77498148992876 41.89024500250554, -87.77497421880929 41.89001730472543, -87.77494039045719 41.88895495201852, -87.77491065936293 41.88802096776939, -87.77489125239379 41.8874114308499, -87.77488920293436 41.88734731442845, -87.77488133320698 41.88709969010267, -87.7748327704105 41.8855748043892, -87.77478438621047 41.88405480443712, -87.77472225025352 41.88210287367755, -87.77471146361313 41.88176440023415, -87.77470867034546 41.88163398013118, -87.77470602009382 41.88155893970026, -87.77470201111784 41.88144574757363, -87.77469263170924 41.88118604093619, -87.77468352966825 41.88089466670465, -87.77467344292299 41.88063608091991, -87.77467116385554 41.88057737139871, -87.77466544361766 41.88035711686279, -87.7746559378828 41.8800868170233, -87.77464451097228 41.87976302153855, -87.77463998922919 41.87960328464736, -87.77463432484168 41.8794273502844, -87.77463376581314 41.87941143145366, -87.77462300034621 41.8791130234768, -87.77461739544202 41.87895539354796, -87.7746139412434 41.87885880679116, -87.77460377864178 41.87859212609483, -87.77459392489723 41.87833236212927, -87.77458268833432 41.87805409488813, -87.77457199479905 41.87782322351495, -87.77456651621353 41.87760909000319, -87.77456131025545 41.87740568920985, -87.77455692244446 41.87728557936288, -87.77454493020247 41.87696768110548, -87.774541754697 41.87684329611299, -87.77453332348674 41.87659715075538, -87.7745263569844 41.87641487046127, -87.77451951248896 41.87623528012285, -87.77451250635056 41.87606164594257, -87.77450309223825 41.87579351347382, -87.77449951652108 41.8756897081343, -87.77449600695509 41.87558678237355, -87.77448768415051 41.87529033476868, -87.77448123731804 41.87504826021899, -87.77447641230661 41.87490262608323, -87.77446948876 41.87469424836111, -87.77446267872465 41.87448540520913, -87.77445334150939 41.87422100577238, -87.77444720331341 41.87404910325989, -87.77444074965335 41.87385845594965, -87.77443803280515 41.87377839310273, -87.77443122958354 41.87355618542984, -87.7744238115128 41.87326547733755, -87.77441899092092 41.87299692937939, -87.77440928133861 41.87277122122959, -87.77440079249401 41.87257397775855, -87.77438750241926 41.87234633173512, -87.77438536250995 41.87230954857677, -87.77437497917089 41.87213178035386, -87.77436229327508 41.87186835243651, -87.77435442390467 41.87163379007337, -87.77434506654546 41.87141401199491, -87.77434296610647 41.87136427613598, -87.77433133646241 41.87124108423467, -87.77431684863345 41.87108758265825, -87.77430141836743 41.87092430611487, -87.77428347052526 41.87073415296633, -87.7742777906285 41.87061052321074, -87.77427581922701 41.87056718126485, -87.77427578308209 41.87050812470168, -87.77427570833106 41.87041136214746, -87.77427552914422 41.87021273175013, -87.77428584602325 41.87002312725452, -87.7742842733408 41.86984144928974, -87.77427829545678 41.86961309888977, -87.77427368810054 41.86943393085549, -87.77426963504466 41.86925435442418, -87.77426403228772 41.86911371144439, -87.77425553882367 41.86890019409986, -87.77425075624934 41.86877898416692, -87.77424381529745 41.86859393313368, -87.77423751796771 41.86841486688426, -87.77422994634945 41.86820533344702, -87.77422330218938 41.86804075544828, -87.77421662745171 41.86786300486123, -87.77420315653856 41.86749191542228, -87.77419537757784 41.86728905030901, -87.7741874517134 41.86708206671448, -87.77417439762661 41.86673479940876, -87.77416285293026 41.86641651934175, -87.77414943810044 41.86601798859465, -87.77414596209277 41.86591544618849, -87.77413887371775 41.86570520040337, -87.7741388517685 41.86545915064991, -87.77385992374728 41.86546179855419, -87.77313700886249 41.86547512494718, -87.77289647138433 41.86547883537698, -87.77260497189918 41.86548331269847, -87.77194988029981 41.86549062294983, -87.77167233234682 41.86549491939008, -87.77146285794126 41.86549817581233, -87.77071024668275 41.86550686767628, -87.77070249957139 41.86550699465394, -87.77044205679923 41.86551156373993, -87.7701965221831 41.86551584858216, -87.76949137498742 41.86552490294132, -87.76921364843136 41.86552872631265, -87.76899481458618 41.86553173906408, -87.76830860956444 41.86554109932609, -87.76798637419859 41.86554610004474, -87.76778435441292 41.86554922152246, -87.76675792640994 41.86556350975591, -87.76657925984701 41.86556600309486, -87.76585550952824 41.86557395566426, -87.76553236231811 41.86557836732473, -87.76530038368996 41.86558152923512, -87.76456856519059 41.86559261712718, -87.76430871096102 41.86560107151411, -87.76399439722914 41.86561128692587, -87.76258269527828 41.8656307348877, -87.76177931371835 41.86564271087479, -87.76165737607062 41.86564440914862, -87.76153084843762 41.86564616621934, -87.7603665322506 41.86566371706548, -87.75974684303792 41.86567360469622, -87.75940908260051 41.86567809244296, -87.7589837150029 41.86568375967205, -87.75743566421561 41.86570679695506, -87.75733531410123 41.86570829784185, -87.75573365098066 41.86573238879339, -87.75501925459832 41.86574481959218, -87.75473895462316 41.8657488414313, -87.75453329892548 41.86575181315534, -87.75432309178797 41.86575484114828, -87.75407025488202 41.86575847812505, -87.75366294736402 41.86576432705273, -87.75330642450797 41.86576892141431, -87.75305392028672 41.8657721736375, -87.75246375387243 41.86578131985924, -87.75235143413082 41.86578305717006, -87.75231977984932 41.86578396674686, -87.75208225439465 41.86578729370772, -87.75172334078809 41.86579230986109, -87.75109083049774 41.86580296318882, -87.75086317947303 41.86580650124569, -87.75061603134114 41.86581032467264, -87.7499710091688 41.86581994785829, -87.74963981464983 41.86582490647022, -87.74930223258347 41.86582996830867, -87.74864226329805 41.86584013919838, -87.74841652077897 41.86584371066125, -87.74818049530818 41.86584744718267, -87.74751113342279 41.86585709719164, -87.74719601569264 41.8658623505746, -87.74693957491223 41.86586663979656, -87.74633117200595 41.86587445391272, -87.74597780191695 41.86587942528185, -87.74573407062213 41.86588287093097, -87.74507615285215 41.86589352633619, -87.74475411267252 41.86589692552791, -87.74317204055113 41.8659136156012, -87.74264157693536 41.86592156311391, -87.74230586474513 41.86592650505442, -87.74212686694064 41.86592912357271, -87.74039140737194 41.8659534914547, -87.74016676920417 41.86595661359448, -87.74006910076027 41.86595797493211, -87.73998567803375 41.8659591348122, -87.73998200652061 41.86595919899458, -87.73997712590109 41.86581995738345, -87.73996768874849 41.86549070790378, -87.73996644150287 41.86544684796683, -87.73995743826877 41.86513428578746, -87.73995224668515 41.864953248068, -87.73994235815687 41.86460923127983, -87.73994107459554 41.86456537114832, -87.7399312993639 41.86422503277498, -87.73992868421357 41.86413456882357, -87.73992607008918 41.86404399455689, -87.73991618213944 41.86369997908786, -87.73991490044247 41.86365611895884, -87.7399050215658 41.86331089439298, -87.73989986969688 41.86312963673021, -87.73988992036604 41.86278430364406, -87.73988863773974 41.86274038834345, -87.73987875025118 41.86239631791143, -87.73987617404751 41.86230574408694, -87.739873596975 41.86221522514167, -87.73982533863293 41.86053433702688, -87.73982375973685 41.86047883995413, -87.73982128298188 41.86039320572966, -87.73981122804639 41.8600436454953, -87.73980994462994 41.8599997853287, -87.73980005376319 41.85965615296871, -87.73979489886392 41.859475114987, -87.73978508089257 41.8591315927419, -87.73978447656846 41.85910969042808, -87.73978379750501 41.85908773256828, -87.73977390685984 41.8587441004278, -87.73977122657715 41.85865297742876, -87.73976794760597 41.85856300309994, -87.73975547955482 41.85822007166291, -87.7397538304597 41.8581762644772, -87.73970790816669 41.85691789248865, -87.73970121384554 41.85673684673535, -87.73964118807058 41.85509052954207, -87.73963791396646 41.85500006203597, -87.73963464125735 41.85490948421626, -87.73962204124933 41.85456523297577, -87.73962046401212 41.85452137261908, -87.7396078653739 41.85417706647596, -87.73960124457543 41.85399602073745, -87.73958864485139 41.85365177136183, -87.73958706766999 41.85360790907713, -87.73957446932086 41.85326360425008, -87.73956784831395 41.85308255848022, -87.73955620211146 41.85276224175018, -87.73955455241683 41.85271832611919, -87.73954319910062 41.85240607983751, -87.73953657714129 41.85222503403431, -87.7395252755508 41.851915148058, -87.73952369880107 41.85187128658485, -87.73951203600677 41.85155300209929, -87.73950453480467 41.85134802147279, -87.73949770887337 41.85114150726818, -87.73946946781435 41.84959952472694, -87.73944942560986 41.84959975040695, -87.73933750335944 41.84605017887307, -87.739329964064 41.84580990603821, -87.73932308974719 41.84559285394466, -87.73931549236103 41.84535093532767, -87.73930674824318 41.84507404881909, -87.73927711605043 41.84413546541917, -87.73927428313674 41.84404489024402, -87.73927159835345 41.84395437044447, -87.73916436053716 41.84038412829788, -87.73910919634883 41.83854820126336, -87.73898808414027 41.83854944045325, -87.73893344245063 41.83672915835264, -87.73893326621314 41.83672440985342, -87.73881215631869 41.83672573065389, -87.7388024339606 41.83631727859544, -87.73876292266165 41.83465481446278, -87.73872494917944 41.83305662809916, -87.73871838843569 41.8327822223848, -87.73868035867706 41.83118216964421, -87.7386724410719 41.8308482052708, -87.73863864267405 41.82942561834383, -87.7385887552733 41.82703767777255, -87.73858536847941 41.82688024809231, -87.73858283957317 41.82675718199319, -87.73848858204491 41.82225037344682, -87.73848582775723 41.82211989653266, -87.73873988755938 41.82211632656767, -87.73886097285414 41.82211464837185, -87.73899967086135 41.82211273186716, -87.7410973096562 41.82208333508917, -87.7412079002665 41.82208181875367, -87.74141095762791 41.82207891383288, -87.74330216298702 41.82205241918754, -87.74370555966532 41.82204675173708, -87.74476347919499 41.8220318433466, -87.74476144844455 41.82189299885049, -87.74476082393949 41.82184906004334, -87.74474283925635 41.82058393872875, -87.7447425139049 41.82056330036515, -87.74474117800497 41.82046888980538, -87.74474086623195 41.82044676877748, -87.74474009274579 41.82039506343765, -87.74473839971523 41.82027551318256, -87.74473669260058 41.82015750129521, -87.74473558739146 41.8200778017913, -87.74473484055578 41.82002318682641, -87.74473451607963 41.82000249330528, -87.74471984105898 41.81947381574315, -87.74470700079307 41.81873663530414, -87.74468968917282 41.81781485437489, -87.74468875544071 41.81776462900324, -87.74468511036255 41.81756982082189, -87.74468190461441 41.81739902733137, -87.74467298302824 41.81692309635058, -87.74466050007135 41.8162593013847, -87.74465491932028 41.81596321929267, -87.74473026695648 41.81593951062794, -87.74582396727925 41.81559482945971, -87.74814156796302 41.81486444205233, -87.74824663801742 41.81483127777836, -87.74836465696073 41.81479411808905, -87.74934155235358 41.81448618443591, -87.75055402844329 41.81410402749711, -87.75176767009818 41.81372147777051, -87.75200767487503 41.81364579729766, -87.75307704035835 41.81331033656062, -87.75306868468789 41.81297220017861, -87.75302367318581 41.81114768243673, -87.75297860415968 41.80932154633338, -87.75293351787647 41.80749359832394, -87.75289164960168 41.80585378565738, -87.75288699381436 41.80567058052485, -87.75288430732444 41.80555385303288, -87.75288066290297 41.80542927204107, -87.75287523680366 41.80524213839923, -87.75287115010593 41.80510166547624, -87.75286672474009 41.80494991209778, -87.75286298882055 41.8048273877469, -87.75285915091941 41.80470401324705, -87.75285490398616 41.80456902697525, -87.75285132247852 41.80444565349084, -87.75284672309705 41.80426458886619, -87.75284331181503 41.8041304863503, -87.75283717002681 41.8039575655495, -87.75283555942751 41.80384347742494, -87.75283399358095 41.80373251773643, -87.7528290280293 41.80358369825214, -87.7528258569529 41.80345557838097, -87.75282184184105 41.80329090061748, -87.75281707948871 41.80312770181114, -87.75281152696242 41.80294644077616, -87.75280761036544 41.80283582594787, -87.7528015193117 41.80263304724622, -87.75279768118342 41.80248928236936, -87.75279295601399 41.8023382683624, -87.75278879822176 41.80212452226385, -87.75278600882869 41.80201503772469, -87.75278376641565 41.80192613892604, -87.75277843418866 41.80174092746592, -87.75277488540178 41.80161793773595, -87.75276992785066 41.80146025473556, -87.75276449590565 41.80128991652056, -87.75275856061756 41.80110245167556, -87.75275382354262 41.80093233705333, -87.75274865635693 41.80074514968376, -87.7527465180702 41.80064464277884, -87.75274499340307 41.80057383244381, -87.75274290899769 41.80043924175085, -87.75274008421417 41.80030508777102, -87.75273612470563 41.80018714487505, -87.75284826670419 41.80018439156035, -87.75294436636426 41.80018320311036, -87.75326403191836 41.80017869795355, -87.75334718700843 41.80017730686182, -87.75348884854179 41.80017492040164, -87.75395754358458 41.80016855865591, -87.75430398484755 41.80016385578131, -87.75456639851996 41.80015952466334, -87.75460993741227 41.8001588112288, -87.75482848185138 41.80015513749423, -87.75518028365282 41.80014963623392, -87.75546664952702 41.80014517689295, -87.75579060820498 41.80014027465693, -87.75583374537365 41.80013961362459, -87.75600635992438 41.80013718838405, -87.75639097893237 41.80013250587029, -87.7567024604534 41.8001287192649, -87.75700735103325 41.80012366299557, -87.75701464971323 41.80012350668893, -87.75728224341486 41.80011823595036, -87.75763077135879 41.80011386281974, -87.75790051106858 41.80011049358234, -87.75823476774994 41.8001065422006, -87.75823641794902 41.80010652219571, -87.75845576889814 41.80010292792561, -87.75884881861873 41.8000986372265, -87.75911400677982 41.80009576406663, -87.75941228334199 41.80009215054263, -87.75946011321082 41.80009151186744, -87.75956098208098 41.80009017677471, -87.75977167342509 41.80008692121386, -87.76006969077757 41.80008341336232, -87.76024593467726 41.80008132917645, -87.7606130908048 41.80007715024654, -87.76068428700998 41.80007616074975, -87.76091071078207 41.8000730350371, -87.76129589250654 41.8000668580846, -87.76148699757604 41.80006377544921, -87.76187301770535 41.80005812214968, -87.76190430786356 41.80005742679693, -87.76211433732163 41.80005438318947, -87.76222547861894 41.80005274007825, -87.76253174662658 41.80005769237189, -87.76287274010461 41.80006320104603, -87.76308155616898 41.8000604229756, -87.7631327973843 41.80005988148492, -87.76336596350811 41.8000573606992, -87.76360563738626 41.80005317053087, -87.76374200779105 41.80005170650784, -87.76390067976581 41.80005002438099, -87.76428075153967 41.80004529318177, -87.76444779527833 41.80004241574164, -87.76464660589842 41.80003900925067, -87.76479318160325 41.80003649650985, -87.76495589890351 41.80003351541306, -87.76511773738517 41.80003058325249, -87.76527693253688 41.80002766605894, -87.76550102029897 41.80002331198443, -87.765559528762 41.80002192825815, -87.76578512955531 41.80001659371229, -87.76594917030158 41.80001320473138, -87.76617516621585 41.80000880525247, -87.76658849283103 41.80000074589726, -87.76678261180257 41.79999695580494, -87.76682182289369 41.79999618840547, -87.76708846805882 41.79999028530155, -87.76739754658847 41.79998423415064, -87.76774466367388 41.79997741005231, -87.76798587688857 41.79997327211063, -87.76800685985964 41.79997277245324, -87.76817559969761 41.79996882711674, -87.76836613074229 41.79996422195036, -87.76862000143313 41.79995943223448, -87.76900291729275 41.79995223035353, -87.76913783171571 41.79994968167551, -87.76922766523029 41.7999479005175, -87.76940557047553 41.79994440974979, -87.76960412712872 41.79994050055658, -87.76984127655142 41.79993543398399, -87.77014804828551 41.79992889786139, -87.77033795229987 41.79992466972713, -87.77045052722596 41.79992247655086, -87.77062494752037 41.79991910513059, -87.7710618227847 41.79991059671651, -87.77127857310003 41.79990638837936, -87.77128444231373 41.79990625243816, -87.77145120136848 41.79990251241558, -87.77163640790052 41.79989853276368, -87.77166788128247 41.79989791852449, -87.77181559698171 41.79989507280657, -87.77207728120267 41.79988998487558, -87.77227959493307 41.79988422247318, -87.77246320094106 41.79987897184689, -87.77269249275402 41.79987056842464, -87.77288576021431 41.79986344382031, -87.77289988309646 41.79986293642006, -87.77311989487768 41.79985487107125, -87.77329327703929 41.79984849898628, -87.77349663437914 41.79984079025897, -87.77365090465427 41.79983492851098, -87.77384715238196 41.79983072575556, -87.77396622369606 41.79982814912934, -87.77410572377691 41.79982515041544, -87.77430751155745 41.79982078256576, -87.77447962327366 41.79981709053899, -87.77470463933155 41.79981127019698, -87.77484528281202 41.79980764495845, -87.77500018732557 41.79980483009404, -87.7751911099601 41.79980139500458, -87.7753255092757 41.79979897431888, -87.77533453287536 41.79979879860717, -87.77555340504294 41.79979508593262, -87.77566278665157 41.79979322999779, -87.77593291445909 41.79978718660163, -87.77613536432938 41.79978265244927, -87.77632031123539 41.79977902101722, -87.77652352484786 41.79977501239665, -87.7765454248139 41.7997745691377, -87.77673814934637 41.79977072746898, -87.7768928342078 41.79976763538212, -87.77715239438193 41.79976216939075, -87.77735271383193 41.79975795209283, -87.7774487461608 41.79975594642191, -87.77756807189647 41.7997534773388, -87.77776365727954 41.79974942934449, -87.77778449235612 41.79974900869096, -87.77794574626685 41.79974567225074, -87.77816069950792 41.79974122269454, -87.77838130049525 41.79973696467707, -87.77854937443497 41.79973371412389, -87.77868634418958 41.79973080904473, -87.77883593175282 41.79972763347209, -87.77898133817024 41.79972452067999, -87.77903892936492 41.7997232892204, -87.77924174224209 41.79971897198507, -87.7794034000233 41.79971552430265, -87.77959336719918 41.79971229710593, -87.77991611809088 41.79970682648074, -87.7801498512015 41.79970229928941, -87.78020091184781 41.79970125561697, -87.78033501906887 41.79969855295958, -87.78045503970917 41.79969611221409, -87.78055594979013 41.79969404591542, -87.78081301473341 41.79968875119609, -87.78110591769905 41.79968269375473, -87.78126276523597 41.79967968823313, -87.78142152085462 41.79967660961024, -87.78145585580279 41.79967595155116, -87.78164296604052 41.79967226650654, -87.78179361714638 41.79966914844743, -87.78203113206395 41.79966828494148, -87.78230257196874 41.79966731007249, -87.78251497908933 41.79966915147244, -87.78263984699285 41.79967021665976, -87.78264098423898 41.79967022211147, -87.7827836039925 41.79967145539156, -87.78292112300437 41.79967277245477, -87.78310866504692 41.79967454850307, -87.78325094093246 41.79967297983122, -87.78345212161854 41.79967073273168, -87.7835394182413 41.79966955832558, -87.78373954119699 41.79966683940549, -87.7838599600036 41.7996651648594, -87.78390181107657 41.79966459580788, -87.78407053684838 41.79966213778383, -87.7842641318535 41.79965930385972, -87.78447500195935 41.79965633178126, -87.7847699775328 41.79965214211376, -87.78491335957234 41.79965000008952, -87.78508043387396 41.79964753269785, -87.78508806388086 41.7996474307999, -87.78528169502113 41.79964465021897, -87.78544997932543 41.79964229783404, -87.78568883217997 41.79963934748532, -87.78587530884539 41.79963702494527, -87.78603119984153 41.7996344194483, -87.78619435310816 41.79963173992956, -87.78629801141443 41.79963000989263, -87.78641304036736 41.79962811545217, -87.78655660454731 41.79962589073835, -87.78675074689407 41.79962327339936, -87.78690656025007 41.79962138095809, -87.78711405163993 41.79961888138666, -87.78727690829483 41.79961652741141, -87.78745795864148 41.79961387491075, -87.78751730582192 41.79961300430191, -87.78768914959977 41.79961052751071, -87.78793890036798 41.79960691100467, -87.78812755999483 41.79960218129426, -87.78837740624425 41.7995959028751, -87.78856501888899 41.79958916332418, -87.78873406949329 41.79958299355141, -87.78875901631137 41.79958206925898, -87.78891442171114 41.7995762744444, -87.7891335128505 41.79956809227701, -87.78934176588527 41.79956674516573, -87.78960169624769 41.79956504046368, -87.78962491388803 41.79956462893939, -87.79052431726058 41.79954014845454, -87.79178161437092 41.79952215254756, -87.79417970613942 41.79948842678331, -87.79665533458554 41.79945353019639, -87.79677630253197 41.79945184494878, -87.79704149278317 41.79944808921108, -87.79856399283999 41.79896560525254, -87.79874159964702 41.79890935009551, -87.79904180077811 41.79881420310231, -87.79924117520387 41.79875102274901, -87.79944062193559 41.798687788322, -87.7995924888712 41.79863964618747, -87.79974457240957 41.79859144969733, -87.79989621721423 41.79854341562535, -87.80004800784988 41.79849532604595, -87.8001997245386 41.79844723701985, -87.80035144100216 41.79839914779164, -87.80050323058634 41.79835105870154, -87.80065560963151 41.79830275232571, -87.800901974898 41.7982246384741, -87.80102088980179 41.79818698901728, -87.80119959569237 41.79813035231127, -87.80136734266291 41.79807717535147, -87.8016260597866 41.79799516728004, -87.80161971391945 41.79777800926415, -87.80160562320783 41.79576014737697, -87.80161538420926 41.79481431734014, -87.80161015210263 41.79465770402037, -87.80160484901448 41.79450054123445, -87.80160027380796 41.79433531304138, -87.80159394006095 41.79410780923055, -87.80158941917662 41.79394488785615, -87.8015866277328 41.79384341757299, -87.80157939576669 41.79361363209501, -87.80157479283869 41.79346064384716, -87.80156771027846 41.79322605680473, -87.80156281177773 41.793064422926, -87.80155632378425 41.7928289862413, -87.80155256198908 41.79270295138436, -87.80155087027813 41.79264586168301, -87.80154437711613 41.79241997619908, -87.80153801976476 41.79219518877805, -87.80152796189643 41.79193918295955, -87.80151529803062 41.79161675203336, -87.80150776986031 41.79144503354841, -87.80149943962519 41.79125500799504, -87.80148535448299 41.79093619291749, -87.80147066482499 41.79063263220603, -87.80145728889111 41.79032990331773, -87.80145026771267 41.79013241820363, -87.8014372355985 41.78976593951836, -87.80142944548722 41.78956828698927, -87.80141593850034 41.78929028216804, -87.80140117033061 41.78901430254587, -87.80139045969393 41.78871742961099, -87.80137947908453 41.78844887654782, -87.80137292849308 41.78830540184588, -87.80136035448186 41.78803000835977, -87.80135248068282 41.78784687243356, -87.80134358529268 41.78764564712964, -87.80133503037646 41.78745611327574, -87.80132975700388 41.78733127906032, -87.80132546688054 41.78722908874688, -87.80131654641033 41.78702204484495, -87.80130794203868 41.78682090291198, -87.80130052601527 41.78664891063622, -87.80129544225738 41.78652319828164, -87.80129124549049 41.78641873089281, -87.80128144560777 41.78617598095003, -87.8012729363858 41.78598570602058, -87.80126625031365 41.78584088591872, -87.8012581873994 41.78566746412941, -87.80125170182482 41.78551614068743, -87.80124192352517 41.78528859366458, -87.801233681263 41.78510131177327, -87.80122571344391 41.78492075412768, -87.80121615884937 41.78468843248801, -87.8012124914775 41.78459983065637, -87.80120394568634 41.78439171728816, -87.80119536416056 41.78419227710116, -87.80118309924806 41.78390631900961, -87.80117574660818 41.78373564529439, -87.80117233463061 41.78365620976393, -87.80115944607448 41.78335696607456, -87.80114622466094 41.78304921377442, -87.80113432643029 41.78277212222918, -87.80112595887262 41.78257797892118, -87.80111740058564 41.78236256695276, -87.801106640233 41.78209426130345, -87.80109533333699 41.78185649771029, -87.80108096221699 41.7815546954015, -87.80106796423483 41.78119340329359, -87.80105671953784 41.78094389461912, -87.8010381545197 41.78053301406955, -87.80102359214769 41.78021910870955, -87.80101769306896 41.78003249585439, -87.80100660069952 41.77968015916854, -87.80100388367215 41.77934827859037, -87.8010105967534 41.77913499520535, -87.80098959676752 41.77897885668514, -87.80096778881713 41.77863236887914, -87.80095591276942 41.77846353988472, -87.80094928053913 41.77827234086391, -87.80094738546266 41.77821785852793, -87.80094380014282 41.77811484829056, -87.80093689021766 41.77792194705278, -87.80093383172846 41.77783082125274, -87.800923819844 41.77760963932513, -87.80089887892656 41.77729154152514, -87.8008718818096 41.77694760974449, -87.80086722663489 41.77680123510428, -87.80086716511941 41.77679980750779, -87.80088639109108 41.77656745397013, -87.80085030479363 41.77633835705036, -87.80084684159986 41.77625606737231, -87.80084332182294 41.77617202080297, -87.80083722381481 41.77602736699286, -87.8008193342185 41.77560161562197, -87.80083101346592 41.77547046493197, -87.80083133829025 41.77543119601827, -87.80082779865607 41.77538049234536, -87.80081964867512 41.77520465427544, -87.80081370937603 41.7750717196223, -87.80080900292391 41.77496689343756, -87.80080448014601 41.7748666777074, -87.80079995565194 41.77476662662556, -87.80079443691746 41.77464047343661, -87.80078694229242 41.77446941374131, -87.80078173494947 41.77434998498214, -87.80077529439762 41.77420236623722, -87.80077689897706 41.77413700437631, -87.80079318485758 41.77408707866519, -87.80082111701151 41.77405199927571, -87.80081842381045 41.77396993185435, -87.80081174392795 41.77385562810785, -87.80080874333069 41.77377735216017, -87.80074265475419 41.77377841192422, -87.79935464897625 41.77380066848601, -87.7983796017888 41.77381629298079, -87.79730912493413 41.77383343825159, -87.79594215536653 41.77385531592966, -87.79544110371849 41.77386332760683, -87.79428924788523 41.77388173498034, -87.79364372416198 41.77389204475711, -87.79321279253949 41.77389892681797, -87.79240276792675 41.77391185660387, -87.7912881736322 41.77392963762033, -87.79077551465139 41.77393781266242, -87.78978763537349 41.7739535539074, -87.78850871361537 41.77397392124824, -87.7881477272299 41.77397966781458, -87.78692574703756 41.77399911042049, -87.78607125594505 41.77401269901796, -87.78572890181292 41.77401814225855, -87.78510218117488 41.77402810506455, -87.78387352339934 41.77404762768681, -87.78336068837456 41.77405577126513, -87.7824678857867 41.77406994604944, -87.78170812270575 41.77408200103987, -87.78112031409624 41.77409132486755, -87.75661249170604 41.77457921551834, -87.75540160968923 41.77460128689056, -87.7543016784748 41.77462143459974, -87.7521314566182 41.77466065871864, -87.74875044875677 41.7747219212447, -87.7471556181744 41.77475078694079, -87.74704005701447 41.77474801609539, -87.74703799821101 41.77466741894704, -87.74690357782643 41.77466999915529, -87.74670710286925 41.77467495177905, -87.74650322760526 41.77467953530445, -87.74643330449376 41.77468082552677, -87.74628531504274 41.77468358103355, -87.74600826681895 41.77468806638425, -87.74583204209324 41.77469152824104, -87.745661100437 41.7746948801855, -87.74540505044058 41.77470007468711, -87.74523139189176 41.77470365852663, -87.74521166553031 41.77470405206926, -87.7450130729196 41.77470811113231, -87.74477382206929 41.77471248435836, -87.74461193893106 41.77471522278992, -87.74446391629077 41.77471770244772, -87.74417547545791 41.77472503256851, -87.74400870287137 41.77472944681623, -87.74399865501991 41.77472975203323, -87.7439161109707 41.77473196228234, -87.74355275100541 41.77474162440332, -87.74339200469858 41.77474431179117, -87.74307630739762 41.77474960534602, -87.74287055482695 41.77475480588856, -87.74278584883744 41.77475689444211, -87.74268185645595 41.77475948807521, -87.74216537195549 41.77478021384243, -87.74214527342052 41.77445381263788, -87.74213654055464 41.77418713235102, -87.74212898679872 41.77390379912007, -87.74212168130008 41.77358956550909, -87.74211449513182 41.77333420020644, -87.74210385097003 41.77296429524359, -87.7420994348652 41.77281037187034, -87.74209299339554 41.77257380827523, -87.74208721024046 41.77232133056467, -87.7420807336274 41.77209634607132, -87.74207332550422 41.77185316260825, -87.74207273282627 41.77182999875531, -87.74207122798009 41.771786246578, -87.74206572360087 41.77154721624483, -87.74205453829012 41.77112870790005, -87.74204247687791 41.77067767504175, -87.7420339484904 41.77039683475556, -87.74201210484888 41.76940635837766, -87.74197348331651 41.76765641971377, -87.74193593080743 41.76595385242251, -87.74189941065107 41.76429868462408, -87.7418951568777 41.76413128786257, -87.74189743371933 41.76411881162493, -87.7418783453389 41.76360706627507, -87.7418683583042 41.76327753360622, -87.74186683980567 41.76297191969527, -87.74185496225955 41.76258902796049, -87.74184169558092 41.762269522506, -87.74182152501537 41.76191951997004, -87.74178484741404 41.76128358551524, -87.74179224272551 41.76083759069676, -87.74177885736552 41.76046322611818, -87.74177438818273 41.76039511714001, -87.74175838289176 41.76015027003668, -87.74173376673956 41.75977376081121, -87.74172445662859 41.75959423603361, -87.74169036066884 41.75893682584159, -87.74168509601374 41.75879963880936, -87.74168240146591 41.75872991925672, -87.74167485103526 41.75853445770714, -87.74165825824282 41.75810634318387, -87.74164791558437 41.75784766692596, -87.74163562324857 41.75749808776968, -87.74163105463725 41.75735313707124, -87.74162949881513 41.75730310114971, -87.74162603943425 41.75719292654027, -87.74162172988805 41.75705560661871, -87.74160032233763 41.75650929727033, -87.74159473966021 41.75636694814165, -87.74157308710735 41.7557915755085, -87.74155383274295 41.7552702769533, -87.74153950445402 41.75488325477114, -87.741532565244 41.75469710053725, -87.74153182558784 41.75467376927671, -87.74152154359433 41.75434898330674, -87.74150929805091 41.75396243804577, -87.74150194788156 41.75372520951419, -87.74149656958423 41.75344496090265, -87.7414956341017 41.75329154971858, -87.7414943320813 41.75307419339934, -87.74150162461672 41.75284285709548, -87.74150149065207 41.75283753271848, -87.74149737331541 41.75267139869852, -87.74148327461955 41.75248666172204, -87.74146741312508 41.75227847869901, -87.74146648422111 41.75178123286714, -87.74145644779216 41.75167684165731, -87.74144411279069 41.75149167546375, -87.7414346566889 41.75124441814667, -87.74142469971767 41.75098802160394, -87.74141308873992 41.75066813924263, -87.74140963615416 41.75027011483948, -87.74140071454114 41.74982109872432, -87.74139193592256 41.74952414671334, -87.74135816829417 41.7491470964239, -87.74131987859558 41.74871966495137, -87.7413068631533 41.7483692587427, -87.74130354137351 41.74804876112592, -87.74130387362629 41.74764556874292, -87.74129443670614 41.74735659889706, -87.74129131404972 41.74726168447732, -87.74127417255184 41.74673819972671, -87.74125824628435 41.74629397887189, -87.74124641112117 41.74597491837525, -87.74123416535507 41.74562059268921, -87.74122779702211 41.74543616862866, -87.74121678491693 41.74507932422239, -87.74120606028171 41.74469531119562, -87.74119713713743 41.74439413219503, -87.74119666000306 41.74438241096656, -87.74119528960557 41.74433207318564, -87.74117793259428 41.74390400795058, -87.74116359788192 41.74354986319317, -87.74115907697836 41.74342371175238, -87.74114368741886 41.7429930483888, -87.74113147926117 41.74266671444108, -87.74112119637968 41.74238608241724, -87.74111426838373 41.74219893986393, -87.74110352429696 41.74189673512187, -87.74109437804029 41.741640012452, -87.74108250907098 41.74129260300592, -87.74107107325636 41.74095803944207, -87.74105756167896 41.74056232152842, -87.7410454102078 41.74021395087087, -87.74103708515817 41.73997556411237, -87.74102980682352 41.73976682162824, -87.74101626207536 41.73937069181051, -87.74100713062056 41.73910831553256, -87.74099898717385 41.73887415563542, -87.74099062675093 41.73864762381019, -87.74098202677213 41.73841510895324, -87.74097489983326 41.7381859766413, -87.74096578452077 41.73789404436997, -87.74095322993949 41.73752184996854, -87.74095309700833 41.73752036707748, -87.74095187812313 41.73748550796476, -87.74094401794655 41.73723244138692, -87.74093262845204 41.73686513838173, -87.74092919588719 41.73662060171323, -87.74092863382228 41.7363589006789, -87.74092795511261 41.73603808642367, -87.74093557186396 41.73580724594698, -87.74095302478915 41.73555867381236, -87.74099276443091 41.73522898418589, -87.74102366046189 41.73496475527222, -87.74103006467054 41.73490981958592, -87.74106728299365 41.73452355731512, -87.74016474940535 41.73454700301395, -87.73963783298294 41.73455778322054, -87.73952657837063 41.73456003525287, -87.7390353212095 41.73456185948827, -87.7387083033578 41.73456453029768, -87.73860654355568 41.73456617248621, -87.7385107106473 41.73455697639298, -87.73841906009095 41.7345591636268, -87.7383987576996 41.73455966308352, -87.73825787099987 41.73454913577097, -87.7375674329197 41.73456078952758, -87.73720975552318 41.7345666454446, -87.73630568599206 41.73458146423929, -87.7359935566711 41.73458664822029, -87.73495965654109 41.73460385340539, -87.73478402311609 41.73460683640278, -87.73358452734686 41.73462717401681, -87.73356360399215 41.73462750444438, -87.73235162730647 41.73464644676556, -87.73233301228548 41.73464676077575, -87.73113825136006 41.73466602815297, -87.73111051102599 41.73466648596828, -87.73001594424738 41.73468686995618, -87.72992363787661 41.73468849892887, -87.72940105608762 41.73469777356462, -87.72871152911534 41.73470946066811, -87.72866550525721 41.73471023333219, -87.72790960968571 41.73472368002189, -87.72750001709407 41.73472926015157, -87.72715956899562 41.73473388606399, -87.72627968505347 41.73476744207694, -87.7259557610424 41.73477978092253, -87.72507286259672 41.7347948699427, -87.7247405370953 41.73480057452477, -87.7238578996279 41.7348150510169, -87.72358779879185 41.73481949141883, -87.72264715698485 41.73483828951115, -87.72233912029417 41.7348444178332, -87.72143877752843 41.73485966059647, -87.72107226482881 41.73486585913479, -87.7209189102131 41.73486844374639, -87.71969644868362 41.73488695133629, -87.71901032361001 41.73489898176182, -87.71898272940291 41.73489946542199, -87.71817157315351 41.73491365029316, -87.71695490685434 41.73493542355282, -87.71657303615608 41.73494207448942, -87.71561610836943 41.73495872396875, -87.71470176232329 41.73497446874752, -87.71414319251582 41.73498430323725, -87.71399093510618 41.7349869956583, -87.71277852638134 41.73501528020817, -87.71197804927078 41.73502733735729, -87.71192817762179 41.7350281105008, -87.71167546268448 41.73503401597062, -87.71121005355226 41.7350448898691, -87.71002531991347 41.7350678600809, -87.70934160175783 41.73508045039029, -87.70915746193268 41.73508384116872, -87.70845683865383 41.73509419353874, -87.70769086286744 41.73509982345023, -87.70706723872829 41.73511165658293, -87.70691180237679 41.73511344438853, -87.70610061481315 41.73512271331353, -87.70464696193469 41.7351485240669, -87.70450863261318 41.73515045710573, -87.70346664730357 41.73516492071965, -87.70279800503668 41.73517541415895, -87.70215111591617 41.73518709293605, -87.70209495506477 41.73519018737505, -87.70170322627484 41.73521185508504, -87.70147060146977 41.73522037386, -87.7012439801064 41.73521040078201, -87.70092706453413 41.73521738363282, -87.70063346508917 41.73521903253528, -87.70048521370231 41.73522038467483, -87.70020755632865 41.73522538657564, -87.69952871928443 41.73523690281345, -87.69906530476628 41.73524567945262, -87.69899520140713 41.73524702138003, -87.69855476305774 41.73525545654049, -87.69805468007728 41.73526358688505, -87.69763023597289 41.73526993946817, -87.69722612578916 41.73527642994787, -87.69710860488624 41.73527860621135, -87.6965967098203 41.73528817445298, -87.69603561603519 41.73529702873407, -87.69547013745023 41.73530478674822, -87.69495262977738 41.73531497541689, -87.69465491997305 41.73532092222286, -87.69452669797633 41.73532350262305, -87.69401065514084 41.73533369546652, -87.69360071834525 41.73534011387978, -87.69322569662884 41.73534672561569, -87.69278817157182 41.73535301295907, -87.69233317227831 41.73535917563373, -87.69224771681974 41.73536070277998, -87.69168195679109 41.73537079931659, -87.69100453474184 41.735383343644, -87.69043035031824 41.73539319536956, -87.68998845549463 41.73540051870694, -87.68966088528464 41.73540639624905, -87.68952617973369 41.73540879698328, -87.6889490636805 41.73541862460482, -87.68861328870815 41.73542437054294, -87.68826437618618 41.73543221002949, -87.68823964016238 41.7354326474648, -87.68766548719837 41.73544300696781, -87.68744170952488 41.73544570055895, -87.68717417934324 41.7354489438063, -87.68695322987253 41.73545096736929, -87.6867380834654 41.73545519062296, -87.68649533922722 41.73546035584367, -87.68629765306396 41.73546250800427, -87.68602724760778 41.73546751646879, -87.68571327158743 41.73547337618373, -87.68533822164031 41.73547889118075, -87.68518984892822 41.7354811825852, -87.68482362863415 41.73548685520608, -87.68441082140789 41.73549434952075, -87.68414187076483 41.7355004325952, -87.68388164994968 41.73550549355106, -87.68343975356841 41.73551279141536, -87.6829934496573 41.73552116200636, -87.68252104538423 41.73552718593574, -87.6825140479143 41.73552728434091, -87.68251198282633 41.73545954295817, -87.68249978406759 41.73505649813224, -87.68248531886077 41.73465662494264, -87.68247878983311 41.73432727014367, -87.68246755939435 41.73392988401089, -87.68245518671884 41.73354407348928, -87.6824455288712 41.73325240697718, -87.6824313942174 41.73280903784821, -87.68241515928312 41.73229770669486, -87.68240167579367 41.73187703694946, -87.68238912483224 41.73147991786787, -87.68237622223084 41.73107050332457, -87.68236162730911 41.73059655999761, -87.68235455651521 41.73034253257472, -87.68234570556508 41.73005803368283, -87.68233794386242 41.72980724198788, -87.68232562955117 41.72942310417261, -87.6823150123786 41.72907767247236, -87.68230534162561 41.7287473656576, -87.68229904514835 41.72852194025216, -87.68228955043766 41.72823603829219, -87.68227914680712 41.72792384005012, -87.68227701562979 41.72785919955469, -87.68226509801521 41.72743579347893, -87.68224384198052 41.72668924581312, -87.68223501116631 41.72638828093537, -87.6822334638906 41.7263384904964, -87.6822164706945 41.72579370120143, -87.68220091161992 41.72531747417798, -87.68219940488979 41.72527067444417, -87.6821834126697 41.72477213360207, -87.68217544729501 41.72449804014219, -87.68216997433203 41.72431073728765, -87.68215550600938 41.72382079361987, -87.68214491301615 41.72348054792656, -87.68214066844692 41.72334577719409, -87.68213163526408 41.72306492768224, -87.68212201984849 41.7227765545947, -87.68211329881005 41.72251552109356, -87.6821049304011 41.72223426256136, -87.68209293746982 41.72182570268519, -87.68208720835858 41.72163137200384, -87.68207714535083 41.72130018516667, -87.68206715820436 41.72100865336219, -87.68206472686815 41.72093742471358, -87.68206279735757 41.72088186870988, -87.68205235918836 41.72057708089985, -87.68204165145779 41.72022999989894, -87.68203112736902 41.71988643402164, -87.682020759191 41.71954212803992, -87.68200767525991 41.71911114039475, -87.6819970673122 41.7187613978754, -87.68198774204888 41.71850971890108, -87.68197782842931 41.71812021504165, -87.6819690744717 41.71777569855537, -87.68196390000914 41.71756973522174, -87.68195557722338 41.71728405916225, -87.68194679147513 41.7169825723379, -87.68193492025539 41.71662384886095, -87.6819347909662 41.71661855240689, -87.68192879560847 41.71643245245743, -87.68191753310664 41.7161221710829, -87.6819072436476 41.71583889880555, -87.68189702316805 41.71549813194316, -87.6818900861654 41.71526743217321, -87.68188091960673 41.71495658603276, -87.6818704717561 41.71464542963778, -87.68186083332516 41.71438128974149, -87.68185335618546 41.71412426769123, -87.68184854415163 41.71391539784301, -87.68184169075793 41.71366175619958, -87.6821031280307 41.71366908323935, -87.68230075013119 41.71366663457803, -87.68249644237794 41.7136667287899, -87.68252947809302 41.71366675095655, -87.68272418078344 41.71366683752967, -87.68291397197468 41.71366368593643, -87.68312361820227 41.71366023472778, -87.68336963926188 41.71365619275441, -87.68363992676565 41.71365379655818, -87.68372519961912 41.71365301625065, -87.68381087606983 41.71365223788664, -87.68404515733526 41.71365016000777, -87.68433702984474 41.71364642881467, -87.68456765698232 41.71364350483756, -87.68471168250255 41.7136423427817, -87.68495428702249 41.71363934841736, -87.6851076582705 41.7136374688133, -87.68531432751878 41.7136349847711, -87.68545938070771 41.71363349715644, -87.68555857135595 41.7136327939551, -87.68588340995178 41.71362888927119, -87.68615945627056 41.71362558679816, -87.68639351824001 41.71362355668448, -87.68662211957353 41.71362160486124, -87.68677200615917 41.71362030799919, -87.68715016554204 41.7136170559452, -87.68745661284896 41.71361367486281, -87.68770855128381 41.71361119285618, -87.68801306059699 41.7136074149213, -87.68823115838512 41.71360471510432, -87.68851126148692 41.71360233565661, -87.68871824500718 41.71360121969222, -87.68897084736179 41.71359840959736, -87.68925595759549 41.71359342103523, -87.6895110279347 41.71358897836242, -87.68962852408974 41.71358897823193, -87.68978850584804 41.71358899527597, -87.6900188653596 41.71358726663556, -87.69027166414217 41.71358278129073, -87.69048602425853 41.71358035787854, -87.6906715200458 41.71357824000263, -87.69086052854165 41.7135763873582, -87.69101818283598 41.71357477134855, -87.69110173596621 41.71357383928639, -87.69124972053258 41.71357219581052, -87.69140704582547 41.71357044076156, -87.69172881574576 41.71356567828791, -87.69173048430494 41.71341642345728, -87.69172388300554 41.71322582089509, -87.69171911605281 41.71309376531321, -87.6917185161665 41.71307674632023, -87.69171110946409 41.71288600285632, -87.69170271175611 41.71265496549946, -87.69169505625958 41.71244520107928, -87.69169089139606 41.71231161146894, -87.69168560016708 41.71212183920702, -87.69168094574518 41.71195638550066, -87.69167401900131 41.71174689983026, -87.69166765855468 41.71155423976153, -87.69166393734589 41.71142024123263, -87.69165863575979 41.71123137505937, -87.6916531972945 41.71104187554807, -87.69164596221087 41.7108375200128, -87.69163816157695 41.71061664073768, -87.69163495345983 41.7104899441382, -87.69162733246729 41.71028407759643, -87.69162094894999 41.71012668167881, -87.69161507543319 41.70992927713553, -87.69161294374307 41.7098121927752, -87.69160375487702 41.70956509655582, -87.69159697317642 41.7093669187837, -87.69159107150539 41.70919437711627, -87.69158689938727 41.70901728896921, -87.69158406826391 41.70889696292608, -87.69157495849186 41.70867857305206, -87.69156504096942 41.70844220330954, -87.69156100478203 41.70832145750274, -87.69155631540293 41.70810808779294, -87.6915546595433 41.70803162128703, -87.69154459578141 41.70774000737018, -87.69153927812852 41.7075860219462, -87.69153330119994 41.70741367163154, -87.69151939194465 41.70720313221376, -87.69206127479819 41.70719759370725, -87.69207530255382 41.70719745353754, -87.69234801839764 41.70719474821409, -87.69264696183437 41.70719158526819, -87.69283932357702 41.70718952931126, -87.69303578823948 41.70718727579811, -87.69338506110763 41.70718299174191, -87.69369206983258 41.7071789373701, -87.69398127333763 41.70717552463521, -87.69414140450016 41.70717364366294, -87.69451975932293 41.70716899731539, -87.69481771884068 41.70716535644172, -87.69511589551897 41.7071619355477, -87.69553445727733 41.70715764609345, -87.69585002132534 41.70715497815942, -87.69630527247021 41.70715149262453, -87.6966800758785 41.70713551256635, -87.69716610895304 41.70712991553651, -87.69728632929944 41.70712361123115, -87.69740777132927 41.70712708199971, -87.69786516254406 41.70712187292511, -87.698322627301 41.70711666380829, -87.69843987584757 41.7071181895575, -87.69856062558065 41.70711386138825, -87.69878097088878 41.7071113465775, -87.69901838377461 41.70710859486342, -87.70120153844684 41.70708357652507, -87.70137474479867 41.70705170671482, -87.7013623439606 41.70687073232803, -87.70134721500452 41.70662807812194, -87.70132812759182 41.70624371294285, -87.70132488578203 41.70617824202733, -87.70173371072539 41.7061728338598, -87.70213574401375 41.7061693060434, -87.70254693526854 41.70617653107333, -87.70261042578625 41.70617764906973, -87.70308524981904 41.7061716638672, -87.70341455454532 41.70616770606198, -87.70379441834532 41.70616166499705, -87.70381665075909 41.70616132052716, -87.70416126333812 41.70615747105751, -87.70451355676553 41.7061507821351, -87.70502280977715 41.70614783251499, -87.70503903766281 41.70614756447726, -87.70542487089979 41.70614141261909, -87.70582699375433 41.70613213892229, -87.70624430840887 41.70612752728034, -87.70634773930301 41.70612639098361, -87.70683789665561 41.70611756548943, -87.70736635870215 41.70610897318103, -87.70746107707576 41.70610751344169, -87.7079177801503 41.70610050476904, -87.70853424946584 41.70609524122079, -87.70872983153392 41.70609301123039, -87.70921201019192 41.70608745205774, -87.70995940569689 41.70608931331554, -87.7101882848584 41.70608989656582, -87.71112354113697 41.70607372258436, -87.71112580966511 41.70613474187959, -87.71113341646024 41.70634082738336, -87.71113892039924 41.70650699920798, -87.71114589282728 41.70671807539499, -87.7111531022384 41.7069351909318, -87.71115794895051 41.70708198476017, -87.711165611338 41.70731282383519, -87.71117262007775 41.70752417489546, -87.71117703751324 41.70765856146539, -87.71118250231562 41.70784753786809, -87.71118774267558 41.708029597888, -87.71119637690973 41.7083115964835, -87.71120148237274 41.7084771900571, -87.71120752449421 41.70867478647666, -87.71121288610217 41.70884792860549, -87.71121872901332 41.7090396787872, -87.71122288506923 41.70917464021457, -87.71122659046218 41.70929549225097, -87.71123163718971 41.70945963153197, -87.71123843171686 41.7096702947199, -87.71124424827504 41.70984955803562, -87.71124642393583 41.70992015476743, -87.7112470956025 41.70994161905264, -87.71125285654369 41.71012661715613, -87.71125916330341 41.71033113120608, -87.71126514589069 41.71052345949954, -87.71127189983778 41.71073458951209, -87.71128243060441 41.71104793928622, -87.7112908596276 41.71129799319032, -87.71129797079016 41.71149123154085, -87.7114502370386 41.71148947681612, -87.71252474068429 41.71147377142704, -87.7125804940537 41.7131856539561, -87.71257762390127 41.71329746977926, -87.71283341513994 41.71329388402805, -87.71296961030751 41.71329184759007, -87.71313250795696 41.71328946195409, -87.71330342919363 41.71328692617894, -87.71353336228066 41.71328355583408, -87.7138017941637 41.71327954267495, -87.71404070207542 41.71327597298516, -87.71415964210347 41.71327430867956, -87.7143220622759 41.71327205407962, -87.71441367619479 41.71327076482893, -87.71454437486929 41.71326891536874, -87.71473258371847 41.7132663069948, -87.71502113045034 41.71326152025879, -87.71528740715119 41.71325708065775, -87.71550884039188 41.71325385219812, -87.71563012560212 41.71325206203697, -87.71573899231026 41.71325047861576, -87.71594364922439 41.71324746212772, -87.7162415270345 41.71324373840879, -87.71643878084885 41.71324128421233, -87.71661713649516 41.71323867364453, -87.71684750805872 41.71323530039263, -87.71685307594714 41.71323522021538, -87.71706040705858 41.71323221663324, -87.71710143226844 41.71323161197015, -87.7172254293053 41.71322980777286, -87.71746261319883 41.71322635866628, -87.71762239861324 41.71322403149122, -87.71798431102862 41.71321872420796, -87.71808193189857 41.71321727127239, -87.71817208174561 41.71321594291958, -87.71836835003769 41.71321304214, -87.71842805854406 41.71321215415224, -87.71868748707973 41.71320750372017, -87.71895746388753 41.71320269092408, -87.71915277855373 41.71320000188005, -87.71928893421865 41.71319815094071, -87.71946820444747 41.71319570549383, -87.71957989099616 41.71319421637532, -87.71990616687938 41.7131888776327, -87.72014247658245 41.7131850066782, -87.72033921892117 41.71318246154961, -87.7204840560252 41.7131804074165, -87.72054358146619 41.71317954534515, -87.72078600943814 41.71317540409158, -87.72112510709265 41.71316991013789, -87.72111666607771 41.71291689331839, -87.72110981853959 41.71270441924776, -87.7211040753218 41.71252625460253, -87.72110091568385 41.7123883070234, -87.72109864079293 41.71224987127333, -87.7210952970789 41.71204633463697, -87.72109429684252 41.71183528964741, -87.72109355738937 41.71168171337229, -87.72109295269462 41.71156013585657, -87.72108807066809 41.71134904492008, -87.72108387089742 41.71116633122362, -87.72107941634447 41.71098735009737, -87.72107198293476 41.71069001736435, -87.7210649145608 41.71046217379158, -87.72105795258409 41.71023883037819, -87.72105079907223 41.71000854305711, -87.72104967286671 41.70997280589164, -87.7210472691612 41.70989419577251, -87.72104106040896 41.70969549960299, -87.72103479572354 41.70949502068645, -87.72102644265135 41.70922902376563, -87.72102138467129 41.70907498504354, -87.72101553334605 41.70889668166726, -87.7210098501472 41.70872370427842, -87.72100254257953 41.70850192294726, -87.7209988586502 41.70838444654935, -87.7209922703607 41.70817567716137, -87.72098803231987 41.70803566658002, -87.7209829826547 41.70786930555247, -87.72097725261777 41.70770124050748, -87.72097103543038 41.70751898357625, -87.72096243657846 41.70726739161347, -87.7209540110335 41.7070245568981, -87.72094512379739 41.70676851756036, -87.72093650368964 41.70651906712578, -87.7209292082685 41.70630774194594, -87.72092253366858 41.70611582303501, -87.72091710522955 41.70593900403018, -87.72091515300814 41.70587466650704, -87.72091316738774 41.70580983537444, -87.72090978143194 41.70569976992712, -87.72090289320599 41.70550721838602, -87.72089150879026 41.70519413928717, -87.72088474051596 41.70500817476307, -87.72087296179537 41.70480225913342, -87.72085249021315 41.70448199575085, -87.72083951913024 41.70427829747642, -87.72083123364656 41.70409015790919, -87.72082919086289 41.70404305373712, -87.72081816697342 41.70379251952709, -87.72081038988424 41.70360086908409, -87.72079614940877 41.70324924367687, -87.72079390617074 41.70319239742141, -87.72079083703306 41.70311059995885, -87.72079024162501 41.70309624310369, -87.72078539968265 41.70297362796733, -87.7207715775849 41.70263578239667, -87.72076411191314 41.70245000562124, -87.72075837441746 41.70222502235968, -87.72075276736867 41.70200547383802, -87.72074367851077 41.70181340359185, -87.72074340464016 41.70180761186382, -87.72073537596687 41.70159636547161, -87.72072471940541 41.70131078852008, -87.72071914355882 41.70116146597086, -87.72071077566814 41.7009049919545, -87.72070291725619 41.70068732359696, -87.72068941296965 41.7003970940417, -87.72068272041025 41.70025355688151, -87.7206786543027 41.69997196628623, -87.72067664481497 41.69980567638435, -87.72066871389819 41.69948375034701, -87.72066373967481 41.69928256496158, -87.72065403556964 41.6990203469785, -87.72064866046161 41.69888057681878, -87.72063757042275 41.69856763557733, -87.72062982347789 41.69834980443407, -87.72061226111347 41.6980089747542, -87.72060339051021 41.69775916499847, -87.7205897889506 41.6974483788751, -87.72056711455643 41.69710982629746, -87.72055117209626 41.69679131661991, -87.72053709068591 41.69649644475166, -87.72053089025358 41.69630491250581, -87.72052908174483 41.69624836845173, -87.72052596545763 41.69615246474844, -87.72051806987065 41.69595046753231, -87.7205110809515 41.69577246036405, -87.72050327538895 41.69557252104246, -87.72049156594623 41.69526712377998, -87.72048103171885 41.69497671656863, -87.72047618338546 41.69485484246021, -87.72046613911243 41.69460143149409, -87.7204427001291 41.69401577485944, -87.72042670451989 41.69360255737536, -87.72041102398214 41.69319867173722, -87.72040546444256 41.69305527846738, -87.72039572468366 41.69280466771635, -87.72038711697189 41.69255823520761, -87.72038527399069 41.69250553519892, -87.72037283958441 41.69221462290178, -87.72035098593804 41.69189792038276, -87.72033795352594 41.69172034628529, -87.7203316323905 41.69163425041158, -87.72031697227659 41.69128487371695, -87.72031618559632 41.69126611718521, -87.72034051200562 41.69126567831578, -87.72517656540042 41.69117839983286, -87.72517115780423 41.69104120342561, -87.72881109227959 41.69097540961887, -87.73002392490912 41.69095346137259, -87.73245222848162 41.69090943049562, -87.73488506856278 41.69086531703653, -87.73927666793347 41.69078551795755, -87.73973540298151 41.69077717274809, -87.73966400805013 41.68913142219895, -87.73948732672186 41.68913463576855, -87.73928250074476 41.68396695899116, -87.73927380070559 41.6837474478484, -87.7390727362357 41.68375094998387, -87.73459918818837 41.68382877747337, -87.72974611369669 41.68391300163325, -87.72853338795399 41.68393401491917, -87.72489376700319 41.68399702330473, -87.72489353522667 41.68399702756634, -87.72489278199112 41.68397785078452, -87.72368726251894 41.68399868586629, -87.72183167824394 41.68403070689512, -87.72182797927009 41.68393671190536, -87.72156608109677 41.6839414739292, -87.72126023421653 41.6839467066004, -87.72078427476002 41.68395481854424, -87.72050475456041 41.68395958521595, -87.72006475363558 41.68396684306659, -87.7193067868469 41.68397931714957, -87.71896210089605 41.68398428157752, -87.71861628457327 41.68398888188661, -87.71782033385102 41.68399957734464, -87.7176046688438 41.6840033890918, -87.71738035661349 41.68400797903854, -87.71726567710783 41.6840100529861, -87.71693520152792 41.68401431936687, -87.71654532809325 41.68402043324355, -87.7163153828027 41.68402454958971, -87.71584978925577 41.68403192887936, -87.71547643003734 41.68403766131826, -87.7151539734228 41.68404185563845, -87.71468956425956 41.68404786387825, -87.71447657598583 41.68405127306071, -87.71411295526242 41.68405686058144, -87.71383260472554 41.68406031698296, -87.71350211509389 41.6840659996484, -87.71321351534871 41.68407086427527, -87.71272526911112 41.68407725875123, -87.71232335754826 41.68408252405747, -87.7121202607905 41.68408510413181, -87.711933385186 41.68408749712062, -87.7116105221692 41.68409195263607, -87.71150730378415 41.68409358995394, -87.71150216384328 41.68400777432417, -87.71148825491719 41.68366531509437, -87.71148647962588 41.68362145091938, -87.71147257120727 41.68327899166785, -87.71146545557488 41.68309793641878, -87.71145131438091 41.68274921795277, -87.71144977896525 41.68271089973579, -87.71143571165645 41.68236212675779, -87.71142845050677 41.6821810704162, -87.71141376560831 41.68183558864442, -87.7115974830851 41.68183262998833, -87.71159588516375 41.68179425654218, -87.71158149439499 41.68144855653696, -87.71158062833666 41.68135617619976, -87.71157979508781 41.68126742296116, -87.7113907998465 41.68127051439799, -87.71137680297625 41.68092953673801, -87.71137495590654 41.68088572676479, -87.71136096171038 41.68054463822351, -87.71122527443177 41.68054675876789, -87.71108797599535 41.6805489239436, -87.71095067953158 41.6805510354513, -87.71081330750411 41.6805531999003, -87.71067601014173 41.68055536486405, -87.71053878540327 41.68055747626817, -87.71040148693109 41.68055964062344, -87.71026389687043 41.68056174834701, -87.71002232551538 41.68056554374661, -87.70885122908311 41.68058383002082, -87.70830306262336 41.68059237612703, -87.70783280745626 41.68059969734759, -87.7077165731519 41.68060150687128, -87.7077149163684 41.68056170296921, -87.70771280604616 41.68051097411585, -87.70723352778406 41.68051844053615, -87.70539944776824 41.6805470203191, -87.70528666914893 41.68054876201455, -87.70528496016102 41.6805788594803, -87.70527890947525 41.68068538921678, -87.7052845234866 41.68083836207481, -87.70528978751655 41.68098595492927, -87.70529089527827 41.6810187822139, -87.705296001417 41.68117139634619, -87.70529909163066 41.68126768419347, -87.70530668235163 41.68144706840397, -87.70531233415959 41.6815811057647, -87.70532017206547 41.68175766436924, -87.70532460905552 41.68185851654456, -87.70532657321303 41.68190880224081, -87.70533193825932 41.68204602173347, -87.70534079178796 41.68227261576286, -87.70534387148469 41.68236242771135, -87.70534665860843 41.68244471774153, -87.70534890581762 41.68251476588205, -87.70535505246838 41.68267334032952, -87.7053607848032 41.68282554481407, -87.70536249820654 41.68287146645238, -87.70536976372451 41.68306166290802, -87.70537795774109 41.68327689117189, -87.70538598234884 41.68348706946357, -87.70539134633961 41.68360186600105, -87.70539715370474 41.68373864932212, -87.70539783781892 41.68375492645076, -87.70540482511413 41.68393241313782, -87.70541706426729 41.68418729201792, -87.7050405140907 41.68419349481294, -87.70486981791861 41.68419579982945, -87.70450528907111 41.68420072188036, -87.70401333314202 41.68420818367174, -87.70368445741958 41.68421335213284, -87.70327791403798 41.68422056543934, -87.70298140854361 41.68422505832492, -87.70265366974365 41.68423003961436, -87.70215590981351 41.68423932720527, -87.70164580893487 41.68424552521189, -87.70124238310123 41.68425203382573, -87.70055274105317 41.68426278458601, -87.70056046882405 41.68453144313123, -87.70056717141969 41.68470670668437, -87.70057857352349 41.68496657555887, -87.70058793600654 41.68519555991976, -87.700595499801 41.68541745452105, -87.70060239487117 41.68561796565567, -87.70060530698852 41.68570198646572, -87.7006153968873 41.68593506414035, -87.7006239898442 41.68613421325728, -87.70062407850358 41.68613629944313, -87.6993977778986 41.68614740730894, -87.69939783850958 41.68614850537888, -87.69941144505215 41.68632891067567, -87.69941815726347 41.68652146280202, -87.6994203041028 41.68658259079621, -87.69942866140047 41.68676804442465, -87.69943599420527 41.68698335134961, -87.69944331444722 41.68719975565786, -87.69945167164806 41.68740384277402, -87.69945872976464 41.68757628131895, -87.69946403648392 41.68772189746424, -87.69947334702576 41.68793334574299, -87.69936265161753 41.68793539716577, -87.69903713185924 41.68793977467532, -87.69886921063146 41.68794200429641, -87.69881553155733 41.68794272389288, -87.69843632879564 41.68794817546516, -87.69826020549441 41.68794679124444, -87.69798597031429 41.68794464430139, -87.69766233651045 41.68795089531573, -87.69764728660536 41.68795119682716, -87.69742588921328 41.6879556522265, -87.69709200829973 41.68796887126302, -87.69678098341282 41.68798119969929, -87.69660587393217 41.68798448393302, -87.69658805569033 41.68798721215449, -87.69642529141512 41.68801207820034, -87.69624509045828 41.68802309959499, -87.69621279328994 41.68802747642604, -87.69614779237378 41.68803279765861, -87.69608962945375 41.68805736548957, -87.69605842346583 41.68807363185984, -87.69605432523053 41.68807703900802, -87.69576463357305 41.68807556919345, -87.69576154319364 41.68799176796004, -87.69570058334226 41.68636046892119, -87.69567223470382 41.68567203143615, -87.69564410411479 41.68481344575403, -87.69562541592454 41.6843332203965, -87.69519224201829 41.68434165666276, -87.69469135353908 41.68434908218421, -87.69440305105852 41.684353133352, -87.6940619547168 41.68435738357843, -87.69357051924766 41.68436392536655, -87.693136133056 41.6843690808202, -87.69232292210789 41.68438076755353, -87.69184048371973 41.68438820204603, -87.69139158768742 41.6843945874706, -87.69099344163504 41.68440021167793, -87.6905276270455 41.68440715725853, -87.69009645663982 41.68441292241344, -87.68991938958807 41.68441538938472, -87.68989602932724 41.68441569824345, -87.68951763134567 41.68441969759282, -87.68927126062871 41.68442232499785, -87.68898835683609 41.68442823154212, -87.68890004566413 41.68442919088378, -87.68861760905891 41.68443229908561, -87.68829197485358 41.68443895243434, -87.68807260194347 41.6844434563568, -87.68773120239767 41.68444883843625, -87.68749693836068 41.68445251663582, -87.68706268662962 41.68445880360552, -87.68682278691716 41.68446225690987, -87.68647414252113 41.68446723800913, -87.68621403227375 41.68447093277609, -87.68584271185404 41.68447361679574, -87.68572195738069 41.68447529571458, -87.68571837144594 41.68436555710775, -87.68570821430623 41.68404957209956, -87.68556625490625 41.67962445676141, -87.68556587215247 41.6793852026511, -87.68558013022289 41.67914619543546, -87.68560902233985 41.67890798531117, -87.68565239809618 41.67867095343172, -87.68565537318328 41.67865900486773, -87.68571025199796 41.67843570515154, -87.68578243051867 41.67820267812954, -87.68586886029063 41.67797242106833, -87.68592980532304 41.67783033358383, -87.68600509234949 41.67765231970143, -87.68614286601928 41.67734649796389, -87.68616405381772 41.6772995251819, -87.68620516229146 41.67720825890704, -87.68592233437428 41.6772096859298, -87.685602136634 41.67721441331084, -87.68522218955832 41.67722003728195, -87.6849862592824 41.67722474358643, -87.68498131462411 41.67722485373267, -87.68496974496085 41.67722509006743, -87.68472264688292 41.67723003552903, -87.68437749432297 41.6772372806732, -87.68375844134437 41.67725027644084, -87.68315121286038 41.67726300754434, -87.68285362076132 41.67726925448937, -87.68254375444492 41.67728037152631, -87.68240665411494 41.67728527562682, -87.68192844485132 41.6772959305953, -87.68168785566284 41.6773012911787, -87.68137448357858 41.67728612155609, -87.68132114959175 41.67728252551196, -87.68112840685968 41.67726954834241, -87.68083929864279 41.67726219883477, -87.68069641239366 41.67725655831352, -87.6803832327667 41.67724415859236, -87.6798848774815 41.67723775828341, -87.67955501929504 41.67722770476367, -87.67935575255733 41.67722640612445, -87.67891076607735 41.67722757804155, -87.67879805562815 41.67722786900134, -87.67875683639245 41.67722799210846, -87.67849927397022 41.67722866547039, -87.67816768055839 41.67723456850553, -87.67783722088022 41.67724042326089, -87.67749013874239 41.67724659358819, -87.67723429465197 41.67725114399133, -87.67689288962558 41.67725701670012, -87.67661555713453 41.67726177066316, -87.67633118593965 41.67726722661317, -87.67601188392811 41.67727377110064, -87.67570090829649 41.6772787154656, -87.67544514028766 41.67728296016841, -87.67516205989445 41.67728770598735, -87.67488505526188 41.67729237614433, -87.67462774590936 41.67729699495383, -87.67439079262422 41.67730123399694, -87.67408808306503 41.6773067436381, -87.67393291876186 41.67730964013999, -87.67386917791691 41.67731081151896, -87.67359902054748 41.67731529831772, -87.6733853555836 41.67731884703773, -87.67322916585753 41.67732181985465, -87.67298305683444 41.67732647104368, -87.67267280805351 41.67733174098526, -87.67260296107848 41.67733219075022, -87.67239836107818 41.67733351084798, -87.67217031406059 41.67733656239948, -87.67187766741972 41.67734231593625, -87.67168709876563 41.67734604958408, -87.67160655037024 41.67734758838624, -87.67136488467649 41.67735091751418, -87.67126378297252 41.67735428644844, -87.6711578468269 41.67735779374694, -87.67109724426012 41.67735980359989, -87.67084819798004 41.67736539435992, -87.67054386577863 41.67737198194953, -87.67016974835001 41.6773800882173, -87.6698338722758 41.67738572404141, -87.66946416685255 41.67739193134609, -87.66892978142583 41.67740089180109, -87.66859401519382 41.67740646947478, -87.66849959436149 41.67740803700719, -87.66828807725716 41.6774115455875, -87.66803352710552 41.67741576794165, -87.66795069271841 41.67741714659557, -87.66798441507247 41.67732553016503, -87.66801871688351 41.67723234100217, -87.66840088960703 41.67619714414086, -87.66840417726421 41.67618823554339, -87.66840310921388 41.67618801475663, -87.66743299988211 41.67598730719761, -87.66731578161009 41.67596305545473, -87.6671858200572 41.67631520731531, -87.66680352362989 41.67735068066114, -87.66677140861276 41.67743766456022, -87.66676885297704 41.67743772026771, -87.66645202110846 41.67744455658902, -87.66623610902725 41.67744936744425, -87.66604589967986 41.67745358835197, -87.66558106392201 41.67746263404726, -87.66529793676327 41.67746815170106, -87.66502785453444 41.67747237151954, -87.66463948096188 41.6774784281253, -87.66433801593851 41.67748375356346, -87.66403655086447 41.67748907930812, -87.66373647099216 41.67749496065162, -87.66334017323098 41.67750277741396, -87.66298740762582 41.67750972237075, -87.66253395107459 41.67751904165722, -87.66246284762998 41.67752057566131, -87.66233631078845 41.67752329373582, -87.66213570372517 41.67752763956846, -87.66175810617409 41.6775357796311, -87.66155687681052 41.67754011889579, -87.66136505848245 41.67754426776332, -87.66138220773955 41.67744746508497, -87.66138359041621 41.67730753950697, -87.66137365200807 41.67702426432358, -87.66136132992345 41.67670277560739, -87.66135171464605 41.67637778939304, -87.66133203982793 41.67571093688404, -87.66132278090949 41.67539767130944, -87.66132090777191 41.67533486899491, -87.66131885806892 41.67526092499185, -87.66131845120576 41.6752472549675, -87.66131301282675 41.67508399026558, -87.66130833129243 41.67494625126653, -87.6613019417467 41.6747616854489, -87.66129490269169 41.67457291506276, -87.66128760983803 41.67437311202814, -87.6612799063968 41.67416359079038, -87.66127085549954 41.67389014675394, -87.66126564862942 41.67373261876714, -87.66125874219973 41.6734816088032, -87.66125099695451 41.67328322931859, -87.66124239692272 41.6730578410438, -87.66123460485289 41.67279708262976, -87.66122600861135 41.67254663932848, -87.66121565602276 41.67229632186422, -87.66120824798686 41.67206547872775, -87.66119544943992 41.67166667795735, -87.66118800393895 41.67145359130188, -87.66118027198273 41.67124327475001, -87.6611712100169 41.67099551701379, -87.66116185893856 41.67074051305649, -87.66115664027956 41.67053141868761, -87.66115004323014 41.67033513151313, -87.65936381280238 41.67031838629765, -87.65906973981798 41.67032357775765, -87.65888644253272 41.67032672801834, -87.65879485671995 41.67033464284226, -87.6587239973589 41.67031007781765, -87.65865229805627 41.67028523173764, -87.65849841630828 41.6702861390295, -87.65828727648223 41.67028830237164, -87.65807023466476 41.67029144660609, -87.65774864690881 41.6702955937367, -87.65749418789714 41.67029977796076, -87.65725429692723 41.67030371909392, -87.65712433597373 41.67030616487048, -87.65711525846547 41.67030630411518, -87.65687705225342 41.67031017149405, -87.65655853510995 41.67031474611594, -87.65628094580586 41.67031870966773, -87.65595781552587 41.67032333640716, -87.65567110851002 41.67032760096423, -87.65563779644675 41.67032809090568, -87.65524288056127 41.67033385742943, -87.65506665268404 41.67033655042174, -87.6548490584628 41.67033984860048, -87.65461014831872 41.67034464157351, -87.65445313345398 41.67034755521129, -87.65435278993924 41.67034943291817, -87.65406519700214 41.67035431998256, -87.65384376867331 41.67035685262463, -87.65350549436394 41.67036069876934, -87.65345790832414 41.67036113026708, -87.65327992618313 41.67036356232771, -87.65323823013733 41.67036419352044, -87.65274377660722 41.67037169592795, -87.65262718175312 41.67037342045617, -87.6524165807469 41.670376509005, -87.65203831126439 41.67038291118133, -87.65201455291553 41.67038329149499, -87.65159407828705 41.67038996535523, -87.65141118814117 41.67039277651131, -87.65121489978806 41.6703957828424, -87.65083498531938 41.67040189795795, -87.65079522949017 41.67040245800426, -87.65047191633224 41.67040706905436, -87.65019004456435 41.67041093783575, -87.64996015424614 41.67041407188815, -87.64959299309656 41.67041858467697, -87.64958321991014 41.6704187447104, -87.64929291607761 41.67042379717936, -87.64897164338629 41.67042929379342, -87.64873174962165 41.67043340894728, -87.64836413926628 41.67043898460448, -87.64834041740072 41.67043933775579, -87.64793415747769 41.67044479172857, -87.64775635291232 41.67044762766983, -87.64748951993745 41.67045188223422, -87.647201346572 41.67045622723091, -87.64714500852715 41.67045715245977, -87.6467311557938 41.67046398390556, -87.64653325638197 41.67046697288943, -87.64632668082615 41.67047007542945, -87.64593275213738 41.67047565119689, -87.64592030653172 41.67047582320194, -87.64555495948227 41.67048181506048, -87.64531887661391 41.67048558885543, -87.6450962643232 41.67048914205076, -87.64471054911368 41.67049341780952, -87.64469338089098 41.67049361654702, -87.64439206448863 41.67049828739986, -87.64430606949901 41.67050004914064, -87.6440982787786 41.67050371560384, -87.64384923242129 41.67050812326487, -87.64354441953863 41.67051109687022, -87.64348940184459 41.67051172635342, -87.6432913990004 41.67051402387083, -87.64308852377721 41.6705168669995, -87.64288347364779 41.67052101426329, -87.64268186572697 41.67052507179806, -87.6424101645074 41.67052925677186, -87.64227358299993 41.67053120795487, -87.64214091819036 41.67053312709462, -87.64192017685025 41.67053635359758, -87.64173557192095 41.67053864642844, -87.6416650312248 41.67053953921602, -87.64160396803378 41.66871486935818, -87.64154302378627 41.66689305455799, -87.64148184966939 41.66506572054455, -87.64142093347874 41.66324511250251, -87.64150519471126 41.66324397256901, -87.64188838712236 41.66323889700569, -87.64205467286158 41.66323660403634, -87.64216056533289 41.66323515544929, -87.64238121325 41.6632314865879, -87.64265120965163 41.66322635803111, -87.64283258672191 41.66322289194985, -87.64304428749907 41.66322098015473, -87.64325825635748 41.66321891692647, -87.64331506159282 41.66321837978184, -87.64364507029553 41.66321396729722, -87.64386447452218 41.66321039789863, -87.64423146178967 41.66320444834435, -87.64438387891076 41.66320217862741, -87.6444540087479 41.66320117137162, -87.64447586292188 41.66320075430923, -87.6446616318501 41.6631972567763, -87.64488880063342 41.66319301896429, -87.64508461926881 41.66319081672239, -87.64542926149592 41.66318692473814, -87.64564543484379 41.66318410101882, -87.64569258165754 41.66318328536199, -87.64587267322406 41.66318019154864, -87.64608984466217 41.66317649614444, -87.64629376445512 41.66317318738184, -87.64666858557899 41.66316711193651, -87.64689278072481 41.66316362053978, -87.64690094353436 41.66316350435584, -87.64717832501032 41.66315920540281, -87.64718000756648 41.66315918882736, -87.64721678667047 41.66315940838911, -87.64721294991737 41.66301398945434, -87.64713762546738 41.66120847267565, -87.64713536757746 41.6611523106694, -87.64712136353202 41.66081675719067, -87.64710963353369 41.66053588604843, -87.6470956434802 41.66019907021491, -87.64709517669763 41.66018781642468, -87.64709323854798 41.66014290704581, -87.64708979436351 41.66006061065494, -87.64708588013006 41.65996733461299, -87.64708204093083 41.65987405819413, -87.6470768367313 41.65975058716624, -87.64707170793888 41.65962711576245, -87.6470648198775 41.65946246890188, -87.64705793148545 41.65929782148545, -87.64704598233251 41.65908937121421, -87.64704767038148 41.6590508783589, -87.64704422651423 41.65896852679187, -87.64703681487384 41.6587911981688, -87.64701261378947 41.65820980059817, -87.64699831299549 41.65786793398514, -87.64698898364124 41.6576436649555, -87.64698846521084 41.65763048870941, -87.64681693916467 41.65766234190135, -87.64663458843488 41.65768661037416, -87.64645200198869 41.65770549822447, -87.64626960662501 41.65772702150178, -87.64608685725993 41.65774744463303, -87.64591056126599 41.65778042030151, -87.64572707584556 41.65776680825598, -87.64554386311036 41.65776189960552, -87.64536044111189 41.65775103993633, -87.64517667513287 41.65774018559915, -87.64499396459648 41.65774029382785, -87.64475182790305 41.65772480310227, -87.6445742010651 41.65772209715069, -87.64439110958786 41.65772220982243, -87.64420806556009 41.65772226540452, -87.64402503188251 41.65771688914177, -87.64384195579414 41.65771974620314, -87.64366354576948 41.65774994520091, -87.64142151652995 41.65777103786716, -87.64141554443438 41.65777113172341, -87.64141811803371 41.65784013053695, -87.64142062024996 41.65790721748105, -87.64142375279521 41.65799066481093, -87.64142631198078 41.65805923376995, -87.6414276136821 41.65809410251682, -87.64106161164936 41.65809623775186, -87.64106033693338 41.65806207967555, -87.64105777740717 41.65799351043257, -87.64105521861723 41.65792494119316, -87.64105215421911 41.65784204340768, -87.64104963404434 41.65777408083141, -87.64065422819407 41.65777367183135, -87.64059624531711 41.65778896516691, -87.64041681550991 41.65778629187025, -87.64023356854285 41.65777788703012, -87.64005001288409 41.65776404622565, -87.6398669257547 41.65776118534936, -87.6396838386455 41.65775832418151, -87.63950059150297 41.65774991817184, -87.63931766544275 41.65775254609002, -87.63913514028873 41.65775188292017, -87.63895278080713 41.65773579722248, -87.63874612977253 41.65772587644206, -87.63856147368237 41.6577123559325, -87.63837701077746 41.65768774912635, -87.63819308590558 41.65766073025155, -87.63800932193149 41.65763914586597, -87.63782555881586 41.65761756119186, -87.63764156142696 41.65759048610899, -87.63745752128531 41.65756066612428, -87.63727359813564 41.65753359090201, -87.63708971744789 41.65750931487823, -87.63690583822527 41.65748498368226, -87.63672141434654 41.65745680680325, -87.63653690834938 41.65742950732211, -87.6364185305124 41.65741215885844, -87.63629408799001 41.65739389530234, -87.63623106324737 41.65739279679745, -87.63611085934185 41.65739069826392, -87.63610252709431 41.65738965975407, -87.63592697396673 41.6573669692218, -87.63574325638696 41.65734801607034, -87.63556009398228 41.65734547719975, -87.63537723743507 41.65734853836548, -87.63519439323277 41.65735719778609, -87.63501184112579 41.65736591357735, -87.63482946332456 41.65738555264457, -87.6346966405444 41.65739786335876, -87.63464711580835 41.65740244725784, -87.63446462124062 41.65741939557439, -87.63428199591962 41.65742805487365, -87.63409973478623 41.6574504378399, -87.63391750262784 41.65747013123178, -87.63373473110109 41.65747878877126, -87.63355187198177 41.65748201167708, -87.63336949380664 41.6575016484235, -87.63318718834215 41.65752128532317, -87.63300437327256 41.65752725197412, -87.63282355407573 41.65758531822742, -87.6326418644248 41.65762241206864, -87.63264606461485 41.65773457173898, -87.63263624610148 41.65773560962182, -87.6326021258465 41.65774368954001, -87.63256303742035 41.65775097073991, -87.63254096866336 41.6577545684989, -87.63252590055056 41.65776051419116, -87.63248979657575 41.65776929523667, -87.63244164130063 41.65777575296585, -87.63237850013402 41.65778738815721, -87.63228628855495 41.65780032795432, -87.63227381874439 41.65780288646982, -87.63232725751544 41.65787610228988, -87.63227543980786 41.65786936446693, -87.63222394279009 41.6578667999858, -87.6321744092387 41.6578654000856, -87.63215346615999 41.6578664249499, -87.63214973199837 41.65786656683036, -87.63211607334199 41.65787256371538, -87.63206969515855 41.65788386190821, -87.63203401765077 41.65789379831268, -87.63202338052281 41.6578961484353, -87.63201531683163 41.65789730674624, -87.63200952994693 41.65789776541872, -87.63200359669443 41.65789822319782, -87.63191951729608 41.65790363792051, -87.63187828106054 41.65790645966173, -87.63186091183039 41.65790865918412, -87.6318579065267 41.65790907993816, -87.63185182213974 41.65790997588271, -87.63183796774766 41.65791197757975, -87.63180352427442 41.65791610287762, -87.63177766736192 41.65791803075967, -87.63175246328241 41.65792056637733, -87.63172589216884 41.65792089816637, -87.63171067528091 41.65792031129511, -87.63169904738245 41.65791947189599, -87.63169422022438 41.65791916799378, -87.63166885103207 41.65791665298492, -87.6316452200162 41.65791579519075, -87.63160779839434 41.65791787197985, -87.63157806272984 41.65791977613605, -87.63154499876974 41.65792495318016, -87.63151486549121 41.65793651499698, -87.63149932275252 41.65794575087364, -87.63138957355355 41.65799650974847, -87.63125516221348 41.65781895306375, -87.63107539331212 41.65783273715803, -87.63102761155372 41.65785299762787, -87.63090019547924 41.65790702363206, -87.63081853681852 41.65795778348168, -87.63080775047997 41.65798719184967, -87.63080982918966 41.6580383591992, -87.63083552859109 41.65807787022465, -87.63084043400946 41.65811138124982, -87.63080794730762 41.65821151604283, -87.63076028338163 41.65827379582804, -87.63066274267462 41.65834597965474, -87.63055731871506 41.65841619415277, -87.6304047589949 41.65852020508882, -87.63023909525916 41.65863791228506, -87.6300984969812 41.65872547500713, -87.62995492263239 41.6588359209218, -87.62973309778184 41.65901123057138, -87.62962169803029 41.65907888281851, -87.62953897358611 41.65912102345762, -87.62952088473573 41.65913005685766, -87.62914651504779 41.65931703594437, -87.62885495125592 41.65955636757474, -87.6288228911295 41.65958350476109, -87.62879752661358 41.65961408569684, -87.62876786524441 41.65962927221861, -87.62875416032345 41.65963769595623, -87.62871042154032 41.65968869222336, -87.62868308511973 41.65969873360389, -87.62865550700606 41.65972419672566, -87.6286302530025 41.6597513755854, -87.62860040608119 41.65977682478305, -87.62855683458635 41.65981920476681, -87.62854469085353 41.65983208361946, -87.62804201489284 41.66029499019261, -87.62796612294953 41.66036807295512, -87.62793220300617 41.66041797627587, -87.62788046235141 41.66046442265421, -87.62777242532496 41.66057227123052, -87.6276533904781 41.66066171912222, -87.62754358388877 41.66075731682925, -87.62690827610606 41.66119942497542, -87.62688629705939 41.66121476799653, -87.62687173214808 41.66124151790331, -87.62683597761462 41.66124486605447, -87.62663723097914 41.66135462556424, -87.62658683269318 41.66139191321367, -87.62653655161699 41.66141844372534, -87.62644066416307 41.66148585946765, -87.62636117429642 41.6615730792974, -87.62631557252371 41.66160677379276, -87.6262744922018 41.66166201175599, -87.62625772704016 41.66166909880533, -87.62621485378824 41.66167415907544, -87.62610949435438 41.66173081276513, -87.62606406830967 41.66174842637454, -87.62596847873706 41.66178181311454, -87.62590145205678 41.66181361920225, -87.62588487738167 41.66180994953415, -87.62586819047161 41.66180984682222, -87.62572729226801 41.66185008948758, -87.62570346642673 41.66185356561115, -87.62569408955983 41.66184093877654, -87.62567983708327 41.66183909464438, -87.62566076908287 41.66184254489144, -87.62557479123895 41.66186704391617, -87.62555078965394 41.66188660051698, -87.62544098620238 41.66191457525511, -87.62537419442114 41.66192486681098, -87.62535414827134 41.66192807222394, -87.62535271153089 41.66192830213201, -87.62535221597014 41.66192974260522, -87.6253477970123 41.66194259734358, -87.62533429358443 41.66194523655464, -87.62518796700068 41.66197383132197, -87.62516882039441 41.66198441630145, -87.62507096580929 41.66201065303811, -87.62486095129651 41.66205947070757, -87.62475592533991 41.66208560780873, -87.62469616758506 41.66210851143665, -87.62467210667251 41.66213344640391, -87.62465046587835 41.66215120610313, -87.62460509849207 41.66216344057801, -87.62459072861854 41.66217229853419, -87.62452387142991 41.66219515829353, -87.6244688917164 41.66221628000076, -87.62445210622123 41.66222512304111, -87.62444711359109 41.66224655302939, -87.62443038744489 41.66225001751555, -87.62442319001967 41.66225891968882, -87.62439457324299 41.66225874317995, -87.62436372638366 41.66225498525889, -87.62433752520153 41.66225482363618, -87.6242897367716 41.66227423323646, -87.62427063415431 41.66227411538973, -87.62424692670582 41.66226677894583, -87.62421836287828 41.66226841398769, -87.62420159793929 41.66227544585017, -87.62418504311819 41.66226996479169, -87.62413989079357 41.6622625508964, -87.62409713917599 41.66224977260615, -87.62406871400495 41.66223883963902, -87.62402595766719 41.66223319686559, -87.62399730114952 41.66223664254848, -87.62395432916686 41.66225064793044, -87.62391842183408 41.6622611841395, -87.62388985836778 41.66226281910195, -87.62385414229045 41.6622625986376, -87.62380883335905 41.66226945424144, -87.62367997957051 41.66229912095683, -87.62365141631891 41.66230070097438, -87.62362995250146 41.66230237970954, -87.62361312290392 41.66232192530141, -87.62358450610436 41.66232174859064, -87.62354644832297 41.66232151357009, -87.62352016742598 41.66232854145342, -87.62350583618243 41.66233383186307, -87.62348684688904 41.6623300920501, -87.62347235904514 41.66234970697084, -87.62345796908394 41.66236037592861, -87.6233171658869 41.66239172481341, -87.62328614852959 41.66239685722858, -87.62304571742159 41.66240256182791, -87.62288672896868 41.66240361000293, -87.62221014893582 41.66240809801889, -87.62203529527527 41.66241420622326, -87.6220241102499 41.66213025177768, -87.62025290265353 41.66136568972977, -87.61985720926796 41.66114142895965, -87.61951184246301 41.66078668689394, -87.6193855247099 41.66006056263444, -87.61945847291655 41.65953715654947, -87.6196564574761 41.65912071958969, -87.6196841460707 41.65906247699868, -87.61886387791017 41.65908054628566, -87.6186627396923 41.65908497631874, -87.61845341048533 41.65910743069207, -87.61770957432179 41.65910496214904, -87.61773056283745 41.65901490372289, -87.6182131256908 41.65677681850483, -87.6183711954215 41.65604368885464, -87.61848179120739 41.65555297406765, -87.61860677971478 41.65506421506097, -87.61874601382239 41.65457768311231, -87.61889963662657 41.65409354728799, -87.61906742490075 41.65361207948036, -87.61924930387292 41.65313349624108, -87.61944519687738 41.65265802069176, -87.61965502673344 41.65218592342358, -87.61978053480915 41.65192066537355, -87.61982331720996 41.65183042229812, -87.61986880909717 41.6517401411367, -87.61999046465975 41.65149878965408, -87.62007692697557 41.65132983460609, -87.62013579991165 41.65122042591248, -87.62031640121259 41.6508864828856, -87.6202051841864 41.65084672434013, -87.61908245372356 41.65059588817632, -87.61803444837739 41.65037364420204, -87.61734763093082 41.65028495310193, -87.61722997933143 41.65027560373285, -87.61722914903794 41.64961200275882, -87.61722987785664 41.6493613100196, -87.61722700435043 41.64893350376848, -87.61722463685427 41.64858558706003, -87.61722823387107 41.64805600507224, -87.61723088940492 41.64792190432928, -87.61723677187915 41.64762519477409, -87.61723266562326 41.64703452921882, -87.6172305242646 41.646722482764, -87.61721571363714 41.6445840193874, -87.61684742664859 41.64458449610233, -87.61672526097911 41.64458463934589, -87.61643584373832 41.64458604606896, -87.61604994250237 41.64458786487483, -87.61554333378284 41.64459030083984, -87.61508462069824 41.64459248472282, -87.61473490665591 41.64459414129581, -87.61439707607883 41.644599575186, -87.61398616908328 41.64460619793476, -87.6138294917962 41.64460735761952, -87.61358830483942 41.64460798849345, -87.61335930154114 41.64460853106466, -87.61329271763162 41.6446081957636, -87.61290027230528 41.64460620323545, -87.61257449326693 41.64460388589719, -87.61210424279498 41.64460055281699, -87.61195598657334 41.64460113185986, -87.61156039264901 41.64460268292926, -87.61123944812257 41.64460316396072, -87.61088524508558 41.64460371032114, -87.61071625732224 41.64460314122601, -87.61062215848173 41.64460282459815, -87.61041450025779 41.64460209506457, -87.61022147635246 41.64460140233456, -87.60985170253336 41.64460379604277, -87.60948566206926 41.64460618435386, -87.60925548031092 41.64460747809876, -87.60918350923085 41.64460787572285, -87.60897818278643 41.64460795442109, -87.60865815825774 41.64460802321218, -87.6080599993689 41.64460812002111, -87.60799514209339 41.64461029072985, -87.60750239838882 41.64460962246263, -87.60734965847107 41.64460879452493, -87.60698392205411 41.64460684071139, -87.60653675979199 41.64460662099447, -87.60578292304434 41.64460687410961, -87.60510038050549 41.64460826035553, -87.60493112427062 41.64460869728698, -87.60442873657021 41.6446099586529, -87.60362790197685 41.64461198529725, -87.60273300062211 41.64461085652077, -87.60249698435356 41.64461042558914, -87.60199136500776 41.64460954315057, -87.6012377034953 41.64461358249147, -87.60037752402501 41.64461243642212, -87.60007781874366 41.64461118300199, -87.59981235796114 41.64461009382519, -87.59924721506903 41.64460887295809, -87.59884844931486 41.64460948099843, -87.59863883476669 41.6446102830656, -87.5984389487664 41.64461136430349, -87.59823892415 41.6446117326968, -87.59783792883097 41.64460570908425, -87.59782876320089 41.64460557201519, -87.5978285808358 41.64461948382008, -87.59782998180755 41.64470250942654, -87.5978861538993 41.64802547930275, -87.59784301411972 41.64801565292835, -87.59783454737642 41.64801373259274, -87.59782052365441 41.64799854892936, -87.5978004377602 41.6479759167258, -87.59778969056617 41.64796218108778, -87.59776156052197 41.6479583785299, -87.59771632681695 41.64797856189939, -87.5976486127352 41.64801934865478, -87.5976137983493 41.64804991742876, -87.5975895941932 41.64806123388362, -87.59751645302482 41.64808376333618, -87.59741589292429 41.64811072770274, -87.59730247753629 41.6481355788435, -87.59703867840274 41.6482057363812, -87.59688578038046 41.64824691005845, -87.59676055819494 41.64828035716054, -87.59667196398098 41.6483039396729, -87.59663758607918 41.64830898849119, -87.59657992494965 41.64831515046124, -87.59643729555623 41.64834777210086, -87.596339397592 41.64837222778794, -87.5961721375254 41.64841500992097, -87.59600148130932 41.64845398282134, -87.59589136319538 41.6484783597346, -87.59572320140568 41.64851674446751, -87.59551666739141 41.64856388411326, -87.59527035567015 41.64862108672747, -87.59523300171988 41.64863039720052, -87.59517977965317 41.64865151623512, -87.59509528894118 41.64867452012442, -87.59485913806822 41.64873261041983, -87.59467303618891 41.64877828809331, -87.59452964258088 41.64880717013277, -87.59434956094536 41.64883828603735, -87.59422849886818 41.64885968198111, -87.59418293442596 41.6488638348257, -87.59410230691617 41.64886891478006, -87.59391955139863 41.64889704891559, -87.5937791927216 41.64891645409144, -87.5936716123083 41.64892355579939, -87.59355507080274 41.64892077496089, -87.5933776825898 41.64891469329723, -87.59324234538195 41.64890454605652, -87.59318044222493 41.64890403767925, -87.59314404753279 41.64891900700901, -87.59311589566647 41.64892349110922, -87.59302741963816 41.6489301663743, -87.59298367504555 41.64892236510077, -87.59298038100006 41.64892245365736, -87.59295183344351 41.64892314796644, -87.59289641439675 41.64892515111721, -87.59279927071297 41.64892189071234, -87.59271720409038 41.64892481983024, -87.59258806893656 41.64892903728551, -87.59242885837217 41.64893317050888, -87.59233005332472 41.64893439973132, -87.59225494796877 41.6489301283145, -87.5921603366682 41.64891661983035, -87.5920917102107 41.6489089322043, -87.59206939286881 41.64890236650263, -87.59202519516347 41.64888303567489, -87.59191149222626 41.64884969942502, -87.5918085881635 41.64881912219285, -87.59170462626211 41.64878508016289, -87.59153227527086 41.64873544788442, -87.59145919522605 41.6487077522747, -87.59133461384762 41.64865952575305, -87.59128798345233 41.64863540376948, -87.59123361148922 41.64862270316569, -87.5911847598325 41.64861997272467, -87.59113761615255 41.64860232403062, -87.59103611547199 41.64855155675664, -87.59094251821936 41.64850078555342, -87.59090496596882 41.64848270473038, -87.59082707985239 41.64845936863419, -87.59073965924875 41.64843152504053, -87.59068114126558 41.64841028990595, -87.59066192269931 41.64840154847293, -87.59062229523326 41.64837939251151, -87.5905931686419 41.64835406606017, -87.59058359814274 41.64834632003738, -87.59054040483549 41.64832243950842, -87.59050013971463 41.6482984681133, -87.59046935802036 41.64827708279724, -87.59044848673021 41.64824659539839, -87.59042607695089 41.64821626271279, -87.59041092958282 41.64819711902278, -87.59038836776701 41.64815454554771, -87.59034484400573 41.64809575468168, -87.59030150263116 41.64804020331037, -87.59028757466943 41.64802320807915, -87.59025521240349 41.64799928768568, -87.5902481014636 41.64799402744998, -87.59023212154656 41.64797092648998, -87.59023124345134 41.64795182013925, -87.59022372753535 41.64793722649488, -87.59020770098408 41.64793092064998, -87.59016245674155 41.64788177870081, -87.59011120049671 41.64782090692968, -87.59009804156494 41.64780704520636, -87.59005603785231 41.64776884668051, -87.59001092880338 41.64773342729388, -87.59000032521763 41.64773280985047, -87.58999001368794 41.64773224918184, -87.58996851512251 41.64771816884868, -87.58995027967039 41.64769395552147, -87.5899288681977 41.64767230133868, -87.58990401761082 41.64766972558269, -87.58988894173449 41.64766984762059, -87.58988088339211 41.64767061880373, -87.5898762735804 41.64767058898695, -87.58985889346756 41.64766109089249, -87.58984496254804 41.64764437002275, -87.58981885997326 41.64761703211472, -87.58980786221476 41.64760614823549, -87.58979963301927 41.64758995822584, -87.58979753060805 41.64758824312887, -87.58979039513889 41.64758512329882, -87.58977592484896 41.6475834928555, -87.5897657364422 41.6475658631073, -87.58976385544041 41.647563984781, -87.58974550335371 41.64754992476078, -87.58970422853395 41.64751837214737, -87.58969487146742 41.64751117630066, -87.58967526354805 41.64749178410552, -87.58957908933753 41.64739807348228, -87.58949990861693 41.64732203658996, -87.58946498170971 41.64728574980061, -87.58933046876579 41.64721067197183, -87.58931167129523 41.64716537829607, -87.58929336323365 41.64712843060899, -87.58927440684049 41.64708423363787, -87.58921733014182 41.64700784564999, -87.58919538808787 41.64698146761181, -87.58913290340492 41.64693627527343, -87.58904565215062 41.64686852863477, -87.5889115833325 41.64676134429187, -87.58889025125187 41.64674556333678, -87.58879243660043 41.64681359342073, -87.5887823954037 41.64683405611689, -87.58878668427811 41.64685559960335, -87.58880041609549 41.64687050804213, -87.58819903176591 41.64635808293283, -87.58815591611994 41.64632130356046, -87.58793229165326 41.64606737333913, -87.58781967498561 41.64594676973782, -87.58758752880468 41.64572769170093, -87.58747424598222 41.64560785185241, -87.5873044310201 41.64542820235027, -87.58716171801824 41.64525893751866, -87.58709197826101 41.64515787687035, -87.58692969933 41.6449531374766, -87.58700429340841 41.64491251146776, -87.58693885475722 41.64483826351376, -87.58689410388044 41.64477847538205, -87.58686453216779 41.6447349225945, -87.58684273627692 41.64469603080867, -87.58683810422465 41.64468774798232, -87.58567208245745 41.64470423010583, -87.57857791650851 41.64469595751432, -87.57837707055727 41.64469463851191, -87.57725061654718 41.64470874975163, -87.57486903974991 41.64467999617305, -87.57372555908472 41.64468281789735, -87.57326779388276 41.64468204354664, -87.57321650139508 41.64468197797206, -87.57275866097154 41.64468125488528, -87.57251719402311 41.64468086436036, -87.5720593536569 41.64468013848808, -87.57200806145057 41.64468001831234, -87.57155029648935 41.64467929091162, -87.57130875524307 41.64467884246831, -87.57085311140892 41.64467807009367, -87.57079464599553 41.64467801211502, -87.57033900365757 41.64467723853539, -87.57009753534824 41.64467684319209, -87.56964189131614 41.64467612227329, -87.56958342555222 41.64467606340171, -87.56912778369002 41.64467528419993, -87.56888631608879 41.64467483088064, -87.56843059890603 41.64467404978605, -87.5683721345352 41.64467393542052, -87.56791641857521 41.6446732071822, -87.56767502317422 41.6446728072346, -87.56721930617729 41.6446720759507, -87.56716084328301 41.64467196097885, -87.56670519979413 41.64467117216167, -87.56646373099697 41.64467082379283, -87.56600801336511 41.64467008797607, -87.56594955048404 41.64466997238814, -87.56549390708001 41.64466918014136, -87.56525243898697 41.64466877434528, -87.56479672328494 41.64466803373832, -87.56473825897015 41.64466791615242, -87.56428268883894 41.64466711931916, -87.56404122079635 41.64466671125303, -87.56358550519526 41.64466596584397, -87.56352704087752 41.64466584901407, -87.56307139731476 41.64466504688748, -87.56282992932928 41.64466463600231, -87.56237421484954 41.64466383091069, -87.56141292734007 41.64464903555573, -87.55931836251962 41.64464354033557, -87.55875351827069 41.64463882913788, -87.55298618768957 41.64459721211568, -87.55199850475726 41.64459020085609, -87.54980303097943 41.64457306012277, -87.54248234327059 41.6445650498369, -87.54214462933011 41.64456363081042, -87.54180702335141 41.64456224062514, -87.54165362313471 41.64456159526566, -87.54108568075132 41.64455921864114, -87.54068068243785 41.64456084539284, -87.54010982804853 41.64456319213433, -87.53973867364513 41.64456471842666, -87.53902451088774 41.64456387476578, -87.53797964304516 41.64456285111817, -87.535029752567 41.64455809939604, -87.53020107341766 41.64457284429623, -87.52993767584725 41.64457369356813, -87.52784347706597 41.64456005150208, -87.52516587091122 41.64454312178303, -87.52511602695519 41.64867183202863, -87.52511438623023 41.64880898199662, -87.52507798942742 41.65182602312922, -87.52499066393553 41.65905597979955, -87.52483544865379 41.67364558873752, -87.52483114152241 41.67398393545896, -87.52481257520751 41.67544982676955, -87.52480830724122 41.67578679420264)), ((-87.52454008405145 41.70535130961687, -87.5244616011821 41.7028939295397, -87.52465485050482 41.69222140354702, -87.52465931195958 41.69227548414981, -87.52465917729486 41.69234463961325, -87.52465656697082 41.69266993392658, -87.52465298934854 41.69291758340532, -87.52464746503807 41.69327381314839, -87.52464538075589 41.6936126944134, -87.52464307951018 41.69398640068881, -87.52464184729979 41.69412574873886, -87.52463880402256 41.69465505258447, -87.52463845008951 41.69474769842751, -87.52463770144811 41.69494764368149, -87.52463735311105 41.69503683143846, -87.52463672127588 41.69520134939957, -87.52463639008909 41.69528913821778, -87.52461324448755 41.69542100523524, -87.5245962199448 41.69723883376017, -87.52456217259946 41.69911450251254, -87.52455742011951 41.6992307448301, -87.52455490568495 41.69932337539461, -87.52455094529125 41.69954300347936, -87.52454853531844 41.69979667203912, -87.52454698100182 41.6999674121563, -87.52454501271268 41.70017711979821, -87.52454298324828 41.70039176595469, -87.52454170944941 41.70052836841435, -87.52454043851144 41.70066491710222, -87.52453837126929 41.70086784555264, -87.52453725654327 41.70097719857468, -87.52453575037623 41.70121728822747, -87.52453403667869 41.70148547906459, -87.52453206528698 41.70178311407345, -87.52453140849403 41.70189688920424, -87.52453040157785 41.70206197954037, -87.52453017559432 41.70210926256973, -87.52452738884649 41.70260335815456, -87.52452945146612 41.70305231606782, -87.52453298088972 41.70381978849921, -87.52453328756832 41.70388592955382, -87.52453533192153 41.70432462294128, -87.52453715662851 41.70471962493357, -87.52454008405145 41.70535130961687)), ((-87.52498580471448 41.70886773992004, -87.5248680019171 41.7088008306192, -87.5246467020678 41.7086896303405, -87.52464185074844 41.70853773049724, -87.52486207636343 41.70865243201794, -87.52487371701494 41.70865874378754, -87.52488859478957 41.70866677925725, -87.52491126352295 41.70868442085335, -87.52492939405666 41.70870480090618, -87.52494236806552 41.70872728634526, -87.52494979828101 41.70875115900474, -87.52495044832256 41.70876063174263, -87.52495143245515 41.70877562235722, -87.52495116629574 41.70877929841728, -87.52496138801942 41.70881704978397, -87.52497726145916 41.70885368856855, -87.52498580471448 41.70886773992004)), ((-87.52542496560891 41.70924363180946, -87.5253277022331 41.7090619299091, -87.52515741600918 41.70896521121956, -87.52537429883073 41.70882695173766, -87.52539126763729 41.70882602793073, -87.52540766800573 41.70882943669311, -87.52542156537294 41.7088367791217, -87.5254312894107 41.70884705661803, -87.52555898311884 41.70897081678954, -87.52555611498784 41.70898048406372, -87.52567873974012 41.70908502624793, -87.52542496560891 41.70924363180946)), ((-87.52584161691678 41.70977765694717, -87.5258041021083 41.7097617302591, -87.52587460265519 41.709487130315, -87.5256873025827 41.7093490304081, -87.52553477031866 41.70931113820743, -87.52576947560975 41.70915652218181, -87.52622020679605 41.70953944758732, -87.52584161691678 41.70977765694717)), ((-87.52688072088768 41.71688647110234, -87.5271748040016 41.7168091319155, -87.5279567035536 41.7163669315488, -87.5283865040151 41.7157563313207, -87.5282353035917 41.7151335309197, -87.5279950036998 41.7144100307188, -87.5274096032326 41.7134051308215, -87.52691480324761 41.7129560309253, -87.5263923032303 41.7124484310208, -87.5254169028242 41.7132183309184, -87.5253495031051 41.7131318314149, -87.52665420293449 41.7120419310762, -87.5279847034214 41.7109496304451, -87.5271385031176 41.7103128303338, -87.5264720135666 41.70982823871974, -87.52654791305973 41.70983538980661, -87.52664838183979 41.70984926785798, -87.52668624416765 41.70985565346976, -87.52679398364877 41.70988259076262, -87.52689969352801 41.70991363073356, -87.52700312041576 41.70994874169706, -87.5271040440734 41.70998778874032, -87.52720209864592 41.71003068368071, -87.527204028204 41.7100315751235, -87.52840800713254 41.71080310043663, -87.52643276116325 41.71231346735096, -87.52742306745611 41.7130845436098, -87.52750256038831 41.71316531740026, -87.52757729921734 41.71324863663127, -87.52764706291109 41.71333430958126, -87.52771178140395 41.71342219745449, -87.5277713116851 41.71351210742926, -87.52782554447825 41.7136038727165, -87.52785131585431 41.71365174954569, -87.5279151394079 41.71376161298255, -87.5279735210362 41.71387313944242, -87.5280264613361 41.71398619117306, -87.52804198153801 41.71402214177666, -87.52806684755424 41.71409586282302, -87.52809736710257 41.71416836246836, -87.52813339977186 41.7142394179838, -87.52817483623087 41.71430881069963, -87.52822157158587 41.71437631840909, -87.52827342504601 41.7144416665055, -87.52827912862462 41.71444834797816, -87.52837594447463 41.71457350829189, -87.52846760746958 41.71470077299456, -87.52848444344455 41.7147251778185, -87.52852619080997 41.71479627350631, -87.52856253193364 41.7148690336135, -87.52859332523541 41.71494320879322, -87.52861846319983 41.71501857875183, -87.52865011025727 41.71514470892303, -87.52867277401651 41.71525398152868, -87.52868903833861 41.71530964032093, -87.52869957947416 41.71536605488589, -87.52870425431585 41.71542292207409, -87.52870310391123 41.71547988623623, -87.52869606020333 41.71553661757793, -87.5286832364161 41.71559275875811, -87.52867301434182 41.71562539939107, -87.52867354274662 41.71564754942406, -87.52867373033401 41.71565603093972, -87.52866858856858 41.71568645681012, -87.52865770857167 41.71571599121131, -87.52864131615148 41.71574406078937, -87.52861971079004 41.71577011740442, -87.52861637146276 41.71577349696725, -87.52856500083212 41.71581331367401, -87.52856719639948 41.71585155815492, -87.52842673661321 41.71599204260043, -87.52835398797538 41.71607191419575, -87.52827647107998 41.71614922729491, -87.52819437401804 41.71622381909634, -87.52818882085917 41.71622860962847, -87.52809759626388 41.71630670555987, -87.52800205057142 41.71638183334665, -87.5279023283241 41.71645388641421, -87.52779865756754 41.71652270031951, -87.52775727335201 41.71654859094402, -87.52764140593787 41.71662022872012, -87.52752191724564 41.71668843721262, -87.52739899285827 41.71675310931164, -87.52729016880089 41.71680602503162, -87.52730132382672 41.71682498469819, -87.52700288186853 41.71694803125675, -87.52698853865077 41.71694971441084, -87.52697443456738 41.71694703523563, -87.52696286888974 41.71694045087728, -87.52696276185671 41.71694034007928, -87.52693548640978 41.71693054385236, -87.52692394558972 41.71692777288667, -87.52691394823519 41.71692471115176, -87.52690743682621 41.71691824403855, -87.52690603219976 41.71691337674797, -87.52689508841979 41.71689812444552, -87.52688072088768 41.71688647110234)), ((-87.52488204237149 41.71739683060795, -87.5248963026057 41.7170877321665, -87.5256438031211 41.7170709318873, -87.52632570346771 41.7170324318662, -87.52677222610225 41.71691500350725, -87.52676871875704 41.71692182599689, -87.52679668179752 41.71699090478609, -87.52651900555276 41.71708462191785, -87.52660512662301 41.71723619091497, -87.52651698749804 41.71726658336961, -87.52642112987222 41.71711442451666, -87.52580496007398 41.71731761979469, -87.52581256064543 41.71733076440012, -87.52554925541482 41.71736782779457, -87.52534557399251 41.71738176419972, -87.52514143841978 41.71739130560127, -87.52493702453877 41.71739648232361, -87.52488204237149 41.71739683060795)), ((-87.52546304663116 41.71877854699682, -87.5253861033684 41.7186603319064, -87.5250497028389 41.718397131793, -87.52484490284409 41.7181635319633, -87.52485097565877 41.71803924102973, -87.52485829860461 41.71804051242687, -87.52490509672113 41.71805028183826, -87.52499797946142 41.71813845082073, -87.52510257697173 41.71825354259176, -87.52520230970909 41.71837101367503, -87.52529414887974 41.71848703038686, -87.52535220923149 41.71857443442831, -87.52540493500838 41.71866372002875, -87.52545225493644 41.7187546981601, -87.52546304663116 41.71877854699682)), ((-87.52490324560264 41.72125198867082, -87.524906602952 41.7212373325055, -87.52533254340941 41.72106016952868, -87.52526781571099 41.72117086444844, -87.52523637393179 41.72122149540445, -87.52515350578233 41.72133647467579, -87.52509273636596 41.7214153858043, -87.52490324560264 41.72125198867082)), ((-87.52481554399539 41.72170897947378, -87.52481820320401 41.7216232325662, -87.5248684787078 41.72140375999523, -87.52500267167744 41.72152279520706, -87.52495303909942 41.72158106533517, -87.52489852937629 41.72163685814291, -87.52483943898503 41.72168992844714, -87.52481554399539 41.72170897947378)), ((-87.52967879762068 41.73914216229244, -87.529558206296 41.738347435348, -87.5292304059124 41.7373385352515, -87.52926330573349 41.7364541346027, -87.5289317055383 41.7336696343364, -87.52901724476324 41.7336332995785, -87.52899210413553 41.73369994955407, -87.52900071177625 41.73371480161183, -87.52899444262744 41.7337181609387, -87.52898301920506 41.73372332286778, -87.52896777468585 41.7337324644567, -87.52896143083041 41.73373878655894, -87.52895809641515 41.73374760015172, -87.52895527562634 41.73376819068154, -87.52896099670049 41.73379122827977, -87.52897858658558 41.73379681176005, -87.52900618471415 41.73379302571245, -87.52902079243022 41.73379968740464, -87.52902414293418 41.73381598376148, -87.52900945171193 41.73384540964012, -87.52901387789767 41.7338635258658, -87.52902140949189 41.7338912134604, -87.52902250857332 41.73391183143048, -87.52901898108212 41.73394199353969, -87.52902967518328 41.73397165227614, -87.52903557021351 41.73400127714415, -87.52904139514195 41.73404248276065, -87.52904417255928 41.73407537869603, -87.52904998958421 41.7341142233311, -87.52905663686686 41.73415112614428, -87.52906679661857 41.73418256493392, -87.52907632057303 41.73422382415664, -87.52908249162827 41.73425485054837, -87.52907343130947 41.73427037368192, -87.52906302890453 41.73429955625206, -87.52906529992724 41.73433480819387, -87.52907155142553 41.73438575879857, -87.52908130588504 41.73446434244665, -87.52909564549965 41.73449265426468, -87.52909178724678 41.7345199319896, -87.52909449313046 41.73454975489736, -87.52910323308666 41.73459216096124, -87.52910416026243 41.73462350713133, -87.52910538466732 41.73466336245043, -87.52911440645823 41.7347008584552, -87.52912402608692 41.73473745219967, -87.52911997953105 41.73477982392617, -87.52912455907935 41.7348474756344, -87.52913963554992 41.73486953449138, -87.52914994534984 41.73489770914721, -87.52914105136151 41.73492045151713, -87.52912983228562 41.73493873266336, -87.52912182775076 41.734972349799, -87.52913316014035 41.73504205128746, -87.52917268099706 41.73533080995171, -87.5292159561379 41.73553885579715, -87.52921627144666 41.73553998371724, -87.52922229957125 41.73557073438972, -87.52923060430426 41.73561288985489, -87.529228404276 41.73565999470384, -87.52922990129446 41.73570440774124, -87.5292387984686 41.73576665592886, -87.52924874663098 41.73582671602708, -87.52926552062806 41.73586544576504, -87.52927734746477 41.73589544198714, -87.52928084306085 41.7359353680754, -87.52927273610828 41.73595378105676, -87.52925292098166 41.73598267657498, -87.52924227141962 41.73601097842236, -87.52925847389417 41.73603085079468, -87.52927689583028 41.7360549915435, -87.52929106847294 41.73609084954555, -87.5292998964173 41.73615548423531, -87.52930931895926 41.73623431223147, -87.52931916985158 41.73631695725625, -87.52933471788118 41.73655387172209, -87.52936199377504 41.73669654763165, -87.52938193286612 41.73681930179423, -87.52939539493532 41.73693550745705, -87.52940250819073 41.73700268343227, -87.52940795812852 41.73707102754441, -87.52942512829992 41.73716025410325, -87.52943138432914 41.73723159541719, -87.52943590871998 41.73729765472937, -87.52944181911563 41.73735837262139, -87.52944587940023 41.73743532384199, -87.52945321977431 41.7374990153095, -87.52944182772626 41.73751641736477, -87.5294309185165 41.73755086409498, -87.52944926331681 41.73759588951398, -87.52946796134188 41.73763304152611, -87.52947523925896 41.73766360909195, -87.52948537241699 41.73770602375471, -87.52949070865111 41.73775408638714, -87.52949152365612 41.73777972442626, -87.52948549034552 41.73783456744981, -87.52950138295579 41.73787057411521, -87.52951123944433 41.73790288862785, -87.52951071242487 41.73793054778716, -87.52951251627748 41.73796494679452, -87.52953296543937 41.73801473428833, -87.52953990540965 41.73805460267018, -87.52954684162181 41.73809186421194, -87.52953984156017 41.73812142571942, -87.52954661285713 41.73817194218663, -87.52955755513113 41.73820527858366, -87.52957201917876 41.73825598525463, -87.52956874579112 41.73828930453498, -87.52955759435046 41.73832251491634, -87.52958803031754 41.738408790252, -87.5296358229334 41.73880142177281, -87.5296432959658 41.73885441163713, -87.52965754526031 41.73895462427782, -87.52967493864001 41.73906421615368, -87.52967879762068 41.73914216229244)), ((-87.52972544739842 41.7394495959774, -87.52970019858989 41.73928320002396, -87.52970436242495 41.73930803338369, -87.52971127952756 41.73933212184973, -87.5297128661941 41.73935773728576, -87.52972469721713 41.73938737561772, -87.52972505436084 41.73942047494753, -87.529726052736 41.73944619604057, -87.52972544739842 41.7394495959774)), ((-87.5297828218261 41.73982770778377, -87.52977927901351 41.73980435976401, -87.52979805846066 41.73980716369341, -87.52981391056328 41.73981405270386, -87.52980346279136 41.73982037441151, -87.5297828218261 41.73982770778377)), ((-87.52955500761527 41.74133224804854, -87.5294519066853 41.7412571356779, -87.5296597067908 41.7411998353524, -87.5301234072673 41.7412046359086, -87.5305332068443 41.7410376353506, -87.5345733079983 41.7409707354483, -87.53985261184145 41.74086378363929, -87.53985520153661 41.74101300660979, -87.5368487389789 41.74104544411111, -87.53380414854898 41.74107918364965, -87.53362751279799 41.74108108260976, -87.53176001583857 41.74110116626266, -87.5313334642458 41.74110477573763, -87.5306869584257 41.74111672788512, -87.5306671403056 41.74111933401552, -87.53062695963665 41.74112986585828, -87.53053478874475 41.74116078108463, -87.53018877760557 41.74126404546371, -87.53016677226562 41.74123921982606, -87.53014689743327 41.74123458161254, -87.53012605334909 41.74123424297744, -87.53010952739244 41.74123717417321, -87.52988666811493 41.74132875649084, -87.52955500761527 41.74133224804854)), ((-87.53003292372365 41.74194418103618, -87.52997830705129 41.7416406354136, -87.52967400964809 41.74141894488822, -87.52971126452073 41.74141405900268, -87.52972981440151 41.74141026411792, -87.52977133375714 41.74142166864408, -87.52981049461454 41.74143709078415, -87.52984660426922 41.74145625126626, -87.52987900805937 41.74147881702255, -87.52990715974911 41.74150439866225, -87.52991969809567 41.74151845510312, -87.52992540976061 41.74152768844991, -87.52997880579784 41.74160051052672, -87.5300031440125 41.74165265857953, -87.53002189862141 41.74170608403028, -87.53003492534005 41.74176048509746, -87.53003987301696 41.74179268207287, -87.5300367016259 41.74189134578622, -87.53004236512336 41.74190732867253, -87.53004226806026 41.74192387695179, -87.53003635338213 41.74193983514885, -87.53003292372365 41.74194418103618)), ((-87.53005148537112 41.74204734196842, -87.5300451264617 41.74201200076151, -87.53004541960173 41.74201204565572, -87.53005560631608 41.74201793534057, -87.53006128817545 41.74202653700794, -87.53006135279962 41.7420361691325, -87.53005574708845 41.74204482992443, -87.53005148537112 41.74204734196842)), ((-87.53007004104309 41.74215046969034, -87.53005442175804 41.74206366166274, -87.53006007948055 41.7420706843586, -87.53007047526918 41.74209208052914, -87.53007521503977 41.74211456122285, -87.53007412435612 41.74213730440052, -87.53007004104309 41.74215046969034)), ((-87.53092742998767 41.74798692678593, -87.5308886080128 41.7479690370885, -87.53084852450201 41.74766288800275, -87.53086410873844 41.74772404377345, -87.53088537177437 41.74784109965907, -87.53090240676852 41.74788567598938, -87.5309117712642 41.74795764200285, -87.53092742998767 41.74798692678593)), ((-87.53570145781597 41.75000201705647, -87.5356023089753 41.7499614366009, -87.5355313092812 41.7499989365419, -87.5333627085538 41.7491091364843, -87.53273200574242 41.74881849999549, -87.5327362554151 41.74882000429029, -87.53302853579433 41.74895047341823, -87.53319402012411 41.74902188056052, -87.53323271268296 41.74903422382565, -87.53339476593402 41.74908694528571, -87.53352744108432 41.74914459439978, -87.53356680733543 41.74916169097499, -87.53376212319812 41.7492508670813, -87.53389916598191 41.74931109808309, -87.53421381688054 41.74941098343512, -87.53432200049328 41.74943423961687, -87.5343808145599 41.74944631105419, -87.53443514233398 41.74948628986418, -87.53446240510968 41.74949526118015, -87.53452406854564 41.74950803870482, -87.53456220574945 41.74951500020967, -87.53458946953764 41.74953571682224, -87.53467830173933 41.74957395866793, -87.53474273162834 41.74958538370541, -87.53476726873006 41.74959255193593, -87.53480286930466 41.74961486379722, -87.53483307615592 41.74962887782598, -87.53486761355741 41.74964525432086, -87.53490989286821 41.7496638794948, -87.53495686038012 41.74968259266537, -87.53499931986437 41.74970143915102, -87.53504714174812 41.74972224498507, -87.53507191398693 41.74973718059768, -87.53510614515294 41.74973986057346, -87.53513465237214 41.74974601374904, -87.53515722564289 41.7497609074468, -87.53518214320378 41.74978476411016, -87.53520380876837 41.74980184693113, -87.53523049813801 41.74979508953062, -87.5352578540322 41.74979363352251, -87.53528588214134 41.74980598489635, -87.53530849093386 41.74981511551164, -87.53533473496842 41.74982943147388, -87.5353781763335 41.74984600632317, -87.5354387363802 41.74986497808091, -87.53548652686652 41.74987656212509, -87.53551886589007 41.74989585955651, -87.53554127176893 41.74990954452539, -87.53557716198949 41.7499232679572, -87.53560470770564 41.749939101985, -87.53562240272566 41.7499572551522, -87.53564729116117 41.74996862495817, -87.53567720675368 41.74997366179185, -87.53568843231882 41.74998452536324, -87.53570021935565 41.75000035863785, -87.53570145781597 41.75000201705647)), ((-87.53623420974225 41.75022006616049, -87.53618515606409 41.75019998906636, -87.53621159641975 41.75020086614878, -87.53623272501443 41.75021402070437, -87.53623420974225 41.75022006616049)), ((-87.53685627908196 41.75047467183241, -87.53679362817216 41.75044902955113, -87.53680020960184 41.7504472701034, -87.53682409651788 41.75044192021392, -87.53683844599983 41.75044627252914, -87.53684938988391 41.75046212810643, -87.53685627908196 41.75047467183241)), ((-87.53712949870712 41.75058649741489, -87.53708364938565 41.7505677318263, -87.5370911124549 41.75056882753873, -87.53711902883057 41.75057242393075, -87.53712714112284 41.75058326606052, -87.53712949870712 41.75058649741489)), ((-87.53762364829622 41.75078874703585, -87.53753880408162 41.75075402129566, -87.53753990855147 41.7507542683109, -87.53758935067768 41.75076564282752, -87.53761810728084 41.75078154019305, -87.53762364829622 41.75078874703585)), ((-87.53866459438991 41.75118562343633, -87.5381130101246 41.7509890370826, -87.53799407368072 41.75094035779352, -87.53806123507397 41.75095719659172, -87.53812700537178 41.75098182880551, -87.53819185689461 41.75100376548621, -87.53826231130876 41.75102897944787, -87.53834074885418 41.75106056021396, -87.53840986293115 41.75108754860408, -87.53850128389948 41.75111842296828, -87.53859759947156 41.75115649373219, -87.53866202256414 41.75118342153046, -87.53866459438991 41.75118562343633)), ((-87.54108626338382 41.75774406891985, -87.54105541172861 41.7577442380755, -87.5404232114724 41.7571964375755, -87.5437335123749 41.7551380372629, -87.5442624127097 41.7540272367192, -87.5438147119708 41.7529502372545, -87.5427823114459 41.7519797364564, -87.54211186157842 41.7519245387587, -87.54214296681124 41.75192571017207, -87.54218430947368 41.75192832833769, -87.5422586168133 41.75192057965369, -87.5423309586286 41.7519176742468, -87.54242868147819 41.75191897872951, -87.5425000737668 41.75191872864798, -87.54252861570315 41.75192526497886, -87.54260213469404 41.75192793842685, -87.54265487638477 41.75192146837247, -87.54266658524709 41.75192003958677, -87.54279550639602 41.75192040529757, -87.54286940904724 41.75192475523564, -87.54290594706157 41.75191869500068, -87.54293804412518 41.75192813719988, -87.54297801603489 41.75192607869815, -87.54301414499383 41.75191757346475, -87.54302699751099 41.75191546643484, -87.54304426713658 41.7519207711226, -87.54306856724502 41.75193240997798, -87.54310403090241 41.75194829694246, -87.54315216181831 41.75197126732702, -87.54318778243099 41.75198924101097, -87.54322607875227 41.75201318793479, -87.54328438515502 41.75205497288621, -87.54335562912176 41.75211471186385, -87.54335584488874 41.75211498805157, -87.54337588267126 41.75213367699845, -87.54340942038888 41.75216019822499, -87.54344422320709 41.75218513589799, -87.54347043982243 41.75220499329244, -87.5434910299556 41.75222039340865, -87.54351442718743 41.75224593845122, -87.54353716119775 41.75227787396535, -87.54356758746735 41.75232201903802, -87.54360702558738 41.75237536482975, -87.54367076585817 41.75246425190169, -87.54371608660797 41.75253418537364, -87.54378993554046 41.75268335117583, -87.54386732864435 41.75285444214145, -87.54392694294106 41.75297807068058, -87.5439976469284 41.75313890488659, -87.54406478168494 41.75326788233153, -87.54414661294756 41.75344471062184, -87.54420987176736 41.7536119434661, -87.54425409969646 41.75372034451968, -87.54434089739333 41.75397056181102, -87.5444132131323 41.75413157173808, -87.54434162049468 41.75432148021542, -87.54433089086631 41.7543177017325, -87.54432132142 41.75431821259249, -87.54431321621495 41.75432158675205, -87.54430503519758 41.75433130026341, -87.54429230930079 41.75435286563857, -87.54427532654761 41.75437799680508, -87.54426263912977 41.75440255290052, -87.54425938772391 41.75441959981773, -87.54425004410952 41.75444936559617, -87.54423886639809 41.75447022890413, -87.54422577149549 41.75449516694673, -87.544212357036 41.75451612301467, -87.5442042506883 41.75454370239881, -87.54420961209077 41.7545637179728, -87.54420735080564 41.75458373635155, -87.54419076430472 41.75460340852803, -87.54417129348721 41.7546102175851, -87.54415139268484 41.75461625583136, -87.54415412330873 41.75462667546959, -87.54416485736614 41.75465399951036, -87.54416944283115 41.7546715676071, -87.54416878628717 41.75468629969262, -87.54415437321286 41.75470796268519, -87.54415831393659 41.75472440064338, -87.54417196006175 41.75474724513776, -87.54418075359473 41.75476846378501, -87.54417809562901 41.75479695848905, -87.54416328284142 41.75481540791053, -87.54414104159144 41.7548448671892, -87.54411374735639 41.75487708873987, -87.54408230963938 41.75491540297236, -87.54404224380691 41.75496090313808, -87.54404036815608 41.75496146628036, -87.54403004768349 41.75496027076739, -87.54401018060618 41.75496043587829, -87.54399515808082 41.75496620464079, -87.54398153502447 41.75497728115376, -87.54396536188227 41.75499311467066, -87.54395141138259 41.75501000601418, -87.54394067567584 41.75502475090134, -87.54393347810549 41.75504679275562, -87.54393994975743 41.75505707316967, -87.54395022116118 41.75506828585661, -87.54395794307099 41.7550931204507, -87.54395330142805 41.75511303904146, -87.54395952816877 41.75513119390051, -87.54397072109677 41.75514194638658, -87.54397819523581 41.7551510813561, -87.54397716844535 41.7551660855995, -87.54397321360719 41.75518074068484, -87.54397016724253 41.75520207092477, -87.5439678147994 41.755217586653, -87.54396130288494 41.75523466714149, -87.54394554286674 41.75525566222025, -87.5439149799528 41.75527630942453, -87.54385099775607 41.75532049176211, -87.54380463617478 41.75535041476847, -87.5437616536511 41.75538263956646, -87.54367597252077 41.75543007615755, -87.54356718383954 41.75550284780273, -87.54348216359116 41.7555530322629, -87.54347623999813 41.75555496715806, -87.5434514782344 41.75556585725825, -87.54342446846223 41.7555806551424, -87.54339766776097 41.75560234236494, -87.54336605333891 41.75562218689489, -87.54333740984681 41.75563867443967, -87.5433023965825 41.75566066302606, -87.54327080595785 41.75567850271088, -87.54324128845532 41.75569169216442, -87.54320394951921 41.75571196244077, -87.54317418805546 41.75572997961032, -87.54314431104558 41.75575161872175, -87.54310090653586 41.75577343948733, -87.54306385279014 41.75579725291804, -87.54303047205781 41.75582067858566, -87.542970943678 41.7558512252955, -87.54291941809346 41.75588377465717, -87.5428074407036 41.75595051390486, -87.54270933648343 41.75601260087988, -87.54260981325423 41.75607380010402, -87.54260386418002 41.75607765551077, -87.5424952504391 41.75614793029844, -87.54237081811384 41.75621702525764, -87.5423234717065 41.75624057466345, -87.5422828035333 41.75626631195295, -87.54223634869132 41.7562946414369, -87.54219101623097 41.75632122420267, -87.54210203981144 41.75636814212016, -87.54203007341879 41.75641180218989, -87.54193896964864 41.75646477021998, -87.54184948387288 41.75650830949751, -87.54182770864722 41.75652064602893, -87.54180021432158 41.75653609964331, -87.54176816125332 41.75656164801682, -87.54169628857846 41.7566124977503, -87.54156510083347 41.75668538784275, -87.54143511398556 41.75676786447343, -87.54137908979237 41.75680545907469, -87.54129544424752 41.75685098689723, -87.54126108032521 41.75686762740723, -87.54123380688924 41.75688601770511, -87.54121608774088 41.75689926053197, -87.54120321573231 41.75690888713765, -87.54114053155062 41.75693968446366, -87.54112276084895 41.75694820591028, -87.54109840092218 41.75696812672491, -87.541083722816 41.75697244327657, -87.54105582759563 41.75698479234464, -87.54104359888981 41.75699563030918, -87.54105326266259 41.75700250307523, -87.54099638542246 41.75703504265176, -87.54098591039666 41.75703140227942, -87.54097542939856 41.75702836670747, -87.54095900646197 41.75702570125017, -87.54094789790368 41.75702598139531, -87.54093622443706 41.75702749312468, -87.54092878585993 41.75703032223283, -87.540915949668 41.75703992161007, -87.540888859227 41.75705825929916, -87.54084133511284 41.75708710225383, -87.54078047536194 41.75712150836435, -87.54073231880862 41.75715402481429, -87.54067019474222 41.75719588607597, -87.5406236590338 41.75722775558049, -87.54060503101408 41.75724642519535, -87.54059675912495 41.75726928352876, -87.54059888186609 41.7572903738084, -87.54060672738427 41.75730510964231, -87.54062495335148 41.75732518724064, -87.54064643539793 41.75734556118181, -87.54068223689359 41.75737591309741, -87.54074121072064 41.75742629273, -87.54082615673977 41.75750462429684, -87.54096983159506 41.7576413206189, -87.54102201942328 41.75768331229679, -87.54105908496901 41.75771512728695, -87.54108626338382 41.75774406891985)), ((-87.54222709918238 41.75836440611864, -87.54155296400737 41.75783251909526, -87.54164910998276 41.75782402655206, -87.54222709918238 41.75836440611864)), ((-87.54252619378701 41.75904308619526, -87.54247031204621 41.7589911383363, -87.54259160632479 41.75870519393501, -87.54262731203941 41.75873857461663, -87.54256973250885 41.7588580221914, -87.54253283879473 41.75893455414747, -87.54252661807119 41.75895761746078, -87.54252283125027 41.75898248241533, -87.54252197312826 41.75900747757459, -87.54252404904855 41.75903243667123, -87.54252619378701 41.75904308619526)), ((-87.54366583343577 41.76005322223165, -87.54363231302079 41.7600713384493, -87.54323878838174 41.75970551658782, -87.54324615698917 41.75970503683407, -87.54328026086016 41.75970071555557, -87.54366583343577 41.76005322223165)), ((-87.55480084838281 41.76176944507957, -87.55454881621409 41.7619091378369, -87.55417151624749 41.7610677374025, -87.55297791584231 41.7605716380676, -87.5520041152101 41.7601176377914, -87.5510973149995 41.7597667376072, -87.55002081445581 41.7592973373433, -87.5492647145978 41.75908393722, -87.5481781137875 41.7588404377003, -87.5471914133205 41.7588383378375, -87.5466145133679 41.7590551374724, -87.54530576675646 41.75951902562432, -87.54531540788682 41.75951147771453, -87.54533778055176 41.75949233850341, -87.54536856681869 41.75946554607517, -87.54540861518599 41.75943052884254, -87.54544757899563 41.75940033438974, -87.54550977526073 41.75936157238013, -87.54574504116482 41.75925080710907, -87.54606427075613 41.75909547199574, -87.54636392917655 41.75896703350493, -87.54659034461027 41.75887846209255, -87.54668017889338 41.75885144272618, -87.54669829451643 41.75884761388041, -87.54680192705031 41.75883108949251, -87.5468507363763 41.75882000791101, -87.54688283969732 41.75880227943588, -87.54690960019624 41.75878371935828, -87.54693676058186 41.75877161082867, -87.54697262268256 41.7587642823117, -87.54701754668878 41.75875624767588, -87.54708826642181 41.75874262571478, -87.54719984136446 41.75872697805949, -87.54724584425794 41.75872054158386, -87.54729304367487 41.75872108433247, -87.54733771743878 41.75871856334194, -87.54738256669904 41.75871950161989, -87.54744599648738 41.75872067647556, -87.54754290396666 41.75872054291651, -87.5475434910748 41.75872054693201, -87.54758111279945 41.75871913049262, -87.54764302546147 41.75871532769084, -87.54773259091534 41.75870734996645, -87.54781335955627 41.75869925741635, -87.54784760279719 41.75869227513914, -87.54788771072079 41.75868234029243, -87.54793793113342 41.75867571255426, -87.54797422912564 41.75867464424408, -87.54802778166966 41.75867723235586, -87.54806409028981 41.75867835870221, -87.54809638320434 41.7586780859929, -87.54813678111832 41.75867740131334, -87.54818129391469 41.75867893990143, -87.54840100245177 41.75869243427228, -87.54850628357921 41.75870289519944, -87.54857120474449 41.75871107757233, -87.54857892813253 41.75871184466656, -87.54861568513266 41.75871527910854, -87.54865279130748 41.75871706852365, -87.54870886675366 41.75872620634181, -87.5488047587348 41.75875227389285, -87.548875359333 41.75877256867346, -87.54892362691703 41.75878478157124, -87.54896660135886 41.75879226447108, -87.54899638521624 41.75879650219074, -87.549162281009 41.75884365725006, -87.54954549553207 41.7589563732809, -87.54998598674635 41.75912417180952, -87.55000971398856 41.75913212765453, -87.5501799970901 41.75919547422716, -87.5502456915241 41.75921834317439, -87.55033870653907 41.75925511865452, -87.55043300417573 41.75928594822969, -87.55050359909133 41.75930975631427, -87.5506233835994 41.75935448141053, -87.55069697123866 41.75938242593158, -87.55077293306289 41.7594140362139, -87.55082462549515 41.75943406406449, -87.55086481158465 41.75944484895707, -87.55099066624292 41.75949373117811, -87.55106913381412 41.75952124188583, -87.55116391663871 41.75956351787143, -87.55125692550796 41.75960073275066, -87.55135848763928 41.75963709933022, -87.55145267360557 41.75967709352524, -87.55152262012417 41.75970317406052, -87.55153783616028 41.75970876619056, -87.55161867843569 41.75973997058495, -87.55171714237011 41.75978087184433, -87.55172141040735 41.75978260179387, -87.55179680070854 41.75981297285259, -87.55195245278469 41.75987122210466, -87.55213676584007 41.75993559397934, -87.55225173844507 41.75997523570592, -87.55238231828889 41.76003970867318, -87.55246072095694 41.76007270685145, -87.55246801755099 41.76007549991462, -87.55256220099085 41.76011593309242, -87.55269391759425 41.76018636827387, -87.55278507534804 41.76022540773985, -87.55282392741725 41.76024350905127, -87.55286766893826 41.76026345534731, -87.55315811948681 41.76039213293082, -87.55324807514715 41.76043069623837, -87.55332707536945 41.76046279199059, -87.55338788798058 41.76048653188802, -87.55348631499329 41.76053066937149, -87.55358596778358 41.76057341577428, -87.55366800420258 41.7606091818091, -87.55373244846385 41.76063568991407, -87.55387278559837 41.76069976031484, -87.55390680236502 41.76071464644607, -87.55394088823917 41.76072676102476, -87.55399562611323 41.76074680810716, -87.55403269010922 41.76076122007522, -87.55406972476121 41.76078114868632, -87.55410598170643 41.76081385849918, -87.5541491925128 41.76087773794951, -87.55417542039714 41.76093052366247, -87.55420961261224 41.76098243059381, -87.55425114544062 41.76103348112662, -87.55427903409404 41.76105197495345, -87.55434745511747 41.76105287852351, -87.5543825255285 41.76108055814546, -87.55443406397058 41.76116507587597, -87.5545278058581 41.76129378727664, -87.55466960527184 41.76148866019611, -87.5547368034406 41.76163914614053, -87.55476748482982 41.76172991574726, -87.55480084838281 41.76176944507957)), ((-87.55630440236588 41.76202383953941, -87.5556105165599 41.7618654379138, -87.55508153259721 41.76170223528142, -87.5551653787733 41.76166125926266, -87.55526973776225 41.76162126845804, -87.55532482866154 41.76161203678206, -87.55535102009365 41.76161633030786, -87.55543434891483 41.76159675225207, -87.5554539881779 41.76158821325679, -87.55552131524949 41.76157950232204, -87.55559338110353 41.76157908457392, -87.55563970012346 41.76158900396985, -87.55573391001047 41.76162759439979, -87.5558067251026 41.7616683465419, -87.55589466687591 41.76172198913208, -87.55594319571809 41.76174932080821, -87.55598815223406 41.76177201788021, -87.55601787777765 41.76178731371224, -87.55607856966247 41.76182112127773, -87.55614758660849 41.76187282468189, -87.55619347659395 41.76191841539583, -87.5562109631165 41.76193637213409, -87.55623198824058 41.76196486344852, -87.55626159561069 41.76199297317072, -87.55630440236588 41.76202383953941)), ((-87.5573617443106 41.76270140031293, -87.5567406167319 41.7625231382571, -87.5565770164939 41.7623220381309, -87.55641436103426 41.76207987882976, -87.55641813864432 41.76208138217761, -87.5564576320965 41.76209811527184, -87.55653594540186 41.76213890275645, -87.55659841582883 41.76217727897072, -87.55664934595841 41.76220918265783, -87.55671948633159 41.76226819285711, -87.55675625237505 41.76231053828889, -87.55678517085342 41.76234410518578, -87.55683702836816 41.76239934072751, -87.55686964584439 41.76243021575254, -87.55690846379068 41.76245427138915, -87.55699217614739 41.76250330048526, -87.55702069207896 41.76251858651104, -87.55705890617647 41.76254126586977, -87.55710863187473 41.76257269440988, -87.55716624297277 41.76260694848575, -87.55723855031316 41.76264127491491, -87.55731807877062 41.76268160381765, -87.5573617443106 41.76270140031293)), ((-87.55843288704514 41.76399393677784, -87.5583256175935 41.7640151379824, -87.55791371714341 41.763611638214, -87.557462716852 41.7631550379684, -87.55745317472862 41.76274298266436, -87.55749484301251 41.76276512697514, -87.55751003853048 41.7627725557535, -87.55753312879158 41.7627818512691, -87.55760242890338 41.76281022756792, -87.55764110643587 41.76284616534655, -87.55764249516737 41.76293168634874, -87.55764967663968 41.7629957874601, -87.55770097823562 41.76304871309947, -87.55757247521532 41.76311508176334, -87.55831805454139 41.76387663319475, -87.55843288704514 41.76399393677784)), ((-87.55980299314305 41.76492868497542, -87.55963131767091 41.7649734376609, -87.5594064178209 41.764724437839, -87.55935441806869 41.7643514376638, -87.5586737173743 41.7639463380161, -87.55854517966493 41.76397174277232, -87.55866620402054 41.76390963441203, -87.55872407925931 41.76387937143503, -87.55880268459089 41.76383826991496, -87.55882482925406 41.76382563079819, -87.55897907052909 41.76375257486129, -87.55909438388142 41.76386540204449, -87.55911192562917 41.76388198573998, -87.55928041520828 41.76404138386302, -87.55934333130871 41.76404318015862, -87.55940117913138 41.76405773040786, -87.55943939461784 41.76408040898115, -87.55948769921409 41.76412966399397, -87.55952283006386 41.7641527882921, -87.55956524508913 41.76418279520144, -87.55960877186598 41.76422104136249, -87.55965694676624 41.76427809086766, -87.55970158199401 41.76432778788347, -87.55974235025553 41.76439118222792, -87.55954434756585 41.76449734139264, -87.55951661734389 41.76451727165905, -87.55951706467943 41.7645319017512, -87.55952907719278 41.7645480090532, -87.55954536743748 41.76456502278979, -87.55958995894099 41.76461839855076, -87.5595995371264 41.76463262360614, -87.55960115055851 41.76465093807958, -87.55960102955036 41.76466100964133, -87.55959790575857 41.76466784851153, -87.5595878555603 41.76468973544044, -87.55959002047192 41.7647139829462, -87.55960203300998 41.76473006197413, -87.55961414178991 41.76474112096587, -87.55962985886435 41.76475404293199, -87.55964194262388 41.76476419531251, -87.55966977447869 41.76478770977135, -87.5596867265087 41.76480154483908, -87.5597241931961 41.7648347289132, -87.55975668484999 41.76487611180501, -87.55978738956975 41.76491380585306, -87.55980299314305 41.76492868497542)), ((-87.5604456900637 41.76624782182557, -87.56036801827651 41.7654814377518, -87.55999295313129 41.76490250355148, -87.56000135009052 41.76490108598148, -87.56002213145005 41.76490122586234, -87.56004101049118 41.76490728115572, -87.56005741071368 41.76491518533184, -87.56012183540521 41.76494075555605, -87.56016812459461 41.76498407123752, -87.56022020415855 41.76504259988084, -87.5601448615881 41.76508984326814, -87.56035832218171 41.76532344814061, -87.56043423398849 41.76528992908279, -87.56046887387257 41.76533571709067, -87.56051367765971 41.76538980715788, -87.56055078224459 41.7654745801197, -87.56058562117163 41.76556482778588, -87.56061181616249 41.76564239328457, -87.56062105664857 41.76572752876292, -87.5606230537314 41.7658054789907, -87.56059552599926 41.7659002466191, -87.56058200925791 41.76598797359539, -87.56055217649016 41.76609150606803, -87.56051602748303 41.76617139678044, -87.5604456900637 41.76624782182557)), ((-87.55951749150054 41.76909992413685, -87.55935581781731 41.7684372382866, -87.5593198178405 41.7673752385942, -87.55931981776401 41.7666005381686, -87.56033304689161 41.76633532223473, -87.5602826829697 41.76637235535879, -87.56018070420711 41.76643972810646, -87.56007694820315 41.76647196131872, -87.55996959783818 41.76649813252558, -87.5598358912878 41.7665213821708, -87.55974115513756 41.7665350148975, -87.55963982764416 41.76654805347613, -87.55956263905348 41.76656564733148, -87.55952706269606 41.7665977895354, -87.55953980274857 41.7666357460177, -87.55953887945563 41.76671258051049, -87.5595062088663 41.76674693826003, -87.55944459975233 41.76674926701659, -87.55939214225164 41.76678294341083, -87.55939428956545 41.76684827146368, -87.55940582928669 41.76692518990873, -87.55942021335152 41.76700926239504, -87.55942345122881 41.76710588243774, -87.5594290733501 41.76718715215507, -87.55942969538057 41.7673183337408, -87.55942909020808 41.76742974678845, -87.55942410453447 41.76753948567224, -87.5594256783978 41.7676525607921, -87.5594370002576 41.76774758952794, -87.55947513350723 41.76780767166217, -87.55951856469699 41.76785406856141, -87.55951999775257 41.76791774658691, -87.55952002000893 41.76797702232293, -87.55948952050608 41.76801359011505, -87.55944721476662 41.76805666576755, -87.55943649122986 41.76815593713526, -87.55943142671936 41.76827226067469, -87.5594256496249 41.76838693311135, -87.55941307761638 41.76851802469313, -87.55941474342836 41.76862341749764, -87.55943410920551 41.7687201464302, -87.55947973236613 41.76882803075981, -87.55952022005916 41.76893642741705, -87.55952427586115 41.76902591860273, -87.55951749150054 41.76909992413685)), ((-87.55958521536354 41.76916374384358, -87.55955701788351 41.7692619392167, -87.55953673117064 41.76917878580981, -87.55958521536354 41.76916374384358)), ((-87.56358065622075 41.77197226931378, -87.5632171196517 41.7718650388156, -87.5633462199898 41.7710786391005, -87.5631541192992 41.7705820387877, -87.56282291952451 41.7700557383671, -87.5621820191766 41.7690254389164, -87.5615001188028 41.7687885386006, -87.5606632184315 41.7685656389415, -87.56021766729596 41.76851580302399, -87.56023807930703 41.76849997890317, -87.56032238058575 41.76843962289783, -87.5604129518703 41.76840675246905, -87.56052670429284 41.76839708857344, -87.56064175892108 41.76840115560053, -87.56078090326663 41.76841394669277, -87.56079628144425 41.76841536736141, -87.5609235797963 41.76843762848139, -87.56105069611625 41.76847525736284, -87.56116390266483 41.76851114549169, -87.5612836804146 41.76854927200313, -87.56142096786516 41.7685946528395, -87.56154000005479 41.76863387151621, -87.56168084769034 41.76868805796354, -87.56179470183739 41.76873108470178, -87.56191648895332 41.76878514120801, -87.56201782288552 41.76883302392825, -87.56212426923888 41.7688825863543, -87.56223428186271 41.76894040658755, -87.56233766294049 41.76900092567889, -87.56237662545288 41.76905332866885, -87.56240297401131 41.76911826991505, -87.56240882344795 41.76918087981497, -87.56242916899699 41.76925730734067, -87.5624713912938 41.76934376109774, -87.56251732879544 41.76942584917474, -87.56258093476664 41.76950201915282, -87.56264164794131 41.76957487489424, -87.56269788420281 41.76965428824165, -87.56273522904209 41.76971930316642, -87.56279292444417 41.76979927456212, -87.56286299691729 41.76988646463754, -87.56292501972841 41.76997250187595, -87.56298723275354 41.77004262402112, -87.56305962287205 41.7701199500698, -87.56313342794735 41.77020167642227, -87.56319556264728 41.77027838394933, -87.56324006360347 41.77035826693935, -87.56327569993914 41.77044357879941, -87.56330547335152 41.77052885135095, -87.56333221560052 41.77062233589884, -87.56335387976058 41.77071139663915, -87.56337292017564 41.77077134976845, -87.56338496683668 41.77080930115539, -87.56341397537116 41.77089731228887, -87.56344590145422 41.77098644205336, -87.56346772340167 41.77106232986162, -87.56341807217891 41.77110645572208, -87.56342091848956 41.77117508176497, -87.56346381593146 41.77126647910478, -87.56348479729138 41.77135114464672, -87.5634859477714 41.77143896879962, -87.56350535945234 41.77153240526908, -87.56351595408071 41.77162742895263, -87.56352084784342 41.77170869216364, -87.56356994574854 41.77177213952602, -87.56359189092186 41.77183760065044, -87.56358051173875 41.77193083079565, -87.56358065622075 41.77197226931378)), ((-87.57213836216111 41.77913419103445, -87.57178252323 41.7787777395919, -87.57075722285261 41.7777882397128, -87.57021192232909 41.7774074397435, -87.5690710222059 41.7768572398802, -87.5684650219164 41.7764092398614, -87.5680641210766 41.7759975391733, -87.5676086214656 41.7754389396027, -87.5664874209167 41.7740688393789, -87.56636482079369 41.773977339647, -87.56588242067021 41.7735864393866, -87.5655446207661 41.7730296393532, -87.56450431973261 41.7724451393355, -87.563987720184 41.7720923388619, -87.56398146418547 41.77209049356243, -87.5639818529078 41.77209049250558, -87.56407321144482 41.7721147046089, -87.56416720057094 41.77216418198742, -87.56427944039275 41.77222036912886, -87.56437192461723 41.77227312941263, -87.56448498934263 41.7723216364922, -87.56462008045025 41.7723675471039, -87.56471638041234 41.77240770836227, -87.5648000742714 41.7724598611811, -87.564904238759 41.77251709030806, -87.56500085350034 41.77253090908045, -87.565083087681 41.77258250252384, -87.56516900838383 41.77263247453305, -87.56525476333495 41.77269616654078, -87.56535593532828 41.77275831524518, -87.56544988275621 41.7728116341213, -87.56554746690911 41.77286717128401, -87.56565238438517 41.77292275873857, -87.56575712391596 41.77299316324552, -87.56585086737472 41.77306349447275, -87.56593727276099 41.7731343260137, -87.56601414867151 41.77320509246967, -87.56609030321133 41.77327475771267, -87.56614838635571 41.77332289595112, -87.56618633732548 41.77335292463628, -87.56623551749433 41.77340668460671, -87.56626680531461 41.77344218452033, -87.56628524507019 41.77346072209063, -87.5662935770797 41.77346906519831, -87.56629494944517 41.77347080353742, -87.56628637826782 41.77349484116657, -87.56628069665216 41.77351933772289, -87.5662779091454 41.77354412720765, -87.56627805171458 41.77356899114142, -87.56628112962392 41.77359376490203, -87.56628707070215 41.77361822928757, -87.56629591322427 41.77364221824922, -87.56630751352736 41.77366551293068, -87.56632183694471 41.77368794624687, -87.56633873841236 41.77370935531534, -87.56635811105127 41.77372957229419, -87.56637977314895 41.77374843323283, -87.56640357862197 41.77376579911703, -87.56642930875226 41.77378150410252, -87.56645682193977 41.77379546354204, -87.56648585920848 41.77380756730623, -87.56651620223799 41.77381767754564, -87.566547634735 41.77382576317723, -87.56655030275131 41.7738263570108, -87.56658463485462 41.77385573022248, -87.56664018200043 41.77388272108423, -87.56669054235915 41.77390523018716, -87.56673025299715 41.77392004045456, -87.56677217204926 41.77394633574698, -87.56678976084447 41.77395320433051, -87.56681339294224 41.77395731323533, -87.56682592296646 41.77397040448982, -87.5668336366212 41.77398450808484, -87.56683477169047 41.77400002061417, -87.56684417410054 41.7740201440955, -87.56684822096901 41.77404659804577, -87.56686192813322 41.77407462690294, -87.56688552526185 41.77410332411078, -87.5669151626538 41.77413272188551, -87.56693523988606 41.77416158852436, -87.56694813129535 41.77418417713987, -87.56698571425929 41.77422067962398, -87.56700497828822 41.77424386014853, -87.56702662962317 41.77426969193398, -87.56704908413451 41.77429904066403, -87.56707508578207 41.77432901761519, -87.56710097815353 41.77435591885629, -87.56713522456207 41.7743985739606, -87.56716271786549 41.77443256695209, -87.56720149160803 41.77447678903073, -87.5672278476103 41.77451714144543, -87.56727195993543 41.77458102048747, -87.56729848151008 41.77461344272169, -87.56731311596886 41.77464065359617, -87.56734833554971 41.7746785406114, -87.56736290336642 41.7746990000857, -87.56736337211744 41.77469968955754, -87.56736985223958 41.77471589716946, -87.5674297645289 41.77478904768807, -87.56745854997772 41.77482856469199, -87.5674896067917 41.77486510665349, -87.56751769037518 41.77490190184362, -87.56761999376535 41.77503590182137, -87.56769325657571 41.77513630990842, -87.56773793750097 41.77520186657141, -87.56781919410925 41.77531141089158, -87.56790991599529 41.77542368045011, -87.56800955564275 41.77554122294167, -87.56807042129613 41.77562376603186, -87.56813138893487 41.77569777338622, -87.56821117487983 41.77578011255694, -87.56822721542763 41.77580008720634, -87.56831289748985 41.77588924396335, -87.56838202777185 41.77596437580998, -87.56844054155188 41.77601660500491, -87.56849975960378 41.77608047436402, -87.56855872849954 41.7761500504264, -87.56863218019832 41.77622565107126, -87.5686922249338 41.77628483310028, -87.56875211442264 41.77634481018215, -87.56881267157569 41.77640405099228, -87.56889279976045 41.77646707135216, -87.56894452894464 41.77650720826841, -87.56895815997854 41.77651706862695, -87.56918859715884 41.77668844611289, -87.5692898251941 41.77674674887421, -87.56941584434961 41.77681674213053, -87.56950199365308 41.77684805124325, -87.56961477004421 41.77692179905775, -87.56969342685443 41.77696677910628, -87.56981146240535 41.77696811304615, -87.5698435951319 41.77697908420482, -87.56989836555472 41.77699777975025, -87.5699666214349 41.77705421704961, -87.57002957521765 41.77712488964887, -87.57014933317501 41.77722832052899, -87.57022818315644 41.77731885776047, -87.5703252531511 41.77741884570188, -87.5704087950886 41.777484714561, -87.57054828534258 41.77759376573139, -87.57065946168193 41.77767902734843, -87.57066175171228 41.77768068967241, -87.57080122422443 41.77778205668614, -87.57092766749366 41.77787839652321, -87.57114269700297 41.77804832311339, -87.57125366698864 41.77815114621899, -87.57137768541384 41.77826668030346, -87.57147181970626 41.7783671961027, -87.57156598419428 41.77846496828576, -87.57167141650541 41.77854031276721, -87.57175402734954 41.77856116761453, -87.57182346699911 41.77857973966719, -87.57188479486564 41.7786641213447, -87.57193956413261 41.77874516637689, -87.57201991121647 41.77883351658073, -87.57207773007636 41.77890470232249, -87.57209825878921 41.77896685895625, -87.57208680710633 41.77900520363621, -87.57209514508909 41.77904422720726, -87.57212585434988 41.77911358726151, -87.57213836216111 41.77913419103445)), ((-87.57238571600251 41.7792770507703, -87.5723054234475 41.7793015398862, -87.57222163172872 41.77921760393534, -87.57223815785485 41.77922794506565, -87.57237254846855 41.77927274281237, -87.57238571600251 41.7792770507703)), ((-87.57269404825476 41.77739640866069, -87.5728413231311 41.7774160398162, -87.57328542294729 41.7781233396154, -87.57328468183792 41.77816038362667, -87.57327101637765 41.77813651923441, -87.5731567199912 41.778004585794, -87.57299345803304 41.77785915559588, -87.57281824375276 41.77769498571023, -87.57279112112656 41.77766955843983, -87.57286784646797 41.77763000030144, -87.57286796317641 41.77762012163955, -87.57269404825476 41.77739640866069)), ((-87.57240048888563 41.77735727836386, -87.57252541102295 41.77737392998812, -87.57245030454102 41.77740933811695, -87.57240048888563 41.77735727836386)), ((-87.57198705822046 41.7758315169111, -87.5718980229974 41.77678353937, -87.5720425225315 41.7770119393221, -87.57221893049903 41.77723785569999, -87.57215897759772 41.77718073038806, -87.57208166928014 41.77708361977309, -87.57200153919388 41.77697715900394, -87.57193008234506 41.77688118477699, -87.57187315478998 41.77679683226652, -87.5718237669259 41.7766949659886, -87.57177280808774 41.77666389189745, -87.57174566734832 41.77660278906543, -87.57174534416269 41.77650618717038, -87.5717567899737 41.77640637179364, -87.57177922833397 41.77630717704316, -87.5718024964406 41.77619975574616, -87.5718413868271 41.77607322715454, -87.57188508000301 41.77597472129459, -87.57193608339514 41.77587791263345, -87.57198705822046 41.7758315169111)), ((-87.57416527121387 41.77553685931183, -87.5734772229571 41.7755101395159, -87.57330410361234 41.7755207637946, -87.57331555768684 41.77548198965892, -87.57331578276066 41.77540075966233, -87.57331212738099 41.77538125130662, -87.57336702069277 41.77537708691631, -87.5733752866371 41.77545329587396, -87.57339545279289 41.77548416420381, -87.57347583561733 41.77550719988287, -87.57354913154727 41.7755093306729, -87.57360999214532 41.77550918484939, -87.57370457037945 41.77550981025328, -87.57378083098536 41.77550921675874, -87.57386073334725 41.77551084271926, -87.57399260625519 41.77551994674341, -87.5741230056606 41.77552959028513, -87.57416527121387 41.77553685931183)), ((-87.57460211403102 41.77568271513823, -87.57455610026427 41.77566624412049, -87.57458541148458 41.77567589756871, -87.57460211403102 41.77568271513823)), ((-87.57461035244512 41.7760740267115, -87.574561523323 41.7759767392899, -87.57507130012822 41.7759003408438, -87.57507886821679 41.77593135590888, -87.57498396027047 41.7759524090621, -87.57486792470488 41.77596810905469, -87.5747806325753 41.77597137417109, -87.57469687766469 41.77598563973226, -87.57464067733768 41.77602588433891, -87.57461035244512 41.7760740267115)), ((-87.57322398930219 41.7797479496995, -87.5745763237785 41.77909773986, -87.5750592238081 41.7783031392062, -87.5752386241592 41.7776884389822, -87.5751861234606 41.7770728394489, -87.57479862340161 41.7764491388397, -87.5747485458314 41.77634936400301, -87.57475843785971 41.77636146485021, -87.57484814296696 41.77646469378499, -87.57492261219097 41.77655410117752, -87.57500108632077 41.7766770133488, -87.57507098920613 41.7767806607978, -87.57513421686446 41.77689084934138, -87.57519079292635 41.77700538453922, -87.57524488633028 41.77714405442909, -87.57528779367372 41.77729856597732, -87.57530263798355 41.77740788657346, -87.57530185345604 41.77753686346252, -87.57529821152453 41.77765923493023, -87.57529268626045 41.77771133966046, -87.57528583813132 41.77777605910894, -87.57527963141905 41.77786712904551, -87.57525139879257 41.77796079737761, -87.57522912212563 41.7780468209998, -87.57520006491538 41.77814816815352, -87.57516581633621 41.77825441993138, -87.57513018638105 41.77835352798233, -87.57510829046061 41.7784071723228, -87.57510043482772 41.77845157737794, -87.57508005587371 41.77850084031591, -87.57505362489931 41.77856597989427, -87.57502434550186 41.77862396442007, -87.57500021674583 41.77868033718806, -87.57497792973213 41.77870488975117, -87.57495362953179 41.77871351105793, -87.57493077206158 41.77872433731336, -87.57493403605949 41.77875838719584, -87.57494323862201 41.7787864404966, -87.57494367400454 41.77881169081058, -87.57492723926144 41.77883737833911, -87.5747348324077 41.77911108552934, -87.57466058178468 41.77918963011831, -87.57456272342108 41.77928009497025, -87.5744384585454 41.7793714813705, -87.5743026733272 41.77944468005137, -87.57411030979914 41.77952793351206, -87.57395579672328 41.77957246715712, -87.57369617082769 41.77963880984861, -87.57337557616708 41.77971407812846, -87.57322398930219 41.7797479496995)), ((-87.56941579984213 41.78128234687279, -87.56938512264691 41.7812373400782, -87.57146892254001 41.7807466400925, -87.57294212305121 41.7802324399521, -87.57292222371289 41.7798930402513, -87.57292819671015 41.77989016840144, -87.57292761097322 41.77989498054828, -87.57307959829639 41.78025054850674, -87.57266739425299 41.78038338792806, -87.57205272750669 41.78058458925484, -87.57165005655381 41.78071693905105, -87.57128197084683 41.78083908746834, -87.57104808344721 41.78089846028292, -87.57087825289553 41.78093575188939, -87.57068930117478 41.78097785849324, -87.57053047437289 41.78101467487313, -87.57037535513267 41.78104822223877, -87.57022312739451 41.78108508229303, -87.57008484419197 41.78112093762523, -87.56994658761859 41.78115459671574, -87.56982890687172 41.78118455093801, -87.56979361163602 41.78119276848418, -87.5694943373034 41.78126245920238, -87.56941579984213 41.78128234687279)), ((-87.57714808094944 41.78812746798405, -87.5770173255416 41.7881264408923, -87.5772796254042 41.7873173413782, -87.5770842250466 41.7862804408781, -87.57634552512459 41.7845291404354, -87.5759192246975 41.7838415402881, -87.5753806241142 41.783350540074, -87.5748694241031 41.7830612407408, -87.5740727243128 41.7825799406302, -87.57276082345339 41.7820821406837, -87.5716190231244 41.7816918403077, -87.5705752228646 41.7814795400715, -87.5695371225286 41.7814603404993, -87.56946127199259 41.78134905948195, -87.56968346016117 41.78129688257486, -87.56979247513065 41.78128853621713, -87.56986224849344 41.78127917539227, -87.56998750708632 41.78129043686792, -87.57007967854483 41.78130916073846, -87.57021878475399 41.78132764883028, -87.57032633538437 41.78134812178033, -87.57049691791072 41.78137120889527, -87.57060592903414 41.78139223986427, -87.57076606987464 41.78143007639522, -87.57090209482404 41.78146116719412, -87.57103667689032 41.78149005173166, -87.57119179645564 41.78151852378009, -87.57135045997411 41.7815574471355, -87.571498054817 41.78160233484702, -87.57165294103645 41.78165056412897, -87.57181155321777 41.78169387736383, -87.57198841785674 41.7817438976294, -87.57213240473216 41.78178382007805, -87.57230412495697 41.78183490485694, -87.57247865980003 41.78189588634965, -87.57261436838395 41.78195386510102, -87.57284509879348 41.78203772182685, -87.57310061625091 41.78213381615351, -87.57332899341858 41.78223082890316, -87.57337722605921 41.7822448688573, -87.57346256782137 41.78228330423674, -87.57357344932461 41.78233233729407, -87.57370397455288 41.7823946729798, -87.57384471975526 41.78246091725859, -87.57397671811746 41.78252271269258, -87.57406636331658 41.78256940974684, -87.57412040369282 41.78258842830947, -87.57420719990408 41.78262797046096, -87.57430409411631 41.78268184927372, -87.57441482382887 41.78274405335753, -87.57452131348109 41.78279250735385, -87.57464371698499 41.78285972721358, -87.57477125165741 41.78292698082398, -87.57488055454711 41.78298588244319, -87.5749687079564 41.78303476382186, -87.57508594046898 41.7831052422155, -87.57518928707732 41.78317178630336, -87.5753255504992 41.78324513487662, -87.57545238144814 41.78331018793204, -87.57555584970822 41.78336630475803, -87.57567985148829 41.78342255718845, -87.57581557082847 41.78347998419013, -87.57590596561759 41.78352558675998, -87.5759722972323 41.78355950407715, -87.57604590409244 41.78359840957574, -87.57609954591869 41.78365145398438, -87.57613030648385 41.7837169701661, -87.57618608694786 41.78383753803268, -87.5763165895751 41.78408922568192, -87.57638250084803 41.78422138537591, -87.57643019583196 41.78434354558648, -87.57649313616622 41.78447897850438, -87.57654302178055 41.78458029763227, -87.57656117626111 41.78461719036196, -87.57661817960066 41.78475862261881, -87.57668461820329 41.78490834929325, -87.57674895517141 41.7850498295069, -87.5768181790896 41.78521219732475, -87.57692985213052 41.78550657160514, -87.57699663956299 41.7856892317854, -87.57704263799606 41.78583113938826, -87.57709362881921 41.78598515523133, -87.57714453640752 41.78614630563764, -87.57719685021863 41.78631240470914, -87.57724176947347 41.78648394365155, -87.57727069802922 41.786643302485, -87.57731050252514 41.78681316061031, -87.57732155752602 41.78693343324647, -87.57733239795834 41.78707181618749, -87.57733832530361 41.78719150566685, -87.57733750539603 41.78732377499239, -87.57733447194791 41.78745712746117, -87.57733018503487 41.7875723588614, -87.57732140727282 41.78769524469735, -87.57731003155045 41.78778957371407, -87.57729216493291 41.78787452889977, -87.57727694351318 41.78798419976103, -87.57727681381017 41.7880577459599, -87.577273590027 41.78808242301321, -87.57722134308727 41.78809744712707, -87.57715232970601 41.7881041274571, -87.57714808094944 41.78812746798405)), ((-87.57731844557827 41.78836716199296, -87.57739042561229 41.7883501409738, -87.57766792702247 41.78834681376547, -87.57766705359587 41.78836218470362, -87.57733384748052 41.78838414208661, -87.57732587704615 41.78837585623727, -87.57731844557827 41.78836716199296)), ((-87.57787177320813 41.78960338716752, -87.5763632250886 41.7885930416619, -87.5763820841036 41.78858858209669, -87.57638976977591 41.78859088084807, -87.57646037832609 41.7886358042716, -87.57654996890771 41.78868798618882, -87.57664974859183 41.78874682280023, -87.57675163953023 41.78881335721991, -87.57687969564763 41.78889982130589, -87.57701142015364 41.78898631077215, -87.57712425249426 41.78905785605339, -87.57724722295367 41.78914044535272, -87.57734832878076 41.7892113632783, -87.57746255153245 41.78928950392741, -87.57756363202969 41.78936261829368, -87.57768871374358 41.78945290472314, -87.57781664423671 41.78955034461021, -87.57787177320813 41.78960338716752)), ((-87.57939508283874 41.79244979874547, -87.57939682666979 41.7923076414818, -87.5789031264971 41.7912660411404, -87.57792702557268 41.78965654844801, -87.5779464532859 41.78967524089892, -87.57807435884531 41.78977487548383, -87.57818613266201 41.78987440388456, -87.57820865901871 41.78989211508922, -87.57827194268282 41.78993644011399, -87.57830958710568 41.78997840089473, -87.57837407576926 41.79004523607082, -87.57842327756909 41.7901020916143, -87.57846186012142 41.7901890650057, -87.57849732953885 41.7902913848076, -87.57853140965761 41.79038710919239, -87.57858214983946 41.79050050779945, -87.57863904659634 41.79058924747029, -87.57867844105482 41.79066963874294, -87.57870015833454 41.79069393135921, -87.57876162887524 41.79076788216614, -87.57881146946804 41.7908329740295, -87.57884615335523 41.79087727554089, -87.57885121071045 41.7908837301007, -87.57888011995429 41.7909201447556, -87.5789618027525 41.79102167046091, -87.57902241370324 41.79110659288061, -87.57910621446484 41.79121526669616, -87.5791198899305 41.79123731100733, -87.57917050354611 41.79129911547174, -87.57927456118118 41.79143152378799, -87.57929015319885 41.79145906922117, -87.57931691222004 41.79150631148148, -87.57934569720754 41.79155712965848, -87.57938739353104 41.79162208712857, -87.57940619763772 41.79165138115141, -87.57944875253207 41.79171258347016, -87.57950412052806 41.79180680150198, -87.57954725391693 41.79188117985396, -87.57958463593692 41.79194564213809, -87.5796140343746 41.79200291552566, -87.5796281442921 41.79203554539541, -87.57961141142863 41.79209706145915, -87.57958966208855 41.79216285779154, -87.57955948448968 41.79223173882274, -87.57952084218216 41.79230684403858, -87.57946972426568 41.7923724476272, -87.5794060932012 41.7924316890576, -87.57939508283874 41.79244979874547)), ((-87.57554161554449 41.79679673929289, -87.57534482611381 41.7967316423344, -87.57496002583621 41.7962679428359, -87.57539212584381 41.7954477420792, -87.5759920259495 41.7950795424499, -87.5769158260054 41.7949811421608, -87.5782659263982 41.7946578420501, -87.5789066260451 41.7941469424684, -87.57927607864258 41.79361783355923, -87.57927029326397 41.79363653992535, -87.5792360661963 41.79369283452372, -87.57920169184852 41.79376168789155, -87.57916315896631 41.79382737394178, -87.57912889489513 41.79388680799399, -87.57908204419819 41.79394615966921, -87.57906036729106 41.79400567634359, -87.57903861598152 41.79407147253513, -87.57901211748008 41.79418433707853, -87.57901162657167 41.79419968233852, -87.57898953729669 41.79422332964759, -87.57893908794901 41.7942723959435, -87.57883393716381 41.79435787575451, -87.57880504764718 41.79438224731054, -87.57876875027524 41.79441288176991, -87.57857411290891 41.79456144134969, -87.57843299354759 41.79464942819112, -87.57830371063424 41.79472871205666, -87.57811204529537 41.79481142711638, -87.57791826879981 41.7948864439886, -87.57782985069535 41.79492098930002, -87.57760384282159 41.79499250052257, -87.57738839964678 41.79503883344981, -87.57708058150801 41.7950818136026, -87.57691454428348 41.79510541893324, -87.57685295366615 41.79510391537526, -87.57678063755567 41.79507874116863, -87.57671489192774 41.79505635364878, -87.57665120265132 41.79504605474032, -87.57661890241965 41.79504858570522, -87.57656956382637 41.79506527576935, -87.57651212859197 41.79508465647518, -87.57645845456504 41.79509582827217, -87.57639693499584 41.79508828834645, -87.57635815510244 41.79508034882925, -87.57632741020154 41.79507520597878, -87.57622474343681 41.79507398107206, -87.57616592955169 41.7950856685361, -87.57609744057726 41.79510936674872, -87.57604197202751 41.79514851762195, -87.57594504480195 41.79522032813085, -87.5758229243288 41.79531392556621, -87.57578084814813 41.79533670031952, -87.57573216824794 41.7953599809107, -87.57563869607013 41.79538735579752, -87.5755194933877 41.79542004915461, -87.57544371478728 41.79543985771878, -87.57540669602568 41.79546925059817, -87.57540369938911 41.79553674119052, -87.57538691957522 41.79559151551113, -87.57536697835434 41.79566547919686, -87.57535534907035 41.79571864120256, -87.57532641264082 41.79580901147439, -87.57531207177703 41.79584349558277, -87.57527897377591 41.79591353016696, -87.57520040861097 41.79604528620803, -87.57518139337542 41.79610279029355, -87.57516506162467 41.79618171855832, -87.57516436950536 41.79624044128126, -87.57519601996756 41.79635536129338, -87.57521335680485 41.79637797852052, -87.57522218029432 41.79643841088175, -87.57522937382591 41.79651255387807, -87.57527225272014 41.79660833746578, -87.57531214340972 41.79664647119402, -87.57539520920299 41.79669312344291, -87.57548046130256 41.79674088804093, -87.57553272423056 41.79678733697258, -87.57554161554449 41.79679673929289)), ((-87.57980632427544 41.79843222202501, -87.5797357269842 41.7983146425187, -87.5788835263911 41.7975733429671, -87.57779352610039 41.7969054425094, -87.57724824300628 41.79686692006538, -87.577316426545 41.79685671936513, -87.57740809616242 41.79685787093497, -87.57755324730046 41.7968643145588, -87.57768580765162 41.7968811042595, -87.57783074470179 41.79690565781458, -87.57798804135911 41.79693962382729, -87.5780523630371 41.79695870784416, -87.57810864190803 41.7969755443495, -87.57821015388114 41.79701298418923, -87.57829257831942 41.79705194614818, -87.57836321639236 41.79709522033497, -87.57842572334545 41.79714393042445, -87.57846854989126 41.79718208289874, -87.57856831902694 41.79724311286189, -87.57866735566554 41.79730413847481, -87.57875753820296 41.7973694947669, -87.57876990944915 41.79737789119304, -87.57885140803263 41.79743322973622, -87.57889290520059 41.79745984735052, -87.57895110251899 41.79750084514342, -87.57901297270338 41.79754131762004, -87.57906242707746 41.79757731906767, -87.57914171961622 41.79763327382836, -87.5792122234119 41.79768807285264, -87.57933433044292 41.79778327729058, -87.57946718938402 41.79789995807748, -87.57949911901521 41.7979292564132, -87.57954482894111 41.79797181934735, -87.57960801429087 41.7980254723925, -87.57965657409042 41.79807518855509, -87.57993726104222 41.79836914528041, -87.57980632427544 41.79843222202501)), ((-87.5800850655841 41.80348623773909, -87.58033177837622 41.80288305548122, -87.58044752333524 41.80293128315225, -87.58019069663969 41.80335546976298, -87.58011031153374 41.8034720403958, -87.58008753344524 41.80348470680327, -87.5800850655841 41.80348623773909)), ((-87.58010798098462 41.80391822431822, -87.579952827531 41.8038095434208, -87.58001510863311 41.80365727383046, -87.58001880882057 41.80371568089944, -87.58002490893797 41.80377107294104, -87.58004552924466 41.80383613799707, -87.58010798098462 41.80391822431822)), ((-87.58058721721711 41.80417286377602, -87.58048032776711 41.8041790434075, -87.58022074810738 41.80399721471009, -87.58025615223907 41.80401589662821, -87.58033219221718 41.80404658213747, -87.5804379219507 41.80410078853509, -87.5805454797664 41.80415547334199, -87.58058721721711 41.80417286377602)), ((-87.58733487134829 41.81181415698247, -87.58718623055179 41.8117777445615, -87.5870211302485 41.8116207444982, -87.58683943023451 41.8112677443567, -87.5869115306306 41.8105383445853, -87.58642563030369 41.809734043896, -87.58611663031159 41.8093239439964, -87.5855889294491 41.808171544464, -87.5850438294205 41.8072476438178, -87.58388322961861 41.806460244128, -87.5827571287149 41.8056237441421, -87.5816999284953 41.8048323433621, -87.5812719283672 41.8043294433105, -87.58115228746404 41.80424862420215, -87.5811696696003 41.80425470645661, -87.58126929084598 41.80431068303825, -87.5813669190042 41.80437718477797, -87.58144049475622 41.80446183331766, -87.58150553673683 41.80454549290286, -87.58156622778499 41.80459803078852, -87.58162615414956 41.80464875323941, -87.58174245439342 41.80474007532143, -87.58201337562534 41.80493943569976, -87.5822533799296 41.8051166388151, -87.58237479234562 41.80520066618926, -87.58238010577931 41.80520435053659, -87.58246196164076 41.8052616106167, -87.58261241847799 41.8053618538795, -87.58272152027061 41.80544168429235, -87.58281796941237 41.80550589906434, -87.58286889681963 41.8055419081873, -87.58289835017017 41.80558279792795, -87.5829551701748 41.80558544631018, -87.58297895683121 41.80559018572757, -87.58306026945159 41.80564376395348, -87.58311717614339 41.80568941533456, -87.58319828789695 41.80576039057841, -87.58331434752552 41.80581924444178, -87.58341693243311 41.8058825659913, -87.58345930604065 41.80591624071176, -87.58353941689003 41.80596616086623, -87.58364931158305 41.80603180711937, -87.58368261169919 41.80606040024279, -87.5838030831152 41.80615951456707, -87.58390302317403 41.80623884443795, -87.58405361824954 41.80632764308024, -87.58417029409114 41.80638740625488, -87.58425646159529 41.80644010913296, -87.58432254823828 41.80648809834695, -87.58433649923947 41.80649642296032, -87.58435105233437 41.80650658906401, -87.58439769249077 41.80654575126358, -87.58449480476094 41.80660357588241, -87.58452390447103 41.80662434736371, -87.58460136294042 41.80669436501611, -87.58467031777008 41.80675746623329, -87.58472469778717 41.80681226704515, -87.58481861888563 41.80688285836174, -87.58489497868581 41.80694005261909, -87.5849505697349 41.8069948609389, -87.58501276013743 41.80706250054147, -87.58515769804625 41.80721528616719, -87.58521774743778 41.80730944856957, -87.58527438394134 41.80737842426827, -87.58533225270753 41.80744878009967, -87.58538949158915 41.80752278168459, -87.58545808836026 41.80761699933648, -87.58549321448511 41.80769681334501, -87.58552852856057 41.80776059980953, -87.58558142456141 41.80783915678429, -87.58558621701127 41.80784651436922, -87.58562533442604 41.80784814113483, -87.58566689197428 41.80790192525009, -87.58569190549994 41.80795881165004, -87.58569840485534 41.80797713163943, -87.58568675115924 41.80798254461267, -87.58568346765304 41.80799992166391, -87.58570524702728 41.80807322575966, -87.58576350111126 41.8081641662036, -87.58583218717276 41.80825061873315, -87.58590203733297 41.80834116776317, -87.58590323063267 41.80834254711573, -87.58592421473786 41.80838019732636, -87.58596809857356 41.80846094605646, -87.58603104450111 41.80853641091606, -87.58610089039377 41.80864625203785, -87.58610687335758 41.80865542911408, -87.58613078187177 41.80869073850766, -87.5861481793265 41.80871815774483, -87.58613938133779 41.8087429916275, -87.58612269154666 41.80877136752642, -87.58610476604163 41.80879573039152, -87.58610167637283 41.80882158934203, -87.58610541540614 41.8088408227564, -87.58611760895731 41.80886790545257, -87.58614155389817 41.80892215084165, -87.58616422239047 41.80896939029958, -87.58620150835681 41.8090467748976, -87.58622230778704 41.80908450590253, -87.58624555357279 41.80909472994664, -87.58629097907182 41.80909653417766, -87.58628797305268 41.80913433016778, -87.5862805725336 41.80916822842691, -87.58627262427773 41.80919575750648, -87.58629625134344 41.80922683912576, -87.58633957126997 41.80925212035032, -87.58638838786084 41.80925886049109, -87.58643223584431 41.80928280096926, -87.58646720463977 41.80929120544882, -87.58648362909119 41.80931093461546, -87.58647421376565 41.8093350781648, -87.586441417288 41.80937023833269, -87.58643515993722 41.80939736595276, -87.58645096030368 41.80943918191729, -87.58647980371623 41.80948865550523, -87.58651756312265 41.8095505382035, -87.58654877139132 41.80959843601867, -87.58658289767787 41.8096383406146, -87.58660665779732 41.80967065740724, -87.5866421441837 41.8097135071991, -87.58667375863529 41.80974236208917, -87.58671386938613 41.80976910613968, -87.58674735874094 41.80979435157836, -87.58676796339651 41.80982360113612, -87.58678651756216 41.8098683984902, -87.58681415475093 41.80991748214905, -87.58684763927107 41.80997165134195, -87.58688553774564 41.81003424778107, -87.58692474950399 41.81010695193999, -87.58694631700276 41.81017015520155, -87.58694756065005 41.81017364823472, -87.58695599478845 41.81019917016501, -87.58696777050388 41.81024005401824, -87.5869791042404 41.81027486988057, -87.58699210321957 41.81033058020397, -87.58700048747389 41.81036672269136, -87.58700864690672 41.81040014551489, -87.58701744897255 41.81044131273462, -87.58702432637257 41.81048715878325, -87.58702971252954 41.81054139344071, -87.58703316307985 41.81056949004451, -87.58703500887616 41.81060649455988, -87.58703696483713 41.81063710536067, -87.58703268987392 41.81067665054889, -87.58702303162946 41.81073451939718, -87.58701503945146 41.81077203574175, -87.58700671284025 41.81083526484566, -87.58699656756379 41.81087817478844, -87.58698878905145 41.81090685650935, -87.58699257490554 41.810937699015, -87.58700287449332 41.81096378191369, -87.58699705620828 41.81099088496707, -87.58699776221196 41.81102173545294, -87.58701042011813 41.81104357895301, -87.58700792098212 41.81106304732553, -87.5869951920985 41.81107572549016, -87.58696734897028 41.81108221398662, -87.58695234577021 41.81108848293935, -87.58694563658004 41.81111039379993, -87.58694102584491 41.81114738352844, -87.58693432801302 41.81117450803092, -87.58694044542143 41.81119738012828, -87.58696881414745 41.811208816251, -87.58698610877636 41.81122619064557, -87.58699624394797 41.81125051476879, -87.58699789426122 41.81127272775367, -87.58698829754232 41.81130307166525, -87.58698813413412 41.81131714817251, -87.58699204196103 41.81132806887372, -87.58704108290149 41.81140407371092, -87.58706544304113 41.8114292048537, -87.58709624861807 41.81144554212997, -87.58713058016963 41.81145523180029, -87.58715277977795 41.81147357108522, -87.58717966898243 41.81147975570993, -87.58720644151811 41.81148967232983, -87.58722166071843 41.81150895364918, -87.58722096139867 41.81151860867128, -87.58721178500738 41.81153495981581, -87.58719623988954 41.81155950185939, -87.58719226737341 41.81158582134472, -87.58719887236025 41.81160452557451, -87.58721488138481 41.81162844957065, -87.58722747579274 41.81163371832514, -87.58725432068496 41.8116374059055, -87.58727395626418 41.8116491134445, -87.5872922740368 41.81167313557628, -87.5873060531837 41.8116996241441, -87.58731910671276 41.81172542275282, -87.58732557515022 41.81175589929074, -87.58733212373684 41.81178272509437, -87.58733487134829 41.81181415698247)), ((-87.58841269985899 41.81207819179921, -87.58820272508395 41.81202675444095, -87.58822701749148 41.81202591406347, -87.58830020447652 41.81204120757209, -87.58837916514167 41.81206477156937, -87.58841269985899 41.81207819179921)), ((-87.59022090030174 41.81375445556721, -87.58955693153609 41.813174144494, -87.5889048310903 41.8127751449474, -87.58855825333889 41.81215309468592, -87.58856225727521 41.81215542210478, -87.58862641297914 41.8121904156088, -87.58868901184275 41.81223308404427, -87.58875674590604 41.81227578463432, -87.58879442879251 41.8123160949881, -87.58882618545604 41.81236130715784, -87.5888512438176 41.81241470817313, -87.58887252098305 41.81247796433048, -87.58891134713656 41.81254627417802, -87.58896412991731 41.812613027617, -87.58901845587219 41.81267320450333, -87.58905388192093 41.81271844065892, -87.58909000277346 41.81276697305927, -87.58912977782661 41.81281662765481, -87.58917408827121 41.81285478567639, -87.58923157803095 41.81289522529653, -87.58928469954776 41.81293289039994, -87.58934444030277 41.81296895398335, -87.58942034111593 41.81300347370929, -87.5894962987776 41.81303305543594, -87.58956643708687 41.81305820749962, -87.58961587116679 41.81309749595955, -87.58966242805249 41.81313182663028, -87.58969497609512 41.81317210344055, -87.58972447697707 41.81322224126581, -87.5897768438391 41.81326154836694, -87.58982706965006 41.8132959029942, -87.5898646522552 41.81334499373789, -87.58990599700095 41.81338587682588, -87.5899465081409 41.81343553507567, -87.58998130246248 41.81347198490893, -87.59001550806461 41.81349580782106, -87.59003581446832 41.8135162462852, -87.59006021082827 41.81356360617045, -87.59009800168714 41.81359458544208, -87.5901336510568 41.81362061307768, -87.59015442356952 41.81366410669217, -87.59018393736918 41.81371314539894, -87.59021948386948 41.81374795401105, -87.59022090030174 41.81375445556721)), ((-87.59044043721664 41.8146167484277, -87.59027583193399 41.8144755446174, -87.59025187835272 41.81388690577047, -87.59025376170278 41.81389252361924, -87.59028845457993 41.81393775464421, -87.59029891423832 41.81398502404797, -87.5902984131254 41.81402837974644, -87.5902988982465 41.81404978792722, -87.59030579662148 41.81408770431889, -87.59032421110386 41.81414490345456, -87.59037000940978 41.81418142434445, -87.59042394796495 41.81421196019225, -87.59045792581807 41.81425553970233, -87.5904962885601 41.81430079329844, -87.59051701080348 41.81434867707759, -87.59049619664653 41.81437214430677, -87.59046262144102 41.81435710728539, -87.59041579869736 41.81434582737796, -87.5903681129375 41.8143455189546, -87.59034648973854 41.8143755669003, -87.5903568842466 41.81442832414208, -87.59037369209744 41.81449758821702, -87.59039800064353 41.81455263114049, -87.5904304797309 41.81459894413619, -87.59044043721664 41.8146167484277)), ((-87.5921505722161 41.81676136151037, -87.5914073324284 41.8159130454417, -87.5907112316733 41.8148490447943, -87.59051883833399 41.8146840034776, -87.59056069346289 41.81469528740496, -87.59061916104514 41.81471487481215, -87.59064961180185 41.81474635721258, -87.59067781030024 41.81478221477926, -87.59069342994064 41.81482732235949, -87.59073561232584 41.81485942751683, -87.59078937411179 41.81490533078909, -87.59083874062949 41.81495065512446, -87.59089094475435 41.81500423251332, -87.59092690943777 41.81506648494128, -87.59096215433762 41.81512763582046, -87.59100764317114 41.81519104775796, -87.59104797882881 41.81525607376228, -87.5910766582811 41.81531388850374, -87.59110985210879 41.81536185303263, -87.59115515291892 41.81537806196173, -87.5911790471072 41.81540565932757, -87.59120124377746 41.81545300447097, -87.59122328841947 41.81551352140186, -87.59124250182407 41.8155652369983, -87.59125896300137 41.81560101968705, -87.59130194786104 41.81569076060079, -87.59131710620564 41.81571226355894, -87.59133731199289 41.8157414828033, -87.59136762386888 41.81578503780536, -87.59139415471603 41.81583844912358, -87.59141262801658 41.81589070931159, -87.59142333363894 41.81591657405354, -87.59145586443539 41.81595849734951, -87.59150254507315 41.81598239928461, -87.59156198801584 41.81598113715012, -87.59157302485508 41.81597846360251, -87.59159282576481 41.81597914086986, -87.59160599979693 41.81598196966105, -87.59163662820291 41.8159980850265, -87.59167097544641 41.81600983191588, -87.5917117804606 41.81603424523141, -87.59175316645042 41.81607183478194, -87.59177972941185 41.81612250002495, -87.59189790580332 41.81623083883986, -87.59195471766608 41.81626688104951, -87.59198894484472 41.81628905611979, -87.59201423590754 41.8163226994447, -87.59203279771322 41.81636727669243, -87.59204645827849 41.81639151443129, -87.59207037707638 41.81641691580177, -87.59208824961583 41.8164576463389, -87.59211487517092 41.81650282418815, -87.59213639062672 41.81654577364084, -87.5921572673947 41.81658048615703, -87.5921505722161 41.81676136151037)), ((-87.59725153394855 41.82287912993058, -87.59701053463979 41.8223028462779, -87.59588293453049 41.8211108460249, -87.5954812341428 41.8201920454271, -87.59487029898028 41.81976870564228, -87.59491136800914 41.81977924022647, -87.5951150607724 41.81981362206611, -87.59548985030801 41.82016016149787, -87.59554192612809 41.82022580932476, -87.59564573346309 41.82032307513342, -87.59567682707113 41.82036279119304, -87.59568820098654 41.82039469802804, -87.59571579740184 41.82041957338996, -87.59573030761906 41.82043393676594, -87.59571318768388 41.82045523294885, -87.59570055692203 41.82046887235848, -87.59569445441215 41.82048914185608, -87.59569474793078 41.82052756268033, -87.59571049211213 41.82056224212035, -87.59575672867673 41.82062510780514, -87.59577912754379 41.82065543941117, -87.59580054605297 41.82070716836704, -87.5958247863305 41.82076879590104, -87.59586211817559 41.8208403860264, -87.59590253039194 41.82089937392603, -87.59594524634043 41.82094959329413, -87.59597398786968 41.82100246858031, -87.59606336671607 41.82114245059494, -87.5961125969129 41.82120039621201, -87.5961536929286 41.82126377786644, -87.59620352999922 41.82133270441204, -87.59630979366483 41.82140803094839, -87.59632286832586 41.82141964021921, -87.59636135679182 41.82145446495009, -87.5964179499549 41.82151026280894, -87.59648243620349 41.82158147862742, -87.59654625291986 41.82164720130342, -87.59658974056936 41.82169413252952, -87.59663881884443 41.82176524943005, -87.59667920793424 41.82182643094321, -87.59674606052208 41.82188339131781, -87.59678820020706 41.82191988642753, -87.59682525237945 41.82195206756787, -87.59683978332258 41.82196467377887, -87.59685349571038 41.82198452070525, -87.59686817531812 41.82204828239576, -87.59693101284175 41.82213540359245, -87.59696220943898 41.82216633966905, -87.59698474146273 41.82218514513826, -87.59700356616418 41.82220722023353, -87.59702018904427 41.82222928093687, -87.59704019232572 41.82227660974915, -87.59704252943716 41.82232876692012, -87.59705750446155 41.82236673326014, -87.59709294591092 41.82241141730061, -87.5971025579864 41.82246910826291, -87.5971130032586 41.8225180242791, -87.59712817289949 41.8225389768464, -87.59720557990315 41.82257130673308, -87.59719811083356 41.82258278533974, -87.59719641711341 41.82260253348821, -87.59719629140008 41.82261350865631, -87.59727092138687 41.82261782944359, -87.59727447048837 41.82262828040455, -87.59726182814131 41.82264301843681, -87.59724778086166 41.82265225947751, -87.59725110330723 41.8226824674407, -87.59726264146944 41.82270010441998, -87.59728739264348 41.82271727748858, -87.59732679283069 41.82273673983282, -87.59732811568209 41.82274937221557, -87.59732274126219 41.82277019393901, -87.59734362910355 41.82280435670057, -87.59734777329682 41.82282688625028, -87.59734466876419 41.82284168567843, -87.59733800290842 41.82284713122788, -87.59732848318583 41.82284542394307, -87.59730089768976 41.82281945079032, -87.59728127650898 41.82280285945178, -87.59726959338856 41.82279784490315, -87.5972527373347 41.82279609032508, -87.59724686138381 41.82279660179805, -87.59724612139017 41.82279714590947, -87.59724516139239 41.82281689821105, -87.59724842134881 41.82285259458298, -87.59725153394855 41.82287912993058)), ((-87.60476808258417 41.83457084871011, -87.60499286140625 41.8342501584768, -87.60499603131494 41.8342640426192, -87.60503332163134 41.83442211178847, -87.60476808258417 41.83457084871011)), ((-87.60608907448831 41.83856155353356, -87.60610185091183 41.83816728206018, -87.60611500648442 41.83829878378583, -87.60612905110636 41.83843916811514, -87.60613594126701 41.83852105081632, -87.60613498879938 41.83855127583922, -87.60608907448831 41.83856155353356)), ((-87.60639324071221 41.83899634302809, -87.6060847388529 41.8386953482032, -87.60608618332708 41.83865077274249, -87.60615627839611 41.83863502112484, -87.60639324071221 41.83899634302809)), ((-87.6074339891891 41.84038870445783, -87.6071614391041 41.8397458482044, -87.60671671757777 41.83931194845985, -87.60676104002667 41.83933868152847, -87.60682817452056 41.83937313502709, -87.60690335604389 41.83940983506439, -87.60694131397878 41.8394281867006, -87.60700484633588 41.8394565796668, -87.60709523007544 41.83951203668478, -87.60721091962495 41.8396516279898, -87.6073479140525 41.8401049906963, -87.60741319807528 41.84026855728347, -87.60745016250611 41.84034814941911, -87.60743661712895 41.840369491545, -87.6074339891891 41.84038870445783)), ((-87.6074837770186 41.84050613761711, -87.60746674119551 41.84046595569823, -87.60750090432951 41.84050284488112, -87.6074837770186 41.84050613761711)), ((-87.60786926673217 41.84107973702992, -87.6075626394715 41.8406921482764, -87.60750835124159 41.84056410014871, -87.60752760502869 41.84056030772689, -87.60757583127271 41.84063821932318, -87.60765630433343 41.84076193793285, -87.60773349881661 41.84088554850424, -87.60784934362428 41.84103060915571, -87.60786926673217 41.84107973702992)), ((-87.60877810004447 41.84340024644606, -87.60875683930769 41.84334802901976, -87.60877127222787 41.84336451715353, -87.60877810004447 41.84340024644606)), ((-87.60916806286059 41.84449132923991, -87.60908168195211 41.84419998467138, -87.60915012236094 41.84423871660043, -87.60916860600166 41.8444562607383, -87.60916806286059 41.84449132923991)), ((-87.61306215421034 41.85739800037311, -87.61303026468127 41.85693000305672, -87.6130395651754 41.85696238234902, -87.61304015929574 41.85696504772588, -87.61304879545042 41.85700352197557, -87.61305452718324 41.85703245490354, -87.61306542259955 41.85710826515711, -87.61307047731948 41.85718444877092, -87.61306965906529 41.85726070724304, -87.61306333166887 41.85737642013507, -87.61306215421034 41.85739800037311)), ((-87.61221506139927 41.85855671699644, -87.61221424290321 41.85851455116, -87.6126183425838 41.8584353514287, -87.61298238051472 41.85781150076181, -87.6129680171576 41.85785576409425, -87.61292211321205 41.85797841839015, -87.61287057063693 41.85809980265397, -87.61286094448577 41.85812682831085, -87.61283501049016 41.85818725843142, -87.61282702372286 41.8582022482004, -87.61280355128886 41.85824620052597, -87.61276675130387 41.85830337885202, -87.61272472406156 41.85835852216141, -87.61263859077528 41.85843191156808, -87.61253910863051 41.8584941852387, -87.61240293222298 41.85848987220275, -87.61226529558297 41.85850761444649, -87.61221506139927 41.85855671699644)), ((-87.6122911125003 41.86015619609489, -87.6124306424836 41.8600175514692, -87.6122309425046 41.8593748517691, -87.61223081444167 41.85936825444954, -87.6122428523459 41.8594042137785, -87.61225845818883 41.85944308776067, -87.61227631342855 41.85948746491487, -87.61231533374574 41.85956935055003, -87.61233237886412 41.85959758668462, -87.6123518832602 41.85963898212187, -87.61236575811978 41.85968160539478, -87.61239772554383 41.85977050082239, -87.61240334119201 41.8597869184308, -87.61242223823749 41.85985298141088, -87.61242227272471 41.85985314655689, -87.61243539686184 41.85992016095292, -87.61244271440029 41.85998766030708, -87.61240859619586 41.86008382479774, -87.61231561666486 41.86015190254716, -87.6122911125003 41.86015619609489)), ((-87.61236972672449 41.86045556908596, -87.6123072820548 41.86035379037398, -87.61239330263972 41.86035370767655, -87.61239051508991 41.86045487126415, -87.61236972672449 41.86045556908596)), ((-87.61310495457948 41.86206560728407, -87.61263768872057 41.8613719845196, -87.61264823405021 41.86137186136464, -87.61269225345876 41.86142718670374, -87.61274448655755 41.86149636851379, -87.6127845283609 41.8615493916806, -87.61287160558412 41.86167367595045, -87.61295348381559 41.86179995828685, -87.61301636217837 41.86190523820093, -87.61303001700311 41.86192810084666, -87.61310113692183 41.86205796545118, -87.61310495457948 41.86206560728407)), ((-87.6133519247047 41.86376029537283, -87.61331681169676 41.86251188069355, -87.61335489317646 41.86261312751467, -87.61339116040698 41.86272463366119, -87.61339399911768 41.86273335127773, -87.6134274051206 41.86285455384504, -87.6134298273017 41.86286450348761, -87.61344473378958 41.8629395144016, -87.61345376193714 41.86301506640542, -87.61345698679068 41.86309088197828, -87.6134553832392 41.86313568635062, -87.61345430019999 41.86316671702777, -87.61345417718911 41.86318759944913, -87.61344995342374 41.86327931297057, -87.61343989615131 41.86337077093314, -87.61342400751722 41.86346177987917, -87.61340329171476 41.86354787489515, -87.61340228878383 41.86355209420375, -87.6133519247047 41.86376029537283)), ((-87.61269100376217 41.8652831433576, -87.61268424379951 41.8652286523612, -87.61277620724016 41.86521930683758, -87.61274107793479 41.86528832335834, -87.61269100376217 41.8652831433576)), ((-87.60976897262805 41.86261253248067, -87.6097569423609 41.8640061525956, -87.6100093426233 41.864608252392, -87.6104639429193 41.8652355520845, -87.6105544426819 41.8658296524189, -87.61266796807755 41.86581218476639, -87.61266898965124 41.86584273093305, -87.61157844115409 41.86585898931923, -87.61048037231818 41.86586465127672, -87.61047263077683 41.86571114445353, -87.61045628367256 41.86538606765061, -87.61044764102031 41.86531891637193, -87.61042157903834 41.86531864255409, -87.61037087816 41.86532202786027, -87.61030712010385 41.86522642876942, -87.61026080173261 41.86515695464738, -87.61018063771697 41.86503512584475, -87.61010018154387 41.86490325201694, -87.61005195551698 41.8648174100711, -87.61002510568919 41.86476960136068, -87.60995548280482 41.8646343376268, -87.60996908455873 41.86462945620862, -87.6098944708977 41.86446466087234, -87.60988504014121 41.86446767475005, -87.60985976065184 41.86440766315913, -87.60981131414792 41.86429259483674, -87.60979356665091 41.86425159261837, -87.60976388195472 41.86417204218875, -87.60973986437472 41.8640914297569, -87.60972245785666 41.86399743877167, -87.60969913530217 41.8638714131087, -87.6096754500943 41.86375147821003, -87.60965736668024 41.86362979319401, -87.60965300899214 41.86358643376284, -87.60964508568321 41.86350770675374, -87.60961119340401 41.86350845421742, -87.60960534579888 41.86340515098644, -87.6096504488271 41.86340323980396, -87.60965239796163 41.86337698879521, -87.6096282403589 41.86337387332791, -87.60962783104763 41.86328962201424, -87.60965476104963 41.86328778923178, -87.60965726491656 41.86324812261205, -87.60967392021148 41.8632458952223, -87.60967817448402 41.86317451610616, -87.60968080502361 41.86313015789091, -87.60969414733054 41.86299846364164, -87.60971335968993 41.86286719056863, -87.609733448095 41.86276223954113, -87.60973840266129 41.86273644736706, -87.60976897262805 41.86261253248067)), ((-87.61064173531037 41.86065333207239, -87.61046514222861 41.8613459520917, -87.60979678809151 41.86242268588715, -87.60981985418051 41.86234980175073, -87.60988266473434 41.86215121185801, -87.60989760712039 41.86210877038096, -87.60993823714729 41.86200537515678, -87.60996940883913 41.86193658049768, -87.60998446805709 41.86190330608801, -87.61003622567169 41.86180272598369, -87.61009343322978 41.861703799008, -87.6102069656349 41.86152243372435, -87.61028472200606 41.86139827888119, -87.61036128383007 41.86139640081372, -87.61035427504792 41.86110930781174, -87.61034789309259 41.86084820707021, -87.61042486043468 41.86084937790466, -87.61042479503769 41.86083562907125, -87.61042901016656 41.8608068412449, -87.61043901570429 41.86077888514933, -87.61045451301599 41.86075236318734, -87.61047523509163 41.86072790567711, -87.61050062882826 41.86070605633832, -87.61051667525948 41.86069586611597, -87.6105301699836 41.86068730638249, -87.61056312251473 41.86067214596277, -87.61059878553536 41.86066084481627, -87.61063630887639 41.86065372690072, -87.61064173531037 41.86065333207239)), ((-87.61036427219973 41.85987210867988, -87.6106855426291 41.8601833515063, -87.61067062012415 41.86037197588767, -87.61066926669146 41.86028269147351, -87.61066903543704 41.86026743198269, -87.61066811268401 41.86020589951919, -87.61060359780468 41.86019770097397, -87.61058139845595 41.86019349868347, -87.61054660160052 41.86018329172095, -87.61051442096093 41.86016914757109, -87.61048554693116 41.86015140236138, -87.61046078212159 41.86013049995181, -87.61044067456842 41.86010696492779, -87.61042572830266 41.86008135013786, -87.61039005502781 41.85997629492029, -87.61037368035817 41.85992103184689, -87.61036466503468 41.85987415817055, -87.61036427219973 41.85987210867988)), ((-87.61044555250639 41.85947462909407, -87.61036172315853 41.85985328713941, -87.61035746908466 41.85980583645057, -87.61035812190264 41.85974788169958, -87.61036375011012 41.85970776845899, -87.6103774587922 41.85964438049669, -87.61039692004674 41.85958188000978, -87.61042206062136 41.8595205115959, -87.61044555250639 41.85947462909407)), ((-87.61047758230833 41.85928697217474, -87.61048524187839 41.8592953517836, -87.61048200650457 41.85930996600114, -87.61048200286062 41.85930571240758, -87.61047758230833 41.85928697217474)), ((-87.60993522103043 41.85830462912221, -87.60989484211299 41.8586494514175, -87.6103286923497 41.8591240857977, -87.61032471960843 41.85912195438989, -87.61027824146861 41.85909630599821, -87.61022109060218 41.8590599413219, -87.61021111699661 41.85905249648855, -87.6101679423804 41.85902036415257, -87.61007357591015 41.8589476776036, -87.61004803117487 41.85892443785055, -87.61000783242987 41.85888236201291, -87.60997257808927 41.85883790318471, -87.60994245166103 41.85879138964113, -87.60991773962672 41.85874310008565, -87.60989858674883 41.85869344542475, -87.60988552404056 41.85864413236317, -87.60988513642491 41.85864272952988, -87.60987749156347 41.85859136359124, -87.60987572519389 41.85853970646932, -87.60987979397591 41.85848808473174, -87.60988973109637 41.85843693986256, -87.6099054230952 41.85838659908371, -87.60993522103043 41.85830462912221)), ((-87.60934360965047 41.85686013207828, -87.6100108420022 41.8576588516047, -87.60998115803379 41.8579123426533, -87.60997357920175 41.8578305215143, -87.60997142387114 41.85781988768346, -87.60996822528597 41.85780397804318, -87.60994927630414 41.85772973726561, -87.60992459050998 41.85765641973975, -87.60989423823091 41.85758430115349, -87.60985836424697 41.85751363022042, -87.60981703767501 41.857444626911, -87.60979301580068 41.85741006320231, -87.60977043970594 41.85737756596298, -87.60971867970981 41.85731261298861, -87.60966201111832 41.85725001628625, -87.60960054189556 41.85718996807782, -87.60953452690821 41.85713271914009, -87.60946418302959 41.85707840612383, -87.60943937556596 41.85704852978538, -87.60943886777852 41.85704792202597, -87.60940723467714 41.85700488513138, -87.60940579433179 41.8570024059508, -87.60938087094183 41.85695993274972, -87.60935995997846 41.85691336873177, -87.60934467663679 41.85686560444843, -87.60934360965047 41.85686013207828)), ((-87.60948018439009 41.85624964158315, -87.60946874208599 41.8564678513658, -87.60934357423281 41.85685995042786, -87.60933520486515 41.85681702553127, -87.60933153546988 41.85676799032311, -87.60933374088775 41.85671890982132, -87.60934181609581 41.85667019563176, -87.60935568145169 41.85662220345514, -87.60936862277207 41.85659111087143, -87.60937518871508 41.85657531764798, -87.60940025684768 41.85652992024544, -87.6094058602622 41.85652142203725, -87.60942605857126 41.85648466659453, -87.60944069468171 41.85644650396336, -87.60944961641408 41.85640734427572, -87.60948018439009 41.85624964158315)), ((-87.60651719382078 41.85335798582755, -87.6065141406287 41.8533261503619, -87.6089216408192 41.8530714504899, -87.6096335416879 41.8533250503176, -87.60962845498283 41.85342205602555, -87.60959768235442 41.85340005474572, -87.60958810551465 41.85339321636307, -87.60952530972003 41.85335352208723, -87.6094588767114 41.85331729087867, -87.60941619353463 41.85329731810431, -87.60938917239388 41.85328468941844, -87.60931652855169 41.85325588280082, -87.609241305197 41.85323103904303, -87.60916394401447 41.85321024435029, -87.60908473750655 41.85319366384598, -87.60900419735727 41.85318132792142, -87.60892265478012 41.853173321265, -87.60884058657381 41.85316967542251, -87.60875835656451 41.85317044729879, -87.60867640948693 41.85317558234458, -87.60839363197501 41.8531950388731, -87.60783820009188 41.8532408456077, -87.60750340337435 41.85326738027072, -87.60689621328623 41.85332317455345, -87.60651719382078 41.85335798582755)), ((-87.60681774674596 41.85649183462954, -87.60652261725112 41.85341453563703, -87.60659665988791 41.85341071443363, -87.60661784521453 41.85373766168367, -87.60662584731425 41.8538607923742, -87.60664048686435 41.85405270806539, -87.6066473987263 41.85414893761511, -87.60667687939183 41.85456032336859, -87.60669698153339 41.85484031033649, -87.60670540530442 41.85497167587738, -87.60673187562101 41.85538312401822, -87.60674760205679 41.85562814809204, -87.6067543495422 41.85570659353662, -87.60676150424472 41.85579440055595, -87.60679494992881 41.85620578270867, -87.60681370207551 41.85643661065181, -87.60681774674596 41.85649183462954)), ((-87.60597678421306 41.86007993052082, -87.6059706412491 41.8599997515437, -87.60703504399875 41.85997406130816, -87.60683766189673 41.86005692502448, -87.60641760915588 41.86006870242537, -87.60597678421306 41.86007993052082)), ((-87.60629459457002 41.86486174804538, -87.6067148417559 41.8650559522534, -87.6073229418807 41.8646712527593, -87.6074255418761 41.8637686525238, -87.606988641992 41.8626399519253, -87.6067325410578 41.8624346519697, -87.60660606407484 41.86217242983156, -87.60662433788167 41.86216472334392, -87.60665002347579 41.86217218639801, -87.60668890782793 41.86218135096632, -87.60671406782798 41.86218329418956, -87.60671924086168 41.8621835733521, -87.60673748121529 41.86218733833178, -87.60675332574284 41.86219506758804, -87.60676527895843 41.86220603870477, -87.6067721777253 41.8622192267569, -87.60678355699177 41.86224525907457, -87.60679034874188 41.86227441880574, -87.60680670369661 41.86231167941159, -87.60684452392529 41.8623272315719, -87.60687838543329 41.8623325229019, -87.60691879267178 41.86234639051565, -87.6069548920858 41.86237441809183, -87.60695777568887 41.86237915561623, -87.6069770247323 41.86241050774543, -87.60699031276775 41.86244947670715, -87.60700285227161 41.86249626424023, -87.60708686961679 41.86273765753509, -87.60710658695577 41.86278918256236, -87.6071935907178 41.86301643459967, -87.60726641156431 41.86319927809409, -87.60733423835939 41.86336952075673, -87.60743387713772 41.86360917469113, -87.60748051776577 41.86372132590029, -87.60749679064908 41.86376272887405, -87.6075250305655 41.86384625019551, -87.60754749722749 41.8639307506708, -87.60756422864999 41.86401598328663, -87.60756466478607 41.86401955384098, -87.60757511607414 41.86410172699494, -87.60758012748474 41.864187791693, -87.60757930024563 41.86427395533349, -87.60757256326852 41.86435994441424, -87.60756879614888 41.86437482187957, -87.60755244507683 41.86443133322578, -87.60754220762179 41.86446661500259, -87.60750986574202 41.86455738218083, -87.60747192015593 41.86464690646194, -87.6074510758354 41.86468195626064, -87.60740697207478 41.86474797826895, -87.60735767912956 41.86481190899618, -87.60732822492156 41.86484534012358, -87.6073033823305 41.86487355860763, -87.60724430357531 41.86493265297656, -87.60718059381239 41.86498905638657, -87.60711254909823 41.8650424951795, -87.60709217191879 41.86505891414223, -87.60704202119422 41.86509465642218, -87.6069880183445 41.86512710948464, -87.60693049291183 41.86515602595643, -87.60686996317965 41.86518122003202, -87.60680675985935 41.86520247124032, -87.60674136463302 41.86521964925394, -87.60667429083269 41.86523261708607, -87.60664326131842 41.86523014220028, -87.60661337967507 41.86522336692283, -87.60658571179944 41.86521257132685, -87.60656124139126 41.86519806432965, -87.60654084620906 41.86518040006332, -87.60652521550203 41.86516021269701, -87.60652454423632 41.86514489554255, -87.60650147300922 41.86512688474929, -87.6064720626957 41.86510449764429, -87.60638692503058 41.86504309189166, -87.60636284162197 41.86502372888403, -87.60633569131562 41.86499619741128, -87.60631345260327 41.86496628153698, -87.60629655787918 41.8649345059573, -87.60629459457002 41.86486174804538)), ((-87.60606665362793 41.86479290397061, -87.6060667415054 41.8647564528157, -87.60614027853813 41.86479043567551, -87.60606665362793 41.86479290397061)), ((-87.60598334648328 41.86572510024174, -87.60605080067042 41.86564407776034, -87.60605108627973 41.86568086889101, -87.60601104302565 41.8657057526148, -87.60598334648328 41.86572510024174)), ((-87.60561337418359 41.86629028991408, -87.60562964132311 41.8661499525894, -87.60565712341574 41.86611694252089, -87.60565690733499 41.8661174242363, -87.60563614582868 41.86617744687503, -87.60562113496681 41.86623841072673, -87.60561337418359 41.86629028991408)), ((-87.60564742969358 41.86658815560692, -87.6055859414909 41.8665269529548, -87.60561163353108 41.86630530659941, -87.60560857382382 41.86636195949792, -87.60561110200447 41.86642394145638, -87.60561950827852 41.86648560308527, -87.60563368349354 41.86654669698384, -87.60564742969358 41.86658815560692)), ((-87.61535609278516 41.86830918318084, -87.61535151591202 41.86830431016207, -87.61535246001087 41.86830427918702, -87.61535609278516 41.86830918318084)), ((-87.61625681893432 41.87159174726238, -87.61624034559929 41.871444752823, -87.6162066450445 41.8689330529388, -87.615722645042 41.8686994531248, -87.61560289251099 41.86857195203131, -87.61563313152323 41.86859717157832, -87.61570508056076 41.86865063922623, -87.61578106787368 41.86870089439708, -87.61586080130625 41.86874779749655, -87.6159440616297 41.86879115505013, -87.6160305207503 41.86883085605436, -87.61611988645298 41.86886673265023, -87.61621190376508 41.86889870091142, -87.61621232825823 41.86892310129287, -87.61621951327416 41.86933467264721, -87.61622666131409 41.86974627228139, -87.61622780242182 41.86981167481052, -87.61623582952545 41.8701578018985, -87.61624536574071 41.87056936122625, -87.61625047287916 41.87078912465704, -87.6162548282543 41.8709809195152, -87.61626414203199 41.87139258716444, -87.61626785065535 41.87155674368059, -87.61626647183061 41.87157180153115, -87.61625949479259 41.87158825033708, -87.61625681893432 41.87159174726238)), ((-87.61626009313211 41.87196804609422, -87.6162601451273 41.8719274532211, -87.61627009041661 41.8717765539155, -87.61627346228772 41.87178065458355, -87.61627441288906 41.87180404204339, -87.61628092246336 41.87196739193918, -87.61626009313211 41.87196804609422)), ((-87.61618022202653 41.87537295603028, -87.6161491456776 41.8750021539917, -87.61619485145131 41.87452908633885, -87.61619083702138 41.87462633320749, -87.61618935146156 41.87468702634693, -87.61618232998438 41.87497345418581, -87.61618140528159 41.87509883239024, -87.61617973476885 41.87532061206056, -87.61618022202653 41.87537295603028)), ((-87.61582014277711 41.8805651855859, -87.615817945609 41.8805624545316, -87.61647634414049 41.88050387855656, -87.616477676979 41.88055681431354, -87.61644641496781 41.88055453408888, -87.61637013558702 41.88055210892678, -87.61629384414651 41.88055404753209, -87.61582014277711 41.8805651855859)), ((-87.6165044767161 41.88168445883239, -87.6164722683756 41.88141386816302, -87.61649765898835 41.88141377388551, -87.61650439881952 41.88168137866656, -87.6165044767161 41.88168445883239)), ((-87.60118510038927 41.89114622933916, -87.6012302426964 41.891097257689, -87.6094877454961 41.8909253571873, -87.609497072254 41.88977694217512, -87.60949717394195 41.88978035157065, -87.6095094184065 41.89019214661577, -87.60952761185797 41.89080249747655, -87.60953544573921 41.89101294664188, -87.60746207467167 41.89104681425857, -87.60521340779037 41.89108348286384, -87.60452218486093 41.89109476870048, -87.60389620712391 41.89110127685808, -87.60275998020788 41.89112056343411, -87.60170350891335 41.89113795862109, -87.60118510038927 41.89114622933916)), ((-87.5985831830628 41.89144928009956, -87.59858334201429 41.8914225579645, -87.60010980610286 41.89142307801933, -87.5985831830628 41.89144928009956)), ((-87.60231751965519 41.89329328987404, -87.6023176431943 41.8932449577532, -87.60982101126784 41.89309898364329, -87.60982147697032 41.8931153253702, -87.60982234621434 41.89314576386929, -87.6051891279148 41.89322125365759, -87.60429304647259 41.89323582013814, -87.603711170941 41.89324450212767, -87.60235195625154 41.89326458985885, -87.60231751965519 41.89329328987404)), ((-87.60231420497475 41.89459009001113, -87.60231555393619 41.89406233675706, -87.60231900184972 41.89417683783712, -87.60231646621615 41.89440248064004, -87.60231515352939 41.89451759318818, -87.60231487379066 41.89454212403873, -87.60231420497475 41.89459009001113)), ((-87.6023518298326 41.89629569012808, -87.6023098436758 41.896296357979, -87.60231394254092 41.89469276181537, -87.60232057408197 41.89484089738919, -87.60232289142834 41.89489222959396, -87.60232496606059 41.89504520668343, -87.60232878219365 41.89527102655001, -87.60233014580355 41.89535111115333, -87.60233144718178 41.89560413779878, -87.60233400708164 41.89568576870467, -87.60233779126787 41.89580791146911, -87.60234170973899 41.89603770972386, -87.60234188504971 41.89607393499751, -87.60234267446022 41.89622366595322, -87.60235074299349 41.89626334432106, -87.6023518298326 41.89629569012808)), ((-87.60685686488212 41.89626090820566, -87.6068558451246 41.8964085580633, -87.6058695448603 41.8964510582866, -87.60567997066806 41.89628287167302, -87.6056870261102 41.89628154198371, -87.60570441395276 41.89627385826402, -87.6057180927435 41.89626274819017, -87.60572695273817 41.8962491937537, -87.60586310326572 41.89624986316251, -87.60586507649468 41.89629600732945, -87.60655409270849 41.89628349317448, -87.60655281621348 41.89623724365883, -87.60668907902112 41.89623464935599, -87.60669818953605 41.89624738481701, -87.60671213796215 41.89625790061837, -87.60672961530501 41.89626492722586, -87.60674908587963 41.89626784871977, -87.60676882656708 41.89626638279191, -87.60678714574711 41.89626070864028, -87.60679284190124 41.89626365329315, -87.60680982733305 41.89626880959944, -87.60682807217705 41.89626969396411, -87.60684581347923 41.89626626559549, -87.60685686488212 41.89626090820566)), ((-87.60768111285297 41.89624561647865, -87.60767974955995 41.89624559867972, -87.60768048723673 41.89624528108558, -87.60768111285297 41.89624561647865)), ((-87.60775763717274 41.89624661556798, -87.60775644131267 41.89624659995502, -87.60775713058507 41.89624634195086, -87.60775763717274 41.89624661556798)), ((-87.60783415874872 41.89624761462148, -87.60782658692204 41.89624751576492, -87.60783063443517 41.89624576355746, -87.60783415874872 41.89624761462148)), ((-87.60791536905975 41.89624867489037, -87.60789421241942 41.89624839867264, -87.60790356555422 41.89624386399222, -87.60791536905975 41.89624867489037)), ((-87.61191530675288 41.89349690714854, -87.6102374462812 41.8962019574855, -87.61018700083973 41.89620299946658, -87.61027136370042 41.89609787631409, -87.61032156672096 41.89601720924284, -87.61033038108354 41.89600425706993, -87.61038315699555 41.89592654387222, -87.61041530084987 41.89587065353965, -87.61045271309065 41.89581337045838, -87.61047761175307 41.89577143091583, -87.61048810692968 41.89574954310693, -87.61051495293944 41.89570418497209, -87.61054062802857 41.89567152526534, -87.61060674485415 41.89556705649535, -87.61067775987804 41.89545485287552, -87.61078968624028 41.89527454748042, -87.61082778275994 41.89520214701997, -87.61085681970241 41.89515798342924, -87.61095465434599 41.89500914790161, -87.61098274906729 41.8949703567756, -87.61104439866477 41.89487428498727, -87.61112065003977 41.89475670732244, -87.61113683530407 41.89473175342741, -87.61119947764641 41.89462569987749, -87.61123407133221 41.89457402393514, -87.61127583664623 41.89451168891374, -87.61135186176315 41.89439468732546, -87.61137335947679 41.89434803300342, -87.61138013513421 41.89433336705127, -87.61151785122946 41.89411971565671, -87.61159254911507 41.89399658426617, -87.6116307446585 41.89393804280536, -87.6117187578172 41.8938032233393, -87.6117842154511 41.89370473186794, -87.61185126733142 41.89358201987304, -87.61185656939531 41.89357425928205, -87.61188402538768 41.8935432298229, -87.61190087036042 41.89351828133079, -87.61191530675288 41.89349690714854)), ((-87.61922221593399 41.90108487776132, -87.6190462485345 41.9009252583448, -87.6162543478697 41.896755657239, -87.61488810619119 41.89464785980321, -87.61490652226868 41.89467120744143, -87.61497860017219 41.89477791431516, -87.61513085329712 41.89500326302385, -87.6152580121517 41.89519656659756, -87.61553042647913 41.89561064534953, -87.61554313868028 41.89562996217035, -87.61581309366035 41.89603763161061, -87.61587236610336 41.89612710752093, -87.61607221024806 41.89643277359693, -87.61621203581817 41.8966466815161, -87.61632172407208 41.89681643900333, -87.61633842372939 41.89684387612112, -87.61636158015678 41.89688186259491, -87.61653849702078 41.89711949314856, -87.61664215748928 41.89726347150324, -87.6168485744253 41.8975502425367, -87.61693994029211 41.8976834682839, -87.61720092231556 41.89806404766372, -87.61721392185163 41.89808413392144, -87.61747844313577 41.89849346629621, -87.61749483503452 41.89851884369457, -87.61764842192294 41.89875122195008, -87.61773015314625 41.89887599019303, -87.61795057172236 41.89921240646274, -87.61799663294244 41.89928146443908, -87.61819042005982 41.89957210517449, -87.61826487507041 41.89968414962582, -87.61848679499757 41.90001818724339, -87.61854031664694 41.90009692369325, -87.61880337501792 41.90048404322624, -87.6188102300695 41.90049514458976, -87.61893211245943 41.90069252718759, -87.61897908855971 41.90077210058316, -87.61903537189487 41.90085700050596, -87.6190678696253 41.90090078078811, -87.61909685832001 41.90093984714643, -87.61916340202849 41.90102047493556, -87.61922221593399 41.90108487776132)), ((-87.61982818167689 41.90152419817854, -87.61974194904531 41.9015204585164, -87.6195069489131 41.9013431580571, -87.61944550988937 41.9012874269322, -87.6194860316361 41.90131677249668, -87.61956192829562 41.90136565281664, -87.61964157342219 41.90141112581897, -87.61963812410677 41.90141733433673, -87.61976421413493 41.90148793066167, -87.61979717905888 41.9015063838254, -87.61982818167689 41.90152419817854)), ((-87.62467062221316 41.91121102542391, -87.62460745188589 41.9111908599712, -87.62467980494138 41.91101002939867, -87.62467838989208 41.91104100219111, -87.62467062221316 41.91121102542391)), ((-87.62025747201619 41.91364738537359, -87.6202572502398 41.9135686600165, -87.62097645081739 41.9129154597794, -87.6229356514265 41.9126617599785, -87.62405275174051 41.9117666600339, -87.62466320217044 41.91137344092191, -87.62466179675801 41.91140420372525, -87.62466294392912 41.91141637624153, -87.6242449766313 41.91173404729589, -87.62375790583393 41.91210425338529, -87.62368919636506 41.91215799954415, -87.62334053024769 41.9124307060051, -87.62329009103962 41.9124701579701, -87.62320658637798 41.91252617265604, -87.62311926062769 41.91257884340612, -87.62311268086566 41.9125823973723, -87.62302837247643 41.91262803432615, -87.62293414529006 41.91267358131284, -87.6228368731371 41.91271540522289, -87.62273681428877 41.91275334217013, -87.62263422762597 41.9127873391395, -87.6225293714753 41.91281726078655, -87.62242257635896 41.91284305454137, -87.62232301233398 41.91286304804333, -87.62220791868849 41.91288179051283, -87.62209173394045 41.9128962190442, -87.62197468062358 41.91290627930504, -87.62192313694443 41.91290933482438, -87.62180937745352 41.91291378993294, -87.62169759949177 41.91292193365013, -87.62158650667349 41.91293444455165, -87.62147646775496 41.91295129748321, -87.62136774078046 41.91297241254428, -87.62126065771197 41.91299773663811, -87.62115551083065 41.91302721697189, -87.62112402030762 41.91303747606165, -87.62105256035464 41.91306074374945, -87.62095206475817 41.91309826506776, -87.62085701182512 41.91313848196875, -87.62077863531559 41.91317572874414, -87.62070352692248 41.91321659068525, -87.62063205113039 41.91326093284938, -87.6205644720156 41.91330856423775, -87.62050108259945 41.91335926823577, -87.62044206914264 41.91341288080424, -87.62039163061951 41.91346514702762, -87.62038772762631 41.91346918397505, -87.62033828379172 41.91352793299937, -87.62029388514119 41.91358888181899, -87.62025747201619 41.91364738537359)), ((-87.62175090450953 41.91352917771228, -87.6212921507926 41.913642559962, -87.6202589509185 41.9141723606202, -87.62025784716658 41.91378055486955, -87.62025855931419 41.91377872674855, -87.6202917368866 41.9137092025349, -87.62033040872409 41.91364127553867, -87.62037442472189 41.91357519399809, -87.6204235965573 41.91351117408058, -87.62044228702521 41.91348897983413, -87.62046939154389 41.91346346242269, -87.62051929239207 41.91341648863572, -87.62057238161533 41.91337035925806, -87.62057501749133 41.91339098393841, -87.62056468015828 41.9134317813048, -87.62054796576416 41.91346126058735, -87.62054766599542 41.91346175323616, -87.62051894442901 41.91350729375607, -87.62049365136822 41.91356155392754, -87.62048589670597 41.91358485887926, -87.62048642418762 41.91361002753425, -87.62049504348474 41.91364109025385, -87.62050565951149 41.91366763876672, -87.62051533854105 41.91369928407642, -87.62052019773306 41.91371487400544, -87.62055320475488 41.9137598095185, -87.62059011984849 41.91378701468402, -87.62061724177421 41.9137928630319, -87.62066613443355 41.91379366128053, -87.62072102250416 41.91378727912645, -87.62077322783958 41.91377091793291, -87.62084308080151 41.91373699285314, -87.62089556008453 41.91370927275482, -87.62094778673487 41.91367790179257, -87.62101197709863 41.91364100488103, -87.62109438608306 41.91360463394734, -87.62116360314955 41.91358818627317, -87.62125905311518 41.91357826757125, -87.62132440990342 41.91356882162972, -87.6213827341267 41.91356051179024, -87.62142011353424 41.91355594142529, -87.62146496132748 41.91355029111865, -87.62150831329384 41.91354389222692, -87.62155260994783 41.91353837622955, -87.62162372941765 41.91353255967825, -87.62173208490023 41.91352842883869, -87.62175090450953 41.91352917771228)), ((-87.6267982712054 41.91629545159509, -87.62680702278503 41.91628098730743, -87.62681257645652 41.91628594388982, -87.6267982712054 41.91629545159509)), ((-87.62754776675955 41.9178188352982, -87.62777465346331 41.9175821601085, -87.6276718737698 41.91732178246043, -87.62769224105752 41.91734689175206, -87.62774497345241 41.91741620589841, -87.62785651490394 41.91753039296363, -87.62788987560012 41.9175705810123, -87.62790894677434 41.91760033471183, -87.6279175737351 41.9176310415291, -87.62791503949468 41.9176544059344, -87.62790852659818 41.91766194025791, -87.62789132249655 41.91767270141924, -87.62785578722271 41.91764306500313, -87.62759290043726 41.91779273892411, -87.62754776675955 41.9178188352982)), ((-87.62826259210891 41.91928940254493, -87.62842570926833 41.91917612112906, -87.62843620227031 41.91921948338781, -87.6284358035991 41.91923888123448, -87.62841927328617 41.9192585664347, -87.62839652901626 41.91927190094385, -87.62836525029685 41.91923957404211, -87.62826259210891 41.91928940254493)), ((-87.62861278018315 41.92092789031808, -87.62861255329381 41.9209275613045, -87.62891856884364 41.92073549924633, -87.62892835831181 41.9207814962426, -87.62893387889036 41.92082077278234, -87.62879459709625 41.92088953947525, -87.62876114847033 41.92085717138344, -87.62861278018315 41.92092789031808)), ((-87.62947812201696 41.92244521217258, -87.62938027427272 41.92222003412804, -87.62942690699794 41.92228556145995, -87.62944752069818 41.92232567143194, -87.62945610820944 41.92234671683192, -87.62947969747138 41.92239699921213, -87.62948572847596 41.92242661890494, -87.62948583417935 41.92244050504642, -87.62947812201696 41.92244521217258)), ((-87.62959804788446 41.92656211973524, -87.629466054752 41.9261172618903, -87.6299635544481 41.9260119620249, -87.6298525546535 41.9256215617674, -87.6293122542976 41.9246016611748, -87.6295134542486 41.9242629611346, -87.6286975538415 41.9229328609482, -87.62878800049425 41.92287768223004, -87.62900319972562 41.92274997529028, -87.62901312121457 41.9227599422447, -87.62904217483889 41.92278114120319, -87.62910558940814 41.92283528948002, -87.62915017135596 41.92288128165299, -87.62920757063506 41.92294110096493, -87.6292731682196 41.92299718431429, -87.62935061807373 41.92304488699058, -87.62943261452901 41.92310008149798, -87.62949513421246 41.9231456074872, -87.62958141445071 41.9231988536785, -87.62970559820675 41.9232973905718, -87.6297881658452 41.92337426850499, -87.62986447586829 41.92344894010435, -87.62990313152726 41.92348243607997, -87.62992119889836 41.9235101821824, -87.62993780264854 41.92354746756322, -87.62997946917415 41.92367823774722, -87.63004463368409 41.9239078895091, -87.63008003775217 41.92403681037869, -87.63008375329325 41.92405637172308, -87.63008118406508 41.9240862136733, -87.63008315180862 41.92411114298807, -87.6300853185146 41.92412806011821, -87.63008366939931 41.92415442281588, -87.63009200074076 41.92418883095517, -87.63012266566163 41.92430087407968, -87.63014638014266 41.9244167977681, -87.63016969191354 41.92456943722186, -87.63019161558594 41.92467769409865, -87.63020527090862 41.92478288059503, -87.6302175295792 41.92487447573519, -87.63023103007338 41.92498704378232, -87.63024864504762 41.92515641551574, -87.63025924531799 41.92528202801569, -87.6302701116538 41.92543020046558, -87.63027257137009 41.92557104924929, -87.63027331574011 41.92568729847908, -87.63027496880015 41.92574762686203, -87.63027129832496 41.92576738994904, -87.63026518861126 41.92578055663381, -87.63025869712156 41.92579047429219, -87.6302188511649 41.92584171216537, -87.63017717453701 41.92588577639555, -87.63010246333468 41.92595650284501, -87.63002739535803 41.92602945207399, -87.62996059297143 41.92609268108725, -87.62994013509278 41.92610850023736, -87.62990629771312 41.92613798404448, -87.62987647312991 41.92616014010893, -87.62982117530512 41.92613266041187, -87.62976855662636 41.92611225057647, -87.629728326555 41.92609485267582, -87.62970627045512 41.92609224771738, -87.62969966773014 41.92610120830071, -87.62969772013057 41.92611445067947, -87.62969124792571 41.92612497654262, -87.62966281467401 41.9261542752296, -87.6296505433907 41.92617409535392, -87.62964297325806 41.92619089902634, -87.62963841808097 41.92619959142078, -87.62962913102784 41.92622322371767, -87.62962011841729 41.92626424837377, -87.62961273035398 41.92631450548934, -87.62960300253013 41.92635042205762, -87.62960194318552 41.92637659543383, -87.62959956606427 41.92639889138161, -87.62960255845108 41.92643085272927, -87.62959865919562 41.92645475843806, -87.62960019526699 41.92649217131063, -87.62959931193799 41.92652095277492, -87.62959804788446 41.92656211973524)), ((-87.63048729017042 41.92804564020483, -87.6301892548294 41.9276030617344, -87.62998586338551 41.92729677966428, -87.63003116036738 41.92734277667576, -87.63006015359994 41.92736627959007, -87.63008665907157 41.92738880705723, -87.63010567815547 41.92740371516706, -87.6301225351784 41.92741894125972, -87.63014311932892 41.92743753455758, -87.63022819705014 41.92752405916608, -87.63027713211113 41.92757551108354, -87.63032679727986 41.92762737880867, -87.63038481514424 41.92769496714818, -87.6304712532733 41.92779481019485, -87.63052054022897 41.92785776244443, -87.63054480461201 41.92789353067119, -87.63057297970664 41.92793472921711, -87.63059553007017 41.92796947185489, -87.63061144274893 41.92799639976221, -87.63048729017042 41.92804564020483)), ((-87.63182111363358 41.93245798548082, -87.63176620787758 41.93244392958226, -87.63182144880348 41.9324442008045, -87.63182111363358 41.93245798548082)), ((-87.63103269110171 41.93298000471059, -87.6312690554693 41.932895463104, -87.6313600560599 41.9325412632518, -87.63182079531857 41.93247107695882, -87.63181810306664 41.9325818023616, -87.63159975655228 41.93257920575426, -87.63151791937585 41.93259530890217, -87.63149591679257 41.93260433955063, -87.63147374583752 41.93261531865826, -87.63144137912548 41.9326377880441, -87.63142165162063 41.93266060883582, -87.63140048765081 41.93269025394559, -87.63139073457272 41.93271834999862, -87.6313838519285 41.9327528310088, -87.63138043013971 41.93280014785232, -87.63138353004659 41.93296995047993, -87.63103269110171 41.93298000471059)), ((-87.63112985626958 41.93443381137119, -87.63037415519889 41.9336258629671, -87.6304638553607 41.9331834629895, -87.63066398830048 41.93311188045601, -87.63063582759466 41.93314394341387, -87.63061245059482 41.93316435480771, -87.63058190214193 41.93317517187609, -87.63053027196153 41.93321478422681, -87.6304964239869 41.93327500526274, -87.63047162737742 41.93332773482344, -87.6304655048354 41.93337986410113, -87.63044265658165 41.93342239736592, -87.63042364396053 41.93350386546877, -87.63043115393285 41.9335963646884, -87.63044280947653 41.93366613917139, -87.63050377630756 41.93374678075009, -87.63057528140588 41.93381162631731, -87.63063841241809 41.93386933913582, -87.63070741521365 41.93391440901791, -87.63072980871294 41.9339435806985, -87.63080158511517 41.93404064270174, -87.63093766619976 41.93420569005965, -87.63112985626958 41.93443381137119)), ((-87.6324121944872 41.93718750760328, -87.63222215675501 41.9368957633097, -87.6320572560187 41.9359147637774, -87.63126065274453 41.93458906113742, -87.63144812218594 41.93481157926696, -87.63145588478854 41.93483097235373, -87.63150163551261 41.93489532976556, -87.63155922761476 41.93498234471661, -87.63157991204706 41.93500974893329, -87.63159215086877 41.93504656770501, -87.63162388685434 41.93507845792234, -87.63165279087941 41.93512391360053, -87.63172147413709 41.93518816463729, -87.63177284916699 41.93527593621185, -87.63181919160523 41.93533974771645, -87.63184240694613 41.93539784769345, -87.63187484077251 41.9354465092689, -87.6319195095205 41.93548226488393, -87.63193544676564 41.9355370539374, -87.63194220680522 41.93555408221029, -87.63195983968622 41.93559513418928, -87.631998092496 41.93564945527255, -87.63202766750349 41.93568728740062, -87.63204290645788 41.93575225294209, -87.63210056055958 41.93584042083513, -87.63210692410837 41.93586677618961, -87.63212489201926 41.93594100776752, -87.63214732637573 41.93604364044671, -87.63216975567015 41.9361366422607, -87.63218511391904 41.93625449065809, -87.63219560295707 41.93634044841427, -87.63220604202874 41.93638738337918, -87.6321879839104 41.93647574691877, -87.63220839669742 41.93656835139083, -87.63223642142404 41.9366974729741, -87.63225900098392 41.93676682053368, -87.63227770274472 41.93682428829851, -87.63229374024013 41.93691041849886, -87.63242555923296 41.93718220749534, -87.6324121944872 41.93718750760328)), ((-87.63557724602141 41.93968735104032, -87.63499507839173 41.94013930847399, -87.63504248966898 41.94008095734232, -87.63510234015024 41.94001576258549, -87.63516690304557 41.9399531482975, -87.63523606783649 41.93989330561913, -87.63530957367642 41.93983642587465, -87.63538719833008 41.93978269897664, -87.63546868404163 41.93973223558852, -87.63550174804151 41.93971333599234, -87.63557724602141 41.93968735104032)), ((-87.63760190111886 41.94035706729591, -87.6374271585307 41.9402046634718, -87.63702854915481 41.93985926273187, -87.63703756638631 41.93981308243222, -87.63711549994333 41.93982634307831, -87.63752919650801 41.94028803988768, -87.6375412578395 41.94028407980895, -87.63760190111886 41.94035706729591)), ((-87.64022963867404 41.94528290371827, -87.640376359699 41.9447384648774, -87.63993355552603 41.94409023152114, -87.64037114083806 41.94471287537768, -87.64040569192152 41.94475037721553, -87.64043525467173 41.94479018261688, -87.64045954179385 41.94483190346659, -87.64047833262364 41.94487521022845, -87.64049145253135 41.94491966332608, -87.64049879276072 41.94496484992429, -87.64050024814777 41.94501033086428, -87.64049586070965 41.94505572111235, -87.6404855977583 41.94510060966188, -87.64046960879762 41.94514452952521, -87.64044805040987 41.94518707303158, -87.64042106807008 41.94522790790847, -87.64041264253424 41.94523888906866, -87.64030270355171 41.94525869706083, -87.64022963867404 41.94528290371827)), ((-87.64014324700319 41.94560347795875, -87.6401726904913 41.94549422178002, -87.6401780169769 41.94549990187252, -87.64019677576968 41.94551579434624, -87.64014324700319 41.94560347795875)), ((-87.63692149589801 41.94494201591817, -87.63746445848049 41.9457744644289, -87.638574858963 41.9460780647499, -87.63901765920311 41.9462226649431, -87.6397520592934 41.9462338650608, -87.640309759662 41.9464987648803, -87.6403091598294 41.9466530651875, -87.6410718598664 41.9473868652954, -87.6417929599275 41.9478830644803, -87.64211086047079 41.9478713651801, -87.6420773210323 41.94776350995065, -87.64212737864337 41.94784541406865, -87.64213434162852 41.94785766855349, -87.642140972501 41.9478800458075, -87.64214182880495 41.94790296567908, -87.64213676941235 41.94792557414325, -87.64212609724255 41.94794705222928, -87.64211011502361 41.94796663283001, -87.64208942036622 41.94798357668321, -87.64206482622575 41.9479972871499, -87.64203714912342 41.94800724609724, -87.64200745732381 41.94801307741246, -87.64196134375955 41.94801469351491, -87.64191527793925 41.94801189115684, -87.6418701025631 41.94800475801431, -87.64182648167744 41.94799338178638, -87.64178514624581 41.94797798476804, -87.64177753688438 41.94797459099579, -87.64165709394013 41.94792186237029, -87.64153974123154 41.94786539266845, -87.64142566430809 41.94780529306077, -87.64132859889784 41.94774944018298, -87.64131508106597 41.94774167463873, -87.64120817471721 41.94767459230338, -87.64110516260602 41.94760424166545, -87.64103072469349 41.94754992423317, -87.64096054539651 41.94749255891679, -87.64089477254207 41.9474323104451, -87.64083369889197 41.94736940177486, -87.64077746528689 41.94730399731804, -87.64047352594959 41.94691484517123, -87.64040944920553 41.94683391529358, -87.6403044080359 41.94672035696228, -87.64029376067838 41.94672257093693, -87.64026910505113 41.94669791656538, -87.64020324913402 41.94676088231449, -87.64017463638315 41.94675876200558, -87.64009952522976 41.94671585499872, -87.64026476056868 41.94651858324187, -87.6400214496071 41.94639675362362, -87.63997373514431 41.94645458679917, -87.63987395032738 41.94641589475479, -87.63988431569923 41.94639556868703, -87.63984058511413 41.94637406379999, -87.63979390864507 41.94635632866297, -87.63974483834366 41.94634264020539, -87.63969406806514 41.94633311348232, -87.63964214591302 41.94632791453489, -87.63942439081775 41.94632418347576, -87.63920658664168 41.94632484381856, -87.63913518871374 41.94632600323505, -87.63888712232163 41.94631075400791, -87.63868269153944 41.94624094027483, -87.63869816093985 41.94620083115922, -87.6381007588061 41.94603492249019, -87.63742144988002 41.9458193928776, -87.6373632156032 41.94579318885742, -87.63731341289377 41.9457521633888, -87.63682233071503 41.94504526731956, -87.63692149589801 41.94494201591817)), ((-87.63391761842934 41.94251519810279, -87.63393815782059 41.9425616648445, -87.6342313576265 41.9430429640831, -87.6346684579558 41.9431268645248, -87.63542465759301 41.9428793647179, -87.63592315824501 41.9428759643529, -87.6361433582307 41.943061264284, -87.6370099588511 41.9445767642355, -87.63692025690594 41.9448045194568, -87.63657885902859 41.94409184947356, -87.63618351204664 41.94324340917164, -87.63615211499042 41.94319332872151, -87.63611547751293 41.94314527432517, -87.63607388881984 41.94309952325306, -87.63602752948172 41.94305640480444, -87.63597676687303 41.94301613964444, -87.63592188944439 41.94297900613609, -87.63590195076563 41.94296692024928, -87.63584719883882 41.94294529337209, -87.63578997816751 41.94292762920771, -87.63573076253037 41.9429140972049, -87.63567010432031 41.94290483710459, -87.63560851557487 41.94289988028159, -87.63554658676652 41.94289931237505, -87.63551028428286 41.94290156207926, -87.63548486848229 41.94290313508437, -87.63542390861141 41.94291132786566, -87.6353642648553 41.94292377940232, -87.6353064837826 41.94294038819033, -87.63525105023157 41.94296101722547, -87.63519851541069 41.94298547694454, -87.63515990527461 41.94300703122048, -87.63511630466003 41.94304471910125, -87.6350684373773 41.94307936265599, -87.6350166344909 41.94311077042595, -87.63496137887033 41.94313861658622, -87.63490303984241 41.94316274064676, -87.63484205978973 41.94318292328729, -87.63477895653533 41.94319900464804, -87.63471417094442 41.94321090480727, -87.63468928653953 41.94321429314795, -87.63462297648728 41.94321095304591, -87.63455730104866 41.94320325401743, -87.63449277678694 41.94319125299442, -87.63442995191727 41.94317503674009, -87.63436930467347 41.94315477227254, -87.63431127228328 41.94313054430918, -87.63426787780344 41.94310892984548, -87.63405568561457 41.94291323813616, -87.63399032745828 41.9427388014763, -87.63397500567604 41.94269787479857, -87.63391761842934 41.94251519810279)), ((-87.63370688440135 41.94221821921614, -87.633792157307 41.942231364493, -87.633826203232 41.94230838737955, -87.63379432050891 41.94227978611676, -87.63371490510735 41.94222238860247, -87.63370688440135 41.94221821921614)), ((-87.63551609040749 41.94496490146133, -87.63432633729769 41.94379149512605, -87.63434554611278 41.94380842629172, -87.63447763983602 41.94392874884907, -87.63460595362001 41.94404620249985, -87.6347116476876 41.94414075292747, -87.63480597037277 41.94422954284986, -87.6348965439096 41.94431261866359, -87.63499461615893 41.94440712246177, -87.63506247661631 41.94447583152082, -87.63515280247137 41.94458167078974, -87.63524703292381 41.94467899675551, -87.63534879364639 41.94478490566787, -87.6354278412766 41.94487644768297, -87.63549951425273 41.94494517965447, -87.63551609040749 41.94496490146133)), ((-87.64149695203326 41.95932371847183, -87.64162496127101 41.9583407670408, -87.6413310607734 41.9566659665189, -87.6407661602541 41.9546396659303, -87.63933545010235 41.95092231987834, -87.63933782994926 41.95092728910207, -87.63939374183988 41.95104429962075, -87.63943831108783 41.95115270459457, -87.63949053522263 41.95125831006543, -87.63952391581003 41.95134388224234, -87.63958373291709 41.95145237902393, -87.63963211434903 41.95156080722564, -87.63968055801874 41.95166354401575, -87.63973262834348 41.95178337689197, -87.63979235402756 41.95190041051605, -87.6398444552274 41.95201739777841, -87.63988533764761 41.95211439758608, -87.63992615813768 41.95221708849422, -87.63995960147741 41.95229696914606, -87.63999679559382 41.95238256392861, -87.64003020863535 41.95246529012002, -87.6400601181019 41.95251953879108, -87.64008603059629 41.95259083696652, -87.64014197610364 41.95270500182375, -87.64022386540341 41.95288761923683, -87.64025721733758 41.95297603647668, -87.64029053816769 41.95306729926274, -87.64032779481339 41.95314720281581, -87.64034602209018 41.9532241460258, -87.64039440648862 41.95333257362048, -87.64042760477896 41.95343521832645, -87.64045720646926 41.95351792137751, -87.64048680860236 41.95360062442254, -87.64059164162106 41.95361244982352, -87.6406385981962 41.95376099945479, -87.64066813504698 41.95389196637676, -87.6407137260729 41.9540036565767, -87.6407401971789 41.95406460076582, -87.64078151853516 41.95417678630387, -87.64081149262066 41.95421983137461, -87.64084524829963 41.95424972731978, -87.64089103844751 41.95439731261757, -87.64095043534013 41.95458875117599, -87.64101250297031 41.95476107702429, -87.64105867542307 41.95492436059686, -87.64109109942038 41.95504678341394, -87.64112624684026 41.95517907462145, -87.64116532016666 41.9553429201611, -87.6412001096556 41.95550152449031, -87.64122613204336 41.95564484779495, -87.64124258379819 41.9557010377585, -87.64128800042012 41.95585613974175, -87.6413046586334 41.95600862628906, -87.64132854763916 41.95614532233529, -87.64134718778057 41.9562913716527, -87.64138652378368 41.95642418113398, -87.64140465853885 41.95650477858813, -87.64142416132376 41.95659153075905, -87.64152714595564 41.95715177456486, -87.64154417743109 41.95739910556368, -87.64158208468844 41.95766412169245, -87.64161807146445 41.95784490816205, -87.64165673280915 41.95799590809307, -87.64170283256408 41.95814250610601, -87.64172746223693 41.95824825238001, -87.64174514105105 41.95836430133731, -87.64173141175134 41.95850859107102, -87.64169937584178 41.9587851784814, -87.64166563002208 41.95908719545842, -87.64163938530342 41.95929748997392, -87.64162913071506 41.95933115476026, -87.64149695203326 41.95932371847183)), ((-87.64153269569046 41.96002913925376, -87.64147056123871 41.9595263668879, -87.64148957180241 41.95938038944814, -87.64160513660448 41.9593876228945, -87.64161309331715 41.95939645229461, -87.64159334781381 41.95952577753, -87.64155665138057 41.95982190350716, -87.64154564294824 41.95995580931006, -87.64153269569046 41.96002913925376)), ((-87.64161246263706 41.96223428384291, -87.6421237617343 41.9617027670446, -87.64211926146081 41.9610577668679, -87.6415763611984 41.9603824667576, -87.6415624451668 41.96026986262352, -87.64159571864843 41.9603468582373, -87.64167305983617 41.96044353634029, -87.64172419579177 41.96048841002864, -87.64194921926814 41.96067793556615, -87.64211993810713 41.960867600508, -87.64219566855617 41.96095702302864, -87.64223334706938 41.96103257883882, -87.64226092842112 41.96111029638846, -87.64228192045704 41.96118542105889, -87.64229126941822 41.96127976877874, -87.64229598112539 41.9613710688604, -87.64228919473996 41.96145132400769, -87.64227592175038 41.96151589704078, -87.64224003840624 41.96162130604819, -87.64221197744129 41.96168318388037, -87.64217011048335 41.96176037330605, -87.64212307797499 41.96183245456132, -87.64206437002647 41.9619096794861, -87.64200457496865 41.96197191474838, -87.64185142726126 41.96209865284643, -87.64176110990884 41.9621577675634, -87.64163946606587 41.96222388383863, -87.64161246263706 41.96223428384291)), ((-87.63836513254805 41.96264679793735, -87.63999065402281 41.9624952053023, -87.63956887996567 41.96254206697681, -87.63899765642937 41.96261589019488, -87.63837500358272 41.96269551910866, -87.63836513254805 41.96264679793735)), ((-87.63565605296716 41.96272775678577, -87.63646856055171 41.9628236681608, -87.63806385101115 41.9626748948052, -87.63789166893507 41.96269358019988, -87.63790693460531 41.96275388051666, -87.63787094510381 41.96276002906053, -87.63778566751061 41.962767882535, -87.63743805565839 41.96280540258464, -87.63719744484378 41.96283569375618, -87.63684601888291 41.96287565839351, -87.63659170867173 41.96288756188988, -87.63631013265842 41.96288793827975, -87.63612867732759 41.96287478994712, -87.63592262611714 41.96284455907907, -87.63572601186111 41.9627683379866, -87.63565605296716 41.96272775678577)), ((-87.63543052755799 41.96251489651272, -87.63560514095103 41.96269326950082, -87.63551009290934 41.96261474986751, -87.63543052755799 41.96251489651272)), ((-87.64000477923076 41.96076626980798, -87.6397871612929 41.9607216669095, -87.6386059607498 41.9602957668672, -87.6375869602353 41.9604207673139, -87.6365935598578 41.960827267299, -87.63543368086717 41.96188371205889, -87.63544184225552 41.96187084740271, -87.63556513777681 41.96169451330871, -87.63582978953859 41.9613844898847, -87.6360657769731 41.96117086059249, -87.6363908636752 41.96093562461957, -87.63676861693939 41.96072573579445, -87.63703130394558 41.96060205577893, -87.63731905110164 41.960493236853, -87.63764332193844 41.96040324396937, -87.63786222700236 41.96035978509942, -87.6381653123694 41.96030761503561, -87.63840103156706 41.96028335579498, -87.63847163174738 41.96027590752713, -87.63851861852653 41.96027602697721, -87.6385197091812 41.9602760132708, -87.63857932778505 41.96027526870464, -87.63864107691751 41.96027372197031, -87.63871136108087 41.96027837304065, -87.63876153648377 41.96028287551948, -87.63882159947129 41.96028749267515, -87.63898326565014 41.96031099965447, -87.63907496651491 41.96032758121078, -87.63920718900791 41.96036062512962, -87.63926743884018 41.96038198228713, -87.63932353736391 41.9604030676141, -87.63943648275527 41.96045753729859, -87.63968273211759 41.96058443557882, -87.64000477923076 41.96076626980798)), ((-87.64017105548578 41.95966567169796, -87.64065196122159 41.9597536670223, -87.64069591946028 41.95990435796206, -87.64066692543008 41.95988312297239, -87.64057388884628 41.95983835300409, -87.64047162670788 41.95978189322279, -87.64038462321513 41.95975577373407, -87.64025719697595 41.95972010290895, -87.64016089239374 41.9596892734759, -87.64017105548578 41.95966567169796)), ((-87.63960013534097 41.95956120568816, -87.64010573470551 41.95965371941123, -87.64009875060783 41.95967028431329, -87.64004890087409 41.95966765648301, -87.63999077177134 41.95964572308577, -87.63977033605718 41.95960942878106, -87.63960013534097 41.95956120568816)), ((-87.6363483319015 41.95968229627787, -87.6364609600565 41.9596041675308, -87.63651211243686 41.95959970960105, -87.63642406811147 41.95964410617081, -87.6363483319015 41.95968229627787)), ((-87.63421587275322 41.96138665226252, -87.6351508597802 41.9605129677604, -87.63554108502495 41.96024227337276, -87.63548683348499 41.96029271370564, -87.63537868093374 41.96039194916914, -87.63529098380104 41.96045634662993, -87.63522325202638 41.96053085461106, -87.63514197346463 41.96062026435104, -87.63505394860394 41.96071462772704, -87.63495254075082 41.96080890934931, -87.63485102313822 41.96091317966756, -87.63476327020774 41.96098257083127, -87.63465527904091 41.96106682294641, -87.63457410813557 41.96114624347423, -87.63447966298821 41.9612155939598, -87.63434485079901 41.96130467696116, -87.63422352991098 41.96138385277857, -87.63421587275322 41.96138665226252)), ((-87.63191230082488 41.9651838082227, -87.6317615590719 41.9651770687153, -87.631460858851 41.9640285686323, -87.6314233501824 41.96387492914447, -87.63149537752824 41.96392138614273, -87.6315888675275 41.96397384805702, -87.63167404426723 41.9640384683218, -87.63173443431569 41.96423373236505, -87.63177562842472 41.96437662751718, -87.63177562811765 41.96437665550618, -87.63180625567558 41.96449407475099, -87.63184041021231 41.96464535183548, -87.63186871068731 41.96479727946253, -87.63189119577206 41.96494974755575, -87.63190782949158 41.96510264640487, -87.63191230082488 41.9651838082227)), ((-87.63244321016261 41.96520754462846, -87.63199954541317 41.96518770883775, -87.63199476143367 41.96506838128691, -87.63212057669696 41.96510293013885, -87.63239366922839 41.96518975182719, -87.63244321016261 41.96520754462846)), ((-87.64662379505566 41.97746890197006, -87.6464064647454 41.977296969684, -87.646169764886 41.9770397698433, -87.6460347645303 41.9731927689543, -87.64508886343 41.9713095685356, -87.64326416252629 41.9691538681434, -87.6419427620496 41.9679099680185, -87.6403859621828 41.9669021678631, -87.6403727384899 41.96689579886611, -87.64048626623021 41.96692491395759, -87.64070272602576 41.96704109160049, -87.64092989079367 41.96717399180535, -87.64111722313814 41.96728414920726, -87.64152678185941 41.96752495192577, -87.64167850292898 41.96762325784142, -87.64197256785687 41.96783004764873, -87.64255778267913 41.96829624316644, -87.64266889790213 41.968215409446, -87.64261553367153 41.96816517080435, -87.64280244532551 41.96804577040517, -87.64281012084768 41.96804312670557, -87.64283909910787 41.96803619351819, -87.64286935400561 41.96803352206386, -87.6428997082146 41.96803524055024, -87.64292906033816 41.96804129021388, -87.64296523771289 41.96805536654227, -87.64327193053452 41.96830580660684, -87.64301256736212 41.96849436619092, -87.64274414877134 41.96827162432381, -87.64262750118517 41.96835423573018, -87.64279234171853 41.96849090126347, -87.64314572373925 41.96877307240129, -87.64345137685578 41.96903832440035, -87.64367189582593 41.96923956554473, -87.6439729210185 41.9695424124396, -87.64424033680395 41.96982820685439, -87.64440341157066 41.9700237220217, -87.64458709657683 41.97024493683384, -87.64464663164378 41.9703166424037, -87.64484130210874 41.97056618797232, -87.64495194928678 41.97071341927472, -87.64502748019873 41.97079883234251, -87.6453229310213 41.97125172310844, -87.64545043372868 41.97145588787747, -87.64557258383103 41.97166185822346, -87.64568931310875 41.97186957941941, -87.64580061790946 41.97207894251498, -87.64590642670908 41.97228989193902, -87.64600505191817 41.97249876671859, -87.64605670469047 41.97261639051955, -87.64610275390253 41.97273532513455, -87.64614312876169 41.97285537997338, -87.64617779368135 41.97297644176783, -87.646206637128 41.97309834657555, -87.6462297012734 41.97322093137327, -87.64624691182389 41.9733440571377, -87.64625827199824 41.97346755814096, -87.64628736635116 41.97440114432731, -87.64630848646138 41.97509467581187, -87.64633150784263 41.97586807422265, -87.64636060022747 41.97684309738949, -87.64636084529339 41.976854376962, -87.64636574720639 41.97691853875718, -87.64637649836597 41.97698229593281, -87.64639306901537 41.97704537334056, -87.64641538766415 41.97710744371621, -87.64641951945418 41.97711641507404, -87.64644330954091 41.97716825701701, -87.64647672711659 41.97722748548721, -87.64651549892611 41.97728487910248, -87.64655940433214 41.97734016294101, -87.64660830073248 41.97739308971501, -87.64666189580731 41.97744338105444, -87.64662379505566 41.97746890197006)), ((-87.64698179715266 41.9776470475831, -87.6468114646878 41.9776173693828, -87.64665612141809 41.97749447569415, -87.64669120397289 41.97747097171867, -87.64669455907256 41.97747368055148, -87.64675635873094 41.97752017986551, -87.64682220210612 41.97756349343961, -87.64689179774366 41.97760339834178, -87.64696477677296 41.97763975707091, -87.64698179715266 41.9776470475831)), ((-87.64861190934607 41.97787291339476, -87.6480995988026 41.97781283180498, -87.64836312626304 41.97782704040942, -87.64860387243924 41.97784001953399, -87.64861190934607 41.97787291339476)), ((-87.6501609760095 41.97929469167577, -87.64949895863241 41.97869728672874, -87.64952481448758 41.97871580403883, -87.6496471271292 41.97880950622704, -87.64976514191943 41.97890620103529, -87.64987874977999 41.97900580522689, -87.65011349201457 41.97924062395882, -87.65014784514942 41.97928166233628, -87.6501609760095 41.97929469167577)), ((-87.64995761610284 41.98519190463722, -87.6499516660333 41.9851895703988, -87.6503972666181 41.9838871701656, -87.6504157665634 41.9837482707501, -87.6505530659899 41.9826436701567, -87.6505866664007 41.9819523701731, -87.65046216585149 41.9802857696618, -87.650075165833 41.9798443697572, -87.65029736585031 41.9794177699641, -87.65022020433777 41.97934813936088, -87.65023256977389 41.97935837205222, -87.65028138006866 41.97939225255698, -87.65047512851302 41.97945745448899, -87.65047255871801 41.97946734520631, -87.6504586390045 41.97950686199593, -87.65043910705313 41.97954502708617, -87.65041418671758 41.979581403832, -87.65038421557674 41.97961558260863, -87.65034949075573 41.97964715382471, -87.65031042023217 41.97967576288328, -87.65026745074694 41.97970111057646, -87.65022109913346 41.97972284350407, -87.65051754585441 41.98021016536382, -87.65055485990951 41.98027707155994, -87.65058927970945 41.98034988717215, -87.65060778806507 41.98041100129105, -87.65064106367467 41.98053578599195, -87.65066855674445 41.9806613592786, -87.65067938400266 41.98075291518207, -87.65069533228535 41.98092636009679, -87.65070543104669 41.98110007318039, -87.65070964266123 41.98127388818666, -87.65070992637891 41.98128179370828, -87.65072610540867 41.98179862152847, -87.6507363991776 41.98231555261393, -87.65074132017584 41.98248146796802, -87.65074084947958 41.98252553613136, -87.65073531389424 41.98269259812185, -87.65072389600444 41.98285945938044, -87.65070659412325 41.98302606693227, -87.65068344969832 41.98319225499733, -87.65065446290306 41.98335796868918, -87.65061967176969 41.98352301778238, -87.6505790389858 41.98368734469605, -87.65053859067815 41.98383438617878, -87.65048722914501 41.98400117417079, -87.65048646579784 41.98400366657332, -87.65042857771691 41.984171926391, -87.65036503748526 41.98433902660431, -87.65029584409436 41.98450488569529, -87.65022107665142 41.98466939408231, -87.65012205956434 41.98486578284808, -87.64995761610284 41.98519190463722)), ((-87.65423406101603 41.98852557526597, -87.65393366769101 41.9883259707373, -87.65261263334095 41.98693283422697, -87.65266418403938 41.98696828306402, -87.65301724178352 41.98723919772079, -87.65309224765321 41.987297215117, -87.65314985413666 41.98733141981124, -87.65323250991011 41.98738273117674, -87.65331198539512 41.98742175792069, -87.65338840502822 41.98745774817074, -87.6534662062996 41.98750894883651, -87.65354250910103 41.98756966375083, -87.65367887277678 41.98770356457429, -87.65393073176496 41.98797612661287, -87.65416564871259 41.98823703562439, -87.65430789347603 41.9884062330928, -87.65430389398615 41.98850214643143, -87.6542941829304 41.98850560183035, -87.65427656988494 41.9885120557165, -87.65425741237398 41.98851822686751, -87.65424400374251 41.98852020624084, -87.65423406101603 41.98852557526597)), ((-87.65431529516728 41.99130463600097, -87.6540931680874 41.9902053715634, -87.65419590128045 41.99019136476652, -87.65418695290333 41.99021580678495, -87.65418890048792 41.99029205197203, -87.65419825338023 41.99045730823289, -87.65421174496791 41.99057231399917, -87.65422231460157 41.99071301720292, -87.6542384489496 41.99086656732718, -87.65428071163437 41.99088416035583, -87.65433157906864 41.99090534427385, -87.65438202643374 41.99092466128064, -87.65441315135155 41.99099336674416, -87.65432458563349 41.99099591612998, -87.65431529516728 41.99130463600097)), ((-87.65412823206817 41.99273919495754, -87.654017468783 41.9927151720836, -87.6540360685259 41.9923047717926, -87.6542290135114 41.99221056740825, -87.65421216298326 41.99230469733706, -87.65420990808981 41.99237147789039, -87.65420018046655 41.99239370262602, -87.65417675031041 41.99245931510701, -87.65414864030453 41.99258436679376, -87.65412823206817 41.99273919495754)), ((-87.65449760597834 41.99346079378714, -87.65451188765849 41.99345637284242, -87.6545111280862 41.99345987690275, -87.65449760597834 41.99346079378714)), ((-87.65441175179198 41.99386532686113, -87.65442886834791 41.9934820717643, -87.65445198460318 41.99347491604554, -87.65445119607895 41.99347793682582, -87.65446577596049 41.9934802450214, -87.65447108789668 41.9935203962315, -87.6544721541587 41.99357240522831, -87.65447118062541 41.99362593808529, -87.6544651276426 41.9936381416544, -87.65444913980423 41.99365415577785, -87.65450616390022 41.99365701793431, -87.65450480099803 41.9936675741861, -87.65449450351858 41.99369166244471, -87.65447241108367 41.99373162442384, -87.65443270704212 41.99386080537585, -87.65441294435777 41.99386513408358, -87.65441175179198 41.99386532686113)), ((-87.65452950046634 41.99502286464966, -87.6545101689926 41.9945692722596, -87.65427386845739 41.9945628717967, -87.6541450683589 41.9940014717193, -87.6542612654339 41.99395733363619, -87.6542591349034 41.99396600743142, -87.65426608824362 41.99410778573549, -87.65427799535883 41.99420930944634, -87.65430035667467 41.99426506564119, -87.65432587716752 41.99427635828791, -87.65436054759125 41.99429547055946, -87.65439068751027 41.99453321394199, -87.65460042679207 41.99453140793391, -87.65460055288527 41.99456115625215, -87.65459007178998 41.99463699776231, -87.6545813736998 41.99468708323821, -87.65455683776206 41.99476303373157, -87.65455131452555 41.99487787315069, -87.65455864960427 41.99495609877736, -87.65453231857062 41.99501766018848, -87.65452950046634 41.99502286464966)), ((-87.65448531989556 41.99534642783117, -87.6544092686991 41.9952925719294, -87.65449634846919 41.995194616218, -87.65449760645936 41.99527247168275, -87.65448602676969 41.99532012494335, -87.65448531989556 41.99534642783117)), ((-87.65453692678898 41.99607472258914, -87.65456694375969 41.99551011649648, -87.65457951465658 41.99553011434491, -87.65463897997499 41.9956843321207, -87.65465324997523 41.99578510191224, -87.65465736230927 41.99586547582801, -87.65466375345032 41.9959425430201, -87.65464433045753 41.99594596791655, -87.65464434337842 41.99601742614426, -87.65463828444346 41.99604066187339, -87.65459463475939 41.99606315312292, -87.65453692678898 41.99607472258914)), ((-87.65439481665429 41.99705073820856, -87.654259568577 41.9969908726641, -87.6542484690865 41.9964477728877, -87.65430191084621 41.99643932941864, -87.65431006646359 41.99647663889154, -87.65432778782522 41.99659191624973, -87.65434444408899 41.9967206354773, -87.65435690718654 41.99679762882248, -87.65437848305403 41.99696896695645, -87.65439481665429 41.99705073820856)), ((-87.65717267665202 42.00421277778197, -87.6571770706538 42.0037076734888, -87.65715424982496 42.00361930382053, -87.65719728806874 42.00367961096603, -87.6572375391456 42.00371088452043, -87.65724929993286 42.00377810430064, -87.65727200169339 42.00387867522114, -87.65728924753573 42.00392479651831, -87.65728686741456 42.00421220937056, -87.65717267665202 42.00421277778197)), ((-87.6553550603908 42.0066951737084, -87.65530286977049 42.0067255740651, -87.6559574702364 42.0061614741788, -87.6572022708126 42.0061802744397, -87.6573359702646 42.0052038740249, -87.65729812564017 42.00504613080357, -87.65731198381759 42.00507172623485, -87.65738242992192 42.00537888669621, -87.65742145656692 42.00561621329906, -87.65741510999095 42.00580566219593, -87.65741206080656 42.00620132884521, -87.65739408848182 42.0062794868972, -87.65584442443624 42.00630731441669, -87.65534818457104 42.00668914690361, -87.6553550603908 42.0066951737084)), ((-87.65910898599752 42.01010700002138, -87.6586830719074 42.0097580748435, -87.65877444094419 42.00968602405317, -87.65878977051813 42.009710187964, -87.65884143091762 42.00974852701631, -87.65890012734663 42.00974771907425, -87.65895283775596 42.00975988363788, -87.65895417680777 42.00976886471041, -87.65893058679346 42.00976932901164, -87.65889015621465 42.00978251161764, -87.65888310173816 42.00981644343311, -87.65889695829435 42.00984920752505, -87.65901596053745 42.00998107891201, -87.65905172911539 42.01002950405791, -87.65910898599752 42.01010700002138)), ((-87.6602352077921 42.01117475380532, -87.66012102397022 42.01098225161709, -87.66027803257704 42.01115509295273, -87.6602352077921 42.01117475380532)), ((-87.66119285280529 42.01273285690016, -87.6611597725766 42.012733475008, -87.66040430133678 42.01145982814272, -87.66046810863915 42.0115020012486, -87.66052116839833 42.0115370816131, -87.66064315783366 42.01161990280683, -87.66078783995177 42.01175274579823, -87.66091203149011 42.01189927208419, -87.66096973863836 42.01196815952814, -87.66101439041003 42.01202593886174, -87.66102639576161 42.01204584934379, -87.66107364981124 42.01216758389555, -87.66093868575487 42.01220888885977, -87.6609335298391 42.01222680615888, -87.66094144232002 42.01224405880718, -87.66097643347791 42.01228976204359, -87.66105178759106 42.0123811999456, -87.66113172495133 42.0124985705415, -87.66117611307104 42.01258842885188, -87.66119690942314 42.01269277384676, -87.66119285280529 42.01273285690016)), ((-87.6615815060098 42.01337573674984, -87.66156447307409 42.0133458747325, -87.6615688377843 42.01322737059277, -87.66157074199053 42.01323024241554, -87.66160876219335 42.01329234649489, -87.66162119844998 42.01332040937852, -87.66160484614591 42.01336016002961, -87.6615815060098 42.01337573674984)), ((-87.66386773423736 42.0192962912666, -87.66380147405781 42.0191783756729, -87.66382049035552 42.01886030349532, -87.66389008341827 42.01894935629572, -87.66392867480398 42.01899609547989, -87.66398035092874 42.01907597804187, -87.66399341735506 42.01916002612528, -87.66398583178413 42.01918506368952, -87.66396725974853 42.01919433993055, -87.66394508273538 42.0191930860361, -87.66392200633594 42.01919319815856, -87.6638848754108 42.01919674113056, -87.66386926693502 42.01920408709607, -87.66386773423736 42.0192962912666)), ((-87.66391607974282 42.02031868600638, -87.6638411738415 42.0201683756283, -87.66397223935324 42.02005715840788, -87.66400752243379 42.02017668923769, -87.66389164879638 42.02017546382897, -87.66388492417343 42.02018148924405, -87.66392681966668 42.02031688493951, -87.66391607974282 42.02031868600638)), ((-87.66469364135583 42.02175585771497, -87.6642746742947 42.0213572759546, -87.66427267456524 42.0213034243326, -87.66434242900077 42.02135493382487, -87.66444860876254 42.02142819130547, -87.66452549204568 42.02149112370383, -87.66459607027073 42.0215620882976, -87.66464506357936 42.02161045173284, -87.66471799275367 42.02173444664505, -87.66470676138718 42.02174930927507, -87.66469364135583 42.02175585771497)), ((-87.66524127424763 42.02278130708096, -87.66516467147258 42.0224713561128, -87.66516507575133 42.02247177574995, -87.66518403404325 42.02250660050714, -87.66523029147956 42.02255964057894, -87.66524865132735 42.0225882868929, -87.66526711080888 42.02261789369706, -87.66528227438073 42.02266334321136, -87.66528403116909 42.02269609092758, -87.66526726410433 42.02274728256857, -87.66524127424763 42.02278130708096)), ((-87.83793609156 41.9733191830467, -87.83793501726025 41.9734578159035, -87.83793394252766 41.97359650363927, -87.83793286821282 41.97373513648927, -87.83793390092427 41.97384370408369, -87.83794433958094 41.97384354605659, -87.83814697099183 41.9738395717846, -87.83841830257995 41.97383388128594, -87.83865906969071 41.97382882682997, -87.83899021517135 41.97382564294708, -87.83938794506359 41.97381956132645, -87.83959909433003 41.97381627096803, -87.83993305045827 41.97381098341133, -87.84036263290204 41.97380415715738, -87.84066006725251 41.97379942414826, -87.841026618878 41.97379719888976, -87.84115537385648 41.97379641699182, -87.84135186676947 41.97379574990038, -87.84155036483111 41.9737878875107, -87.8415582336942 41.97394084232057, -87.84155775500335 41.97408916481672, -87.84155680003616 41.9743853981764, -87.84155597022551 41.97455574528158, -87.84155493604644 41.97469032073413, -87.84143160973986 41.97469226820444, -87.84125057722427 41.97469538916108, -87.8410731497694 41.97469841552191, -87.84089579621634 41.97470144192549, -87.84085636760608 41.97470215094019, -87.8407184418584 41.97470452293579, -87.84054108754133 41.97470754878805, -87.84036366002961 41.97471057405196, -87.84024596376746 41.97471259327096, -87.8401863059963 41.97471365424149, -87.84000895200431 41.97471667927296, -87.83983159836589 41.97471970403194, -87.83965424355401 41.97472278339603, -87.83962143553325 41.97472335580383, -87.83947688951939 41.97472580760528, -87.83929953547062 41.97472883154042, -87.83912218061563 41.97473191008228, -87.83894475335789 41.9747349331541, -87.83876739926652 41.97473795626689, -87.83859004473621 41.97474103398796, -87.83841269061638 41.97474405655257, -87.83823533648241 41.9747470788431, -87.83805798190907 41.9747501557419, -87.83788062774669 41.9747531774843, -87.83770319965416 41.97475619863346, -87.83752584540557 41.97475927471149, -87.8373484912006 41.97476229563151, -87.83717113698154 41.97476531627743, -87.83699378274832 41.97476833664926, -87.83681311760925 41.97477145219592, -87.83663074714569 41.97477088036971, -87.83662837546561 41.97493961455757, -87.83662718355723 41.97509301042241, -87.83662700836776 41.97521021483804, -87.83662687505986 41.97530305003249, -87.83662686011444 41.97530980099664, -87.83644116547116 41.97532109965477, -87.83625108437579 41.97532428332853, -87.83607630289109 41.97532720365228, -87.83600185940463 41.97532847282446, -87.83590166959399 41.97533012435196, -87.83581803084164 41.9753315182147, -87.83572696236686 41.97533304446554, -87.83563427544725 41.97533456335336, -87.83555232830747 41.97533596463045, -87.8353776214213 41.97533888421382, -87.83529964682731 41.97534013788883, -87.83520298813055 41.97534174896642, -87.83502828048225 41.97534466801476, -87.83496509165361 41.97534576582854, -87.83485364710495 41.9753475871197, -87.83467901334653 41.97535050595728, -87.83463046293981 41.97535133786398, -87.83450430602583 41.97535342420959, -87.83432967224066 41.97535634251559, -87.83429583417711 41.97535690892371, -87.83415496489106 41.9753592605105, -87.83398033071353 41.97536217800887, -87.8339612785467 41.97536247932603, -87.83380562413595 41.97536504031683, -87.83362664968625 41.97536804843453, -87.83362901788968 41.97506509806864, -87.83363009086264 41.97492783739933, -87.83363013394532 41.9749129639733, -87.83350883127103 41.97491512523911, -87.83341371684202 41.97491685157703, -87.83341464521338 41.97469468416924, -87.83341493273613 41.97462970282048, -87.83341572145916 41.97447246306146, -87.83341657588461 41.97425029558897, -87.83341755672905 41.97403081798374, -87.83341847548736 41.97392029071083, -87.83306953074012 41.97392590014459, -87.83287742778558 41.97392940284818, -87.83271603695961 41.97393234487433, -87.83248569216755 41.97393537445153, -87.83235666911253 41.97393751563921, -87.83219505926185 41.9739401980214, -87.83190994113227 41.97394515498511, -87.83174461480924 41.97394816105712, -87.83172260263711 41.97394856103434, -87.83172218844734 41.97383956086789, -87.83172413341433 41.97361975817496, -87.83172420532782 41.97361059310327, -87.83172629604161 41.97338167998743, -87.83172770480408 41.97322093033083, -87.83172794939593 41.97318975744973, -87.83172914474883 41.97305606449105, -87.83146484830053 41.97306067252796, -87.83119643243674 41.97306531627397, -87.83117561497424 41.97306571917681, -87.83092801573888 41.97307001454728, -87.8308065705731 41.97307211745414, -87.8306596737209 41.97307465763573, -87.83051145259498 41.97307724671343, -87.83039125732216 41.97307935492966, -87.83021633457689 41.97308237521379, -87.83012284095871 41.97308399616302, -87.8299211426049 41.97308750263112, -87.82985442486348 41.97308869192855, -87.82962595132626 41.97309262956679, -87.82958608234591 41.97309333250509, -87.82933083355826 41.97309775551722, -87.82931766581659 41.97309802701354, -87.82904924968688 41.97310266601193, -87.82903571501274 41.97310288097994, -87.82878090707061 41.97310730470574, -87.82874052324877 41.97310800536207, -87.82851249044191 41.973111997331, -87.82844533101148 41.97311318386721, -87.82824414739208 41.9731166347681, -87.8281502130815 41.97311830705601, -87.82797565751858 41.9731213258158, -87.82785502082992 41.97312342915883, -87.82770731477068 41.9731259619993, -87.82755982853847 41.97312855050236, -87.8274388976439 41.97313065183749, -87.82726471012116 41.97313367141241, -87.8269695184855 41.97313879124076, -87.82679005311324 41.97314189631025, -87.82674256140895 41.9731426999167, -87.82674279185325 41.97321460142211, -87.82674269159897 41.97335255239697, -87.82674123500318 41.97351780199359, -87.82673969452034 41.97369358952832, -87.82673779357668 41.97374325160465, -87.82673530653423 41.97384362353431, -87.82673538218945 41.97403362221776, -87.82644421455008 41.97404079958974, -87.82614067974751 41.974049420748, -87.82588337142511 41.97405330559106, -87.82575455192296 41.97405754396838, -87.82541867173687 41.97406859399798, -87.82516535259194 41.97407493722661, -87.82501601880148 41.97407839242744, -87.82491810264762 41.97408101424163, -87.82479576419163 41.97408419628196, -87.8246855984156 41.97408717087626, -87.82425891821316 41.97409869277473, -87.82405075885929 41.97410508367975, -87.82404023717342 41.97410540664855, -87.82384069806919 41.97410982967127, -87.82349973378702 41.97411738723279, -87.82330055945323 41.97412212844149, -87.82305338398638 41.97412813728176, -87.82275558414248 41.97413680214678, -87.82225913503801 41.97414894536677, -87.82204568583326 41.97415438895304, -87.82185040836484 41.97415976903355, -87.82151409666781 41.97416903432791, -87.82116805018111 41.97417742508472, -87.82080452642407 41.97418694450006, -87.82039149107784 41.97419799730093, -87.82009450969937 41.97420545146685, -87.8196807018861 41.97421649853101, -87.81939805900446 41.97422257104485, -87.81942609197829 41.9740309756758, -87.81942659963319 41.9739587229269, -87.81942723718832 41.97389766713126, -87.8194306349579 41.97374952277042, -87.81943240930502 41.97363850001356, -87.81943317853006 41.97355200543629, -87.8194336821238 41.97346470177346, -87.8194325145118 41.9732195917348, -87.81934287679218 41.97322059493571, -87.81896778816906 41.97321924457913, -87.81896870728829 41.97309586877189, -87.81896900908038 41.97304921856497, -87.81896955329496 41.97297243802462, -87.81897036333258 41.97286261839248, -87.81897049492032 41.97284626349366, -87.81897171757146 41.97267601821388, -87.81897184783328 41.97265982796117, -87.81897307179698 41.97248941802916, -87.81897313494029 41.9724815698725, -87.81897400949856 41.97236368254354, -87.8189744263769 41.97230281783997, -87.81897484144054 41.97225122855654, -87.81897559828786 41.97213882911508, -87.81897578020802 41.97211621764134, -87.81897642948469 41.97202637512034, -87.81897713439353 41.97192961743826, -87.81897726067666 41.97191392112339, -87.81897809186371 41.97180146712418, -87.818978488198 41.9717430172274, -87.81897892223661 41.97168906800334, -87.81897968023721 41.97157661367186, -87.81897984272454 41.97155641701371, -87.81898051140529 41.97146416021481, -87.81898119687004 41.97136981679225, -87.81898134213577 41.97135176054014, -87.81898217330068 41.97123930625542, -87.81898255100216 41.97118321656467, -87.81898300445626 41.97112685251731, -87.81898376206544 41.97101439817305, -87.81898459240432 41.97090199903667, -87.8189848785137 41.97086643535171, -87.81898542391528 41.97078954501912, -87.8189867395198 41.9706077744106, -87.81885182945487 41.97061106668649, -87.81870463492746 41.97061468792696, -87.81865571680051 41.9706158956229, -87.81855744038303 41.97061830925299, -87.81841024582826 41.9706219298414, -87.81826584678298 41.97062545356347, -87.81826216836947 41.97062554655431, -87.81811593065547 41.97062911644836, -87.81800242683887 41.97063190006424, -87.81797373832219 41.97063259482201, -87.81796873605424 41.97063273674481, -87.81782161461379 41.97063635718101, -87.81775607242777 41.97063798380243, -87.81767934944641 41.97063983458848, -87.81767434643973 41.97063997676958, -87.81752722570516 41.9706435968316, -87.81752089965417 41.97064373306168, -87.81738496014158 41.97064707387244, -87.8173799571338 41.97064721604068, -87.8172894048958 41.97064938832895, -87.817232836445 41.97065078084141, -87.81707644699759 41.97065463280466, -87.81695334603405 41.97066019931151, -87.81695200037245 41.97077385817854, -87.81695068719442 41.97088180978081, -87.81694937528425 41.97100326235236, -87.81694835566634 41.97109763115582, -87.81694980021827 41.97114961299436, -87.81694744645957 41.97121930534536, -87.81694726666238 41.97126883752955, -87.81694660723663 41.97144158258802, -87.81694489629051 41.97156772622633, -87.81694342515375 41.9716761982286, -87.81694199200743 41.97184907700512, -87.81694082115689 41.97198488268225, -87.81693947855264 41.97213298219195, -87.81693810293029 41.9723305585512, -87.81693617627661 41.97247497736081, -87.81693484027802 41.9725751352734, -87.81693260036455 41.97277504085658, -87.81693126225727 41.97288606527459, -87.81693022609464 41.97309624015301, -87.81692935434108 41.9732679411276, -87.81692842179528 41.97338338608589, -87.81692727467052 41.97352539402024, -87.81692595331033 41.97368897010107, -87.81692470873254 41.97382483001273, -87.8169245700399 41.9738360809198, -87.81692364325515 41.97391115775706, -87.81692230963516 41.97398524567138, -87.81692048253954 41.97408400152752, -87.81691746331354 41.97428894823128, -87.81668655332157 41.97429373168819, -87.81595875054406 41.97431487751143, -87.81572243226319 41.97431956797124, -87.81568330036441 41.9743200089778, -87.81548357762836 41.97432420588938, -87.81538308692392 41.97432677118402, -87.81516231430894 41.97433280037153, -87.81487480833552 41.97434165535532, -87.81467257146333 41.97434749288735, -87.81445468011898 41.9743526064628, -87.81423133358861 41.97435784753614, -87.8139955603643 41.97436331241857, -87.81383632732366 41.97436744374044, -87.8138100275707 41.97436812603668, -87.81359087505245 41.97437382995373, -87.81340607723342 41.97437870113049, -87.81321616634452 41.97438368687862, -87.8129835139667 41.97438979416594, -87.81271414805171 41.97439740928579, -87.81258437651584 41.97440081765114, -87.81254487266952 41.97440185551015, -87.81232634487769 41.9744075872889, -87.81222460154203 41.97441044599646, -87.81200478849854 41.97441639141458, -87.81173354444178 41.97442380306093, -87.8114296867731 41.97443092914497, -87.81117577286163 41.97443762298474, -87.81098891448434 41.97444259066616, -87.81075368648506 41.9744488131625, -87.81056848385704 41.97445371195624, -87.8102648789548 41.97446133006524, -87.81004565425954 41.97446683506441, -87.8097750074831 41.97447355902737, -87.80951892517122 41.97448009386989, -87.80933707066254 41.97448473317897, -87.8089651153731 41.97449505448577, -87.80876350431491 41.97450069209588, -87.80857919006453 41.97450476189058, -87.80829129282867 41.97451184532077, -87.80805529546647 41.97451765125362, -87.80771685078216 41.974526859502, -87.80753073161374 41.97453135842271, -87.80729193003725 41.97453827753769, -87.80706668497977 41.97453821309559, -87.80706254370472 41.97433514246658, -87.80706516798888 41.97414591417046, -87.80706894400055 41.97385693889776, -87.8070656206109 41.97368790786357, -87.80706243795271 41.97352603985389, -87.80706025477264 41.97336239296011, -87.80705996690878 41.97319633929752, -87.80706404667836 41.97300898342375, -87.80707157923656 41.97288465040791, -87.80707546169438 41.97272572362402, -87.80708050600849 41.97251946455531, -87.80708236913016 41.97235567125436, -87.80708399482796 41.97222960973706, -87.80708548170436 41.97211603373275, -87.80708664813109 41.97202789507752, -87.80708823901837 41.97188528581095, -87.80708945826207 41.97177585232924, -87.80709043558443 41.97167503455421, -87.80709190168517 41.97152372556261, -87.80709322755436 41.97138498440422, -87.80709465731694 41.97123362007979, -87.80709734833329 41.97110772834791, -87.80709740896305 41.97110487464769, -87.80709857561888 41.97105691120898, -87.80710110967746 41.97091367479874, -87.80710366400874 41.9707693139006, -87.80710656081443 41.97059603032103, -87.80710826311329 41.97049647818486, -87.80710998594786 41.97037209103986, -87.80711230258093 41.97019776198899, -87.80711446293058 41.97007436475855, -87.80711841082071 41.9698487813267, -87.80712175264397 41.96970192650505, -87.80713087557946 41.96951832575665, -87.80713849008167 41.96937944872023, -87.80714582450622 41.96918085637267, -87.80714599950724 41.96909806406428, -87.80714643524514 41.96889216820372, -87.80715338348243 41.96862878324939, -87.80715600156758 41.96826957802827, -87.80684323863464 41.9682728641851, -87.80666080282917 41.9682743050955, -87.80657846646542 41.96827483540488, -87.80644030811962 41.96827573570549, -87.80618510034114 41.96827733581144, -87.80602517727809 41.96827774078194, -87.80591162418153 41.96827802816219, -87.80566314936219 41.96827908175486, -87.80540974911443 41.96828002983423, -87.8051391696399 41.9682817215809, -87.80485582754162 41.96828371507908, -87.80471707981783 41.96828469099808, -87.80441362380823 41.96828675105123, -87.80407630114742 41.96828876394887, -87.8038783209832 41.96828898114701, -87.80377822774341 41.96828909084129, -87.80369041110521 41.96828981827701, -87.80346759623417 41.96829166413759, -87.80313020492449 41.96829312509615, -87.80284362829615 41.96829498507935, -87.80270116813811 41.96829586292296, -87.80254225121119 41.96829705034066, -87.80252817071153 41.96840941445992, -87.80252913269375 41.96847514303045, -87.80252892879975 41.96847755644438, -87.80252732426494 41.96859110446631, -87.80252546875683 41.96871716520062, -87.80252463698429 41.96877310730283, -87.80239683438232 41.96877546114009, -87.80228415747969 41.96877609338175, -87.80194643125223 41.96877798755938, -87.80188759420405 41.96877831893719, -87.80144910523443 41.96878078878822, -87.80120639725227 41.9687824082914, -87.80075415472582 41.96878497639124, -87.80069531730365 41.96878530715622, -87.80024417787122 41.96878787836476, -87.80000139714254 41.96878944009824, -87.79954922068676 41.96879293678963, -87.7994901609827 41.96879337567182, -87.79903504322191 41.96879685666076, -87.79878933650967 41.96879648126602, -87.79833703356796 41.9687975030787, -87.79827819760625 41.96879761280034, -87.79782140867012 41.96879861194852, -87.79757866888178 41.96880401009479, -87.79712354697027 41.96880792255958, -87.79712352368134 41.96880907721452, -87.79706478131982 41.96880841645158, -87.79660973253992 41.96881232722086, -87.79636694673286 41.96881443008607, -87.79591145775385 41.96881833602838, -87.79585269172324 41.96881882929812, -87.79539720196735 41.96882273319631, -87.79515441610766 41.96882483349678, -87.7946989262953 41.968828734624, -87.79464016025227 41.96882922727291, -87.79418481784813 41.96883312705186, -87.79394203295723 41.96883506014007, -87.79348823346727 41.96883912905764, -87.79342946731613 41.96883967597046, -87.79297581486389 41.96884374355381, -87.79273301683381 41.96884726566503, -87.79227811338116 41.96885132460029, -87.79221934721495 41.96885187089356, -87.79176451725921 41.96885592813994, -87.79152173030469 41.96885813052052, -87.79106506203613 41.96886223120396, -87.79100629595617 41.96886272171802, -87.79054962764098 41.96886682062515, -87.79030698817829 41.96886896624967, -87.78985245259082 41.96887301731243, -87.78979368639703 41.9688735623638, -87.78933922430676 41.96887761174234, -87.78909651161861 41.96887975418686, -87.7886396217814 41.96888384444546, -87.78858085604148 41.96888433399385, -87.78812411361216 41.96888842262567, -87.78800940765902 41.96888823924086, -87.78800992879881 41.96885472969962, -87.78801039923326 41.96882992455117, -87.78801218274059 41.96870690981727, -87.78801462634976 41.9685713848929, -87.78801351164351 41.96841062651692, -87.7877638534161 41.96841070051825, -87.7875493764594 41.96841357787121, -87.78732018016896 41.96841770176554, -87.78713849970171 41.96842098157977, -87.78692533566615 41.96842520867599, -87.78677237467021 41.96842701511675, -87.78675922478193 41.96838369388289, -87.78675177339704 41.96832622209077, -87.78675224747198 41.96821914526959, -87.78675284944653 41.9681100108977, -87.78675381629414 41.96794958897145, -87.78675451608096 41.96786342410036, -87.78675517674682 41.96777753373722, -87.78675666029149 41.96762979223928, -87.78674649981085 41.96748696232205, -87.78673624225836 41.96734276038184, -87.78673637127534 41.96729325543767, -87.78673710902558 41.96711236058667, -87.78673760265242 41.96706313179752, -87.78673793695792 41.96702987383008, -87.78673859221024 41.96696461934469, -87.78673979501572 41.96685397849384, -87.78674016969381 41.966780078681, -87.78674085380604 41.96665703152398, -87.78674174803501 41.96653856819691, -87.78674280556653 41.96639859103076, -87.7867440092472 41.96631799928402, -87.78674648185569 41.96612767257575, -87.78674776478512 41.96602916150359, -87.78674946378226 41.96586915463691, -87.78675087869986 41.96573373454671, -87.786752330377 41.96559831462829, -87.78675285695229 41.96553670931514, -87.78675366698262 41.96545054520731, -87.78675496597324 41.96530283048771, -87.78675605529932 41.96517973034398, -87.78675658139186 41.96511818018272, -87.78675788595397 41.96498275982378, -87.78675887421223 41.96488430193883, -87.78675934408363 41.96483367346949, -87.7867602103012 41.96473658738127, -87.78676160700007 41.96464661105117, -87.78676364744351 41.96451517329172, -87.78676491745047 41.96435793599451, -87.78676509307057 41.96433312942881, -87.78676596507181 41.96415797025268, -87.78676601959698 41.96414935368639, -87.78676724870122 41.96396906507687, -87.78676801102145 41.9638626204414, -87.78676873183365 41.96374390952641, -87.78676873501038 41.96361869086049, -87.78676906704511 41.96354981271169, -87.78677852589639 41.96347129094218, -87.78680435699133 41.96344751182189, -87.78680993653171 41.96344429993763, -87.786831677655 41.9634301338012, -87.78685345543231 41.96341602299376, -87.7870478336489 41.96330364227907, -87.78700518042139 41.96323749259536, -87.78697785764476 41.9631820118057, -87.78696275045327 41.96314272511387, -87.78695495829692 41.96310366532786, -87.78693911222264 41.96300880481083, -87.78692901151452 41.9629344434607, -87.78691600823134 41.96288130850029, -87.78689042601324 41.96269095791501, -87.78688154890935 41.96250348592798, -87.78687642769127 41.96235840245911, -87.78686912247494 41.96212897892521, -87.78686646066225 41.96204405783384, -87.78686296841485 41.96193265607147, -87.78685666203158 41.96177993803356, -87.78684794691796 41.96153050244942, -87.78684311320036 41.96137334578441, -87.78683540427524 41.96114811893443, -87.78682785250977 41.96096057087594, -87.78682034884697 41.96071133311622, -87.7868200038574 41.96067318086617, -87.78681944136987 41.96061100006825, -87.78680971922488 41.96033741110411, -87.78680540389269 41.96021702055827, -87.78698085121435 41.96021476316035, -87.78717355966798 41.96020895646534, -87.7872902147561 41.96020545018889, -87.78735803462956 41.96020291874631, -87.78756153145409 41.96019526947072, -87.7876629340895 41.96019103142209, -87.78786799912221 41.9601848709322, -87.78801702906961 41.96018125757702, -87.78814719145585 41.9601781016305, -87.78826170259782 41.96017584158468, -87.78830274192084 41.96017497084, -87.78849435754019 41.9601722582502, -87.78865501804526 41.96016923328462, -87.78883513353135 41.96016558714314, -87.78895457330079 41.96016318979319, -87.78909813756484 41.96016002853225, -87.78924673763909 41.96015718650884, -87.78947300171491 41.96015285878029, -87.78964222438555 41.96015031234133, -87.78987392832289 41.96014608559835, -87.79007286820161 41.96014249917862, -87.79028735957407 41.96013893108973, -87.79046560671588 41.96013542009361, -87.79070528581229 41.96013069865154, -87.79097761813451 41.96013072313571, -87.7912713986711 41.96012920171396, -87.79151143881802 41.9601261635944, -87.79168950022157 41.96012186717138, -87.79180063289351 41.96011918562888, -87.79197955668474 41.96011294937114, -87.7922586568818 41.9601039468215, -87.79252278150814 41.96009624515822, -87.79277226777458 41.96008918746096, -87.79305273949991 41.96008323551602, -87.79327591869591 41.96007838550399, -87.79350983568004 41.96007331140007, -87.7940330882374 41.96006133375119, -87.79419714097503 41.96005757794651, -87.79432793030192 41.96003031110986, -87.79444277462022 41.95998845211341, -87.79459613970546 41.95992608238915, -87.7946333833018 41.95990910964959, -87.79472690633087 41.95986648759489, -87.79484784611975 41.95980810944802, -87.79491595351119 41.95977566294963, -87.79514587144284 41.95966552563132, -87.79527517838443 41.95960358337427, -87.79568678096371 41.95940672099889, -87.79589844124058 41.95930661485619, -87.79610619685664 41.95920731358858, -87.79627397785474 41.95912659538415, -87.79650927149694 41.95901400353581, -87.79666925852226 41.95893744640662, -87.79691367047063 41.95882031295844, -87.79725345914341 41.95865850947271, -87.79744950676699 41.95856505087765, -87.79746700517128 41.9585564951345, -87.79770938168352 41.95843827529737, -87.79800251944943 41.95829529513594, -87.79822461330004 41.95819526104254, -87.7983933908055 41.95811344667878, -87.79860690751073 41.95801010626029, -87.79874524445894 41.9579456028054, -87.79890825537089 41.95786722677286, -87.79900312553018 41.95782161217218, -87.79930736156169 41.95767615727671, -87.79962115362837 41.95752657478495, -87.79972292036672 41.95747809100916, -87.79992607833037 41.95738205447402, -87.79999222938734 41.95735102284771, -87.80012833074845 41.95728405015939, -87.80012289433287 41.95711009540218, -87.80137878018466 41.95650969312902, -87.80248449915447 41.95598106140495, -87.80710124346324 41.95377363176644, -87.80725629196183 41.95369949322446, -87.80729692619904 41.95367984063208, -87.807300248221 41.95358205000014, -87.80729154361174 41.95344249406877, -87.80728712496827 41.95325106432995, -87.8072817339031 41.9530617437261, -87.80727842853175 41.95295404545304, -87.80727540751703 41.95291012411678, -87.80726923353913 41.95276460697126, -87.80757380732709 41.95276741977975, -87.80793978693436 41.95275756914247, -87.80827146121976 41.95274849404001, -87.80847417260971 41.9527437738568, -87.80867931040284 41.95273899717021, -87.80891662623496 41.95273300130116, -87.80917277386723 41.95272577369267, -87.80971949305965 41.95270832010492, -87.81030412814175 41.95269191040595, -87.81044894840892 41.95269037439945, -87.81072833788731 41.95268223408073, -87.81097616105772 41.95268152241554, -87.81105737128101 41.95268128868842, -87.81129906497868 41.95267481412376, -87.81146897757029 41.95267083907033, -87.81166871033001 41.95266612111171, -87.8118560920113 41.95266134670742, -87.81200699672731 41.95265755894378, -87.81222540021669 41.95265151097179, -87.81235437164852 41.95264793933258, -87.81255052071782 41.95264542639517, -87.81282673299133 41.95263910490963, -87.81303845480211 41.95263386263431, -87.81343280252112 41.95262369850625, -87.81370131331371 41.95261677692785, -87.81388741187598 41.95261152688316, -87.81416855889677 41.9526040719233, -87.81448514941378 41.95259611784112, -87.81463410835693 41.95259378022495, -87.81474854980792 41.95259198409171, -87.81498612168144 41.95258605915789, -87.81525744457022 41.95257962742258, -87.81558849446598 41.95257019878031, -87.81563865145654 41.95256861376812, -87.81584930449431 41.95256300775946, -87.81589850802368 41.95256171965151, -87.81611538078137 41.95255594360171, -87.81654102331497 41.95254616992413, -87.81672685403873 41.95254212152691, -87.81706654871824 41.95253381916999, -87.81720478814876 41.95253043990166, -87.81742342463501 41.95252469949571, -87.81762712239134 41.95252053783032, -87.81774654710577 41.95251811043742, -87.817952467891 41.95251184485457, -87.81815442151711 41.95250517745925, -87.81830320796136 41.9525012100047, -87.81856227921779 41.95249430112123, -87.81880105548278 41.9524891969578, -87.81900518500527 41.95248626940623, -87.81931157582679 41.95248484272098, -87.81952406305581 41.95248034053348, -87.81978633446118 41.95247478293667, -87.82001497513031 41.95246839601325, -87.82026303557893 41.95246086052033, -87.82055189871117 41.95245378115637, -87.82075189075759 41.95244865557937, -87.82086010543279 41.95244588181663, -87.82101094500429 41.95244087446922, -87.821179861765 41.95243715502027, -87.82133267154452 41.95243410416007, -87.82152326725054 41.9524271057651, -87.82170883592862 41.95241901440679, -87.8219782969319 41.95241083264963, -87.82213365617706 41.9524061148112, -87.82228528955802 41.95240300310942, -87.82250685716215 41.9523983909238, -87.82271089710146 41.95239276732359, -87.82298233863891 41.95238513784311, -87.8233046247622 41.95237723963907, -87.82369563165715 41.95236786185195, -87.82410311735298 41.95235726604005, -87.82442676550559 41.95234915118906, -87.82442783036217 41.95234912407609, -87.82471658574579 41.95234179143993, -87.82509097781659 41.95233208804103, -87.82512326196289 41.95233124303076, -87.82544738777783 41.95232312741653, -87.82572187383651 41.95231627330728, -87.82607898078203 41.95230734124554, -87.82640203684448 41.95229979449561, -87.82669339571981 41.95229301283526, -87.82698932005601 41.95228525633929, -87.82722751718779 41.95227901272712, -87.82738959372426 41.95227550096764, -87.82772831717145 41.95226689467015, -87.82790448364626 41.9522620718473, -87.82811929231914 41.95225671792621, -87.82830968554902 41.9522519721766, -87.82847114114345 41.95224793530585, -87.82869712744234 41.95224228779884, -87.82893925681873 41.95223607932309, -87.82916524449229 41.95223029368959, -87.82935015701271 41.95222583032287, -87.82963467849548 41.95221896233622, -87.82982970692964 41.95221383594008, -87.83005709209974 41.95220791691965, -87.83034698601003 41.9522003783506, -87.83057323198413 41.95219434492033, -87.83081455438744 41.95218790886891, -87.83104057664258 41.95218231179025, -87.831218797856 41.95217793249437, -87.8314125739774 41.95217312641527, -87.83157590345728 41.95216912026302, -87.83179303153825 41.95216382064623, -87.83179009654607 41.95230781625076, -87.83179142026466 41.95236874343865, -87.83179648200249 41.95252491165446, -87.83179957158984 41.95266699312968, -87.83180015717083 41.95269391648244, -87.83180277161722 41.95283874064629, -87.83180438413639 41.95300820324643, -87.83180568293959 41.95318954694125, -87.83180721504395 41.95332237385168, -87.83180907099944 41.95347954339864, -87.83181526603519 41.95378195652211, -87.83182390904737 41.95397845237113, -87.83182665556878 41.95404078558211, -87.83182482405229 41.95409621080545, -87.83182189814086 41.95421620271934, -87.83182078511406 41.95434879851415, -87.83182085711651 41.95443340309539, -87.83182189549969 41.95455418070003, -87.83182314368683 41.95463881785818, -87.8318250339761 41.95473534028672, -87.83182599884123 41.95485611756387, -87.83182730093476 41.95495263741945, -87.83183157107362 41.95507381332031, -87.8318338622685 41.95517083145196, -87.83183191345857 41.95525523509884, -87.83182859640368 41.95532668005704, -87.83182497067033 41.95539993485002, -87.83182248909884 41.95547261835397, -87.8318204626428 41.95555754360736, -87.83181927028994 41.95562995777009, -87.8318186950597 41.95566576730536, -87.8318211769149 41.95570090413979, -87.83182721419426 41.95573745605574, -87.83184249231722 41.95579575510113, -87.83185618515458 41.95584993089238, -87.83186599150366 41.95588869464262, -87.83187151727653 41.9559247503617, -87.8318758517684 41.95595336404612, -87.83188686520562 41.95600234155733, -87.83190094895008 41.95606294024451, -87.8319204950938 41.9561489747931, -87.83193878982848 41.95623519569287, -87.83194389943117 41.95625873595505, -87.83196664975311 41.95637694644533, -87.8319746421636 41.95645005986934, -87.83199274064162 41.95655669715415, -87.83200203632074 41.95660904226191, -87.83201252857934 41.95666825341218, -87.83202105260153 41.95671576559921, -87.83203001976197 41.95676317022581, -87.83204196935293 41.95682403399129, -87.83205669708184 41.95689632610452, -87.8320616582802 41.9569200029218, -87.83207494884461 41.95697864999521, -87.83208383209264 41.95701795880727, -87.83209491960967 41.95706690889666, -87.83211838119065 41.95716001285576, -87.83213393739992 41.95722048099442, -87.83215379551622 41.95729493597648, -87.83217073506576 41.95735716644666, -87.83218706375686 41.95742219334907, -87.83219864667588 41.95746884071941, -87.8322054420634 41.95749614790121, -87.8322155063375 41.95753658670939, -87.83223955491853 41.95762994045259, -87.83224819843933 41.95766293952026, -87.83227953693788 41.95778258338675, -87.8323044147897 41.95787330573432, -87.8323422596489 41.95801715836798, -87.83237752454306 41.95815710294022, -87.83240911317391 41.95828303540303, -87.83243309288106 41.95838055972907, -87.8324624009116 41.95850187221258, -87.83248772169686 41.95860661966464, -87.8325069573792 41.95868554492669, -87.83253370234193 41.95879619864407, -87.83255789662839 41.95889449252687, -87.83257006260658 41.95894196538217, -87.83257044931642 41.95894347721133, -87.83257687632721 41.95896861400327, -87.83260405199687 41.9590758941912, -87.83262249677091 41.95916669843622, -87.83263877202495 41.95927154360416, -87.83267784223321 41.95943814989472, -87.83297252270887 41.95943331549363, -87.83385274175389 41.95941088668096, -87.83422641342641 41.95940440340458, -87.83437291102128 41.95940186978249, -87.83447377159909 41.95940063383122, -87.8346009149039 41.9593984784948, -87.8347752969208 41.9593955221311, -87.83487579609124 41.95939343382448, -87.83511260763802 41.95938891840714, -87.83541303182989 41.95938355382284, -87.83556423683719 41.95938088871298, -87.83578181207591 41.95937722127146, -87.83598339373656 41.95937375883114, -87.83641895772763 41.95936538175754, -87.83651949315204 41.95936334677874, -87.83669886448804 41.95936021177394, -87.83669874719291 41.95976354304942, -87.83669985861059 41.95988462268776, -87.83670077917434 41.95998487289559, -87.83670056797425 41.96002158959722, -87.83670051365763 41.96008531006616, -87.83670171545495 41.96018627498327, -87.83670198681787 41.96026023278585, -87.83670127900254 41.96037032767943, -87.83670113453144 41.96038893311479, -87.83670009260528 41.96048997037291, -87.83669887456753 41.96061842186066, -87.83669816481911 41.96075428492754, -87.8366973599217 41.9609081770785, -87.83669894029357 41.96100494441967, -87.83669985141179 41.96106076626109, -87.8366958166928 41.96114005664624, -87.8366955788707 41.9612227662824, -87.83669552872895 41.96131445058919, -87.8366952955309 41.96136398871194, -87.83669462343656 41.96150678847807, -87.83669466849891 41.9615986172635, -87.83669477149785 41.96180964075177, -87.83669383715282 41.96188266028461, -87.83669318443253 41.96202826553308, -87.83669339645262 41.9621571622471, -87.83669382656463 41.96225806917996, -87.83669396222083 41.96234945186151, -87.83669386612068 41.96254179349067, -87.83669360051188 41.9626612208593, -87.83669315402113 41.96279925267226, -87.83669256784437 41.96295517643377, -87.83669340531326 41.96304903275689, -87.83669410166701 41.96312697092397, -87.83669329076116 41.96320298216826, -87.83669376004063 41.96320424626273, -87.83668794590677 41.9648920395117, -87.83668699594458 41.96516779762429, -87.83668207224187 41.96659715458004, -87.83678007782255 41.96659521272343, -87.83717502146928 41.96658835169885, -87.8374108625179 41.96658437622244, -87.83759787721281 41.96657969548051, -87.83793449543053 41.96657236208748, -87.83825599977061 41.96656535660203, -87.83851405584251 41.96656059650611, -87.83869068359691 41.96655817462885, -87.8388768744618 41.96655549125506, -87.83915798822558 41.96655044400267, -87.83947477441357 41.96654475534289, -87.83954258441203 41.96654411398625, -87.83979368237539 41.96654047475343, -87.84007118457849 41.96653645226564, -87.84012973031206 41.96653560610535, -87.84039686178389 41.96653171418163, -87.84051814492334 41.96652994728657, -87.84069085152194 41.96652587433936, -87.84098580510108 41.96651891851288, -87.84118524534658 41.96651455935971, -87.84131502694987 41.96651220703779, -87.84143145908899 41.96651012640374, -87.84162251596268 41.96650576514804, -87.84171979167108 41.96650354427086, -87.84196994887745 41.96649783314949, -87.84220457515488 41.96649395239368, -87.84245174293216 41.96648971315111, -87.84254474847685 41.96648810750445, -87.84280027929533 41.96648415847429, -87.84303716445046 41.96648049697269, -87.84314300774875 41.96647825961251, -87.84330773024595 41.96647473672295, -87.84336388803018 41.96647359881096, -87.84343133571905 41.96647219071952, -87.84365507936356 41.9664680959073, -87.8438138741858 41.9664655121482, -87.84404794850116 41.96646170293243, -87.84428312568124 41.96645787516037, -87.84458479850791 41.96645210746918, -87.84492009182016 41.9664448349424, -87.84526074912402 41.96643804746339, -87.84557386175763 41.96643180769843, -87.84582331574327 41.96642666559311, -87.84609104705798 41.96642116061926, -87.84649039402733 41.96640939220175, -87.8465703541022 41.96640703560761, -87.84655501160803 41.96694594262333, -87.84654706855838 41.96757194716301, -87.8465391252995 41.96819795108424, -87.84655304497635 41.96887384631602, -87.84656061008509 41.96954971452399, -87.84654591374459 41.97022785953669, -87.84654076820649 41.97090367396609, -87.8465387826712 41.971581872988, -87.84653354078276 41.9718898780441, -87.8465107891234 41.9718869978411, -87.8463772492033 41.97188939524105, -87.84631472546393 41.97189050212756, -87.84588537066419 41.97189817490085, -87.84587477808485 41.97189840434638, -87.84566433053786 41.97190212086819, -87.84544314287514 41.97190612121597, -87.84539525672351 41.97190695995278, -87.84522202958023 41.97191001058761, -87.84519290070983 41.9719105453478, -87.84502842630174 41.97191346825354, -87.84500091583908 41.97191395496449, -87.84486395146152 41.97191644580603, -87.84477980165316 41.97191795407221, -87.84469947702769 41.97191936824034, -87.84455868749968 41.97192189732107, -87.84453507612611 41.97192229075207, -87.84437052770184 41.97192526728425, -87.84433757368858 41.97192584041986, -87.84420612640679 41.97192818932317, -87.84404165186722 41.97193116597312, -87.84395823790078 41.97193262134009, -87.8438771773712 41.97193408695455, -87.84371270286029 41.97193700797464, -87.84354830109348 41.97193998395363, -87.84338375337948 41.9719429041901, -87.84331637473525 41.97194409849254, -87.84321935237094 41.97194582536585, -87.84305487738995 41.97194880005115, -87.842890476362 41.97195171993256, -87.84272592744011 41.97195469410509, -87.84267546745947 41.97195557614126, -87.84256152675209 41.97195761379123, -87.84239705213847 41.97196053292578, -87.8422325770905 41.971963506707, -87.84206810245119 41.97196642537011, -87.84203463395447 41.97196699564306, -87.84190370097646 41.97196934411074, -87.84173915271225 41.97197231687158, -87.84157511893731 41.97197523671611, -87.84157613011433 41.97186294793625, -87.84157726545132 41.97173446884106, -87.84157840078129 41.97160598974295, -87.84157959525825 41.97146977221991, -87.84157988826101 41.97144128856324, -87.84158086358502 41.97133360989418, -87.84158194232697 41.97121248503089, -87.84158236793071 41.97116671340984, -87.84158302064051 41.97109141504745, -87.8415840993699 41.97097029017898, -87.84158435152861 41.97094702034651, -87.8415851780909 41.97084916558237, -87.84158625596771 41.97072814992473, -87.84158626187532 41.97072738184524, -87.84158740821968 41.97060702563795, -87.84158824537741 41.97050774364889, -87.84158841380258 41.97048584556172, -87.84158956519958 41.97036483076018, -87.84159072535887 41.97023311356788, -87.84158907474111 41.97014335732682, -87.84158247494598 41.97014362213954, -87.8413439440958 41.97014695826061, -87.8412295308763 41.9701486907042, -87.84104916223809 41.97015332160011, -87.84082473078058 41.97015908425674, -87.84068265754121 41.97016171284982, -87.84053248314079 41.9701657883674, -87.84035985712065 41.97016772010123, -87.84018156723667 41.9701697150609, -87.83991765043866 41.97017439881459, -87.839743140165 41.97017756629712, -87.83971471013774 41.97017808154239, -87.83957538663121 41.97018176366269, -87.83936193472914 41.97018474222117, -87.83927397347752 41.97018485767427, -87.83912657067711 41.97018714966325, -87.83896710496454 41.97018962881419, -87.83887875667922 41.97019223980485, -87.83873737878578 41.97019528034244, -87.83863124305506 41.97019652473083, -87.8384418747693 41.97019981775591, -87.83826957131507 41.97020281358863, -87.83813683977765 41.97020523260929, -87.8380681008628 41.97020661006094, -87.83789454362318 41.97021010211262, -87.83772892978401 41.97021343390309, -87.83756019315763 41.9702164374826, -87.83741113438523 41.97021886720735, -87.83730297042423 41.97022084199453, -87.83725291566425 41.97022175111615, -87.83707708550608 41.97022428425973, -87.83692751296716 41.97022657392511, -87.83674075219135 41.97023056092251, -87.83665028725368 41.97023249227794, -87.83664890926217 41.97136623139546, -87.83683830722961 41.971367148885, -87.83795130236537 41.97134671012508, -87.83795105997105 41.97137799308494, -87.83794998577683 41.97151662598908, -87.83794915482332 41.97162386616677, -87.83794891157507 41.97165525888983, -87.8379478369405 41.97179394666959, -87.83794676272365 41.97193257956359, -87.83794568807396 41.97207126733657, -87.83794461384201 41.97220990022379, -87.83794361272271 41.97234858830738, -87.8379432121137 41.97240028750547, -87.8379425384758 41.97248722118783, -87.83794146422136 41.97262585406492, -87.83794041590166 41.97276113911392, -87.83794038953407 41.97276454182101, -87.83793931526453 41.9729031746913, -87.83793824055789 41.97304186298943, -87.83793761963844 41.97312199069945, -87.8379371662775 41.97318049530416, -87.83793609156 41.9733191830467)), ((-87.69602076978408 41.69892817458145, -87.69135882095155 41.69898049457606, -87.69123796907824 41.69898185020636, -87.69100890322041 41.69228343476094, -87.69170815973916 41.69227496422199, -87.69170580256993 41.69220639645793, -87.69210486704783 41.69220154422303, -87.69210288810858 41.69214277640362, -87.69208981238759 41.69175984169647, -87.69576270224641 41.69171534063086, -87.69602076978408 41.69892817458145)), ((-87.82413707733014 41.9783005778378, -87.82415310615983 41.97817844911364, -87.8241693397196 41.97805475764334, -87.82417092074311 41.97804238828083, -87.82417871280286 41.97798152888081, -87.82419098607811 41.97788586536314, -87.82421188397888 41.97773016911766, -87.82422101279786 41.97766215353269, -87.8242278568279 41.97761092208907, -87.82424894787124 41.97744614390771, -87.82426891731107 41.97729011421752, -87.82428252555458 41.97717922624552, -87.82429776127648 41.97705783517056, -87.82431008968179 41.9769598667344, -87.82432823032413 41.97681782519481, -87.82434507536061 41.97668593037606, -87.82436103139221 41.97656618872598, -87.82440324130116 41.9762535915738, -87.82442649152243 41.97607478516938, -87.82454838995766 41.97607432438102, -87.8248668391446 41.97606903938637, -87.825012345525 41.97606655535379, -87.82511371354029 41.9760648636635, -87.82524641928752 41.97606265197119, -87.82536066102244 41.97606074261721, -87.82547115091789 41.97605887143642, -87.82560753536431 41.97605656583204, -87.8256959568111 41.97605509079003, -87.82585440961115 41.97605244339964, -87.82592076231214 41.97605130997606, -87.82610128353139 41.97604826555067, -87.82614556742752 41.9760475281713, -87.8263481569893 41.97604414205289, -87.82637029970432 41.97604374587902, -87.8265951043379 41.9760400183503, -87.8269629161896 41.97603362819719, -87.82719588774228 41.97602960648369, -87.82720692192171 41.97602943531928, -87.82741628012479 41.97602580246105, -87.82742731393108 41.97602563182266, -87.82763674602829 41.97602199916236, -87.82764777983573 41.97602182822835, -87.82785721191188 41.97601819516571, -87.82786824652001 41.9760179693299, -87.82807767777338 41.97601439074556, -87.82808871238028 41.97601416488851, -87.82829807006318 41.97601058557829, -87.82830910466886 41.97601035970004, -87.82851853588051 41.97600678031118, -87.82852957048495 41.97600655441175, -87.82873907565849 41.97600292006156, -87.82875003627889 41.97600274869993, -87.82895946788182 41.97599911362411, -87.82897050205075 41.97599894256455, -87.829179860083 41.97599530676341, -87.82919089425091 41.97599513568265, -87.82940032617944 41.97599149980394, -87.82941135997854 41.97599132870036, -87.82962079151821 41.9759876924177, -87.82963182568403 41.97598752129455, -87.82984125757037 41.97598388461113, -87.82985229136739 41.97598371346518, -87.8300616496809 41.97598007633142, -87.83007268347905 41.97597990488985, -87.83028211495545 41.97597626740032, -87.83029314955004 41.97597604133129, -87.83050258094119 41.97597245832331, -87.83051361516679 41.97597223223147, -87.83072304580152 41.9759686488179, -87.8307340807614 41.97596842270814, -87.83094343782565 41.97596483857038, -87.83095447278428 41.97596461243939, -87.83116390380833 41.97596097333899, -87.8311749383346 41.97596080206909, -87.83138436933751 41.9759571625664, -87.83139540386274 41.97595699127528, -87.83161601646781 41.97595318070081, -87.83185877134144 41.97594897246777, -87.83207923680079 41.97594516036024, -87.83229955513885 41.97594134718723, -87.83252002092156 41.97593753423454, -87.83274048594659 41.97593372085511, -87.83296087776546 41.97592990700759, -87.83318134348373 41.97592609251004, -87.83340180807441 41.97592227785855, -87.83362227374612 41.97591846278835, -87.83362054195676 41.97613058257649, -87.83361911109169 41.97631361533122, -87.83361769437629 41.97649483641533, -87.83361686030028 41.97660152760616, -87.83361886640175 41.97689061123813, -87.83353257689161 41.97689210166315, -87.8333981771536 41.97689437043987, -87.83336728109352 41.97689489450077, -87.83322015493445 41.97689738206524, -87.8332017646313 41.97689768613905, -87.83317741506843 41.97689812889997, -87.83301072135203 41.97690097001802, -87.83295679932725 41.97690188757337, -87.83290780642267 41.97690271682135, -87.83284527804641 41.9769037614611, -87.83273603762822 41.97690559030212, -87.83267998256548 41.97690655331039, -87.83259538511402 41.97690799552579, -87.83251534866065 41.97690934780776, -87.83245936714086 41.97691031130354, -87.8322946604692 41.9769130500083, -87.83228296339428 41.97691327337814, -87.83226825104715 41.97691353847051, -87.83207706028116 41.97691676526766, -87.83207389753761 41.97691680661683, -87.83185328168983 41.97692056289441, -87.83161052478552 41.97692460644992, -87.83139071794753 41.97692836564853, -87.83133098482286 41.97692942168489, -87.83117032298672 41.97693217673787, -87.83116568806398 41.97693226623218, -87.83113354118042 41.97693283910464, -87.83111058985595 41.97693323265952, -87.83094985371541 41.97693598735326, -87.8308901205807 41.97693704288575, -87.83087636444658 41.97693725710054, -87.83085341275169 41.97693765060237, -87.83072953269465 41.97693974276122, -87.83066972606491 41.97694074324753, -87.83061926129736 41.97694161995829, -87.83054113745904 41.97694297923896, -87.83050906374912 41.976943552257, -87.83044925668173 41.9769446075107, -87.8303620844398 41.97694609168556, -87.83028866833213 41.97694736165154, -87.83022878813993 41.97694836158575, -87.83010498116685 41.97695050827504, -87.8300682733251 41.97695111574053, -87.82991651232722 41.97695374340496, -87.82984780431333 41.97695492396595, -87.82962740883012 41.97695873209054, -87.82960423647019 41.97695912437443, -87.82960299520776 41.97726592137348, -87.82960729437102 41.97733620814866, -87.82964687477852 41.97733540514874, -87.8297650167579 41.97733348108431, -87.83001704483165 41.97732926254414, -87.83001844474285 41.9773292390442, -87.83003984507545 41.9773288786993, -87.83003740750144 41.97739609122536, -87.83004478484116 41.977692278192, -87.8300520887462 41.97798841021139, -87.83005429685716 41.9780581447159, -87.83002562042712 41.9780586718445, -87.83002163520814 41.97805881655526, -87.83002312663314 41.97812615194988, -87.82971062519526 41.97813147760293, -87.82971693244316 41.97842321451555, -87.82940457843793 41.97842826528524, -87.82922067671775 41.97843009232709, -87.82892973022842 41.97843479673363, -87.8288713197412 41.97843574753129, -87.82863885615261 41.97843950072104, -87.82863735489623 41.97876940270669, -87.82864026203896 41.97883993639493, -87.8286952646211 41.97883899680498, -87.82869688906358 41.97890561243662, -87.82869622894853 41.97911960279482, -87.82817447032325 41.9791231809641, -87.82808245720213 41.9791228630334, -87.82808224428653 41.9791680816862, -87.82808187823825 41.97927952231524, -87.82808178632806 41.979291149115, -87.82817388591074 41.97929019054752, -87.82869565319818 41.97928573454873, -87.82869508476152 41.97946020871858, -87.82869456008288 41.97962913977797, -87.82869399082989 41.97980366881796, -87.8286934886043 41.97996980087534, -87.82869277463294 41.9801906514733, -87.82869139120152 41.98032794736756, -87.82886118828516 41.98032514428193, -87.82885945677471 41.98060038028974, -87.82919087726481 41.98059481129694, -87.82941157866043 41.98059111513715, -87.82976352397111 41.98058519523629, -87.82977470567246 41.98058502474754, -87.83007257902827 41.98058001927279, -87.8300898674438 41.98057971114574, -87.83030490453392 41.98057609793176, -87.83041613842018 41.98057422553535, -87.83051376133021 41.98057262217259, -87.83051994043416 41.98057253946828, -87.83074248143551 41.98056873903854, -87.83075726891897 41.98056847447169, -87.83079302218958 41.98056791749393, -87.83097237913066 41.98056486060182, -87.83106882550499 41.98056325189292, -87.83107220945959 41.98056321181392, -87.83118741576705 41.98056124600699, -87.83135146988215 41.98055850577471, -87.83139509523706 41.98055776349448, -87.83140245201528 41.98055763100765, -87.83161756216573 41.9805540159284, -87.83163065708315 41.98055379873612, -87.83172143921252 41.98055227449302, -87.83173777097342 41.98055201654747, -87.83181413303092 41.98055075825518, -87.83190999092872 41.9805491465431, -87.83202924393854 41.98054708779804, -87.83204785596023 41.98054678488152, -87.83224435359917 41.9805435264262, -87.8323630173544 41.98054151964651, -87.83236360579313 41.98054152221435, -87.83252721929229 41.98053872349475, -87.83266008126667 41.98053650392478, -87.83268936045494 41.98053602788937, -87.83281023125959 41.98053397566254, -87.83302556231084 41.98053035863339, -87.83324619020821 41.98052659989625, -87.83362057264102 41.98052027194916, -87.83423861006747 41.98050978833594, -87.83448145482974 41.98050568473701, -87.83510037430878 41.98049520031665, -87.83509820813147 41.9806598433065, -87.83509588281446 41.98083546242653, -87.83447696003924 41.98084594680429, -87.83435523849998 41.98084605062215, -87.83435336148197 41.98093821870797, -87.83435100539377 41.98099965124621, -87.83434964536718 41.98115580521452, -87.83452496888466 41.98115070742725, -87.8347555874962 41.98114885526122, -87.83482743926852 41.98114901087129, -87.83500766672027 41.98114940092346, -87.83509199451355 41.98114886986195, -87.83508993982259 41.98128641949482, -87.83508686754 41.98152010275433, -87.83508462477644 41.98168507468691, -87.83508245708478 41.98184999205641, -87.83508028784662 41.98201496429856, -87.83507811902102 41.98217988165359, -87.83507595085555 41.98234485389084, -87.83507378157145 41.98250982611847, -87.83507161269992 41.98267474345897, -87.83445293740593 41.98267013562931, -87.83421037089792 41.98265634814783, -87.83359591866922 41.98266668253804, -87.83359559976701 41.98268874456165, -87.83359449248586 41.98277380983525, -87.83359374799586 41.98283143545642, -87.83359306671871 41.98288088307141, -87.83359255200669 41.98291853105369, -87.83359157694369 41.98299618781957, -87.83358943812355 41.98315676940074, -87.83358706595325 41.98333787661603, -87.83358489613943 41.98350246459205, -87.83358418817235 41.98355536978337, -87.83358399143781 41.98357112066378, -87.83321106309739 41.98356790551976, -87.83320990189755 41.98365988566335, -87.8332082227337 41.9836770570711, -87.83319402853873 41.98371426153611, -87.83318415247882 41.98372607347454, -87.83316648023778 41.98374707197384, -87.83312773273806 41.98377286332742, -87.83308089115168 41.98378950863677, -87.83302971723124 41.98379576195188, -87.83299962740423 41.98379623481367, -87.83277899085056 41.98379960856214, -87.83249582037109 41.98380391680962, -87.83221272341883 41.98380822467953, -87.8319295528766 41.98381253152978, -87.83164638230296 41.98381683768134, -87.83163402230292 41.98381705808639, -87.83136328525659 41.98382114345582, -87.83108011418891 41.98382550309235, -87.83079694352097 41.98382980714796, -87.83051384638031 41.98383411082706, -87.83023067564972 41.98383841348543, -87.82998804216972 41.98384206992953, -87.82970678353738 41.98384637967921, -87.82942567279163 41.98385063450476, -87.82912256489111 41.98385523110272, -87.82881585097797 41.98385992109529, -87.82848309374458 41.98386498914896, -87.8281133312035 41.98387060715465, -87.8281359715038 41.98191858904916, -87.82767749049884 41.98192633987815, -87.82736673656787 41.98193155678169, -87.82697792505009 41.98193812998415, -87.82675177479685 41.98194196206717, -87.8267330149388 41.98194220858797, -87.82644779720337 41.98194594405035, -87.82616250625294 41.9819496784806, -87.82601346008843 41.98195165435455, -87.82587441715066 41.98195372913757, -87.8255828682499 41.9819580933482, -87.82528808240315 41.98196249682858, -87.82527130892635 41.98196275190119, -87.82527388667829 41.9820641342565, -87.82527646443678 41.98216551661001, -87.82527904220191 41.9822668989617, -87.82528161997361 41.9823682813116, -87.82528419694783 41.98246971854024, -87.82528677510051 41.98257110088814, -87.8252893533283 41.98267242835047, -87.82529764300166 41.98299913519931, -87.82446763871476 41.98301148523738, -87.82364594705078 41.98302375641229, -87.82352826134645 41.98302551346085, -87.82353868664269 41.98292352829993, -87.82356731868674 41.98269078221307, -87.8235838969633 41.98255833844136, -87.82360146649071 41.98242131596854, -87.82362004708715 41.98227395452066, -87.82363707868231 41.98213887556297, -87.8236526101996 41.98202226063673, -87.82368059741545 41.98181492323806, -87.8237012238672 41.98166361699241, -87.82372065816212 41.98151872716227, -87.8237392948368 41.98136555424172, -87.82376233909871 41.98117616062323, -87.82379284710989 41.9809431757563, -87.82381013158843 41.9808096370885, -87.82383905137942 41.9805867712652, -87.82385639060944 41.98045542818982, -87.82387459165994 41.98031755772273, -87.82389044513967 41.98019734924523, -87.82390864669176 41.98005939671687, -87.82392441788295 41.979940313263, -87.82393854712649 41.97983337935302, -87.82396171793695 41.97965790783826, -87.82397455071634 41.97955063913572, -87.82398788155511 41.97943920062222, -87.82400954931181 41.97927697720929, -87.82402558629025 41.97915679722147, -87.82404181531807 41.97903549295607, -87.82405856220542 41.9789046685854, -87.82407809249905 41.97875217732204, -87.82409327193065 41.97863767401787, -87.82411080447129 41.97850542667658, -87.82412317058139 41.97841198584573, -87.82413707733014 41.9783005778378)))')

In [12]:
# Append newly created polygons to geometry
# county_gpd_dynamic = county_gpd.append([{'id':"Out Of State", 'geometry':polygonOOS}], ignore_index=True)
# county_gpd_dynamic = county_gpd_dynamic.append([{'id':"Unassigned", 'geometry':polygonIllinois}], ignore_index=True)
county_gpd_dynamic = county_gpd.append([{'id':"Illinois", 'geometry':polygonIllinois}], ignore_index=True)
county_gpd_dynamic = county_gpd_dynamic[county_gpd_dynamic['id'] != 'Cook']
county_gpd_dynamic = county_gpd_dynamic.append([{'id':"Cook", 'geometry':polyCookWithoutChicago}], ignore_index=True)
county_gpd_dynamic

,id,geometry
0,Chicago,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ..."
1,McHenry,"POLYGON ((-88.70742 42.49352, -88.70741 42.493..."
2,Boone,"POLYGON ((-88.70742 42.49352, -88.70750 42.493..."
3,Ogle,"POLYGON ((-89.68809 42.19950, -89.68807 42.184..."
4,Will,"POLYGON ((-88.26146 41.72439, -88.26103 41.708..."
...,...,...
99,Marion,"POLYGON ((-89.13844 38.73633, -89.13847 38.721..."
100,Union,"POLYGON ((-89.04143 37.59650, -89.06017 37.597..."
101,Pope,"POLYGON ((-88.70860 37.59926, -88.70876 37.584..."
102,Illinois,"POLYGON ((-89.64923 39.78269, -89.64923 39.779..."


Zipcode Data generation

In [13]:
zipcode_data

,zip,confirmed_cases,total_tested
0,60000,42,416
1,60001,6,158
2,60002,1573,13692
3,60004,3282,51205
4,60005,2065,31872
...,...,...,...
1419,62995,683,17712
1420,62996,67,328
1421,62997,40,566
1422,62998,31,313


In [14]:
# Merge geometry with zipcode report
zipcode_final_gpd = pd.merge(zipcode_gpd,zipcode_data, how = 'left', left_on=['id'], right_on=['zip'])

In [15]:
# Replace NA values with 0
zipcode_final_gpd['confirmed_cases'] = zipcode_final_gpd['confirmed_cases'].replace(np.nan,0)
zipcode_final_gpd['total_tested'] = zipcode_final_gpd['total_tested'].replace(np.nan,0)
zipcode_final_gpd = zipcode_final_gpd[['id','confirmed_cases','total_tested','geometry']]

In [16]:
# Save file
zipcode_final_gpd.to_file('dph_zipcode_data.geojson', driver='GeoJSON', encoding='utf-8')
print('done')

done


County Data generation


In [17]:
# IDPH data started from 03/17/2020
start_date = '03/17/2020'

# Get last update date from IDPH API
last_update_date_json = requests.get("https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetCountyHistorical?countyName=Illinois").json()
last_update_date = last_update_date_json['values'][-1]['reportDate'][0:10]

# Get JSON file for each county in GeoJson file and append them all together
county_history_list = []
for county in county_gpd_dynamic['id']:
    county_data_url = "https://idph.illinois.gov/DPHPublicInformation/api/COVID/GetCountyHistorical?countyName=" + county
    d = requests.get(county_data_url).json()
    county_history_list = county_history_list + d['values']

# Generate dataframe
county_history_df = pd.DataFrame(county_history_list)
county_history_df

,reportDate,CountyName,tested,confirmed_cases,deaths,latitude,longitude
0,2020-03-17T00:00:00,Chicago,0,0,0,41.789896,-87.627734
1,2020-03-18T00:00:00,Chicago,0,0,0,41.789896,-87.627734
2,2020-03-19T00:00:00,Chicago,0,0,0,41.789896,-87.627734
3,2020-03-20T00:00:00,Chicago,0,0,0,41.789896,-87.627734
4,2020-03-21T00:00:00,Chicago,0,0,0,41.789896,-87.627734
...,...,...,...,...,...,...,...
33899,2021-02-01T00:00:00,Cook,2757584,219443,4244,42.050784,-87.963759
33900,2021-02-02T00:00:00,Cook,2765503,219772,4244,42.050784,-87.963759
33901,2021-02-03T00:00:00,Cook,2776964,220305,4260,42.050784,-87.963759
33902,2021-02-04T00:00:00,Cook,2792830,220869,4263,42.050784,-87.963759


In [18]:
# Change the date format and column names
county_history_df['reportDate'] = county_history_df['reportDate'].apply(lambda x: x[0:10])
county_history = county_history_df.rename(columns={"reportDate":"date", "CountyName": "County", "tested": "total_tested"})
county_history

,date,County,total_tested,confirmed_cases,deaths,latitude,longitude
0,2020-03-17,Chicago,0,0,0,41.789896,-87.627734
1,2020-03-18,Chicago,0,0,0,41.789896,-87.627734
2,2020-03-19,Chicago,0,0,0,41.789896,-87.627734
3,2020-03-20,Chicago,0,0,0,41.789896,-87.627734
4,2020-03-21,Chicago,0,0,0,41.789896,-87.627734
...,...,...,...,...,...,...,...
33899,2021-02-01,Cook,2757584,219443,4244,42.050784,-87.963759
33900,2021-02-02,Cook,2765503,219772,4244,42.050784,-87.963759
33901,2021-02-03,Cook,2776964,220305,4260,42.050784,-87.963759
33902,2021-02-04,Cook,2792830,220869,4263,42.050784,-87.963759


In [19]:
# Eliminate data from last update date or unassigned
county_history = county_history[county_history['date'] != last_update_date]
county_history = county_history[county_history['County'] != 'Unassigned']
county_history

,date,County,total_tested,confirmed_cases,deaths,latitude,longitude
0,2020-03-17,Chicago,0,0,0,41.789896,-87.627734
1,2020-03-18,Chicago,0,0,0,41.789896,-87.627734
2,2020-03-19,Chicago,0,0,0,41.789896,-87.627734
3,2020-03-20,Chicago,0,0,0,41.789896,-87.627734
4,2020-03-21,Chicago,0,0,0,41.789896,-87.627734
...,...,...,...,...,...,...,...
33898,2021-01-31,Cook,2746033,219000,4231,42.050784,-87.963759
33899,2021-02-01,Cook,2757584,219443,4244,42.050784,-87.963759
33900,2021-02-02,Cook,2765503,219772,4244,42.050784,-87.963759
33901,2021-02-03,Cook,2776964,220305,4260,42.050784,-87.963759


In [20]:
# Create pivot tables
county_pivot = pd.pivot_table(county_history, index=['County'],columns=['date'])
county_pivot = county_pivot.replace(np.nan, 0).astype(int)

county_cases = county_pivot['confirmed_cases']
county_deaths = county_pivot['deaths']
county_tested = county_pivot['total_tested']

In [21]:
# Add missing 03/23 data with 03/22
county_cases['2020-03-23'] = county_cases['2020-03-22']
county_deaths['2020-03-23'] = county_deaths['2020-03-22']
county_tested['2020-03-23'] = county_tested['2020-03-22']
county_tested

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

date,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-24,2020-03-25,2020-03-26,2020-03-27,...,2021-01-27,2021-01-28,2021-01-29,2021-01-30,2021-01-31,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2020-03-23
County,,,,,,,,,,,,,,,,,,,,,
Adams,0,0,0,0,0,0,0,0,0,0,...,109847,110637,111434,112115,112640,113015,113687,114446,115339,0
Alexander,0,0,0,0,0,0,0,0,0,0,...,4406,4431,4454,4467,4489,4500,4516,4533,4559,0
Bond,0,0,0,0,0,0,0,0,0,0,...,19042,19339,19626,19800,19977,20036,20147,20374,20507,0
Boone,0,0,0,0,0,0,0,0,0,0,...,45906,46079,46242,46537,46698,46901,47030,47196,47459,0
Brown,0,0,0,0,0,0,0,0,0,0,...,15161,15561,15582,16124,16900,16908,16945,18121,18247,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Whiteside,0,0,0,0,0,0,0,0,0,0,...,48949,49289,49563,50042,50226,50371,50586,51004,51289,0
Will,0,0,0,0,0,0,0,0,0,0,...,613941,618827,623935,628466,632560,635505,638119,641217,645065,0
Williamson,0,0,0,0,0,0,0,0,0,0,...,74828,75361,76138,76369,76789,77035,77301,77747,78392,0


In [22]:
# Get date information
date = county_cases.columns.tolist()
date.sort()
dt_first = date[0]
dt_today = date[-1]
dt_yesterday = date[-2]

In [23]:
# Define function for finding missing dates
def find_missing_date(date):
    missing_dates = []
    dt_range = datetime.strptime(date[-1], "%Y-%m-%d") - datetime.strptime(date[0], "%Y-%m-%d")
    if len(date) != dt_range.days + 1:
        print('Alert! Missing Date or Redundant date')
        for x in range(len(date) - 1):
            if (datetime.strptime(date[x + 1], "%Y-%m-%d") - datetime.strptime(date[x], "%Y-%m-%d")).days != 1:
                missing_date = datetime.strptime(date[x], "%Y-%m-%d") + timedelta(days=1)
                print('Missing Date:')
                print(missing_date)
                missing_dates.append(missing_date)
    return missing_dates

In [24]:
# Looking for missing dates
missing = find_missing_date(date)

In [25]:
# Fill in missing data for dates
for d_missing in missing:
   d_yesterday = d_missing - timedelta(days=1)
   d_key_yesterday = d_yesterday.strftime("%Y-%m-%d")
   d_key_missing = d_missing.strftime("%Y-%m-%d")
   county_cases[d_key_missing] = county_cases[d_key_yesterday]
   county_deaths[d_key_missing] = county_deaths[d_key_yesterday]
   county_tested[d_key_missing] = county_tested[d_key_yesterday]

In [26]:
# Check missing dates again
date = county_cases.columns.tolist()
date.sort()
missings = find_missing_date(date)
if len(missings)> 0:
    raise Exception("Missing dates: {}".format(missings))

In [27]:
# Get date information
date = county_cases.columns.tolist()
date.sort()
dt_first = date[0]
dt_today = date[-1]
dt_yesterday = date[-2]

In [28]:
# Reorder date
county_cases = county_cases[date]
county_deaths = county_deaths[date]
county_tested = county_tested[date]

In [29]:
# Get case time series
cases_ts = county_cases.values.tolist()
deaths_ts = county_deaths.values.tolist()
tested_ts = county_tested.values.tolist()

In [30]:
# Get first case date
county_cases['dt_first_case'] = (county_cases > 0).idxmax(axis=1)
county_cases.loc[county_cases.iloc[:, -2] <= 0, 'dt_first_case'] = np.nan
county_deaths['dt_first_death'] = (county_deaths > 0).idxmax(axis=1)
county_deaths.loc[county_deaths.iloc[:, -2] <= 0, 'dt_first_death'] = np.nan

In [31]:
county_cases['cases_ts'] = cases_ts
county_deaths['deaths_ts'] = deaths_ts

In [32]:
# Add today_case and today_new_case columns
county_cases['today_case'] = county_cases[dt_today]
county_cases['today_new_case'] = county_cases[dt_today] - county_cases[dt_yesterday]
county_deaths['today_death'] = county_deaths[dt_today]
county_deaths['today_new_death'] = county_deaths[dt_today] - county_deaths[dt_yesterday]
county_tested['today_tested'] = county_tested[dt_today]
county_tested['today_new_tested'] = county_tested[dt_today] - county_tested[dt_yesterday]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
# Generate reports
case_report = county_cases[['cases_ts','dt_first_case','today_case','today_new_case']]
death_report = county_deaths[['deaths_ts','dt_first_death','today_death','today_new_death']]
tested_report = county_tested[['today_tested','today_new_tested']]
county_report = case_report.join(death_report, how="outer").join(tested_report, how="outer")

In [34]:
# Reset index for county report
county_report = county_report.reset_index()
county_report.columns = ['NAME','cases_ts','dt_first_case','today_case','today_new_case','deaths_ts','dt_first_death','today_death','today_new_death','today_tested','today_new_tested']
county_report['cases_ts'] = county_report['cases_ts'].apply(lambda x: ','.join(map(str,x)))
county_report['deaths_ts'] = county_report['deaths_ts'].apply(lambda x: ','.join(map(str,x)))

In [35]:
# Illinois, Out of State and Suburban Cook is not in geometry
np.setdiff1d(county_report['NAME'],county_gpd_dynamic['id'])

array([], dtype=object)

In [36]:
# Remove 'Out Of State' and 'Suburban Cook' from county report
county_report = county_report[(county_report['NAME'] != 'Out Of State') & (county_report['NAME'] != 'Suburban Cook')]

In [37]:
# Add start and end dates
county_report['start'] = dt_first
county_report['end'] = dt_today
county_report['dt_unit'] = 'day'
county_report

,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit
0,Adams,"0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,3,3,8,9,11...",2020-03-20,7761,14,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-01,108,0,115339,893,2020-03-17,2021-02-04,day
1,Alexander,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-16,422,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-03,8,0,4559,26,2020-03-17,2021-02-04,day
2,Bond,"0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,2,3,3,3,3,3,3,3,...",2020-03-29,1822,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-16,21,0,20507,133,2020-03-17,2021-02-04,day
3,Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,5765,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,73,0,47459,263,2020-03-17,2021-02-04,day
4,Brown,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-01,646,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-11-12,6,0,18247,126,2020-03-17,2021-02-04,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Whiteside,"1,1,1,1,1,1,1,1,1,1,2,2,3,4,5,5,6,9,15,18,18,2...",2020-03-17,5629,19,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,...",2020-04-02,139,1,51289,285,2020-03-17,2021-02-04,day
100,Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,61943,206,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,854,7,645065,3848,2020-03-17,2021-02-04,day
101,Williamson,"0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,2,3,3,3,8,9,...",2020-03-19,6588,21,"0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-03-19,112,0,78392,645,2020-03-17,2021-02-04,day
102,Winnebago,"1,1,1,2,4,5,5,5,7,8,8,9,12,13,15,18,24,28,36,4...",2020-03-17,27488,60,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,3,...",2020-04-01,405,2,295445,1591,2020-03-17,2021-02-04,day


In [38]:
# Merge geometry with county reports
county_final_gpd = pd.merge(county_gpd_dynamic, county_report, how="left", left_on="id", right_on="NAME")
county_final_gpd

,id,geometry,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit
0,Chicago,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ...",Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,236416,577,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,4683,17,3060439,16093,2020-03-17,2021-02-04,day
1,McHenry,"POLYGON ((-88.70742 42.49352, -88.70741 42.493...",McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,23044,68,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,250,0,241621,1330,2020-03-17,2021-02-04,day
2,Boone,"POLYGON ((-88.70742 42.49352, -88.70750 42.493...",Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,5765,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,73,0,47459,263,2020-03-17,2021-02-04,day
3,Ogle,"POLYGON ((-89.68809 42.19950, -89.68807 42.184...",Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,4824,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,67,0,46093,247,2020-03-17,2021-02-04,day
4,Will,"POLYGON ((-88.26146 41.72439, -88.26103 41.708...",Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,61943,206,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,854,7,645065,3848,2020-03-17,2021-02-04,day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Marion,"POLYGON ((-89.13844 38.73633, -89.13847 38.721...",Marion,"0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,3,3,3,3,4,...",2020-03-30,4201,-6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-28,108,0,57075,991,2020-03-17,2021-02-04,day
100,Union,"POLYGON ((-89.04143 37.59650, -89.06017 37.597...",Union,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,2134,8,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-05,30,0,26254,186,2020-03-17,2021-02-04,day
101,Pope,"POLYGON ((-88.70860 37.59926, -88.70876 37.584...",Pope,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-08,268,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-08-26,3,0,2294,15,2020-03-17,2021-02-04,day
102,Illinois,"POLYGON ((-89.64923 39.78269, -89.64923 39.779...",Illinois,"160,288,422,585,753,1049,1049,1535,1865,2538,3...",2020-03-17,1137559,3328,"0,1,3,5,6,9,9,16,19,26,34,47,65,72,98,141,157,...",2020-03-18,19443,68,16359655,101307,2020-03-17,2021-02-04,day


In [39]:
# Read county population file
pop_df = pd.read_csv('illinois_county_population.txt', sep="\t")
pop_df["Value"]=pop_df["Value"].replace("\,","", regex=True)
pop_df

,County,Value
0,Adams,65691
1,Alexander,6060
2,Bond,16630
3,Boone,53577
4,Brown,6556
...,...,...
97,Whiteside,55626
98,Will,692310
99,Williamson,67056
100,Winnebago,284081


In [40]:
# Correct and add missing information
pop_df.loc[pop_df["County"]== "RockIsland","County"] = "Rock Island"
pop_df.loc[pop_df["County"]== "DeWitt","County"] = "De Witt"
pop_df.loc[pop_df["County"]== "JoDaviess","County"] = "Jo Daviess"
pop_df.loc[pop_df["County"]== "St.Clair","County"] = "St. Clair"
row1 = {'County':'Chicago', 'Value':2693976}
row2 = {'County':'Illinois', 'Value':12671821}
pop_df = pop_df.append(row1, ignore_index=True)
pop_df = pop_df.append(row2, ignore_index=True)
pop_df["Value"] = pop_df["Value"].astype(int)
pop_df

,County,Value
0,Adams,65691
1,Alexander,6060
2,Bond,16630
3,Boone,53577
4,Brown,6556
...,...,...
99,Williamson,67056
100,Winnebago,284081
101,Woodford,38463
102,Chicago,2693976


In [41]:
# Merge geometry with county population
county_final_gpd = pd.merge(county_final_gpd, pop_df, how="left", left_on="id", right_on="County")
county_final_gpd = county_final_gpd.drop(['County'], axis=1)
county_final_gpd = county_final_gpd.rename(columns={"Value": "population"})
county_final_gpd

,id,geometry,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit,population
0,Chicago,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ...",Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,236416,577,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,4683,17,3060439,16093,2020-03-17,2021-02-04,day,2693976
1,McHenry,"POLYGON ((-88.70742 42.49352, -88.70741 42.493...",McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,23044,68,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,250,0,241621,1330,2020-03-17,2021-02-04,day,308570
2,Boone,"POLYGON ((-88.70742 42.49352, -88.70750 42.493...",Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,5765,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,73,0,47459,263,2020-03-17,2021-02-04,day,53577
3,Ogle,"POLYGON ((-89.68809 42.19950, -89.68807 42.184...",Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,4824,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,67,0,46093,247,2020-03-17,2021-02-04,day,50923
4,Will,"POLYGON ((-88.26146 41.72439, -88.26103 41.708...",Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,61943,206,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,854,7,645065,3848,2020-03-17,2021-02-04,day,692310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Marion,"POLYGON ((-89.13844 38.73633, -89.13847 38.721...",Marion,"0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,3,3,3,3,4,...",2020-03-30,4201,-6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-28,108,0,57075,991,2020-03-17,2021-02-04,day,37620
100,Union,"POLYGON ((-89.04143 37.59650, -89.06017 37.597...",Union,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,2134,8,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-05,30,0,26254,186,2020-03-17,2021-02-04,day,16841
101,Pope,"POLYGON ((-88.70860 37.59926, -88.70876 37.584...",Pope,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-08,268,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-08-26,3,0,2294,15,2020-03-17,2021-02-04,day,4212
102,Illinois,"POLYGON ((-89.64923 39.78269, -89.64923 39.779...",Illinois,"160,288,422,585,753,1049,1049,1535,1865,2538,3...",2020-03-17,1137559,3328,"0,1,3,5,6,9,9,16,19,26,34,47,65,72,98,141,157,...",2020-03-18,19443,68,16359655,101307,2020-03-17,2021-02-04,day,12671821


In [42]:
# Check if NA exists for population
county_final_gpd.loc[county_final_gpd["population"].isna()== True,"id"]

Series([], Name: id, dtype: object)

In [43]:
# Calculate weekly change rates of cases and form a time-series list
county_final_gpd['change_ts'] = ""
for x in range(0, len(county_final_gpd)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = county_final_gpd.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
    county_final_gpd.loc[x,'change_ts'] = ','.join(changeRateStrList)

county_final_gpd

,id,geometry,NAME,cases_ts,dt_first_case,today_case,today_new_case,deaths_ts,dt_first_death,today_death,today_new_death,today_tested,today_new_tested,start,end,dt_unit,population,change_ts
0,Chicago,"MULTIPOLYGON (((-87.93514 42.00089, -87.93521 ...",Chicago,"0,0,0,0,0,519,519,782,915,1161,1364,1610,2026,...",2020-03-22,236416,577,"0,0,0,0,0,4,4,5,5,9,9,12,16,16,21,39,41,57,76,...",2020-03-22,4683,17,3060439,16093,2020-03-17,2021-02-04,day,2693976,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.24,1.44,1.37,0.82,..."
1,McHenry,"POLYGON ((-88.70742 42.49352, -88.70741 42.493...",McHenry,"2,4,6,8,11,12,12,14,19,27,45,47,52,52,63,69,81...",2020-03-17,23044,68,"0,0,0,0,0,0,0,0,0,1,1,2,2,2,2,2,3,3,3,3,3,4,5,...",2020-03-26,250,0,241621,1330,2020-03-17,2021-02-04,day,308570,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,3.08,2.33,1.57,0..."
2,Boone,"POLYGON ((-88.70742 42.49352, -88.70750 42.493...",Boone,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,6,7,...",2020-04-05,5765,15,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...",2020-04-08,73,0,47459,263,2020-03-17,2021-02-04,day,53577,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,4.0,..."
3,Ogle,"POLYGON ((-89.68809 42.19950, -89.68807 42.184...",Ogle,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,4,5,7,10,1...",2020-03-31,4824,6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,67,0,46093,247,2020-03-17,2021-02-04,day,50923,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,1.0,4...."
4,Will,"POLYGON ((-88.26146 41.72439, -88.26103 41.708...",Will,"2,3,9,10,12,21,21,28,40,67,104,127,182,223,228...",2020-03-17,61943,206,"0,0,0,1,1,1,1,1,2,3,3,4,4,5,8,9,8,10,11,12,19,...",2020-03-20,854,7,645065,3848,2020-03-17,2021-02-04,day,692310,"0,0,0,0,0,0,0,0,0,0,0,0,0,9.63,6.69,6.62,4.47,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Marion,"POLYGON ((-89.13844 38.73633, -89.13847 38.721...",Marion,"0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,3,3,3,3,3,4,...",2020-03-30,4201,-6,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-09-28,108,0,57075,991,2020-03-17,2021-02-04,day,37620,"0,0,0,0,0,0,0,0,0,0,0,0,0,2.0,2.0,2.0,2.0,3.0,..."
100,Union,"POLYGON ((-89.04143 37.59650, -89.06017 37.597...",Union,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-04-15,2134,8,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-05,30,0,26254,186,2020-03-17,2021-02-04,day,16841,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
101,Pope,"POLYGON ((-88.70860 37.59926, -88.70876 37.584...",Pope,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-05-08,268,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2020-08-26,3,0,2294,15,2020-03-17,2021-02-04,day,4212,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
102,Illinois,"POLYGON ((-89.64923 39.78269, -89.64923 39.779...",Illinois,"160,288,422,585,753,1049,1049,1535,1865,2538,3...",2020-03-17,1137559,3328,"0,1,3,5,6,9,9,16,19,26,34,47,65,72,98,141,157,...",2020-03-18,19443,68,16359655,101307,2020-03-17,2021-02-04,day,12671821,"0,0,0,0,0,0,0,0,0,0,0,0,0,3.51,2.24,2.24,1.44,..."


In [44]:
# Generalize geometry
county_final_gpd['geometry'] = county_final_gpd.apply(lambda x: shapely.wkt.loads(shapely.wkt.dumps(x.geometry, rounding_precision=8)).simplify(0), axis = 1)

In [45]:
# Save file
county_final_gpd.to_file('dph_county_data.geojson', driver='GeoJSON', encoding='utf-8')
print('done')

done
